__[Open and try this file online (Colab)](https://colab.research.google.com/github/djekra/wiktionary_wortschatz/blob/master/1_wiktionary_parsen/x38_d03_reorganisieren.ipynb)__

# Reorganisieren und Reinigen
* speichert in data_03
* lex_test level 3
* 41 min

## ToDo
* schwimmend, erregend: Jeweils beide ADJ-Lemmas unter einer einzelnen lemma_id zusammenlegen
* #TODO_member: member überprüfen wie in d01
* #TODO_Statistik: unterschiedliche Abkürzungen / Genitive in translate_lex sind ohne Statistik ununterscheidbar 
* Nom-Merkmale == lemma löschen
* TODO_verwaiste_lemmas
* TODO_TAGS_IN_MERKMALE


* löschliste am Ende nochmal anwenden
* Prüfen: ändert sich am score in translate_lex_full noch was, wenn echte Statistikzahlen da mit einfließen? Es geht um die Zuordnung lex --> lemma bei gegebenem tagZZ, also z.B. bei 'Alben' (--> Alb, Album, etc.).
* Symmetrie: Es gibt Lexeme für das Lemma "Ackermann/NN". Die gleichen Lexeme sollte es auch für das Lemma "Ackermann/NE" geben.
* Der gleiche Fall triff auch beim Lexem "ertragene" auf. Das sollte nicht nur ein Lexem des Verbs sein, sondern auch ein Lexem des Adjektivs.
* Noch besser: Lemmas zusammenfassen und beide Tag-Felder nutzen

Steigerbare Adverbien prüfen (TODO_Steigerbare Adverbien):
* 'sehr', 'mehr','am meisten'
* 'gern', 'lieber','am liebsten'
* 'bald', 'eher','am ehesten'
* 'oft',  'öfter','am öftesten'  # drin
* 'wohl', 'besser','am besten'   # drin

## Spacy Nachschlagen 
* Kann das Lemma stimmen? >> Nachschlagen in wiktionary_merkmal (hier gibt es alle lex-lemma-Kombinationen, z.B. bei "Alben")
* Wenn nicht fündig geworden: Nachschlagen in translate_lex (hier gibt es nur 1 Lemma pro tagZZ)
* Resultat: Lemma

## Import

In [1]:
# blab init
try:
    import blab
except ImportError as e:
    !pip install blab
    import blab    
startup_notebook = blab.blab_startup()
%run $startup_notebook 

blab init
environment['in_colab']     = False
environment['dropbox_path'] = /home/me/Data_Linux/Dropbox
environment['lib_path']     = /home/me/Data_Linux/Dropbox/31_Projekte/01_Python/libs
Start Time: 22:36:40


In [2]:
# pandas
if environment['in_colab']:
    !pip install --upgrade pandas
import pandas as pd

In [3]:
# alle anderen
try:
    import pandasklar as pak, bpyth as bpy, swifter

except ImportError as e:
    !pip install pandasklar swifter
    import pandasklar as pak, bpyth as bpy, swifter
    
grid       = pak.grid
from functools import partial
gridt = partial(grid, backend='dtale')    # dTale-Nutzung erzwungen
check_mask = pak.check_mask     


## Einstellungen

In [4]:
# Einstellungen

# Was debuggen?
suche_debug = ['besonnte','überlegte',]   
suche_debug = ['besonne',]  # auch gut: besinnen
suche_debug = ['inner-','innere','inner']  
suche_debug = ['Citrone',]  
suche_debug = ['Dezember','December','Citrone',]  
suche_debug = ['Anna',]  
# 'einen','beigen','Montage',
# '24/7','Schriftwerk'
suche_debug = ['gefriergetrocknet','gefriertrocknen','missraten']  
suche_debug = ['calvinistisch','kalvinistisch',]  


suche_debug = ['Euro','€','tugendhaftestes','zu viele','zuviele', 'zu viel']  
suche_debug = ['€']  


gründlich = True # einige Arbeitsschritte können weggelassen werden, wenn gründlich = True

# verbose
pak.Config.set('VERBOSE', True)

# Darstellung der Dataframes
pak.Config.set('GRID_BACKEND', '')   # standardmäßig kein dTale
pd.set_option('max_colwidth', 2000)  # aber dafür breiter

# Load
verzeichnis_load                       = 'data_02'
wiktionary_lemma_filename              = verzeichnis_load + '/wiktionary_lemma.pickle'  
translate_lex_filename                 = verzeichnis_load + '/wiktionary_nolemma.pickle'    # hieß vorher wiktionary_nolemma
wiktionary_merkmal_filename            = verzeichnis_load + '/wiktionary_merkmal.pickle'  
wiktionary_merkmal_text_filename       = verzeichnis_load + '/wiktionary_merkmal_text.pickle'  

# Steuertabellen
lex_test_filename                      = 'Steuertabellen/lex_test.xlsx'  
löschliste_filename                    = 'Steuertabellen/löschliste.xlsx'   
lexeme_manuell_filename                = 'Steuertabellen/lexeme_manuell.xlsx'                              # manuell eingepflegte Lexeme 
     
# Save
verzeichnis_save                       = 'data_03'
wiktionary_lemma_filename_save         = verzeichnis_save + '/wiktionary_lemma.pickle'  
wiktionary_lemma_fix_filename_save     = verzeichnis_save + '/wiktionary_lemma_fix.pickle'  
translate_lex_filename_save            = verzeichnis_save + '/translate_lex.pickle'  
translate_lex_trash_filename_save      = verzeichnis_save + '/translate_lex_trash.pickle'  
translate_lemma_filename_save          = verzeichnis_save + '/translate_lemma.pickle'  
wiktionary_merkmal_filename_save       = verzeichnis_save + '/wiktionary_merkmal.pickle'  
wiktionary_merkmal_text_filename_save  = verzeichnis_save + '/wiktionary_merkmal_text.pickle'  

VERBOSE = True
--> setting verbose=True as default for all pandasklar functions

GRID_BACKEND = 
--> setting backend= as default for all pandasklar functions



## Vorbereiten

In [5]:
# Verzeichnisse erzeugen
os.makedirs(verzeichnis_load, exist_ok=True) 
os.makedirs(verzeichnis_save, exist_ok=True) 

In [6]:
# Steuertabellen und Python-Files von Github herunterladen
import os
if not os.path.exists('Steuertabellen'):
   !git clone https://github.com/djekra/wiktionary_wortschatz.git
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/Steuertabellen Steuertabellen
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/*.py .
#   !rm -r wiktionary_wortschatz

In [7]:
# Lade Ergebnisse der vorherigen Verarbeitungsstufe von Google Drive, falls nicht lokal vorhanden
# Dieses Verzeichnis wurde in der vorherigen Verarbeitungsstufe angelegt 
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av /content/drive/MyDrive/{verzeichnis_load} .

In [8]:
# Weitere Imports
from Steuertabellen.s01_Steuertabellen import *
from x32_Funktionen                    import *

## Daten laden
* Steuertabellen
* wiktionary_lemma
* translate_lex
* wiktionary_merkmal     
* wiktionary_merkmal_text 
* jeweils einen Trash generieren

In [9]:
#blab.help(plan_merkmal_erstellen)

In [10]:
# plan_merkmal Steuertabelle
plan_merkmal = plan_merkmal_erstellen()
grid(plan_merkmal, rows=9, column_definitions={ 'collect': {'maxWidth': 50,}, ' ': {'maxWidth': 50,}                   },     )

103 rows


,merkmal,template,collect,plan,sort,is_lex
0,noSteig,kSt.,E,1,,False
1,noSing,kSg.,E,1,,False
2,nurWortart,Wortart fehlt,P,1,,False
3,istName,Navigationsleiste Anthroponyme,E,1,,False
4,Wikispecies,Wikispecies,E,1,,False
...,...,...,...,...,...,...
98,Superlativ,,,3,x3 Superlativ,True
99,Stamm,,,3,z1 Stamm,False
100,ipa,,,3,z1 ipa,False
101,noFlex,,E,3,z1 noFlex,True


In [11]:
a= pak.analyse_freqs(plan_merkmal,['plan','merkmal'])
grid(a, width=200)

3 rows


,plan,plan_count,plan_percent,merkmal,merkmal_count
0,3,59,57.3,"[Befehl du, Befehl ihr, Gerundivum, PTKVZ, VVIZU, lemma, lex, lexAlt, rhymes, Genus, Nom Sg, Nom Sg f, Nom Sg m, Nom Sg n, Gen Sg, Gen Sg f, Gen Sg m, Gen Sg n, Dat Sg, Dat Sg f]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,1,38,36.9,"[übersicht, noSteig, noSing, nurWortart, istName, Wikispecies, Wikivoyage, Taxonomie, Präfix, Suffix, VMaßeinheiten, def, herkunft, syn, lateral, gegen, klein, up, up_part, down]","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,2,6,5.8,"[def, herkunft, bsp, bsp_re, bsp_gw, bsp_sp]","[1, 1, 1, 1, 1, 1]"


In [12]:
# lex_test, lemma_test
lex_test, lemma_test = lade_lex_text( lex_test_filename, level=3)
grid(lex_test)

185 rows loaded
Delete 3 rows from 185
182 rows


,level,lex,lemma,lex_tag_soll,notiz,check_tag
0,1,…,…,$(,,
1,3,°,°,$(,,
2,3,€,€,NN,,
3,1,Ackermann,Ackermann,NE,,
5,1,Alben,Album,NN,,
...,...,...,...,...,...,...
180,3,o.ä.,o.ä.,KOMBI,,
181,3,jdm,jemand,PIAT,kürzt nicht das Lemma ab,
182,3,jds,jemand,PIAT,kürzt nicht das Lemma ab,
183,3,nen,ein,ART,kürzt nicht das Lemma ab,


In [13]:
# lexeme_manuell laden
lexeme_manuell = pak.load_excel( lexeme_manuell_filename, tabcol='tab' )
lexeme_manuell = pak.rename_col( lexeme_manuell, 'tag',    'lemma_tag' )
lexeme_manuell = pak.rename_col( lexeme_manuell, 'tag_lex','lex_tag' )
lexeme_manuell = pak.move_cols(  lexeme_manuell, 'lex_tag','lemma_tag')
pak.sample(lexeme_manuell)

1794 rows loaded


,lex,lemma,lemma_tag,lex_tag,member,genus,tab
0,abberufen,abberufen,VVFIN,Ptz2,<NA>,<NA>,Partizipien
164,gewußt,wissen,VVFIN,Ptz2,<NA>,<NA>,Partizipien
304,überzeugend,überzeugen,VVFIN,Ptz1,<NA>,<NA>,Partizipien
487,Kids,Kid,NN,<NA>,<NA>,n,Diverses
489,Kontraindikationen,Kontraindikation,NN,<NA>,<NA>,f,Diverses
530,vergleiche,vergleichen,VVFIN,VVIMP,<NA>,<NA>,Diverses
561,zu viele,zu viel,ADV,ADV,<NA>,<NA>,Diverses
565,<NA>,24/7,ADJA,<NA>,<NA>,<NA>,Diverses
585,<NA>,bpm,NN,<NA>,Einheit,<NA>,Diverses
721,❀,_,$(,<NA>,<NA>,<NA>,Symbole


In [14]:
# Laden     
wiktionary_lemma         = pak.load_pickle(wiktionary_lemma_filename)
translate_lex            = pak.load_pickle(translate_lex_filename)
wiktionary_merkmal       = pak.load_pickle(wiktionary_merkmal_filename)
wiktionary_merkmal_text  = pak.load_pickle(wiktionary_merkmal_text_filename)

171220 rows loaded
667548 rows loaded
4839173 rows loaded
657508 rows loaded


In [15]:
# rename_col
wiktionary_merkmal_text = pak.rename_col(wiktionary_merkmal_text,'section_id2','lemma_id')
translate_lex           = pak.rename_col(translate_lex,'lex','data')
#translate_lex           = pak.rename_col(translate_lex,'score','data_score')

In [16]:
# trash erzeugen     
wiktionary_lemma_trash        = wiktionary_lemma.head(0)
wiktionary_lemma_trash        = pak.write_empty_col(wiktionary_lemma_trash, 'msg', 'string')

translate_lex_trash           = translate_lex.head(0)
translate_lex_trash           = pak.write_empty_col(translate_lex_trash, 'msg', 'string')

wiktionary_merkmal_trash      = wiktionary_merkmal.head(0)
wiktionary_merkmal_trash      = pak.write_empty_col(wiktionary_merkmal_trash, 'msg', 'string')

wiktionary_merkmal_text_trash = wiktionary_merkmal_text.head(0)
wiktionary_merkmal_text_trash = pak.write_empty_col(wiktionary_merkmal_text_trash, 'msg', 'string')

## suche_debug

In [17]:
# wiktionary_lemma suche_debug
grid(search_str(wiktionary_lemma, suche_debug))

No rows


In [18]:
# wiktionary_lemma_trash
mask = wiktionary_lemma_trash.lemma.isin(suche_debug) #== 'gefriergetrocknet'
wiktionary_lemma_trash[mask]

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score,msg


In [19]:
pak.analyse_freqs(wiktionary_lemma_trash,['msg','lemma_id'])

'No rows'

In [20]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

No rows


In [21]:
# wiktionary_merkmal
mask = wiktionary_merkmal.lemma_id == suche_debug[0]
a = wiktionary_merkmal[mask].sort_values(['merkmal','lemma_id','data'])
grid(a)

No rows


In [22]:
# wiktionary_merkmal
mask = wiktionary_merkmal.data.isin(suche_debug) 
a = wiktionary_merkmal[mask].sort_values(['merkmal','lemma_id','data'])
grid(a)

1 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
1830918,Euro,abk,Euro~€,€,,NN,,manuell,153.715988,True


In [23]:
# wiktionary_merkmal
# data_id ist bei allen gm-Merkmalen potentiell falsch!
#mask1 = wiktionary_merkmal.data == 'besonnen'
#mask2 = wiktionary_merkmal.merkmal == 'gm'
#mask = mask1  &  mask2
#a = wiktionary_merkmal[mask].sort_values(['merkmal','lemma_id','data'])
#grid(a)

In [24]:
# Genügend Datensätze?
print(wiktionary_merkmal.shape[0])
assert wiktionary_merkmal.shape[0] > 4800 * 1000

4839173


In [25]:
# Prüfen: is_unique
assert wiktionary_lemma.lemma_id.is_unique
assert translate_lex.nolemma_id.is_unique

## wiktionary_merkmal
* is_lex

In [26]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


In [27]:
# korrigieren: Punkt hinten ergänzen
startet_nicht_mit = ( '.', 'Industrie', 'St.')
endet_nicht_mit   = ( '.', 'Tag', 'Ort','Länge','Sieg')
mask1 =  wiktionary_merkmal.data.str.contains('.', regex=False)  
mask2 = ~wiktionary_merkmal.data.str.contains('[0123456789]', regex=True)  
mask3 = ~wiktionary_merkmal.data.str.startswith(startet_nicht_mit)
mask4 = ~wiktionary_merkmal.data.str.endswith(  endet_nicht_mit)
mask5 = (wiktionary_merkmal.data.str.split('.').str[-1].str.len() <= 5) # letztes Wort nicht zu lang
mask6 =  wiktionary_merkmal.data.str.len() > 2
mask7 = ~wiktionary_merkmal.merkmal.isin(['ipa'])
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6  &  mask7
check_mask( wiktionary_merkmal, mask, 233 )

wiktionary_merkmal.loc[mask,'data'] += '.'
pak.sample(wiktionary_merkmal[mask])

check_mask:  241 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
5374,Straßenbahn,abk,Straßenbahn~Str.-B,Str.-B.,,NN,1,,263.622375,True
64801,a._D.,alt_2,a._D.~a. d. D,a. d. D.,,,3,,11.471608,True
78720,p._A.,alt_2,p._A.~p. Adr,p. Adr.,,TODO,1,,6.032382,True
101860,Ingenieur,abk,Ingenieur~Dipl.-Ing,Dipl.-Ing.,,NN,1,,467.962463,True
102378,Geburtstag,abk,Geburtstag~Geb.-T,Geb.-T.,,NN,"1,2",,1581.322998,True
141173,BfA,up,,K. d. ö. R.,,NN,1,,18.144148,False
200438,North_Carolina_1,abk,North_Carolina_1~N.C,N.C.,,NE,1,,1.624077,True
381098,östliche_Länge,abk,östliche_Länge~östl. L,östl. L.,,NN,,,33.685715,True
435149,Reserve,abk,Reserve~d. R,d. R.,,NN,4,,584.237366,True
586750,Mag.,weib,Mag.a,Mag.a.,,TODO,1,,56.269928,True


In [28]:
#wiktionary_merkmal.head(1)

In [29]:
# Merkmale, die eigentlich tags sind
mask = wiktionary_merkmal.merkmal.isin(alle_tags)
check_mask( wiktionary_merkmal, mask, 10 )

wiktionary_merkmal.loc[mask,'lemma_tag'] = ''
wiktionary_merkmal.loc[mask,'data_tag']  = wiktionary_merkmal[mask].merkmal
wiktionary_merkmal.loc[mask,'merkmal']   = 'lex'
wiktionary_merkmal.loc[mask,'is_lex']    = True
grid(wiktionary_merkmal,mask)

check_mask:  11 rows
11 rows out of 4839173


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
1830808,zu_viel,lex,zu_viele,zu viele,ADV,,,manuell,14.845442,True
1830809,zu_viel,lex,zu_viele~zuviele,zuviele,ADV,,,manuell,14.845442,True
1830812,vergleichen,lex,vergleiche,vergleiche,VVIMP,,,manuell,96.039993,True
1830813,angucken_M,lex,anzugucken,anzugucken,VVIZU,,,manuell,2.239461,True
1830814,herauslassen_M,lex,herauslassen_M~herauszulassen,herauszulassen,VVIZU,,,manuell,2.239461,True
1830815,hereinlassen_M,lex,hereinlassen_M~hereinzulassen,hereinzulassen,VVIZU,,,manuell,2.239461,True
1830816,herumpimpeln_M,lex,herumzupimpeln,herumzupimpeln,VVIZU,,,manuell,2.239461,True
1830817,loswerden,lex,loszuwerden,loszuwerden,VVIZU,,,manuell,520.682129,True
1830818,einreden,lex,einzureden,einzureden,VVIZU,,,manuell,77.859055,True
1830819,anstaunen,lex,anzustaunen,anzustaunen,VVIZU,,,manuell,26.530531,True


In [30]:
# Mehrfache Spaces >> Einfacher Space (eigentlich hätten die alle schon von Preprocess erwischt werden müssen)
wiktionary_merkmal['data'] = pak.replace_str( wiktionary_merkmal.data, {'  ':' '})

In [31]:
# Mehrfachbindestrich
mask =  wiktionary_merkmal.data == '--'
wiktionary_merkmal.loc[mask,'data'] = '-'

In [32]:
# Hochkommas entfernen
mask1 = wiktionary_merkmal.data.str.count("\'") > 1
mask2 = wiktionary_merkmal.merkmal.isin(['def','abk']) 
mask  =  mask1  &  mask2 
check_mask(wiktionary_merkmal,mask, 0, 20)
wiktionary_merkmal.loc[mask,'data'] = wiktionary_merkmal[mask].data.str.replace("'", '')

check_mask:  7 rows


In [33]:
# Leerstring data Müll löschen
mask = wiktionary_merkmal['data'] == ''
check_mask(wiktionary_merkmal,mask,0,20)

msg = 'Leerstring data Müll'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg)
grid(wiktionary_merkmal_trash, msg)

check_mask:  6 rows
Move 6 rows from 4839173
6 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg
2152141,Außerm_Pfarrgarten,Genus,,,,NE,,,26.970110,False,Leerstring data Müll
2178055,Covid,Genus,,,,NE,,,44.336594,False,Leerstring data Müll
2178457,Heisei,Genus,,,,NN,,,44.313446,False,Leerstring data Müll
2178464,Reiwa,Genus,,,,NN,,,45.144207,False,Leerstring data Müll
2200952,Am_Bahnhof,Genus,,,,NE,,,26.080570,False,Leerstring data Müll
2200953,Am_Hauptbahnhof,Genus,,,,NE,,,50.251770,False,Leerstring data Müll


In [34]:
# wiktionary_merkmal
mask = wiktionary_merkmal.data.isin(suche_debug) 
a = wiktionary_merkmal[mask].sort_values(['merkmal','lemma_id','data'])
grid(a)

1 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
1830918,Euro,abk,Euro~€,€,,NN,,manuell,153.715988,True


In [35]:
# fillna
wiktionary_merkmal['meta']       = wiktionary_merkmal.meta.fillna('')
wiktionary_merkmal['data_tag']   = wiktionary_merkmal.data_tag.fillna('')
wiktionary_merkmal['lemma_tag']  = wiktionary_merkmal.lemma_tag.fillna('')
wiktionary_merkmal['num']        = wiktionary_merkmal.num.fillna('')

## translate_lex ==========
* hieß bislang wiktionary_nolemma
* Gerundivum als Merkmal speichern
* verwaiste löschen
* Spalten umbenennen
* pflege_translate_tabelle

In [36]:
assert not pak.any_nan(translate_lex)

In [37]:
# lemma_id vorhanden?
mask = translate_lex.lemma_id == ''
check_mask(translate_lex,mask,0)

check_mask:  0 rows


In [38]:
# nolemma_id vorhanden und nicht widersprüchlich?
mask = translate_lex.nolemma_id == ''
check_mask(translate_lex,mask,0)

a = pak.same_but_different(translate_lex,same='nolemma_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


### Gerundivum als Merkmal speichern

In [39]:
# Gerundivum finden
mask1 =  translate_lex.data.str.contains('zu')
mask2 = ~translate_lex.data.str.endswith('zu')
mask3 =  translate_lex.tag.str.contains('LEX_A') 
mask4 =  translate_lex.member.str.contains('Dekliniertes_Gerundivum') 
mask = mask1  & mask2  &  mask3  &  mask4
check_mask(translate_lex,mask,16480)

df = translate_lex[mask].copy()
pak.sample(df)

check_mask:  16480 rows


,nolemma_id,data,tag,tag_0,tag_1,tagZ,tagZZ,member,data_score,lemma_id,lemma,lemma_lower,lemma_tagZZ
359940,aufzuwärmende,aufzuwärmende,LEX_A,LEX_A,not8Rr6fOi,,,Dekliniertes_Gerundivum Adjektiv,1.813820,aufwärmen,aufwärmen,aufwärmen,V
364150,abzustrafende,abzustrafende,LEX_A,LEX_A,t2Po9bLiVt,,,Dekliniertes_Gerundivum Adjektiv,1.779205,abstrafen,abstrafen,abstrafen,V
409567,abzuarbeitende,abzuarbeitende,LEX_A,LEX_A,KYM41OsZld,,,Dekliniertes_Gerundivum Adjektiv,1.799718,abarbeiten,abarbeiten,abarbeiten,V
409570,abzuarbeitenden,abzuarbeitenden,LEX_A,LEX_A,l0mTcSustZ,,,Dekliniertes_Gerundivum Adjektiv,2.179205,abarbeiten,abarbeiten,abarbeiten,V
409835,abzubekommende,abzubekommende,LEX_A,LEX_A,is1e4uP09r,,,Dekliniertes_Gerundivum Adjektiv,1.795872,abbekommen,abbekommen,abbekommen,V
411172,abzudankende,abzudankende,LEX_A,LEX_A,7y92AsDqe3,,,Dekliniertes_Gerundivum Adjektiv,1.759974,abdanken,abdanken,abdanken,V
417646,abzubannende,abzubannende,LEX_A,LEX_A,JB1f1WwITZ,,,Dekliniertes_Gerundivum Adjektiv,1.758692,abbannen,abbannen,abbannen,V
425652,zuzuknipsendes,zuzuknipsendes,LEX_A,LEX_A,00U7XtgF0H,,,Dekliniertes_Gerundivum Adjektiv,1.147154,zuknipsen,zuknipsen,zuknipsen,V
501513,widereinanderzustoßenden,widereinanderzustoßenden,LEX_A,LEX_A,b25PQvdZZ9,,,Dekliniertes_Gerundivum Adjektiv,3.497060,widereinanderstoßen,widereinanderstoßen,widereinanderstoßen,V
513949,hinzuziehendem_1,hinzuziehendem,LEX_A,LEX_A,atTZ3zINht,,,Dekliniertes_Gerundivum Adjektiv,0.102914,hinziehen,hinziehen,hinziehen,V


In [40]:
# anfügen vorbereiten
df = pak.rename_col( df, 'lex',        'data' )
df = pak.rename_col( df, 'nolemma_id', 'data_id' )
#df = pak.rename_col( df, 'data_score', 'score' )
df['merkmal']  = 'Gerundivum'
df['meta']     = ''
df['num']      = ''
df['is_lex']   = True
df['data_tag'] = 'ADJA'
df = pak.drop_cols( df, set(df.columns) - set(wiktionary_merkmal.columns) )
df = pak.update_col(df, wiktionary_lemma, left_on='lemma_id', right_on='lemma_id', col='tag', col_rename='lemma_tag')

update_col: 16480 cells written into new column


In [41]:
# Prüfen: Alle Gerundiva verweisen auf ein Verb
mask_L = wiktionary_lemma.tagZZ == 'V'
verben = wiktionary_lemma[mask_L].lemma_id

mask = ~df.lemma_id.isin(verben)
check_mask(df,mask,0)

check_mask:  0 rows


In [42]:
# anfügen
wiktionary_merkmal = pak.add_rows(wiktionary_merkmal, df)

16480 rows added, now a total of 4855647


In [43]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


In [44]:
mask = wiktionary_merkmal.merkmal == 'Gerundivum'
pak.sample(wiktionary_merkmal[mask])

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
4839167,aufwärmen,Gerundivum,aufzuwärmende,aufzuwärmende,ADJA,VVFIN,,,1.813820,True
4839345,anleuchten,Gerundivum,anzuleuchtenden,anzuleuchtenden,ADJA,VVFIN,,,2.175359,True
4841018,abarbeiten,Gerundivum,abzuarbeitende,abzuarbeitende,ADJA,VVFIN,,,1.799718,True
4841242,abbestellen,Gerundivum,abzubestellende,abzubestellende,ADJA,VVFIN,,,1.813820,True
4849575,widereinanderstoßen,Gerundivum,widereinanderzustoßenden,widereinanderzustoßenden,ADJA,VVFIN,,,3.497060,True
4850313,hinziehen,Gerundivum,hinzuziehendem_1,hinzuziehendem,ADJA,VVFIN,,,0.102914,True
4853816,übriglassen,Gerundivum,übrigzulassendes,übrigzulassendes,ADJA,VVFIN,,,1.262538,True
4855646,entgegenwerfen,Gerundivum,entgegenzuwerfendes,entgegenzuwerfendes,ADJA,VVFIN,,,1.171513,True


### translate_lex pflegen

In [45]:
#grid(translate_lex,pak.sample)

In [46]:
# Verwaiste löschen
mask = ~translate_lex.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lex,mask,0,30)

msg = 'verwaist_lemma_id'
translate_lex, translate_lex_trash = pak.move_rows( translate_lex, translate_lex_trash, mask, msg )
grid(translate_lex_trash,msg)

check_mask:  9 rows
Move 9 rows from 667548
9 rows


,nolemma_id,data,tag,tag_0,tag_1,tagZ,tagZZ,member,data_score,lemma_id,lemma,lemma_lower,lemma_tagZZ,msg
884,zagen_1,zagen,LEX_A,LEX_A,werhb2SkyU,,,Deklinierte_Form,0.328490,zage,zag,zag,A,verwaist_lemma_id
28326,Kremes,Kremes,LEX_A,LEX_A,RO4RB85EnJ,,,Deklinierte_Form,0.753066,Cremes,Kreme,kreme,N,verwaist_lemma_id
177595,Meßgewande,Meßgewande,LEX_A,LEX_A,lw1DNAbRQ3,,,Deklinierte_Form,1.053846,Messgewande,Meßgewand,meßgewand,N,verwaist_lemma_id
204892,Zierates,Zierates,LEX_A,LEX_A,oesaK07Tmk,,,Deklinierte_Form,0.673458,Zierats,Zierat,zierat,N,verwaist_lemma_id
204893,Zierats,Zierats,LEX_A,LEX_A,FJomnScnwM,,,Deklinierte_Form,0.667614,Zierates,Zierat,zierat,N,verwaist_lemma_id
281103,kalvinistischsten,kalvinistischsten,LEX_A,LEX_A,WOfe2AhyRt,,,Deklinierte_Form,2.283051,calvinistischsten,kalvinistisch,kalvinistisch,A,verwaist_lemma_id
281104,kalvinistischsten_1,kalvinistischsten,LEX_A,LEX_A,pM8r0mmpjj,,,Superlativ Adjektiv,0.081220,calvinistischsten,kalvinistisch,kalvinistisch,A,verwaist_lemma_id
654680,Fischgratparkettes,Fischgratparkettes,LEX_A,LEX_A,4MUMdmCoto,,,Deklinierte_Form,0.842930,Fischgratparketts,Fischgratparkett,fischgratparkett,N,verwaist_lemma_id
654681,Fischgratparketts,Fischgratparketts,LEX_A,LEX_A,pvOaEWFFR9,,,Deklinierte_Form,1.526227,Fischgratparkettes,Fischgratparkett,fischgratparkett,N,verwaist_lemma_id


In [47]:
# umformen
translate_lex = pak.rename_col( translate_lex, 'lex',        'data' )
translate_lex = pak.rename_col( translate_lex, 'nolemma_id', 'data_id' )
translate_lex['member'] = ('wiki ' + translate_lex['member'] ).str.strip()
translate_lex = pflege_translate_tabelle(translate_lex, translate_tagZ, wiktionary_lemma)

update_col: 667539 cells written into existing column
update_col: 0 cells written into existing column
update_col: 0 cells written into new column
update_col: 667539 cells written into existing column
update_col: 667539 cells written into new column


In [48]:
# Müll
mask = translate_lex.lemma_tag == 'TODO'
check_mask(translate_lex,mask,0,5)

msg = 'TODO'
translate_lex, translate_lex_trash = pak.move_rows( translate_lex, translate_lex_trash, mask, msg )

check_mask:  5 rows
Move 5 rows from 667539


In [49]:
# not any nan
assert not pak.any_nan(translate_lex)

In [50]:
pak.sample(translate_lex)

,data_id,data,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower
0,lieben_1,lieben,,0.311824,wiki Deklinierte_Form,lieb,lieb,ADJA,A,82.432930,lieb
1,Polen_1,Polen,,0.113761,wiki Deklinierte_Form,Pole,Pole,NN,N,473.835083,pole
1190,führt,führt,,0.746868,wiki Konjugierte_Form Verb,führen,führen,VVFIN,V,1512.875732,führen
1491,Häuser,Häuser,,0.812983,wiki Deklinierte_Form,Haus,Haus,NN,N,108960.914062,haus
7929,freizustellen,freizustellen,,0.565050,wiki Erweiterter_Infinitiv Verb,freistellen,freistellen,VVFIN,V,701.566406,freistellen
188180,überführte,überführte,,1.812539,wiki Deklinierte_Form,überführt_4,überführt,ADJA,A,0.032591,überführt
188314,α-Zerfällen,α-Zerfällen,,0.441083,wiki Deklinierte_Form,α-Zerfall,α-Zerfall,NN,N,345.928619,α-zerfall
194763,1-Cent-Stücken,1-Cent-Stücken,,1.114682,wiki Deklinierte_Form,1-Cent-Stück,1-Cent-Stück,NN,N,53.207848,1-cent-stück
362413,vermarktende,vermarktende,,1.799718,wiki Deklinierte_Form,vermarktend,vermarktend,ADJA,A,1.261264,vermarktend
428878,unterzumischende,unterzumischende,,1.895872,wiki Dekliniertes_Gerundivum Adjektiv,untermischen,untermischen,VVFIN,V,34.946598,untermischen


In [51]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

No rows


In [52]:
# anschauen
#pak.analyse_freqs(translate_lex,['member','lemma_tag','lex'])

## Partizipien ==========
Für fast jedes Partizip gibt es zwei Datensätze: <br> 
1) in wiktionary_merkmal.data: Die Partizip-Flexion des Verbs. merkmal ist 'Ptz1' oder 'Ptz2'.<br> 
2) in wiktionary_lemma.lemma:  Als Adjektiv mit member 'Partizip'. Es gibt ca. 320 Ausnahmen, nicht alle Partizipien sind auch Adjektive.

In [53]:
# Falsche Partizipien löschen
falsche_partizipien = ['sehen','mögen','hören','sollen','können','lassen']
mask1 = wiktionary_merkmal.merkmal.isin(['Ptz1','Ptz2'])
mask2 = wiktionary_merkmal.data.isin(falsche_partizipien)
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0,6)
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal,wiktionary_merkmal_trash, mask, 'Falsche_Partizipien')
#wiktionary_merkmal[mask]

check_mask:  6 rows
Move 6 rows from 4855647


In [54]:
# verben_ptz1: Alle Ptz1-Merkmale der Verben
spalten = ['lemma_id','data_id','data','data_score','merkmal']
mask = wiktionary_merkmal.merkmal == 'Ptz1'
verben_ptz1 = wiktionary_merkmal[mask][spalten]
verben_ptz1 = pak.update_col(df_to=verben_ptz1, df_from=wiktionary_lemma, on='lemma_id', col='lemma', verbose=False)
verben_ptz1 = pak.move_cols(       verben_ptz1,'lemma','lemma_id')

# was anzeigen
a = search_str(verben_ptz1, suche_debug)
if len(a) == 0:
    a = verben_ptz1.sample(5)
a

,lemma_id,lemma,data_id,data,data_score,merkmal
1813664,desinfizieren,desinfizieren,desinfizierend,desinfizierend,0.531992,Ptz1
1822232,anempfinden,anempfinden,anempfindend,anempfindend,0.436950,Ptz1
1822922,anstückeln,anstückeln,anstückelnd,anstückelnd,0.515463,Ptz1
1815885,blaffen,blaffen,blaffend,blaffend,0.453479,Ptz1
1824715,nachverhandeln,nachverhandeln,nachverhandelnd,nachverhandelnd,0.465876,Ptz1


In [55]:
# Kein leeres data_id
mask = verben_ptz1.data_id.isnull()  |  (verben_ptz1.data_id == '')
check_mask(verben_ptz1,mask,0)

check_mask:  0 rows


In [56]:
# alle enden mit 'nd'
mask = ~verben_ptz1.data.str.endswith('nd')
check_mask(verben_ptz1,mask,0)
#verben_ptz[mask]

check_mask:  0 rows


In [57]:
# verben_ptz2: Alle Ptz2-Merkmale der Verben
spalten = ['lemma_id','data_id','data','data_score','merkmal']
mask = wiktionary_merkmal.merkmal == 'Ptz2'
verben_ptz2 = wiktionary_merkmal[mask][spalten]
verben_ptz2 = pak.update_col(df_to=verben_ptz2, df_from=wiktionary_lemma, on='lemma_id', col='lemma', verbose=False)
verben_ptz2 = pak.move_cols(       verben_ptz2,'lemma','lemma_id')

# was anzeigen
a = search_str(verben_ptz2, suche_debug)
if len(a) == 0:
    a = verben_ptz2.sample(5)
a

,lemma_id,lemma,data_id,data,data_score,merkmal
1823308,sandstrahlen,sandstrahlen,sandgestrahlt,sandgestrahlt,0.875071,Ptz2
1805396,aufdecken,aufdecken,aufgedeckt,aufgedeckt,0.412157,Ptz2
1813424,abrüsten,abrüsten,abgerüstet,abgerüstet,0.410091,Ptz2
1825043,vorjammern,vorjammern,vorgejammert,vorgejammert,0.430752,Ptz2
1829754,evangelisieren,evangelisieren,evangelisiert,evangelisiert,0.662157,Ptz2


In [58]:
# Kein leeres data_id
mask = verben_ptz2.data_id.isnull()  |  (verben_ptz2.data_id == '')
check_mask(verben_ptz2,mask,0)

check_mask:  0 rows


In [59]:
# adjektive_ptz1: Adjektive mit Ptz1-Markierung
spalten = ['lemma_id','lemma','lemma_score','tag','member','data_id']
mask1 = wiktionary_lemma.tagZ == 'ADJ'
mask2 = wiktionary_lemma.member.str.contains('Ptz1')
mask = mask1  &  mask2 
adjektive_ptz1 = wiktionary_lemma[mask][spalten]

# was anzeigen
a = search_str(adjektive_ptz1, suche_debug)
if len(a) == 0:
    a = adjektive_ptz1.sample(5)
a

,lemma_id,lemma,lemma_score,tag,member,data_id
136859,gärend,gärend,1.651760,ADJA,Ptz1,gären
166726,zerdätschend,zerdätschend,1.521595,ADJA,Ptz1,zerdätschen
114127,altertümelnd,altertümelnd,1.298455,ADJA,Ptz1,altertümeln
139096,hineinreitend,hineinreitend,1.602174,ADJA,Ptz1,hineinreiten
160863,verhunzend,verhunzend,1.472008,ADJA,Ptz1,verhunzen


In [60]:
# Kein leeres data_id
# data_id verweist auf das Verb
mask = adjektive_ptz1.data_id.isnull()  |  (adjektive_ptz1.data_id == '')
check_mask(adjektive_ptz1,mask,0)

check_mask:  0 rows


In [61]:
# adjektive_ptz2: Adjektive mit Ptz2-Markierung
spalten = ['lemma_id','lemma','lemma_score','tag','member','data_id']
mask1 = wiktionary_lemma.tagZ == 'ADJ'
mask2 = wiktionary_lemma.member.str.contains('Ptz2')
mask = mask1  &  mask2 
adjektive_ptz2 = wiktionary_lemma[mask][spalten]

# was anzeigen
a = search_str(adjektive_ptz2, suche_debug)
if len(a) == 0:
    a = adjektive_ptz2.sample(5)
a

,lemma_id,lemma,lemma_score,tag,member,data_id
133724,gekräht,gekräht,0.529926,ADJA,Ptz2,krähen
149122,pensioniert_1,pensioniert,0.121509,ADJA,Ptz2,pensionieren
116669,aufgehalten,aufgehalten,0.422488,ADJA,Ptz2,aufhalten
128313,entnommen,entnommen,0.472074,ADJA,Ptz2,entnehmen
162272,verseift_1,verseift,0.070115,ADJA,Ptz2,verseifen


In [62]:
# Kein leeres data_id
# data_id verweist auf das Verb
# Das sind Kandidaten für die Aufnahme in lexeme_manuell, Reiter Partizipien
nöö = ['befangen','betreten','durchwachsen','erfahren','erlesen','erwachsen','wollen']
mask1 = adjektive_ptz2.data_id.isnull()  |  (adjektive_ptz2.data_id == '')
mask2 = ~adjektive_ptz2.lemma.isin(nöö)
mask = mask1  &  mask2
#grid(adjektive_ptz2,mask)
check_mask(adjektive_ptz2,mask,0)

check_mask:  0 rows


In [63]:
#for p in adjektive_ptz2[mask].lemma:
#    print(p)

In [64]:
# adjektive_noptz: Adjektive ohne Ptz-Markierung
spalten = ['lemma_id','lemma','lemma_score','tag','member','data_id']
mask1 =  wiktionary_lemma.tagZ == 'ADJ'
mask2 = ~wiktionary_lemma.member.str.contains('Ptz1')
mask3 = ~wiktionary_lemma.member.str.contains('Ptz2')
mask = mask1  &  mask2  &  mask3  
adjektive_noptz = wiktionary_lemma[mask][spalten]

# was anzeigen
a = search_str(adjektive_noptz, suche_debug)
if len(a) == 0:
    a = adjektive_noptz.sample(5)
a

,lemma_id,lemma,lemma_score,tag,member,data_id
130032,evidenzbasiert,evidenzbasiert,204.507187,ADJA,noSteig,
158631,uneinziehbar,uneinziehbar,102.418060,ADJA,noSteig,
151964,routiniert,routiniert,54.805744,ADJA,,
167124,zierlich,zierlich,542.917908,ADJA,,
148687,ovarial,ovarial,450.366577,ADJA,noSteig,


In [65]:
# Nur leeres data_id
mask = adjektive_noptz.data_id.isnull()  |  (adjektive_noptz.data_id == '')
#grid(adjektive_noptz,~mask)
check_mask(adjektive_noptz,~mask,0)

check_mask:  0 rows


In [66]:
# Das sind vermutlich eigentlich Partizipien 
# Partizip (geraten)
mask1 = adjektive_noptz.lemma.str.endswith('nd')
mask2 = ~adjektive_noptz.lemma.str.contains(' ')
mask = mask1  &  mask2
kandidaten = adjektive_noptz[mask].copy()

# Kandidaten aufbereiten
nöö = ['dutzend','flektierend','frappierend','spannend']
kandidaten['verb'] = kandidaten.lemma.str[:-1]
mask =  wiktionary_lemma.tagZZ == 'V'
kandidaten = pak.update_col(kandidaten, 
                            wiktionary_lemma[mask],
                            left_on=['verb'],
                            right_on=['lemma'],
                            col='lemma_id',
                            col_rename='data_id',
                            func='min'
                           )

# und filtern
maskA = kandidaten.data_id.isnull()  |  (kandidaten.data_id == '')
maskB = kandidaten.lemma.isin(nöö)
mask = maskA  |  maskB
kandidaten = pak.drop_rows(kandidaten,mask)

spalten = ['lemma','verb']
grid(kandidaten[spalten])

update_col: func min applied, 382 records less!
update_col: 8 cells written into existing column
Delete 297 rows from 301
4 rows


,lemma,verb
121264,bestimmend,bestimmen
129409,erregend,erregen
147265,naheliegend,naheliegen
147271,nahestehend,nahestehen


### Gibt es Verben ohne Partizip-Lexeme?
* ggf. Partizipen manuell nachtragen

In [67]:
# Verben ohne Ptz1
spalten = ['lemma_id','lemma','lemma_score','tag','member']
mask1 =  wiktionary_lemma.tagZZ == 'V'
mask2 = ~wiktionary_lemma.lemma.isin(verben_ptz1.lemma)
mask = mask1  &  mask2 
check_mask(wiktionary_lemma,mask,26)
verben_ohne_ptz1 = wiktionary_lemma[mask][spalten]
#verben_ohne_ptz1

check_mask:  33 rows


In [68]:
# Kandidaten für die manuelle Aufnahme
# - entweder als Ptz1-Lexem in lexeme_manuell (Reiter Partizipien)
# - oder als lemma in die löschliste

nöö = ['angedeihen','erstinken','gegenanargumentieren','grundsanieren','interdependieren','mittagessen','umfragen','urformen','voranmelden','zusammenreiten']
mask1 = verben_ohne_ptz1.lemma.str.endswith('en')
mask2 = ~verben_ohne_ptz1.lemma.isin(nöö)
mask = mask1  &  mask2
kandidaten = verben_ohne_ptz1[mask][['lemma']]
kandidaten = kandidaten.drop_duplicates().sort_values('lemma')
kandidaten['lex'] = kandidaten.lemma + 'd'
kandidaten = pak.move_cols(kandidaten,'lex')
kandidaten

,lex,lemma
115159,anguckend,angucken
132411,geborend,geboren
2100,handstoppend,handstoppen
145638,milbend,milben
146584,möchtend,möchten
154965,sprützend,sprützen
169161,zustande kommend,zustande kommen


In [69]:
# Verben ohne Ptz2
nöö = ['Schlange stehen', 'gehn', 'stehn', 'umsehn','vergleiche']
spalten = ['lemma_id','lemma','lemma_score','tag','member']
mask1 =  wiktionary_lemma.tagZZ == 'V'
mask2 = ~wiktionary_lemma.lemma.isin(verben_ptz2.lemma)
mask3 = ~wiktionary_lemma.lemma.isin(nöö)
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_lemma,mask,0,11)
verben_ohne_ptz2 = wiktionary_lemma[mask][spalten]
verben_ohne_ptz2

check_mask:  6 rows


,lemma_id,lemma,lemma_score,tag,member
115159,angucken_M,angucken,6.718384,VVFIN,manuell
117142,aufrütteln_M,aufrütteln,6.718384,VVFIN,manuell
132411,geboren_M,geboren,2.239461,VVFIN,manuell
138322,herumpimpeln_M,herumpimpeln,6.718384,VVFIN,manuell
146584,möchten,möchten,7.470430,VVFIN,
169161,zustande_kommen,zustande kommen,12.605865,VVFIN,Wortverbindung


In [70]:
del verben_ohne_ptz1
del verben_ohne_ptz2

In [71]:
#mask = wiktionary_merkmal.lemma_id == 'betreffen'
#wiktionary_merkmal[mask]

In [72]:
#mask = wiktionary_lemma.lemma == 'betreffend'
#grid(wiktionary_lemma[mask])

### Partizipien und Adjektive vergleichen

In [73]:
# merge für Ptz1
a = pd.merge(verben_ptz1, adjektive_ptz1, left_on='data', right_on='lemma', how='outer',indicator=True, suffixes=('_v', '_a'))

In [74]:
# Übersicht
assert a.shape[0] > 13000
pak.analyse_freqs(a,'_merge')

,_merge,_merge_count,_merge_percent,graph
0,both,14180,100.0,##################################################
1,left_only,0,0.0,
2,right_only,0,0.0,


In [75]:
# Als Partizipialadjektiv gekennzeichnet, hat aber kein passendes Partizip
# >> Kandidaten für die Löschliste

mask = a._merge == 'right_only'
check_mask(a,mask,0,20)
a[mask][['lemma_a']]

check_mask:  0 rows


,lemma_a


In [76]:
# Partizip ohne passendes Partizipialadjektiv
mask = a._merge == 'left_only'
check_mask(a,mask,0,1)
#a[mask] 

check_mask:  0 rows


In [77]:
# merge für Ptz2
a = pd.merge(verben_ptz2, adjektive_ptz2, left_on='data', right_on='lemma', how='outer',indicator=True, suffixes=('_v', '_a'))
a = a.drop_duplicates()

In [78]:
# Übersicht
assert a.shape[0] > 14000
pak.analyse_freqs(a,'_merge')

,_merge,_merge_count,_merge_percent,graph
0,both,14197,98.9,#################################################
1,left_only,165,1.1,
2,right_only,0,0.0,


In [79]:
# Als Partizipialadjektiv gekennzeichnet, hat aber kein passendes Partizip
# >> Kandidaten für die Löschliste

mask = a._merge == 'right_only'
check_mask(a,mask,0,30)
a[mask][['lemma_id_a','lemma_a','tag','member']]

check_mask:  0 rows


,lemma_id_a,lemma_a,tag,member


In [80]:
# entkennzeichnen
entkennzeichnen = a[mask].lemma_id_a
mask_L = wiktionary_lemma.lemma_id.isin(entkennzeichnen)
wiktionary_lemma.loc[mask_L,'member'] = pak.remove_str(wiktionary_lemma[mask_L].member, 'Ptz2')
wiktionary_lemma[mask_L]

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score


In [81]:
# Partizip ohne passendes Partizipialadjektiv
mask = a._merge == 'left_only'
check_mask(a,mask,0,350)
fehlende_adjektive = a[mask]
fehlende_adjektive = pak.update_col(df_to=fehlende_adjektive, df_from=adjektive_noptz, left_on='lemma_v', right_on='lemma',col='lemma_id',col_rename='adj_id')
#fehlende_adjektive

check_mask:  165 rows
update_col: 7 cells written into new column


In [82]:
# Das Partizipialadjektiv fehlt. Das kann zwei Ursachen haben:
# (1) Es gibt ein Adjektiv, das ist aber nicht als Partizipialadjektiv markiert
# (2) Es gibt kein Adjektiv.
# Zuerst kümmern wir uns um Fall (1).
# Die in adj_id genannten Adjektive müssen noch als Partizipialadjektiv markiert werden
mask_a = ~fehlende_adjektive.adj_id.isnull()
check_mask(fehlende_adjektive,mask_a,0,20)
fehlende_adjektive[mask_a]

check_mask:  7 rows


,lemma_id_v,lemma_v,data_id_v,data,data_score,merkmal,lemma_id_a,lemma_a,lemma_score,tag,member,data_id_a,_merge,adj_id
14095,beschlagen_1,beschlagen,beschlagen,beschlagen,18.211857,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,beschlagen
14155,betreten_1,betreten,betreten,betreten,11.072776,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,betreten
14163,erwachsen_1,erwachsen,erwachsen,erwachsen,6.950320,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,erwachsen
14210,durchwachsen_2,durchwachsen,durchwachsen,durchwachsen,1.615978,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,durchwachsen
14264,erlesen_1,erlesen,erlesen,erlesen,4.263538,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,erlesen
14287,befangen_1,befangen,befangen,befangen,8.290561,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,befangen
14293,erfahren,erfahren,erfahren_1,erfahren,163.108917,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,erfahren_1


In [83]:
# markieren
mask = wiktionary_lemma.lemma_id.isin(fehlende_adjektive[mask_a].adj_id)
check_mask(wiktionary_lemma,mask,0,20)
wiktionary_lemma.loc[mask,'member'] += ' Ptz2'
#wiktionary_lemma[mask]

check_mask:  7 rows


In [84]:
# Fall (2)
# Dies sind Kandidaten dafür, manuell aufgenommen zu werden.
# Und zwar als Partizipien (Ptz2) in lexeme_manuell, dann werden die Adjektive automatisch erstellt und passend verlinkt.
# Das geschieht in d01_tag_und_member.
#

mask_a = fehlende_adjektive.adj_id.isnull()
kandidaten = fehlende_adjektive[mask_a][['data']].drop_duplicates().sort_values('data')

nöö = ['abbegeben','abbehalten','abbekommen','abgetan','anbekommen','angetan','aufbekommen','aufgetan','ausgetan','begeben','behalten','beibehalten','bekommen',
       'benachrangt','bestoßen','betan','betragen','dabehalten','dargetan','dazugetan','dickgetan','dortbehalten',
       'durchbekommen','durchhauen','durchmarschiert','entraten','entschlagen','erfangen','ermessen','erschlafen','erschollen',
       'ersehen','ertragen','fallenlassen','fortwollen','ge- -zt','gegenanargumentiert','geheimgetan',
       'gemecht','genuggetan','geraidet','getan','gleichgetan','großgetan','gutgetan','hartgetan','hervorgetan','hinbekommen','hindurchmarschieren',
       'hineinbegeben','hineingetan','hinterlaufen','hinunterbekommen','hinzugetan','hinüberrannte','hängen lassen','leichtgetan','leidgetan',
       'lieb behalten','missfallen','mitbekommen','mitwollen','nachbekommen','nachgeraten','nachgetan','notgetan','schwergetan','schöngetan','stehenlassen',
       'umeinandergehetzt','umeinandergelegen','umeinandergestanden','umgefragt','umgetan','umtreten','unterbekommen','unterfangen','unterwaschen',
       'verhebt','verladen','verschleißt','versehen','vertan','vertragen',
       'vorgetan','weggetan','wegwollen','weh getan','wehgetan','widerfahren','widerraten','wiederwollen','wohlgetan','zubekommen','zugetan',
       'zugutegetan','zurechtbekommen','zurückbegeben','zurückbekommen','zurückerhalten','zurückgetan','zurückwollen','zusammenbekommen','zusammengetan','zuvorgetan',
       'überlassen','überschlafen','betroffen','geschehen','umschlagen']
print('len nöö =',len(nöö))
mask1 = ~kandidaten.data.str.startswith('-')
mask2 = ~kandidaten.data.isin(nöö)
mask = mask1  &  mask2  
check_mask(kandidaten,mask, 0,20)
grid(kandidaten,mask)

len nöö = 110
check_mask:  7 rows
7 rows out of 118


,data
14308,fortbegeben
14280,geadert
14263,gesprützt
14278,geädert
14306,herumbekommen
14327,photogeshoppt
14296,zusammengeraten


### Aufräumen

In [85]:
del verben_ptz1
del verben_ptz2
del adjektive_ptz1
del adjektive_ptz2
del adjektive_noptz

## wiktionary_lemma ==========

In [86]:
# lemma_id vorhanden und eindeutig?
mask = wiktionary_lemma.lemma_id == ''
check_mask(wiktionary_lemma,mask,0)

assert wiktionary_lemma.lemma_id.is_unique

check_mask:  0 rows


In [87]:
pak.nnan(translate_lex)

Series([], dtype: int64)

In [88]:
# Dups?
mask = wiktionary_lemma.duplicated(subset=['lemma_id'])
check_mask(wiktionary_lemma,mask,0)

check_mask:  0 rows


In [89]:
# nach Priorität sortieren (laut Steuertabellen)
wiktionary_lemma['tag'] = tag_sortieren(wiktionary_lemma.tag)

In [90]:
# Spalten wiederherstellen
wiktionary_lemma = pflege_tag_felder(wiktionary_lemma, translate_tagZ)  # tagZ, tagZZ, tag_0, tag_1, tagZ_0, tagZ_1
wiktionary_lemma = ergänze_felder(wiktionary_lemma)                     # lemma_lower, isupper, lemma_len

# Fehler?
mask = (wiktionary_lemma.tagZZ.str.len() > 1)
check_mask(wiktionary_lemma,mask,0,10) # ein paar können wir tolerieren

check_mask:  303 rows
check_mask:  0 rows


In [91]:
# suche_debug wiktionary_lemma
search_str(wiktionary_lemma, suche_debug)

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score


## Abkürzungen ==========
* abspalten
* belöschen
* korrigieren
* variieren
* pflegen

In [92]:
geschütze_abkürzungen = ['H2O','DEU','deu','RAF','UNO','CDU','SPD','FDP',
                         'SED','UTC','ORF','RKI',
                         'Mo','Di','Mi','Do','Fr','Sa','So',  'MO','DI','MI','DO','FR','SA','SO',  'Mo.','Di.','Mi.','Do.','Fr.','Sa.','So.', 'NS','US','VR','IR','UV','9/11','OP','KI','AI','IT','IS',
                         'Dax','FDJ','MDR','USB','2D','3D','OEM','VW',
                         'u.','ca.','ev.','PKK','USK','dpi','SUV', 'Fa.','od.','u.','CO2','SO2','VR China','Meck-Pomm','Hbf',
]

In [93]:
bsp_abkürzungen_kurz = ['altisländ.', 'EU','Krh', 'Krh.', 'Krhs', 'Krhs.', 'Krkhs', 'Krkhs.']
bsp_abkürzungen_lang = ['altisländisch', 'Europäische Union','Krankenhaus']

In [94]:
# wiktionary_lemma kann sowohl die Langform als auch die Kurzform als eigenständiges lemma enthalten.
# Eine Kurzform ist dabei mit member 'Abkürzung' gekennzeichnet.
# TODO: Beide müssen nicht unbedingt den gleichen tag haben, siehe 'EU' vs. 'Europäische Union': NN / NE
# TODO: Nicht jede Kurzform hat einen solchen Datensatz.

search_str(wiktionary_lemma, bsp_abkürzungen_kurz + bsp_abkürzungen_lang)

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score
25535,EU,2594,0,EU,eu,True,1,NN,NN,Sv9J9jngfj,NOM,N,Abkürzung noPl,,636.565918
29756,Europäische_Union,12532,0,Europäische Union,europäische union,True,2,NE,NE,K02sn2w8GT,NAM,N,adjNN Wortverbindung adjNN,,108.882866
55175,Krankenhaus,732,0,Krankenhaus,krankenhaus,True,1,NN,NN,AJqSbeXWez,NOM,N,,,52026.234375
55532,Krh.,885631,0,Krh.,krh.,True,1,TODO,TODO,hKXmMclm5U,X_todo,X,Abkürzung,,5.775660
55533,Krhs.,885632,0,Krhs.,krhs.,True,1,TODO,TODO,aD4bHIEhtj,X_todo,X,Abkürzung,,6.462754
55855,Krkhs.,885633,0,Krkhs.,krkhs.,True,1,TODO,TODO,RqCNDygojI,X_todo,X,Abkürzung,,5.818603
114147,altisländ.,21498,0,altisländ.,altisländ.,False,1,TODO,TODO,Q4m8p2dijm,X_todo,X,Abkürzung,,34.896648
114148,altisländisch,609806,0,altisländisch,altisländisch,False,1,ADJA,ADJA,30vObeFRdP,ADJ,A,noSteig,,104.247162


In [95]:
# wiktionary_merkmal enthält für Kurzformen ein Merkmal 'def', das die Langform nennt.

mask1 = wiktionary_merkmal.lemma_id.isin(bsp_abkürzungen_kurz)
mask2 = wiktionary_merkmal.merkmal == 'def'
mask = mask1  &  mask2
wiktionary_merkmal[mask]

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
53844,EU,def,Europäische_Union,Europäische Union,,NN,1,,212.188629,False
197919,altisländ.,def,altisländisch,altisländisch,,TODO,1,,34.896648,False
1451747,Krh.,def,Krankenhaus,Krankenhaus,,TODO,1,,5.775660,False
1451748,Krhs.,def,Krankenhaus,Krankenhaus,,TODO,1,,6.462754,False
1451749,Krkhs.,def,Krankenhaus,Krankenhaus,,TODO,1,,5.818603,False


In [96]:
# wiktionary_merkmal enthält für Langformen Merkmale 'abk', das eine Kurzform nennt.

mask1 = wiktionary_merkmal.lemma_id.isin(bsp_abkürzungen_lang)
mask2 = wiktionary_merkmal.merkmal == 'abk'
mask = mask1  &  mask2
wiktionary_merkmal[mask]

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
19432,Krankenhaus,abk,Krankenhaus~Krh,Krh,,NN,1,,5780.692871,True
19433,Krankenhaus,abk,Krankenhaus~Krhs,Krhs,,NN,1,,5780.692871,True
19434,Krankenhaus,abk,Krankenhaus~Krkhs,Krkhs,,NN,1,,5780.692871,True
1106271,altisländisch,abk,altisländisch~non,non,,ADJA,1,,34.749054,True
1106272,altisländisch,abk,altnordisch,altnordisch,,ADJA,1,,34.749054,True


### wiktionary_merkmal_abk abspalten

In [97]:
#suche_debug = ['au','ei','hm','qm','ui','mg','km','cm','mm','dm','ft','nm','pt','m²','m³','kg','lb','db']

In [98]:
# abk-Merkmale abspalten
mask = (wiktionary_merkmal.merkmal == 'abk')
wiktionary_merkmal, wiktionary_merkmal_abk = pak.move_rows( wiktionary_merkmal, mask, msg=None )
wiktionary_merkmal_abk = wiktionary_merkmal_abk.sort_values(['data_score','data'], ascending=[False,True])

Move 5134 rows from 4855641


In [99]:
gridt(wiktionary_merkmal_abk)

5134 rows


In [100]:
# data_tag setzen
wiktionary_merkmal_abk['data_tag'] = wiktionary_merkmal_abk.lemma_tag

In [101]:
# data_tag ist noch leer
mask = wiktionary_merkmal_abk.data_tag == ''
check_mask(wiktionary_merkmal_abk,mask,0,20)
grid(wiktionary_merkmal_abk,mask)

check_mask:  10 rows
10 rows out of 5134


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
455489,jemandem,abk,jemandem~jdm,jdm,,,,,1.369769,True
455490,jemandem,abk,jemandem~jmdm,jmdm,,,,,1.369769,True
374501,dieselbe,abk,dies,dies,,,,,1.211093,True
523835,jemanden,abk,jemanden~jdn,jdn,,,,,1.107525,True
523836,jemanden,abk,jemanden~jmdn,jmdn,,,,,1.107525,True
524261,jemandes,abk,jemandes~jds,jds,,,,,1.100368,True
524262,jemandes,abk,jemandes~jmds,jmds,,,,,1.100368,True
634282,dieselben,abk,dies,dies,,,,,0.994870,True
95905,einen_1,abk,nen,nen,,,1,,0.926635,True
457320,infiziert_1,abk,infiziert_1~inf,inf,,,,,0.174397,True


In [102]:
#TODO: Direkt durch die Kurzformen in wiktionary_lemma ergänzen?

In [103]:
# suche_debug wiktionary_merkmal_abk
print(bsp_abkürzungen_lang)
search_str(wiktionary_merkmal_abk, bsp_abkürzungen_lang)

['altisländisch', 'Europäische Union', 'Krankenhaus']


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
19432,Krankenhaus,abk,Krankenhaus~Krh,Krh,NN,NN,1,,5780.692871,True
19433,Krankenhaus,abk,Krankenhaus~Krhs,Krhs,NN,NN,1,,5780.692871,True
19434,Krankenhaus,abk,Krankenhaus~Krkhs,Krkhs,NN,NN,1,,5780.692871,True
1106272,altisländisch,abk,altnordisch,altnordisch,ADJA,ADJA,1,,34.749054,True
1106271,altisländisch,abk,altisländisch~non,non,ADJA,ADJA,1,,34.749054,True


In [104]:
# suche_debug wiktionary_merkmal_abk
search_str(wiktionary_merkmal_abk, suche_debug)

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
1830918,Euro,abk,Euro~€,€,NN,NN,,manuell,153.715988,True


### wiktionary_merkmal_abk belöschen

In [105]:
# belöschen: Manuelle Löschungen aus data oder lemma_id
löschmich =  ['Objekt','ECO-Codes','Bibo','Kilom','Land','Basche','DV-Vertrag','Igel','Dolle','Ampere','Baum',
              'Rat','Reibe','Heinrich_1','Diastema mediale','Divisionär','Soda-Zitron','Antillen-Gulden',
              "'H'B","'H'GW","'H'H","'H'L","'H'ST","'H'RO","'H'WI",'gel','gel.','Reg.','Imp.','Reg','Imp','AUR',
             ]

mask = wiktionary_merkmal_abk.data.isin(löschmich)
check_mask(wiktionary_merkmal_abk,mask, 48)

msg1 = 'löschmich_data'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg1)

mask = wiktionary_merkmal_abk.lemma_id.isin(löschmich)
check_mask(wiktionary_merkmal_abk,mask, 65)

msg2 = 'löschmich_id'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg2)

# anschauen
spalten = ['data', 'meta', 'lemma_id',  'lemma_tag',   'merkmal', 'msg']
mask = wiktionary_merkmal_trash.msg.isin([msg1, msg2])
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  48 rows
Move 48 rows from 5134
check_mask:  65 rows
Move 65 rows from 5086
113 rows out of 125


,data,meta,lemma_id,lemma_tag,merkmal,msg
95044,Bibo,umgangssprachliche,Bibliothek,NN,abk,löschmich_data
116869,Reg,,Regierung,NN,abk,löschmich_data
434639,Reg,,Register,NN,abk,löschmich_data
32407,Kilom,,Kilometer,NN,abk,löschmich_data
45837,Land,,Landung,NN,abk,löschmich_data
...,...,...,...,...,...,...
206228,Heyns,,Heinrich_1,NE,abk,löschmich_id
206229,Heynsen,,Heinrich_1,NE,abk,löschmich_id
206230,Hinner,,Heinrich_1,NE,abk,löschmich_id
350072,ANG,,Antillen-Gulden,NN,abk,löschmich_id


In [106]:
# belöschen: zu lang
mask1 =  wiktionary_merkmal_abk.data.str.len() > 6                    # zu lang
mask2 = ~wiktionary_merkmal_abk.data.str.contains('-')
mask3 = ~wiktionary_merkmal_abk.data.str.contains(' ')
mask4 = ~wiktionary_merkmal_abk.data.str.contains('.', regex=False)
mask5 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask6 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask7 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6  &  mask7
check_mask(wiktionary_merkmal_abk,mask, 338)

msg = 'zulang_single'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  338 rows
Move 338 rows from 5021
338 rows out of 463


,data,meta,lemma_id,lemma_tag,merkmal,msg
46514,Energie,,Arbeit,NN,abk,zulang_single
46513,englisch,,Arbeit,NN,abk,zulang_single
76349,Flächeneinheit,,Morgen,NN,abk,zulang_single
76351,Großbritannien,,Morgen,NN,abk,zulang_single
87656,Landsch,,Landschaft,NN,abk,zulang_single
...,...,...,...,...,...,...
272235,Landkreis,,Hann._Münden,NE,abk,zulang_single
587177,unverbr,,unverbrüchlich,ADJA,abk,zulang_single
295492,Aquilin,,Aquilinus,NE,abk,zulang_single
162351,Kraftfahrzeugkennzeichen,,Halle_1,NE,abk,zulang_single


In [107]:
# belöschen: zu lang
mask1 =  wiktionary_merkmal_abk.data.str.len() > 15                   # zu lang
mask2 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask3 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask4 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal_abk,mask, 10)

msg = 'zulang'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  10 rows
Move 10 rows from 4683
10 rows out of 473


,data,meta,lemma_id,lemma_tag,merkmal,msg
97032,Ohmscher Widerstand,Größenzeichen,Widerstand,NN,abk,zulang
67844,Doctor theologiae,,Doktor,NN,abk,zulang
1100563,Muster-ADV-Vertrag,,Muster-Auftragsdatenverarbeitungsvertrag,NN,abk,zulang
201556,Sacer Ordo Cisterciensis,,Zisterzienser,NN,abk,zulang
161836,Residual Current Device,englisch,Fehlerstromschutzschalter,NN,abk,zulang
199212,Ordo Cartusiensis,,Kartäuser,NN,abk,zulang
1152351,Kfz-Haftpflichtversicherung,,Kraftfahrzeug-Haftpflichtversicherung,NN,abk,zulang
331594,chemische Formel,,Natriumsulfat,NN,abk,zulang
245749,Niederländischen Antillen,Internet,Aruba,NE,abk,zulang
903360,chemische Formel,,Natriumnitrit,NN,abk,zulang


In [108]:
# belöschen: Namen
mask1 =  wiktionary_merkmal_abk.lemma_tag == 'NE'                  # ist Name
mask2 = ~wiktionary_merkmal_abk.data.str.isupper()                 # nicht nur groß
mask3 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask4 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask5 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(wiktionary_merkmal_abk,mask, 330)

msg = 'Namen'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  329 rows
Move 329 rows from 4673
329 rows out of 802


,data,meta,lemma_id,lemma_tag,merkmal,msg
4903,Dtl,,Deutschland,NE,abk,Namen
4904,Dtld,,Deutschland,NE,abk,Namen
73358,Bln,,Berlin,NE,abk,Namen
73357,Kfz-Kennzeichen,,Berlin,NE,abk,Namen
151798,Johs,,Johannes,NE,abk,Namen
...,...,...,...,...,...,...
213586,hau,ISO,Hausa_2,NE,abk,Namen
248970,bhw,ISO,Biak_1,NE,abk,Namen
250413,Dore,,Doris_1,NE,abk,Namen
230543,bgk,ISO,Bit_1,NE,abk,Namen


In [109]:
# belöschen: Nur Ziffern
mask1 = wiktionary_merkmal_abk.data.str.isdigit() 
mask2 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask3 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask4 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal_abk,mask, 20)

msg = 'nur_Ziffern'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  20 rows
Move 20 rows from 4344
20 rows out of 822


,data,meta,lemma_id,lemma_tag,merkmal,msg
525716,200,arabische,zweihundert,CARD,abk,nur_Ziffern
661540,400,Arabische,vierhundert,CARD,abk,nur_Ziffern
544347,102,,hundertundzweite,ADJA CARD,abk,nur_Ziffern
1701770,700,arabische,siebenhundert,CARD,abk,nur_Ziffern
1701511,900,arabische,neunhundert,CARD,abk,nur_Ziffern
635132,102,,hundertzweite,ADJA CARD,abk,nur_Ziffern
542226,101,,einhunderterste,ADJA CARD,abk,nur_Ziffern
525682,300,arabische,dreihundert,CARD,abk,nur_Ziffern
544350,200,,zweihundertste,ADJA CARD,abk,nur_Ziffern
542239,101,,hunderterste,ADJA CARD,abk,nur_Ziffern


In [110]:
# belöschen: Endet mit Ziffer
mask1 =  wiktionary_merkmal_abk.data.str[-1].str.isdigit() 
mask2 = ~wiktionary_merkmal_abk.data.str.endswith(('²','³','₂')) 
mask3 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask4 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask5 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5 
check_mask(wiktionary_merkmal_abk,mask, 162)

msg = 'Endet_mit_Ziffer'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  162 rows
Move 162 rows from 4324
162 rows out of 984


,data,meta,lemma_id,lemma_tag,merkmal,msg
20801,E 175,,Gold,NN,abk,Endet_mit_Ziffer
16159,E 949,E-Nummer,Wasserstoff,NN,abk,Endet_mit_Ziffer
126543,E 174,,Silber,NN,abk,Endet_mit_Ziffer
125462,E 948,,Sauerstoff,NN,abk,Endet_mit_Ziffer
126089,E 173,,Aluminium,NN,abk,Endet_mit_Ziffer
...,...,...,...,...,...,...
181742,E 953,,Isomalt,NN,abk,Endet_mit_Ziffer
305638,0-0,,kurze_Rochade,TODO,abk,Endet_mit_Ziffer
305798,0-0-0,,lange_Rochade,TODO,abk,Endet_mit_Ziffer
358976,0-0,,klein_rochieren,TODO,abk,Endet_mit_Ziffer


In [111]:
# belöschen: Bibel
mask1 =  wiktionary_merkmal_abk.lemma_id.str[0].str.isdigit() 
mask2 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask3 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask4 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal_abk,mask, 12)

msg = 'Bibel'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  12 rows
Move 12 rows from 4162
12 rows out of 996


,data,meta,lemma_id,lemma_tag,merkmal,msg
1773844,"2,5G",,"2,5G-Regel",NN,abk,Bibel
663739,2 Kor,,2._Korintherbrief,NN,abk,Bibel
663720,1 Kor,,1._Korintherbrief,NN,abk,Bibel
1217629,1 Thess,,1._Thessalonicherbrief,NN,abk,Bibel
1217648,1 Tim,,1._Timotheusbrief,NN,abk,Bibel
1217654,2 Tim,,2._Timotheusbrief,NN,abk,Bibel
1217633,2 Thess,,2._Thessalonicherbrief,NN,abk,Bibel
1219019,2 Petr,,2._Petrusbrief,NN,abk,Bibel
1219017,1 Petr,,1._Petrusbrief,NN,abk,Bibel
1219084,1 Joh,,1._Johannesbrief,NN,abk,Bibel


In [112]:
# belöschen: meta
löschmich_meta = ['Postleitzahl','Elementsymbol','seltener','selten','Ordenskürzel','Österreich','Makrosprache',
                  'Kirmanjki','Schweiz','ISO','Kfz','Internet','ISO-3166','Kfz-Kennzeichen','fachsprachlich','Dreibuchstabencode',
                  'international','Deutschland','ISO-4217-Code','ISO-639-3','chemisches','Zentral-Quiche','Komi',
                  'Kraftfahrzeugkennzeichen','englisch','veraltet','Dimli','Shimaore','bisweilen',
                  'Katalogbezeichnungen','ISO-639-1','inoffiziell','olympisch','chemische','Zusammensetzungen',
                  'Netzjargon','ISO-4217','bundesdeutsch','Währungscodes','UN LOCODE','römische','schweizerisch',
                  'Währungscode','Römische','bedeutungsgleiche','Summenformel',
                  'Rumänien','Lüftungstechnik','Lebensmittelfarbstoff','internationaler','volkstümlich',
                  'Maya-Sprachen','ISO-3166-1','ISO-3166-Code','üblich','wissenschaftlich','ehem',
                  'Polizeijargon','IOC-Ländercode','Moldawien','taktisches','postalisch','Abkürzung','Römisches',
                  'fachsprachlich schriftsprachlich','Schifffahrt','Codierung','SIL-Code']
mask1 =  wiktionary_merkmal_abk.meta.isin(löschmich_meta)
mask2 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask3 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal_abk,mask, 1035)

msg = 'meta'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
a = wiktionary_merkmal_trash[mask].sort_values(['meta','data_score'],ascending=[True,False])
grid(a)

check_mask:  1035 rows
Move 1035 rows from 4150
1035 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg
749186,Sperrzone,abk,Sperrzone~SpZo,SpZo,NN,NN,1,Abkürzung,54.319572,True,meta
628703,Asthenie,abk,ICD,ICD,NN,NN,1,Codierung,83.081894,True,meta
103075,Rangierbahnhof,abk,Rbf,Rbf,NN,NN,,Deutschland,124.814873,True,meta
1520458,Trinkwasserverordnung,abk,Trinkwasserverordnung~TWV,TWV,NN,NN,1,Deutschland,97.237305,True,meta
1133574,Gesundheitsministerium,abk,Gesundheitsministerium~BMG,BMG,NN,NN,1,Deutschland,59.766357,True,meta
...,...,...,...,...,...,...,...,...,...,...,...
655098,Kubikzentimeter,abk,cbcm,cbcm,NN,NN,1,veraltet,36.631935,True,meta
655097,Kubikzentimeter,abk,ccm,ccm,NN,NN,1,veraltet,36.631935,True,meta
175355,Bauernhof,abk,Hof,Hof,NN,NN,1,volkstümlich,198.280060,True,meta
335119,Wisent,abk,Wisent~B. bonasus,B. bonasus,NN,NN,1,wissenschaftlich,110.699867,True,meta


In [113]:
# Übriggebliebene nach meta
a = pak.analyse_freqs( wiktionary_merkmal_abk, ['meta','lemma_tag','data'] )
grid(a)

16 rows


,meta,meta_count,meta_percent,lemma_tag,lemma_tag_count,data,data_count
0,,2953,94.8,"[NN, ADJA, NE, ADV, TODO, VVFIN, APPR, , KOMBI, CARD, KON, NN NE, PDAT, ZFIX, PTKANT, ADV APPR, PIAT, KOUS, APPO APPR, ITJ]","[1997, 609, 123, 98, 36, 23, 18, 10, 8, 5, 4, 3, 3, 3, 3, 2, 2, 2, 2, 1]","[Str, inf, Pfl, Konv, ind, Ins, Gen, imp, Chemie, erk, Ztr, Adj, Chr, Konj, ged, get, Pat, NaCl, Ver, Erl]","[16, 9, 7, 7, 7, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4]"
1,manuell,139,4.5,"[NN, KOMBI, ADJA, KON, $(, ADV, NN NE, VVFIN, KOUS, ADV ADJD, APPR, CARD, NE]","[89, 18, 8, 5, 5, 4, 2, 2, 2, 1, 1, 1, 1]","[Bd., Feb., Jan., Str., FR, Fr, Fr., Nr., std., SO, So, So., OP, geschl., Abb., Hg., Hrsg., Jun., km, Abschn.]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,umgangssprachlich,4,0.1,"[NN, PIAT]","[3, 1]","[was, Perso, Mini, Kelle]","[1, 1, 1, 1]"
3,kurz,3,0.1,NN,3,"[Mayo, Emu, Snooker-WM]","[1, 1, 1]"
4,international,2,0.1,NE,2,"[DEU, UNO]","[1, 1]"
5,physikalisch,2,0.1,NN,2,"[atm, atü]","[1, 1]"
6,chemische,2,0.1,NN,2,"[CO2, SO2]","[1, 1]"
7,Einheitenzeichen,2,0.1,NN,2,"[min, var]","[1, 1]"
8,Summenformel,1,0.0,NN,1,H2O,1
9,ISO,1,0.0,NN,1,deu,1


In [114]:
# meta ist jetzt sinnlos, bis auf 'manuell'
mask =  wiktionary_merkmal_abk.meta != 'manuell'
wiktionary_merkmal_abk.loc[mask,'meta'] = ''

In [115]:
# suche_debug wiktionary_merkmal_abk
search_str(wiktionary_merkmal_abk, suche_debug)

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
1830918,Euro,abk,Euro~€,€,NN,NN,,manuell,153.715988,True


### Analysieren

In [116]:
# Gelöschte überprüfen, aus suche_debug?
print(suche_debug)
search_str(wiktionary_merkmal_trash, suche_debug)

['€']


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg


In [117]:
# Gelöschte überprüfen, aus lexeme_manuell?
mask1 = wiktionary_merkmal_trash.lemma_id.str.endswith('_M') 
mask2 = ~wiktionary_merkmal_trash.msg.isin(['dup'])
mask = mask1  &  mask2
check_mask(wiktionary_merkmal_trash, mask, 0)
#wiktionary_merkmal_trash[mask]

check_mask:  0 rows


In [118]:
# Gelöschte überprüfen, aus geschütze_abkürzungen?
mask = wiktionary_merkmal_trash.data.isin(geschütze_abkürzungen)
check_mask(wiktionary_merkmal_trash, mask, 0)

check_mask:  0 rows


In [119]:
# Gelöschte anschauen
a = pak.analyse_freqs( wiktionary_merkmal_trash, ['msg','lemma_tag','data'] )
grid(a)

11 rows


,msg,msg_count,msg_percent,lemma_tag,lemma_tag_count,data,data_count
0,meta,1035,51.0,"[NN, NE, ADJA, CARD, ADV, VVFIN, ITJ]","[769, 247, 8, 5, 4, 1, 1]","[chi, zho, kom, nya, ber, ful, nbl, nde, diq, bel, smi, NGC, kpv, srn, roa, fur, ndo, tut, grc, spa]","[11, 11, 5, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2]"
1,zulang_single,338,16.6,"[NN, ADJA, NE, VVFIN, TODO, PIAT, CARD, APPR, ADV]","[192, 105, 30, 4, 3, 1, 1, 1, 1]","[Summenformel, Kraftfahrzeugkennzeichen, Linguistik, Hirnnerv, Wasserl, Ostubaf, Schussw, alphanum, Halbstrukturformel, Gruppenformel, Fernschr, Morphonem, unverbr, Energie, englisch, Flächeneinheit, Großbritannien, Landsch, Rechenzeichen, Vorzeichen]","[14, 4, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1]"
2,Namen,329,16.2,NE,329,"[Kfz-Kennzeichen, Balder, UMa, Benz, UMi, Ofr, Chris, Bert, Baldo, Dtl, Dtld, Bln, Johs, Lzg, Bbg, Sachs, Hmb, Afr, Erzg, Erzgeb]","[20, 6, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,Endet_mit_Ziffer,162,8.0,"[NN, NE, TODO, VVFIN]","[103, 53, 4, 2]","[ICD-10, 0-0, 0-0-0, ISO 3166-2, OR-1, A-2384, A-2325, A-2351, E 507, ISO 639-3, E 175, E 949, E 174, E 948, E 173, E 939, E 501, A-2340, A-3390, R53]","[19, 6, 4, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
4,löschmich_id,65,3.2,"[NE, NN]","[61, 4]","[Sozi, Div, Hein, Heincke, Heindel, Heindl, Heine, Heinecke, Heineke, Heinel, Heinelt, Heinemann, Heinen, Heiner, Heinermann, Heinert, Heines, Heinicke, Heinig, Heink]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
5,löschmich_data,48,2.4,"[NN, ADJA, NE, VVFIN]","[34, 9, 3, 2]","[gel, Reg, Imp, Rat, Reibe, AUR, Bibo, Kilom, Land, Basche, Objekt, ECO-Codes, DV-Vertrag, Igel, Baum, Dolle, Ampere]","[11, 9, 9, 4, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
6,nur_Ziffern,20,1.0,"[ADJA CARD, CARD, ADV]","[12, 6, 2]","[100, 200, 102, 101, 300, 1000, 400, 700, 900, 500, 9000, 10000, 100000]","[3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1]"
7,Bibel,12,0.6,NN,12,"[2,5G, 2 Kor, 1 Kor, 1 Thess, 1 Tim, 2 Tim, 2 Thess, 2 Petr, 1 Petr, 1 Joh, 2 Joh, 3 Joh]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
8,zulang,10,0.5,"[NN, NE]","[9, 1]","[chemische Formel, Ohmscher Widerstand, Doctor theologiae, Muster-ADV-Vertrag, Sacer Ordo Cisterciensis, Residual Current Device, Ordo Cartusiensis, Kfz-Haftpflichtversicherung, Niederländischen Antillen]","[2, 1, 1, 1, 1, 1, 1, 1, 1]"
9,Leerstring data Müll,6,0.3,"[NE, NN]","[4, 2]",,6


In [120]:
# Gelöschte mit dem höchsten score
mask = ~wiktionary_merkmal_trash.msg.isin(['dup'])
grid(wiktionary_merkmal_trash[mask].sort_values('data_score',ascending=False))

2031 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg
46513,Arbeit,abk,englisch,englisch,NN,NN,7,,3426.708008,True,zulang_single
46514,Arbeit,abk,Energie,Energie,NN,NN,7,,3426.708008,True,zulang_single
592835,Blindgänger,abk,Blindgänger~UXB,UXB,NN,NN,1,international,3357.089111,True,meta
592834,Blindgänger,abk,Blindgänger~MEC,MEC,NN,NN,1,international,3357.089111,True,meta
592836,Blindgänger,abk,Blindgänger~UXO,UXO,NN,NN,1,international,3357.089111,True,meta
...,...,...,...,...,...,...,...,...,...,...,...
186226,Heide_3,abk,Heide_3~HEI,HEI,NE,NE,1,Kfz-Kennzeichen,0.967372,True,meta
460570,geliefert,abk,gel,gel,ADJA,ADJA,,,0.901368,True,löschmich_data
249498,Iban_2,abk,Iban_2~iba,iba,NE,NE,1,ISO,0.889556,True,Namen
430967,gelandet,abk,gel,gel,ADJA,ADJA,,,0.854617,True,löschmich_data


In [121]:
# Verbliebene nach tag
pak.analyse_freqs(wiktionary_merkmal_abk,['lemma_tag','meta','data'])

,lemma_tag,lemma_tag_count,lemma_tag_percent,meta,meta_count,data,data_count
0,NN,2106,67.6,"[, manuell]","[2017, 89]","[Str, Pfl, Konv, Ins, Gen, Ztr, Adj, Konj, Pat, NaCl, Ver, Chr, Erl, Fut, StB, Demogr, Geb, Hdl, Anl, Verf]","[16, 7, 7, 6, 6, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3]"
1,ADJA,617,19.8,"[, manuell]","[609, 8]","[ind, ged, get, unv, imp, afr, inf, geh, med, vsl, gebr, ungen, soz, frdl, akt, isl, int, reg, pers, lat]","[7, 5, 5, 4, 4, 4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2]"
2,NE,126,4.0,"[, manuell]","[125, 1]","[LUX, RUS, BAD, ABG, MKD, HAL, GKW, DEU, L.E., BRB, ISO, CHN, AUS, MUC, CHE, AUT, AFG, FRA, GBR, WAR]","[2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,ADV,102,3.3,"[, manuell]","[98, 4]","[bes, pass, insb, insbes, ebd, östl, sod, dto, bisl, sof, eig, eigtl, ntto, zur, teilw, desgl, btto, vll, vllt, vlt]","[3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
4,TODO,36,1.2,,36,"[Chemie, ad lib, s. A., s.A., sel. A., sel.A., m. E., i. E., S. M., gültig, C.c., S. H., m. m., S. Em., HDTV, A. D., z. H., z. Hd., a. l., ad l]","[4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
5,KOMBI,26,0.8,"[manuell, ]","[18, 8]","[i. Allg., i. A., u. a., u.a., z. Z., z. Zt., z.Zt., s. o., s.o., s. u., s.u., s. a., s.a., Anm. d. Red., i.A., k.A., o. B., o. dgl., o.ä., u. dgl.]","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
6,VVFIN,25,0.8,"[, manuell]","[23, 2]","[erk, inf, imp, gez, abgez, rad, abn, zit, entf, herst, anschl, betr, vgl, vgl., beh, dest, vergl, abr, rat, geb.]","[4, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
7,APPR,19,0.6,"[, manuell]","[18, 1]","[entg, innerh, bez, bezgl, bzgl, inf, mod, betr, hins, vs., anl, abzgl, zzgl, gem, inkl, exkl, südl, westl, östl]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
8,,10,0.3,,10,"[dies, jdm, jmdm, jdn, jmdn, jds, jmds, nen, inf]","[2, 1, 1, 1, 1, 1, 1, 1, 1]"
9,KON,9,0.3,"[manuell, ]","[5, 4]","[&, +, u., bezw, bzw, bzw., resp, rsp, d.h.]","[1, 1, 1, 1, 1, 1, 1, 1, 1]"


In [122]:
# suche_debug wiktionary_merkmal_abk
print(suche_debug)
search_str(wiktionary_merkmal_abk, suche_debug)

['€']


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
1830918,Euro,abk,Euro~€,€,NN,NN,,manuell,153.715988,True


In [123]:
# suche_debug wiktionary_merkmal
#mask = wiktionary_merkmal.merkmal.isin(['gm'])
#a = search_str(wiktionary_merkmal[mask], suche_debug)
#grid(a.sort_values('lemma_id'))

### wiktionary_merkmal_abk korrigieren

In [124]:
# Einzelne Worte löschen
r = ['(grad)',]
wiktionary_merkmal_abk['data'] = pak.remove_str(wiktionary_merkmal_abk.data, r)

In [125]:
# korrigieren: Punkt innen ohne Punkt hinten
# Das ist eigentlich schon weiter oben passiert
endet_nicht_mit = ( '.', 'Tag', 'Ort',)
mask1 =  wiktionary_merkmal_abk.data.str.contains('.', regex=False)  
mask2 = ~wiktionary_merkmal_abk.data.str.endswith(endet_nicht_mit)
mask3 = (wiktionary_merkmal_abk.data.str.split('.').str[-1].str.len() <= 5) # letztes Wort nicht zu lang
mask4 =  wiktionary_merkmal_abk.data.str.len() > 2
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask( wiktionary_merkmal_abk, mask, 0,136 )
wiktionary_merkmal_abk.loc[mask,'data'] += '.'
grid(wiktionary_merkmal_abk,mask)

check_mask:  2 rows
2 rows out of 3115


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
186271,Steuerberater,abk,Steuerberater~St.-Ber,St.-Ber.,NN,NN,1,,80.872543,True
313841,Steuernummer,abk,Steuernummer~St.-Nr,St.-Nr.,NN,NN,1,,24.318172,True


In [126]:
# korrigieren: Punkt hinten ergänzen
ausnahmen =  [ 'Admin','Schirm','Kappe','Symbol','Potenz','Schiri','Info','Bus','Kiga','Hof','Teen','Teenie','Teeny','Nitro','Steno','Perso','Navi',]
ausnahmen += [ 'Mayo','Hasch','Britta','Tusse','Tussi','Jäger','Reha','Latte','Pkw','Escape','Spasti','Proll','Prolo','Traufe','Qbit','Qubit',]
ausnahmen += [ 'Rose','Kino','Alster','Hafen','Psycho','Dame','Kelle','Kopie','FeWo','Chemie','Knobi','Makro','Majo','Tief']
mask1 = ~wiktionary_merkmal_abk.data.isin(ausnahmen)
mask2 = ~wiktionary_merkmal_abk.data.str.contains('-')                                                       # Kein Bindestrich
mask3 = ~wiktionary_merkmal_abk.data.str.contains(' ')                                                       # kein Space
mask4 = ~wiktionary_merkmal_abk.data.str.contains('.', regex=False)                                          # kein Punkt
mask5 = ~wiktionary_merkmal_abk.meta.isin(['ISO','kurz','umgangssprachlich','Netzjargon'])
mask6 = (wiktionary_merkmal_abk.data.str.len() > 2)   |   wiktionary_merkmal_abk.data.isin(['Dr','No'])
mask7 =  wiktionary_merkmal_abk.data.str.contains('^[A-Za-zöüäÖÄÜ]')                                         # startswith Buchstabe
mask8 =  wiktionary_merkmal_abk.data.str.count('[A-ZÜÖÄ]')  <=    1                                          # Nicht mehr als 1 Großbuchstabe
mask9 = ~wiktionary_merkmal_abk.data.str.contains('[^A-Za-zöüäÖÄÜß]')                                        # irgendwelche Sonderzeichen

mask  = mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6  &  mask7  &  mask8  &  mask9
check_mask( wiktionary_merkmal_abk, mask, 1815 )

wiktionary_merkmal_abk.loc[mask,'data'] += '.'  
grid(wiktionary_merkmal_abk,mask)

check_mask:  1815 rows
1815 rows out of 3115


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
19432,Krankenhaus,abk,Krankenhaus~Krh,Krh.,NN,NN,1,,5780.692871,True
19433,Krankenhaus,abk,Krankenhaus~Krhs,Krhs.,NN,NN,1,,5780.692871,True
19434,Krankenhaus,abk,Krankenhaus~Krkhs,Krkhs.,NN,NN,1,,5780.692871,True
138965,Garten,abk,Garten~Gart,Gart.,NN,NN,1,,4551.003906,True
21871,Sprache,abk,Sprache~Spr,Spr.,NN,NN,2,,3765.383057,True
...,...,...,...,...,...,...,...,...,...,...
145125,östlich_1,abk,östlich_1~östl,östl.,ADV,ADV,1,,0.779431,True
457318,infiziert,abk,infiziert~inf,inf.,ADJA,ADJA,,,0.769881,True
479907,getilgt,abk,get_M,get.,ADJA,ADJA,,,0.711443,True
457320,infiziert_1,abk,infiziert_1~inf,inf.,,,,,0.174397,True


In [127]:
# variieren: Punkt mit Space
#mask = wiktionary_merkmal_abk.data.str.contains('.', regex=False)
mask = wiktionary_merkmal_abk.data.str.count('\.') > 1
df = wiktionary_merkmal_abk[mask].copy()
r = {'.':'. ',   '-':''}
df['data']    = pak.replace_str(df.data, r).str.strip()
wiktionary_merkmal_abk = pak.add_rows(wiktionary_merkmal_abk, df)
#grid(df)

164 rows added, now a total of 3279


In [128]:
# variieren: Bindestriche weglassen
mask1 =  wiktionary_merkmal_abk.data.str.contains('-')
mask2 = ~wiktionary_merkmal_abk.data.str.startswith('-')
mask3 = ~wiktionary_merkmal_abk.data.str.endswith('-')
mask = mask1  &  mask2  &  mask3
df = wiktionary_merkmal_abk[mask].copy()
r = {'-':' ', '  ':' '}
df['data']    = pak.replace_str(df.data, r)
wiktionary_merkmal_abk = pak.add_rows(wiktionary_merkmal_abk, df)
#grid(df)

123 rows added, now a total of 3402


In [129]:
# variieren: Punkte weglassen
mask = wiktionary_merkmal_abk.data.str.contains('.', regex=False)
df = wiktionary_merkmal_abk[mask].copy()
r = {'.':'',}
df['data']    = pak.replace_str(df.data, r)
wiktionary_merkmal_abk = pak.add_rows(wiktionary_merkmal_abk, df)
#grid(df)

2254 rows added, now a total of 5656


In [130]:
# variieren: Spaces weglassen
mask = wiktionary_merkmal_abk.data.str.contains(' ')
df = wiktionary_merkmal_abk[mask].copy()
r = {' ':'',}
df['data']    = pak.replace_str(df.data, r)
wiktionary_merkmal_abk = pak.add_rows(wiktionary_merkmal_abk, df)
#grid(df)

701 rows added, now a total of 6357


In [131]:
# pflegen: Überfüssige Spaces streichen
r = {'  ':' ', '  ':' ',}
wiktionary_merkmal_abk['data'] = pak.replace_str(wiktionary_merkmal_abk.data, r)
wiktionary_merkmal_abk['data'] = wiktionary_merkmal_abk.data.str.strip()

In [132]:
# data_id neu vergeben
wiktionary_merkmal_abk['data_id'] = (wiktionary_merkmal_abk.lemma_id + '~' + wiktionary_merkmal_abk.data).str.strip()

# data_id vorhanden und nicht widerspüchlich?
mask = wiktionary_merkmal_abk.data_id == ''
check_mask(wiktionary_merkmal_abk,mask,0)

a = pak.same_but_different(wiktionary_merkmal_abk,same='data_id',different='data')
assert a.shape[0] == 0

check_mask:  0 rows


In [133]:
grid(wiktionary_merkmal_abk.sort_values('data'))

6357 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
731,Prozent,abk,Prozent~%,%,NN,NN,,manuell,143.270859,True
387,und,abk,und~&,&,KON,KON,,manuell,245.027344,True
388,und,abk,und~+,+,KON,KON,,manuell,245.027344,True
2923,-prozentig,abk,-prozentig~-%ig,-%ig,ZFIX,ZFIX,1,,14.858729,True
1272,-schaft,abk,-schaft~-sch,-sch,ZFIX,ZFIX,1,,78.539078,True
...,...,...,...,...,...,...,...,...,...,...
3083,Microcontroller_M,abk,Microcontroller_M~μC,μC,NN,NN,,manuell,2.239461,True
2723,Mikrosievert,abk,Mikrosievert~μSv,μSv,NN,NN,1,,26.903440,True
1399,Mikrometer,abk,Mikrometer~μm,μm,NN,NN,,manuell,71.704559,True
2156,Quadratmikrometer,abk,Quadratmikrometer~μm²,μm²,NN,NN,1,,41.616608,True


In [134]:
# pflegen: Dups entfernen
aggwords    =  pak.agg_words_nodup
wiktionary_merkmal_abk = pak.group_and_agg( wiktionary_merkmal_abk, 
              ['lemma_id',  'lemma_tag',  'num',  'merkmal',   'meta',   'data',   'data_tag', 'data_id',  'data_score', 'data_score',   ],
              [ 'group',    'group',      'min',  'group',     aggwords, 'group',   aggwords,  'group',    'max',        'count',        ],
              ['lemma_id',  'lemma_tag',  'num',  'merkmal',   'meta',   'data',   'data_tag', 'data_id',  'data_score', '',             ],        
             )


718 rows less, now 5639 rows


In [135]:
# data_score 
mask = wiktionary_merkmal_abk.data_score_count > 1
wiktionary_merkmal_abk.loc[mask,'data_score'] *= 1+(wiktionary_merkmal_abk[mask].data_score_count / 10)
wiktionary_merkmal_abk = pak.drop_cols(wiktionary_merkmal_abk,'data_score_count')
#wiktionary_merkmal_abk[mask]

In [136]:
# suche_debug wiktionary_merkmal_abk
search_str(wiktionary_merkmal_abk, suche_debug)

,lemma_id,lemma_tag,num,merkmal,meta,data,data_tag,data_id,data_score
1027,Euro,NN,,abk,manuell,€,NN,Euro~€,153.715988


In [137]:
# anschauen
grid(wiktionary_merkmal_abk.sort_values('data_score',ascending=False))

5639 rows


,lemma_id,lemma_tag,num,merkmal,meta,data,data_tag,data_id,data_score
3578,Wasser,NN,1,abk,,H2O,NN,Wasser~H2O,6229.017090
1962,Krankenhaus,NN,1,abk,,Krhs.,NN,Krankenhaus~Krhs.,5780.692871
1961,Krankenhaus,NN,1,abk,,Krhs,NN,Krankenhaus~Krhs,5780.692871
1959,Krankenhaus,NN,1,abk,,Krh,NN,Krankenhaus~Krh,5780.692871
1964,Krankenhaus,NN,1,abk,,Krkhs.,NN,Krankenhaus~Krkhs.,5780.692871
...,...,...,...,...,...,...,...,...,...
4290,getilgt,ADJA,,abk,,get.,ADJA,getilgt~get.,0.711443
4425,infiziert_1,,,abk,,inf,,infiziert_1~inf,0.174397
4426,infiziert_1,,,abk,,inf.,,infiziert_1~inf.,0.174397
4227,gedeckt_1,ADJA,,abk,,ged.,ADJA,gedeckt_1~ged.,0.086008


### wiktionary_lemma pflegen

In [138]:
# suche_debug 
search_str(wiktionary_lemma, suche_debug)

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score


In [139]:
# Alle auflösbaren Abkürzungen mit eigenem Eintrag in wiktionary_lemma
mask_K = wiktionary_merkmal_abk.lemma_id != wiktionary_merkmal_abk.data_id
wiktionary_merkmal_abk[mask_K].tail(3)

,lemma_id,lemma_tag,num,merkmal,meta,data,data_tag,data_id,data_score
5636,üblich,ADJA,1,abk,,übl.,ADJA,üblich~übl.,75.100830
5637,üblicherweise,ADV,1,abk,,üblw,ADV,üblicherweise~üblw,22.103651
5638,üblicherweise,ADV,1,abk,,üblw.,ADV,üblicherweise~üblw.,22.103651


In [140]:
# Obsolet
# wiktionary_lemma.member nachtragen
# Nicht jede Abkürzung hat einen eigenen Eintrag in wiktionary_lemma.
# Aber wenn, dann ist dieser mit member 'Abkürzung' gekennzeichnet.

mask1 = ~wiktionary_lemma.member.str.contains('Abkürzung')                       # nicht als Abkürzung gekennzeichnet
mask2 = wiktionary_lemma.lemma_id.isin(wiktionary_merkmal_abk[mask_K].data_id)   # ist aber eine auflösbare Abkürzung
mask = mask1  &  mask2
check_mask( wiktionary_lemma, mask, 0, 169 )

wiktionary_lemma.loc[mask,'member'] = (wiktionary_lemma[mask].member + ' Abkürzung').str.strip()
grid( wiktionary_lemma, mask )

check_mask:  0 rows
No rows, mask filters them all away


### translate_abk
* translate_abk1: Nimmt die in wiktionary_lemma gelisteten Abkürzungen und löst sie über ihr def-Merkmal auf
* translate_abk2: Basiert auf wiktionary_merkmal_abk

#### translate_abk1

In [141]:
# translate_abk1: Kurzformen aus wiktionary_lemma
mask_l = wiktionary_lemma.member.str.contains('Abkürzung')
spalten = ['lemma_id','lemma','tag','member','lemma_score',]
translate_abk1 = wiktionary_lemma[mask_l][spalten]
translate_abk1 = pak.rename_col(translate_abk1,'lemma',       'kurz')
translate_abk1 = pak.rename_col(translate_abk1,'lemma_id',    'kurz_id')
translate_abk1 = pak.rename_col(translate_abk1,'tag',         'kurz_tag')
translate_abk1 = pak.rename_col(translate_abk1,'member',      'kurz_member')
translate_abk1 = pak.rename_col(translate_abk1,'lemma_score', 'kurz_score')
translate_abk1 = pak.reset_index(translate_abk1)
pak.sample(translate_abk1)

,kurz_id,kurz,kurz_tag,kurz_member,kurz_score
0,24/7,24/7,ADJA,Abkürzung manuell ergänzt,14.942436
1,2D,2D,ADJA,manuell ergänzt Abkürzung,21.063148
22,http,http,TRUNC,Abkürzung manuell ergänzt,19.930384
33,&c.,&c.,TODO,Abkürzung,4.049606
455,BH,BH,NN,Abkürzung,189.630875
667,BerlinFG,BerlinFG,TODO,Abkürzung,5.788859
2975,Stasi,Stasi,NN,Abkürzung noPl,9214.994141
4463,z._A.,z. A.,TODO,Abkürzung,0.102266
4529,μm,μm,TODO,Abkürzung,41.927307


In [142]:
# Langform verknüpfen
mask_m = (wiktionary_merkmal.merkmal == 'def') # alle Definitionen, nicht nur die der Abkürzungen

# lang_id
translate_abk1 = pak.update_col(              df_to =      translate_abk1, 
                                              df_from =    wiktionary_merkmal[mask_m], 
                                              left_on =    'kurz_id',
                                              right_on =   'lemma_id',
                                              col =        'data_id',
                                              col_rename = 'lang_id',
                                              func =       'first',
                                            )
# lang
translate_abk1 = pak.update_col(              df_to =      translate_abk1, 
                                              df_from =    wiktionary_merkmal[mask_m], 
                                              left_on =    'kurz_id',
                                              right_on =   'lemma_id',
                                              col =        'data',
                                              col_rename = 'lang',
                                              func =       'first',
                                            )

# lang_tag
translate_abk1 = pak.update_col(              df_to =      translate_abk1, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'tag',
                                              col_rename = 'lang_tag',
                                            )


# lang_member
translate_abk1 = pak.update_col(              df_to =      translate_abk1, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'member',
                                              col_rename = 'lang_member',
                                            )

# lang_score
translate_abk1 = pak.update_col(              df_to =      translate_abk1, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'lemma_score',
                                              col_rename = 'lang_score',
                                            )

translate_abk1['todo'] = 'lemma' 
translate_abk1['rank'] = 1.8 
print()
pak.sample(translate_abk1)

update_col: func first applied, 178789 records less!
update_col: 2711 cells written into new column
update_col: func first applied, 178789 records less!
update_col: 2711 cells written into new column
update_col: 2616 cells written into new column
update_col: 2616 cells written into new column
update_col: 2616 cells written into new column



,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank
0,24/7,24/7,ADJA,Abkürzung manuell ergänzt,14.942436,Stunde,Stunde,NN,Einheit manuell,6160.494141,lemma,1.8
22,http,http,TRUNC,Abkürzung manuell ergänzt,19.930384,<NA>,<NA>,<NA>,<NA>,NaN,lemma,1.8
31,on,on,FM,manuell ergänzt Abkürzung,21.076181,online,online,ADJA,noSteig noFlex,319.479553,lemma,1.8
33,&c.,&c.,TODO,Abkürzung,4.049606,et_cetera,et cetera,ADV,Wortverbindung manuell,30.591106,lemma,1.8
40,1_Joh,1 Joh,TODO,Abkürzung,15.758732,1._Johannesbrief,1. Johannesbrief,NN,adjNN Wortverbindung adjNN noPl,74.711502,lemma,1.8
455,BH,BH,NN,Abkürzung,189.630875,Büstenhalter,Büstenhalter,NN,manuell,1189.476318,lemma,1.8
605,BVerfGG,BVerfGG,TODO,Abkürzung,11.272240,Gesetz,Gesetz,NN,,15998.839844,lemma,1.8
667,BerlinFG,BerlinFG,TODO,Abkürzung,5.788859,<NA>,<NA>,<NA>,<NA>,NaN,lemma,1.8
2179,MCU,MCU,TODO,Abkürzung,19.673538,Mikrocontroller,Mikrocontroller,NN,,356.384735,lemma,1.8
2975,Stasi,Stasi,NN,Abkürzung noPl,9214.994141,Geheimpolizei,Geheimpolizei,NN,,4278.938477,lemma,1.8


#### translate_abk2

In [143]:
# translate_abk2: aus wiktionary_merkmal_abk
translate_abk2 = wiktionary_merkmal_abk.copy()
translate_abk2 = pak.drop_cols(translate_abk2,['num','merkmal'])
translate_abk2 = pak.rename_col(translate_abk2,'lemma_id',  'lang_id')
translate_abk2 = pak.rename_col(translate_abk2,'lemma_tag', 'lang_tag')
translate_abk2 = pak.rename_col(translate_abk2,'data_score','lang_score')
translate_abk2 = pak.rename_col(translate_abk2,'data',      'kurz')
translate_abk2 = pak.rename_col(translate_abk2,'data_id',   'kurz_id')
translate_abk2 = pak.rename_col(translate_abk2,'data_tag',  'kurz_tag')
translate_abk2 = pak.rename_col(translate_abk2,'meta',      'kurz_member')
translate_abk2 = pak.reset_index(translate_abk2)
translate_abk2 = pak.move_cols(translate_abk2,['kurz_id','kurz'])
pak.sample(translate_abk2)

,kurz_id,kurz,lang_id,lang_tag,kurz_member,kurz_tag,lang_score
0,-prozentig~-%ig,-%ig,-prozentig,ZFIX,,ZFIX,14.858729
253,Artificial_Intelligence_M~AI,AI,Artificial_Intelligence_M,NN,manuell,NN,2.239461
1027,Euro~€,€,Euro,NN,manuell,NN,153.715988
1255,Geburtstag~Geb,Geb,Geburtstag,NN,,NN,1581.322998
2702,Prozent~%,%,Prozent,NN,manuell,NN,143.270859
3198,Strafe~Str,Str,Strafe,NN,,NN,411.752258
3578,Wasser~H2O,H2O,Wasser,NN,,NN,6229.017090
4037,dieselbe~dies,dies,dieselbe,,,,1.211093
4226,gedeckt_1~ged,ged,gedeckt_1,ADJA,,ADJA,0.086008
5636,üblich~übl.,übl.,üblich,ADJA,,ADJA,75.100830


In [144]:
# keine eigene kurz_id
mask = translate_abk2.kurz_id == translate_abk2.lang_id
check_mask(translate_abk2,mask, 0) 
translate_abk2.loc[mask,'kurz_id'] = ''

check_mask:  0 rows


In [145]:
# kurz_id ist nicht in wiktionary_lemma
#mask1 = ~translate_abk2.kurz_id.isin(wiktionary_lemma.lemma_id)
#mask2 =  translate_abk2.kurz_id != ''
#mask = mask1  &  mask2 
#check_mask(translate_abk2,mask,4503) 
#translate_abk2.loc[mask,'kurz_id'] = ''

In [146]:
# War das richtig?
bgc('orange')

In [147]:
# Leere tags
mask = translate_abk2.kurz_tag == 'TODO'
translate_abk2.loc[mask,'kurz_tag'] = ''

mask = translate_abk2.lang_tag == 'TODO'
translate_abk2.loc[mask,'lang_tag'] = ''

#### translate_abk

In [148]:
# kurz_tag
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'kurz_id',
                                              right_on =   'lemma_id',
                                              col =        'tag',
                                              col_rename = 'kurz_tag',
                                              func =       'first',
                                              cond =       'null'
                                            )


# kurz_member
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'kurz_id',
                                              right_on =   'lemma_id',
                                              col =        'member',
                                              col_rename = 'kurz_member',
                                              cond =       'null'
                                            )

# kurz_score
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'kurz_id',
                                              right_on =   'lemma_id',
                                              col =        'lemma_score',
                                              col_rename = 'kurz_score',
                                            )

# lang
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'lemma',
                                              col_rename = 'lang',
                                            )


# lang_member
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'member',
                                              col_rename = 'lang_member',
                                            )


translate_abk2 = pak.move_cols(translate_abk2,['kurz_tag','kurz_member','kurz_score'],'kurz')
translate_abk2 = pak.move_cols(translate_abk2,['lang','lang_tag','lang_member'],'lang_id')
translate_abk2['rank'] = 1.0 
translate_abk2['todo'] = 'merkmal' 
print()
pak.sample(translate_abk2)

update_col: func first applied, but it was pointless!
update_col: 0 cells written into existing column
update_col: 0 cells written into existing column
update_col: 0 cells written into new column
update_col: 5619 cells written into new column
update_col: 5619 cells written into new column



,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,rank,todo
0,-prozentig~-%ig,-%ig,ZFIX,,NaN,-prozentig,-prozentig,ZFIX,Gebundenes_Lexem,14.858729,1.0,merkmal
253,Artificial_Intelligence_M~AI,AI,NN,manuell,NaN,Artificial_Intelligence_M,Artificial Intelligence,NN,manuell,2.239461,1.0,merkmal
1027,Euro~€,€,NN,manuell,NaN,Euro,Euro,NN,Einheit manuell,153.715988,1.0,merkmal
1255,Geburtstag~Geb,Geb,NN,,NaN,Geburtstag,Geburtstag,NN,,1581.322998,1.0,merkmal
2702,Prozent~%,%,NN,manuell,NaN,Prozent,Prozent,NN,Einheit manuell,143.270859,1.0,merkmal
3198,Strafe~Str,Str,NN,,NaN,Strafe,Strafe,NN,,411.752258,1.0,merkmal
3578,Wasser~H2O,H2O,NN,,NaN,Wasser,Wasser,NN,,6229.017090,1.0,merkmal
4037,dieselbe~dies,dies,,,NaN,dieselbe,<NA>,,<NA>,1.211093,1.0,merkmal
4226,gedeckt_1~ged,ged,ADJA,,NaN,gedeckt_1,gedeckt,ADJA,Ptz2,0.086008,1.0,merkmal
4878,okay~O K,O K,ADJA,,NaN,okay,okay,ADJA,noSteig noFlex,51.358014,1.0,merkmal


In [149]:
# translate_abk erstellen
translate_abk = pd.concat([translate_abk1,translate_abk2], ignore_index=True)
translate_abk = pak.change_datatype(translate_abk)
translate_abk_trash = translate_abk.head(0)
translate_abk_trash['msg'] = None

change_datatype 
lang_score           --> np.float32
todo                 --> pd.string 
rank                 --> np.float32
change_datatype before: 5.9 MB after: 5.8 MB



In [150]:
# fillna

translate_abk['kurz_id']     = translate_abk.kurz_id.fillna('')
translate_abk['lang_id']     = translate_abk.lang_id.fillna('')

translate_abk['kurz']        = translate_abk.kurz.fillna('')
translate_abk['lang']        = translate_abk.lang.fillna('')

translate_abk['kurz_tag']    = translate_abk.kurz_tag.fillna('').str.replace('TODO','').str.strip()
translate_abk['lang_tag']    = translate_abk.lang_tag.fillna('').str.replace('TODO','').str.strip()

translate_abk['kurz_member'] = translate_abk.kurz_member.fillna('')
translate_abk['lang_member'] = translate_abk.lang_member.fillna('')

translate_abk['kurz_score']  = translate_abk.kurz_score.fillna(0)
translate_abk['lang_score']  = translate_abk.lang_score.fillna(0)

#### Belöschen und ranken

In [151]:
# konkurrenz: Wie viele konkurrierende lang_id gibt es?
gruppiert = translate_abk.groupby(['kurz'])
translate_abk['konkurrenz'] = gruppiert.lang_id.transform( 'nunique' ) 

In [152]:
grid(translate_abk,pak.sample)

11 rows out of 10169


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,konkurrenz
0,24/7,24/7,ADJA,Abkürzung manuell ergänzt,14.942436,Stunde,Stunde,NN,Einheit manuell,6160.494141,lemma,1.8,1
31,on,on,FM,manuell ergänzt Abkürzung,21.076181,online,online,ADJA,noSteig noFlex,319.479553,lemma,1.8,1
33,&c.,&c.,,Abkürzung,4.049606,et_cetera,et cetera,ADV,Wortverbindung manuell,30.591106,lemma,1.8,1
2975,Stasi,Stasi,NN,Abkürzung noPl,9214.994141,Geheimpolizei,Geheimpolizei,NN,,4278.938477,lemma,1.8,1
2981,Str.,Str.,,Abkürzung,135.142532,Strafe,Strafe,NN,,3705.770508,lemma,1.8,16
3337,Ws.,Ws.,,Abkürzung,11.028746,Wasser,Wasser,NN,,56061.152344,lemma,1.8,1
4529,μm,μm,,Abkürzung,41.927307,,,,,0.000000,lemma,1.8,2
4530,-prozentig~-%ig,-%ig,ZFIX,,0.000000,-prozentig,-prozentig,ZFIX,Gebundenes_Lexem,14.858729,merkmal,1.0,1
5557,Euro~€,€,NN,manuell,0.000000,Euro,Euro,NN,Einheit manuell,153.715988,merkmal,1.0,1
7232,Prozent~%,%,NN,manuell,0.000000,Prozent,Prozent,NN,Einheit manuell,143.270859,merkmal,1.0,1


In [153]:
# ID fehlt
maskA = translate_abk.kurz_id == ''
maskB = translate_abk.lang_id == ''
mask = maskA | maskB
check_mask(translate_abk,mask,1864)

translate_abk.loc[mask,'rank'] *= 0.5
translate_abk[mask].sort_values('kurz_score', ascending=False)

check_mask:  1819 rows


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,konkurrenz
2323,NGO,NGO,NN,Abkürzung,2046.884277,,,,,0.0,lemma,0.9,2
17,USA,USA,NE,Geo Geo_land noSg Abkürzung,1161.254395,,,,,0.0,lemma,0.9,3
2840,SMS,SMS,NN,Abkürzung,870.798462,,,,,0.0,lemma,0.9,1
10,GmbH,GmbH,NN,manuell ergänzt Abkürzung,682.923218,,,,,0.0,lemma,0.9,1
206,ASCII,ASCII,NN,Abkürzung noPl,619.340088,,,,,0.0,lemma,0.9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4370,umg.,umg.,,Abkürzung,0.185524,,,,,0.0,lemma,0.9,1
3930,k._A.,k. A.,,Abkürzung,0.178712,,,,,0.0,lemma,0.9,2
1445,FDA_1,FDA,,Abkürzung,0.151688,,,,,0.0,lemma,0.9,1
4477,zit._n.,zit. n.,,Abkürzung,0.137390,,,,,0.0,lemma,0.9,1


In [154]:
# Anfangsbuchstabe ungleich
mask1 = (translate_abk.lang.str[0].str.lower()  !=  translate_abk.kurz.str[0].str.lower())
mask2 = ~translate_abk.kurz.str[0].str.isdigit() 
mask3 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask = mask1  &  mask2  &  mask3
check_mask(translate_abk,mask,984)

translate_abk.loc[mask,'rank'] *= 0.8
translate_abk[mask]

check_mask:  893 rows


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,konkurrenz
7,BDSM,BDSM,NN,Abkürzung manuell ergänzt,208.074417,Sammelbezeichnung,Sammelbezeichnung,NN,,277.477875,lemma,1.44,2
12,ISBN,ISBN,NN,manuell ergänzt Abkürzung,103.012543,Nummer,Nummer,NN,Einheit manuell,7274.298828,lemma,1.44,1
13,LGBT,LGBT,NN,Abkürzung manuell,134.721130,Sammelbezeichnung,Sammelbezeichnung,NN,,277.477875,lemma,1.44,1
33,&c.,&c.,,Abkürzung,4.049606,et_cetera,et cetera,ADV,Wortverbindung manuell,30.591106,lemma,1.44,1
70,A.C.A.B.,A.C.A.B.,,Abkürzung,59.133762,Parole,Parole,NN,,1149.086792,lemma,1.44,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10113,Ölhafen~Hafen,Hafen,NN,,0.000000,Ölhafen,Ölhafen,NN,,27.923941,merkmal,0.80,1
10114,Österreich~AUT,AUT,NE,,0.000000,Österreich,Österreich,NE,Geo Geo_land,195.422760,merkmal,0.80,1
10115,Überhorizontradar~OTH Radar,OTH Radar,NN,,0.000000,Überhorizontradar,Überhorizontradar,NN,,34.843922,merkmal,0.80,1
10116,Überhorizontradar~OTH-Radar,OTH-Radar,NN,,0.000000,Überhorizontradar,Überhorizontradar,NN,,34.843922,merkmal,0.80,1


In [155]:
# nichtssagende_abk1
mask1 =  translate_abk.kurz.str.len() == 3
mask2 =  translate_abk.kurz.str.startswith('ge')
mask3 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask4 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(translate_abk,mask,20)

msg = 'nichtssagende_abk1'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  24 rows
Move 24 rows from 10169
24 rows


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,msg,konkurrenz
5817,Gegisch~geg,geg,NN,,0.0,Gegisch,Gegisch,NN,,52.228848,merkmal,1.0,nichtssagende_abk1,2
5835,Gemeingermanisch~gem,gem,NN,,0.0,Gemeingermanisch,Gemeingermanisch,NN,,31.228254,merkmal,1.0,nichtssagende_abk1,3
5872,Germanisch~gem,gem,NN,,0.0,Germanisch,Germanisch,NN,,59.976376,merkmal,1.0,nichtssagende_abk1,3
6926,Neuhochdeutsch~ger,ger,NN,,0.0,Neuhochdeutsch,Neuhochdeutsch,NN,,71.346214,merkmal,0.8,nichtssagende_abk1,1
8742,gebildet~geb,geb,ADJA,,0.0,gebildet,gebildet,ADJA,,43.955456,merkmal,1.0,nichtssagende_abk1,3
8744,geboren~geb,geb,ADJA,,0.0,geboren,geboren,ADJA,noSteig,38.656197,merkmal,1.0,nichtssagende_abk1,3
8754,gedeckt~ged,ged,ADJA,,0.0,gedeckt,gedeckt,ADJA,,20.119692,merkmal,1.0,nichtssagende_abk1,5
8756,gedeckt_1~ged,ged,ADJA,,0.0,gedeckt_1,gedeckt,ADJA,Ptz2,0.086008,merkmal,1.0,nichtssagende_abk1,5
8758,gediegen~ged,ged,ADJA,,0.0,gediegen,gediegen,ADJA,,63.110580,merkmal,1.0,nichtssagende_abk1,5
8760,gedient~ged,ged,ADJA,,0.0,gedient,gedient,ADJA,Ptz2,0.910134,merkmal,1.0,nichtssagende_abk1,5


In [156]:
# nichtssagende_abk2
mask1 =  translate_abk.kurz.str.len() == 4
mask2 =  translate_abk.kurz.str.startswith('ge')
mask3 =  translate_abk.kurz.str.endswith('.')
mask4 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask5 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(translate_abk,mask,34)

msg = 'nichtssagende_abk2'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  39 rows
Move 39 rows from 10145
39 rows out of 63


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,msg,konkurrenz
3753,geb.,geb.,,Abkürzung,88.242928,geborener,geborener,,,0.000000,lemma,1.8,nichtssagende_abk2,4
3755,ged.,ged.,,Abkürzung,13.196012,gedämpft,gedämpft,ADJA,Ptz2,0.837086,lemma,1.8,nichtssagende_abk2,5
3756,gef.,gef.,,Abkürzung,34.923553,gefallen,gefallen,VVFIN,,101.318176,lemma,1.8,nichtssagende_abk2,1
3757,geg.,geg.,,Abkürzung,9.004710,gegeben,gegeben,ADJA,Ptz2,0.668355,lemma,1.8,nichtssagende_abk2,3
3759,geh.,geh.,,Abkürzung,21.829100,gehoben,gehoben,ADJA,,340.262726,lemma,1.8,nichtssagende_abk2,3
3760,gek.,gek.,,Abkürzung,18.374691,gekennzeichnet,gekennzeichnet,ADJA,Ptz2,0.453479,lemma,1.8,nichtssagende_abk2,1
3761,gel.,gel.,,Abkürzung,51.948780,geladen,geladen,ADJA,Ptz2,0.538190,lemma,1.8,nichtssagende_abk2,1
3763,gem.,gem.,,Abkürzung,115.115265,gemahlen,gemahlen,ADJA,Ptz2,1.769529,lemma,1.8,nichtssagende_abk2,4
3765,gen.,gen.,,Abkürzung,37.976196,genannt,genannt,ADJA,Ptz2,0.509264,lemma,1.8,nichtssagende_abk2,2
3774,gep.,gep.,,Abkürzung,13.351590,gepaart,gepaart,ADJA,Ptz2,0.807867,lemma,1.8,nichtssagende_abk2,1


In [157]:
# Abk_kürzt_nicht
mask1 =  translate_abk.lang.str.len() <= translate_abk.kurz.str.len() 
mask2 =  translate_abk.lang_id != ''
mask3 =  translate_abk.kurz_id != ''
mask4 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask5 = ~translate_abk.kurz.isin(['okay',]) 
mask6 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6
check_mask(translate_abk,mask,228)

msg = 'Abk_kürzt_nicht'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
gridt( translate_abk_trash, msg )

check_mask:  210 rows
Move 210 rows from 10106
210 rows out of 273


In [158]:
#mask = translate_abk_trash.msg == 'Abk_kürzt_nicht'
#gridt(translate_abk_trash,mask)

In [159]:
# zu_kurz_end_Vokal
mask1 =  translate_abk.kurz.str.len() == 3 
mask2 =  translate_abk.kurz.str.lower().str.endswith(('a.','e.','i.','o.','u.','ä.','ö.','ü.','y.'))
mask3 =  translate_abk.kurz_tag == ''
mask4 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask5 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(translate_abk,mask,21)

msg = 'zu_kurz_end_Vokal'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  21 rows
Move 21 rows from 9896
21 rows out of 294


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,msg,konkurrenz
807,Co.,Co.,,Abkürzung,16.382223,Kompanie,Kompanie,NN,,1337.275879,lemma,1.44,zu_kurz_end_Vokal,1
1624,Ga.,Ga.,,Abkürzung,6.577270,Garage,Garage,NN,,4282.641602,lemma,1.80,zu_kurz_end_Vokal,1
2171,MA.,MA.,,Abkürzung,5.511693,Mittelalter,Mittelalter,NN,noPl,508.786438,lemma,1.80,zu_kurz_end_Vokal,1
2597,Pi.,Pi.,,Abkürzung,7.166071,Piaster,Piaster,NN,,437.788239,lemma,1.80,zu_kurz_end_Vokal,1
2665,Qu.,Qu.,,Abkürzung,13.877033,Quartal,Quartal,NN,,819.089478,lemma,1.80,zu_kurz_end_Vokal,1
2788,Ri.,Ri.,,Abkürzung,3.053587,Richter,Richter,NN,,5028.521484,lemma,1.80,zu_kurz_end_Vokal,1
3067,Ti.,Ti.,,Abkürzung,13.471220,Tiroler,Tiroler,ADJA,noSteig noFlex,41.235859,lemma,1.80,zu_kurz_end_Vokal,1
3313,Wa.,Wa.,,Abkürzung,22.748615,Waage,Waage,NN,,3058.229492,lemma,1.80,zu_kurz_end_Vokal,1
3401,Zi.,Zi.,,Abkürzung,2.227140,Zimmer,Zimmer,NN,,5176.409668,lemma,1.80,zu_kurz_end_Vokal,1
3457,ae.,ae.,,Abkürzung,4.236610,altenglisch,altenglisch,ADJA,noSteig,379.057709,lemma,1.80,zu_kurz_end_Vokal,1


In [160]:
# zu_kurz_start_Vokal
mask1 =  translate_abk.kurz.str.len() == 3 
mask2 =  translate_abk.kurz.str.endswith('.')
mask3 =  translate_abk.kurz.str.lower().str.startswith(('a','e','i','o','u','ä','ö','ü','y'))
mask4 =  translate_abk.kurz_tag == ''
mask5 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask6 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6
check_mask(translate_abk,mask,19)

msg = 'zu_kurz_start_Vokal'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  19 rows
Move 19 rows from 9875
19 rows out of 313


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,msg,konkurrenz
376,Az.,Az.,,Abkürzung,12.935077,Aktenzeichen,Aktenzeichen,NN,,429.356567,lemma,1.80,zu_kurz_start_Vokal,1
1373,Ed.,Ed.,,Abkürzung,8.791534,Edition,Edition,NN,,453.229187,lemma,1.80,zu_kurz_start_Vokal,1
1425,Ew.,Ew.,,Abkürzung,2.276727,Einwohner,Einwohner,NN,manuell,1193.954590,lemma,1.80,zu_kurz_start_Vokal,1
1428,Ez.,Ez.,,Abkürzung,6.786630,Einzahl,Einzahl,NN,,743.405518,lemma,1.80,zu_kurz_start_Vokal,1
2452,Ob.,Ob.,,Abkürzung,13.350096,Oberbefehlshaber,Oberbefehlshaber,NN,,618.614258,lemma,1.80,zu_kurz_start_Vokal,1
3361,Yd.,Yd.,,Abkürzung,2.251934,Yard,Yard,NN,,119.029610,lemma,1.80,zu_kurz_start_Vokal,1
3499,am.,am.,,Abkürzung,7.788982,amerikanisch,amerikanisch,ADJA,,974.893066,lemma,1.80,zu_kurz_start_Vokal,1
3506,an.,an.,,Abkürzung,2.342843,altnordisch,altnordisch,ADJA,noSteig,144.764542,lemma,1.80,zu_kurz_start_Vokal,1
3527,ar.,ar.,,Abkürzung,7.716775,arisch,arisch,ADJA,noSteig,157.932709,lemma,1.80,zu_kurz_start_Vokal,1
3543,as.,as.,,Abkürzung,5.475056,altsächsisch,altsächsisch,ADJA,noSteig,328.152374,lemma,1.80,zu_kurz_start_Vokal,1


In [161]:
# Ranking: Ist die Abkürzung in lexeme_manuell?
mask = translate_abk.kurz_member.str.contains('manuell')
translate_abk.loc[mask,'rank'] *= 2.0

mask = translate_abk.lang_member.str.contains('manuell')
translate_abk.loc[mask,'rank'] *= 2.0

In [162]:
# Ranking: Ist die Abkürzung beidseitig verknüpft?
gruppiert = translate_abk.groupby(['kurz_id'])
translate_abk['rank2'] = gruppiert.todo.transform( 'nunique' ) 

mask = translate_abk.rank2 > 1
translate_abk.loc[mask,'rank'] *= 5.0
translate_abk = pak.drop_cols(translate_abk,'rank2')

In [163]:
# Abkürzung hat eigenen Eintrag
mask = translate_abk.kurz_id != ''
translate_abk.loc[mask,'rank'] *= 2.0

In [164]:
# Ranking: lang_score
translate_abk['rank'] *= (translate_abk.lang_score + 0.2)

In [165]:
# Ranking: Kein tag
mask = translate_abk.lang_tag == ''
translate_abk.loc[mask,'rank'] *= 0.2
translate_abk[mask]

,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,konkurrenz
4,5G,5G,NN,Abkürzung manuell ergänzt,8.974190,,,,,0.000000,lemma,0.144000,1
5,9/11,9/11,NN,Abkürzung manuell ergänzt,55.178623,Terroranschläge,Terroranschläge,,,0.000000,lemma,0.288000,2
10,GmbH,GmbH,NN,manuell ergänzt Abkürzung,682.923218,,,,,0.000000,lemma,0.144000,1
11,HMM,HMM,ITJ,Abkürzung manuell ergänzt,7.572477,,,,,0.000000,lemma,0.144000,1
14,Opa_1,Opa,NN,Abkürzung Person_pre manuell ergänzt,8.245752,,,,,0.000000,lemma,0.144000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10079,zu_Händen~zHd,zHd,,,0.000000,zu_Händen,zu Händen,,Wortverbindung,12.547267,merkmal,5.098907,1
10084,zum_Exempel~z E,z E,,,0.000000,zum_Exempel,zum Exempel,,Wortverbindung,5.617107,merkmal,2.326843,1
10085,zum_Exempel~z. E.,z. E.,,,0.000000,zum_Exempel,zum Exempel,,Wortverbindung,5.617107,merkmal,2.326843,1
10086,zum_Exempel~z.E.,z.E.,,,0.000000,zum_Exempel,zum Exempel,,Wortverbindung,5.617107,merkmal,2.326843,1


In [166]:
# sicherheitshalber einmal sortieren
translate_abk = translate_abk.sort_values(['kurz','lang_score'],ascending=[True,False]).reset_index(drop=True)

In [167]:
# Ranking betrachten
grid(translate_abk.sort_values(['konkurrenz','kurz','rank',],ascending=[False,True,False]))

9856 rows


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,konkurrenz
5955,Straße~Str,Str,NN,manuell,0.000000,Straße,Straße,NN,manuell,1699.207520,merkmal,13595.259766,16
5954,Strand~Str,Str,NN,,0.000000,Strand,Strand,NN,,2786.176025,merkmal,5572.751953,16
5956,Strom~Str,Str,NN,,0.000000,Strom,Strom,NN,,627.335266,merkmal,1255.070557,16
5957,Struktur~Str,Str,NN,,0.000000,Struktur,Struktur,NN,,556.234680,merkmal,1112.869385,16
5958,Strafe~Str,Str,NN,,0.000000,Strafe,Strafe,NN,,411.752258,merkmal,823.904541,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9847,übsch.,übsch.,,Abkürzung,2.284992,überschüssig,überschüssig,ADJA,noSteig,70.928307,lemma,256.061890,1
9848,überschüssig~übsch.,übsch.,ADJA,,0.000000,überschüssig,überschüssig,ADJA,noSteig,23.642769,merkmal,47.685539,1
9851,Mikrosievert~μSv,μSv,NN,,0.000000,Mikrosievert,Mikrosievert,NN,,26.903440,merkmal,43.365505,1
9854,Quadratmikrometer~μm²,μm²,NN,,0.000000,Quadratmikrometer,Quadratmikrometer,NN,,41.616608,merkmal,66.906578,1


In [168]:
# rank!
translate_abk = pak.rank(translate_abk, col_score='rank', cols_group='kurz', on_conflict='first')

rank: 2085 rows less, now 7771 rows


In [169]:
?pak.replace_str

Signature: pak.replace_str(series, translationtable)
Docstring:
Replaces substrings from a Series of strings according to a translationtable.
* translationtable: Can be a dict, a list of tuples or a DataFrame with two columns.
  Example: {'President Trump':'Trump',   'HELLO':'Hello'}
  or      [('President Trump','Trump'), ('HELLO','Hello')]
File:      ~/Data_Linux/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/string.py
Type:      function


In [170]:
# Verwaiste Abkürzungen in wiktionary_lemma kennzeichnen  
mask = translate_abk.lang_id == ''
check_mask(translate_abk,mask,1745)
verwaist_kurz_id = translate_abk[mask].kurz_id

mask = wiktionary_lemma.lemma_id.isin(verwaist_kurz_id) 
check_mask(wiktionary_lemma,mask,1745)

wiktionary_lemma.loc[mask,'member'] = pak.replace_str( wiktionary_lemma[mask].member, {'Abkürzung':'Abkürzung_verwaist' } )
gridt(wiktionary_lemma[mask],pak.sample)

check_mask:  1745 rows
check_mask:  1745 rows
15 rows out of 1745


In [171]:
# Verwaiste Abkürzungen in translate_abk löschen
mask = translate_abk.lang_id == ''
check_mask(translate_abk,mask,1745)

msg = 'Abkürzung_verwaist'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)
grid( translate_abk_trash, msg )

check_mask:  1745 rows
Move 1745 rows from 7771
1745 rows out of 2058


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,msg,konkurrenz
49,5G,5G,NN,Abkürzung manuell ergänzt,8.974190,,,,,0.0,lemma,0.144,Abkürzung_verwaist,1
3045,GmbH,GmbH,NN,manuell ergänzt Abkürzung,682.923218,,,,,0.0,lemma,0.144,Abkürzung_verwaist,1
3190,HMM,HMM,ITJ,Abkürzung manuell ergänzt,7.572477,,,,,0.0,lemma,0.144,Abkürzung_verwaist,1
4812,Opa_1,Opa,NN,Abkürzung Person_pre manuell ergänzt,8.245752,,,,,0.0,lemma,0.144,Abkürzung_verwaist,1
5043,PetA,Peta,CARD,manuell ergänzt Abkürzung,12.209723,,,,,0.0,lemma,0.144,Abkürzung_verwaist,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9777,ÜAG,ÜAG,,Abkürzung,4.995471,,,,,0.0,lemma,0.072,Abkürzung_verwaist,1
9778,ÜD,ÜD,,Abkürzung,5.416959,,,,,0.0,lemma,0.072,Abkürzung_verwaist,1
9779,ÜE,ÜE,,Abkürzung,7.028529,,,,,0.0,lemma,0.072,Abkürzung_verwaist,1
9801,ödp,ödp,,Abkürzung,27.600363,,,,,0.0,lemma,0.072,Abkürzung_verwaist,1


In [172]:
# zu häufig und Anfangsbuchstabe ungleich

erlaubt = ['Ingenieur','okay','per Adresse','Professor','Kalorie','insbesondere','insbesonders']

a = pak.analyse_freqs(translate_abk,'lang')
mask_a = a.lang_count >= 5 
mask1 = translate_abk.lang.isin(a[mask_a].lang)
mask2 = (translate_abk.lang.str[0].str.lower()  !=  translate_abk.kurz.str[0].str.lower())
mask3 = ~translate_abk.kurz.str[0].str.isdigit() 
mask4 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask5A = translate_abk.kurz_member == 'Abkürzung'  # genau gleich 'Abkürzung', also nichts zusätzliches 
mask5B = translate_abk.kurz_member == ''           # oder ganz leer 
mask6 = ~translate_abk.lang.isin(erlaubt)
mask = mask1  &  mask2  &  mask3  &  mask4  &  (mask5A | mask5B)  &  mask6  
check_mask(translate_abk,mask,170)

msg = 'zu_häufig'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)
grid( translate_abk_trash, msg )

check_mask:  176 rows
Move 176 rows from 6026
176 rows out of 2234


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,msg,konkurrenz
181,AKH,AKH,NN,Abkürzung,66.509537,Krankenhaus,Krankenhaus,NN,,52026.234375,lemma,149836.125000,zu_häufig,1
1066,BauFG,BauFG,,Abkürzung,14.304992,Gesetz,Gesetz,NN,,15998.839844,lemma,46077.234375,zu_häufig,1
2171,EEAG,EEAG,,Abkürzung,10.641104,Gesetz,Gesetz,NN,,15998.839844,lemma,46077.234375,zu_häufig,1
2211,ELFG,ELFG,,Abkürzung,22.796688,Gesetz,Gesetz,NN,,15998.839844,lemma,46077.234375,zu_häufig,1
2225,EMVG,EMVG,,Abkürzung,35.377846,Gesetz,Gesetz,NN,,15998.839844,lemma,46077.234375,zu_häufig,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014,BVerfGE,BVerfGE,,Abkürzung,10.220346,Entscheidungen,Entscheidungen,,,0.000000,lemma,0.115200,zu_häufig,1
1017,BVerwGE,BVerwGE,,Abkürzung,10.220346,Entscheidungen,Entscheidungen,,,0.000000,lemma,0.115200,zu_häufig,1
1081,BayObLGSt,BayObLGSt,,Abkürzung,12.903807,Entscheidungen,Entscheidungen,,,0.000000,lemma,0.115200,zu_häufig,1
1082,BayObLGZ,BayObLGZ,,Abkürzung,12.903807,Entscheidungen,Entscheidungen,,,0.000000,lemma,0.115200,zu_häufig,1


In [173]:
a = pak.analyse_freqs(translate_abk,['lang','kurz'])
gridt(a)

3104 rows


In [174]:
# erlaubt_und_verboten

verboten     = ['Bundesministerium','Brandmeister','Braumeister','Rundfunk','Ortsnamengrundwort','Kurzwort','Diminutiv','diminuendo']
doch_erlaubt = []

mask1 =  translate_abk.lang.isin(verboten) 
mask2 = ~translate_abk.lang.isin(doch_erlaubt) 
mask3 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask4 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4 
#grid(translate_abk,mask)
check_mask(translate_abk,mask,67)

msg = 'erlaubt_und_verboten'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  66 rows
Move 66 rows from 5850
66 rows out of 2300


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,msg,konkurrenz
5296,RBB,RBB,,Abkürzung,48.454773,Rundfunk,Rundfunk,NN,noPl,1879.257202,lemma,6766.045410,erlaubt_und_verboten,1
8918,rbb,rbb,,Abkürzung,32.147079,Rundfunk,Rundfunk,NN,noPl,1879.257202,lemma,6766.045410,erlaubt_und_verboten,1
3794,Kfz,Kfz,NN,Abkürzung,172.759323,Kurzwort,Kurzwort,NN,,1048.175781,lemma,3774.152588,erlaubt_und_verboten,2
3959,Kurzw.,Kurzw.,,Abkürzung,4.193744,Kurzwort,Kurzwort,NN,,1048.175781,lemma,3774.152588,erlaubt_und_verboten,1
1411,CSU,CSU,NN,Abkürzung noPl,100.792984,Kurzwort,Kurzwort,NN,,1048.175781,lemma,3019.322021,erlaubt_und_verboten,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278,Brandmeister~BrMstr,BrMstr,NN,,0.000000,Brandmeister,Brandmeister,NN,,50.697273,merkmal,101.794548,erlaubt_und_verboten,2
1256,Brandmeister~Br-M,Br-M,NN,,0.000000,Brandmeister,Brandmeister,NN,,42.247726,merkmal,84.895454,erlaubt_und_verboten,2
1258,Brandmeister~Br-Mstr,Br-Mstr,NN,,0.000000,Brandmeister,Brandmeister,NN,,42.247726,merkmal,84.895454,erlaubt_und_verboten,2
7401,diminuendo~dim,dim,ADV,,0.000000,diminuendo,diminuendo,ADV,,17.771036,merkmal,35.942074,erlaubt_und_verboten,1


In [175]:
# lang_tag_fehlt
mask1 = translate_abk.lang_tag == ''
mask2 = translate_abk.kurz_tag != ''
mask3 = translate_abk.lang_id  != ''
mask4 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask5 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
#grid(translate_abk,mask)
check_mask(translate_abk,mask,6)

msg = 'lang_tag_fehlt'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  5 rows
Move 5 rows from 5784
5 rows out of 2305


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,msg,konkurrenz
4026,LPG,LPG,NN,Abkürzung,376.507599,landwirtschaftliche,landwirtschaftliche,,,0.0,lemma,0.1440,lang_tag_fehlt,1
4865,PAK,PAK,NN,Abkürzung noSg,143.740311,polyzyklische,polyzyklische,,,0.0,lemma,0.1440,lang_tag_fehlt,1
1362,CD-ROM,CD-ROM,NN,Abkürzung,562.880005,scheibenförmiger,scheibenförmiger,,,0.0,lemma,0.1152,lang_tag_fehlt,1
2149,EAP,EAP,NN,Abkürzung,190.905014,Ämtern,Ämtern,,,0.0,lemma,0.1152,lang_tag_fehlt,1
6021,Supergau,Supergau,NN,Abkürzung,87.907776,anzunehmenden,anzunehmenden,,,0.0,lemma,0.1152,lang_tag_fehlt,1


In [176]:
# tag von lang nach kurz übertragen
mask1 = translate_abk.kurz_tag == ''
mask2 = translate_abk.lang_tag != ''
mask = mask1  &  mask2
check_mask(translate_abk,mask,2041)

translate_abk.loc[mask,'kurz_tag'] = translate_abk[mask].lang_tag
grid(translate_abk,mask)

check_mask:  2041 rows
2041 rows out of 5779


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,konkurrenz
1092,Bd.,Bd.,NN,Abkürzung,20.965275,Band,Band,NN,manuell,41571.363281,lemma,299315.250000,1
6664,Ws.,Ws.,NN,Abkürzung,11.028746,Wasser,Wasser,NN,,56061.152344,lemma,201820.859375,1
3802,Kh.,Kh.,NN,Abkürzung,4.933620,Krankenhaus,Krankenhaus,NN,,52026.234375,lemma,187295.156250,1
3932,Krh.,Krh.,NN,Abkürzung,5.775660,Krankenhaus,Krankenhaus,NN,,52026.234375,lemma,187295.156250,1
3935,Krhs.,Krhs.,NN,Abkürzung,6.462754,Krankenhaus,Krankenhaus,NN,,52026.234375,lemma,187295.156250,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7509,erb.,erb.,ADJA,Abkürzung,6.305295,erbaut,erbaut,ADJA,Ptz2,0.449347,lemma,2.337650,1
7527,erschl.,erschl.,ADJA,Abkürzung,3.632099,erschlossen_1,erschlossen,ADJA,Ptz2,0.075022,lemma,0.990078,1
8238,latinis.,latinis.,ADJA,Abkürzung,2.740582,latinisiert_1,latinisiert,ADJA,Ptz2,0.062883,lemma,0.946380,1
9550,vj.,vj.,ADJA,Abkürzung,9.361846,verjährt_1,verjährt,ADJA,Ptz2,0.058751,lemma,0.931504,1


In [177]:
# Weiterer_Schrott
mask1 =  translate_abk.kurz_tag == ''
mask2 = ~translate_abk.lang_member.isin(['','Wortverbindung','Redewendung'])
mask3 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask4 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  
#grid(translate_abk,mask)
check_mask(translate_abk,mask,2)

msg = 'Weiterer_Schrott'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  2 rows
Move 2 rows from 5779
2 rows out of 2307


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,msg,konkurrenz
77,AAA,AAA,,Abkürzung,168.779648,AFIS,AFIS,,Abkürzung,35.151310,lemma,25.452944,Weiterer_Schrott,1
1575,DPA,DPA,,Abkürzung,27.408615,DPMA,DPMA,,Abkürzung,6.408694,lemma,4.758260,Weiterer_Schrott,1


In [178]:
# KOMBI
mask1 = translate_abk.kurz_tag == ''
mask2 = translate_abk.lang_member.isin(['Wortverbindung','Redewendung'])
mask = mask1  &  mask2 
check_mask(translate_abk,mask,98)

translate_abk.loc[mask,'kurz_tag'] = 'KOMBI'
grid(translate_abk,mask)

check_mask:  98 rows
98 rows out of 5777


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,konkurrenz
9020,s._A.,s. A.,KOMBI,Abkürzung,13.263565,seligen_Angedenkens,seligen Angedenkens,,Redewendung,139.364365,lemma,100.486343,1
9037,seligen_Angedenkens~s.A.,s.A.,KOMBI,,0.000000,seligen_Angedenkens,seligen Angedenkens,,Redewendung,195.110107,merkmal,78.124046,1
9047,seligen_Angedenkens~sA,sA,KOMBI,,0.000000,seligen_Angedenkens,seligen Angedenkens,,Redewendung,195.110107,merkmal,78.124046,1
9088,seligen_Angedenkens~sel.A.,sel.A.,KOMBI,,0.000000,seligen_Angedenkens,seligen Angedenkens,,Redewendung,195.110107,merkmal,78.124046,1
9089,seligen_Angedenkens~selA,selA,KOMBI,,0.000000,seligen_Angedenkens,seligen Angedenkens,,Redewendung,195.110107,merkmal,78.124046,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7282,per_Adresse~c/o,c/o,KOMBI,,0.000000,per_Adresse,per Adresse,,Wortverbindung,4.577855,merkmal,1.528913,2
9355,ut_supra~u s,u s,KOMBI,,0.000000,ut_supra,ut supra,,Wortverbindung,2.460861,merkmal,1.064345,1
9373,ut_supra~u. s.,u. s.,KOMBI,,0.000000,ut_supra,ut supra,,Wortverbindung,2.460861,merkmal,1.064345,1
9381,ut_supra~u.s.,u.s.,KOMBI,,0.000000,ut_supra,ut supra,,Wortverbindung,2.460861,merkmal,1.064345,1


In [179]:
# lang_member ist jetzt ausgeschöpft
mask1 = translate_abk.kurz_tag == ''
mask2 = translate_abk.lang_member != ''
mask = mask1  &  mask2 
check_mask(translate_abk,mask,0)

check_mask:  0 rows


In [180]:
# Großgeschrieben => NN
mask1 = translate_abk.kurz_tag == ''
mask2 = translate_abk.kurz.str[0].str.isupper()
mask3 = translate_abk.kurz.str[0].str.isalpha() 
mask4 = translate_abk.kurz.str.len() > 2
mask5 = translate_abk.kurz.str.len() >= 4 
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5 
check_mask(translate_abk, mask, 16)
translate_abk.loc[mask,'kurz_tag'] = 'NN'
grid(translate_abk,mask)

check_mask:  16 rows
16 rows out of 5777


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,konkurrenz
1098,Bde.,Bde.,NN,Abkürzung,3.235405,Bände,Bände,,,0.0,lemma,0.1440,2
2802,GDPdU,GDPdU,NN,Abkürzung,27.685272,Grundsätze,Grundsätze,,,0.0,lemma,0.1440,1
3352,Hrn.,Hrn.,NN,Abkürzung,25.777845,Herrn,Herrn,,,0.0,lemma,0.1440,1
3604,Jhs.,Jhs.,NN,Abkürzung,5.906564,Jahrhunderts,Jahrhunderts,,,0.0,lemma,0.1440,1
4416,Mmes.,Mmes.,NN,Abkürzung,2.805653,Mesdames,Mesdames,,,0.0,lemma,0.1440,1
5496,RiStBV,RiStBV,NN,Abkürzung,13.247355,Richtlinien,Richtlinien,,,0.0,lemma,0.1440,1
6698,Yds.,Yds.,NN,Abkürzung,1.735439,Yards,Yards,,,0.0,lemma,0.1440,1
6799,ZÄPrO,ZÄPrO,NN,Abkürzung,26.821758,Zahnärzte,Zahnärzte,,,0.0,lemma,0.1440,1
1234,BodSchätzDB,BodSchätzDB,NN,Abkürzung,12.183882,Durchführungsbestimmungen,Durchführungsbestimmungen,,,0.0,lemma,0.1152,1
1860,E_163,E 163,NN,Abkürzung,1.642471,Anthocyane,Anthocyane,,,0.0,lemma,0.1152,1


In [181]:
# Großgeschrieben => NN
mask1  = translate_abk.kurz_tag == ''
mask2  = translate_abk.kurz.str[0].str.isupper()
mask3  = translate_abk.kurz.str[0].str.isalpha() 
mask4  = translate_abk.kurz.str[0] ==  translate_abk.lang.str[0]
mask = mask1  &  mask2  &  mask3  &  mask4 
check_mask(translate_abk, mask, 4)
translate_abk.loc[mask,'kurz_tag'] = 'NN'
grid(translate_abk,mask)

check_mask:  4 rows
4 rows out of 5777


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,konkurrenz
5534,RÄ,RÄ,NN,Abkürzung,14.568176,Rechtsanwälte,Rechtsanwälte,,,0.0,lemma,0.144,1
5605,SJZ,SJZ,NN,Abkürzung,13.025398,Süddeutsche,Süddeutsche,,,0.0,lemma,0.144,1
6016,SuS,SuS,NN,Abkürzung,41.070152,Schülerinnen,Schülerinnen,,,0.0,lemma,0.144,1
6370,V4,V4,NN,Abkürzung,5.640099,Visegrad-Staaten,Visegrad-Staaten,,,0.0,lemma,0.144,1


In [182]:
# Aufzählungen => CARD
# nicht nur für kurz_tag == ''
mask1 = translate_abk.kurz.str[:-1].str.isnumeric()
mask2 = translate_abk.kurz.str.endswith('.')  
mask3 = translate_abk.kurz.str.len() <= 3 
mask = mask1  &  mask2  &  mask3
check_mask(translate_abk, mask, 11)
translate_abk.loc[mask,'kurz_tag'] = 'CARD'
grid(translate_abk,mask)

check_mask:  11 rows
11 rows out of 5777


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,konkurrenz
55,8.,8.,CARD,Abkürzung,34.177845,achter,achter,ADJA,,106.679688,lemma,384.766846,1
7,0.,0.,CARD,Abkürzung,29.465649,nullter,nullter,,,0.000000,lemma,0.144000,1
15,1.,1.,CARD,Abkürzung,32.793232,erster,erster,,,0.000000,lemma,0.144000,1
16,10.,10.,CARD,Abkürzung,33.962463,zehnter,zehnter,,,0.000000,lemma,0.144000,1
28,2.,2.,CARD,Abkürzung,55.384083,zweiter,zweiter,,,0.000000,lemma,0.144000,1
39,3.,3.,CARD,Abkürzung,33.316307,dritter,dritter,,,0.000000,lemma,0.144000,1
44,4.,4.,CARD,Abkürzung,34.424000,vierter,vierter,,,0.000000,lemma,0.144000,1
48,5.,5.,CARD,Abkürzung,35.716309,fünfter,fünfter,,,0.000000,lemma,0.144000,1
50,6.,6.,CARD,Abkürzung,34.670155,sechster,sechster,,,0.000000,lemma,0.144000,1
52,7.,7.,CARD,Abkürzung,53.208374,siebter,siebter,,,0.000000,lemma,0.144000,1


In [183]:
# Rest
mask  = translate_abk.kurz_tag == ''
check_mask(translate_abk, mask, 25)

grid(translate_abk,mask)
msg = 'Rest'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  26 rows
26 rows out of 5777
Move 26 rows from 5777
26 rows out of 2333


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,msg,konkurrenz
304,AWZ,AWZ,,Abkürzung,29.302071,ausschließliche,ausschließliche,,,0.0,lemma,0.1440,Rest,1
2514,FFW,FFW,,Abkürzung,19.173544,freiwillige,freiwillige,,,0.0,lemma,0.1440,Rest,1
2842,GOS,GOS,,Abkürzung,16.847281,gymnasiale,gymnasiale,,,0.0,lemma,0.1440,Rest,1
4923,PKV,PKV,,Abkürzung,21.444920,private,private,,,0.0,lemma,0.1440,Rest,1
7361,dass.,dass.,,Abkürzung,1.599041,dasselbe,dasselbe,,,0.0,lemma,0.1440,Rest,1
7369,deins,deins,,Abkürzung,10.623570,deines,deines,,,0.0,lemma,0.1440,Rest,1
7398,dies.,dies.,,Abkürzung,1.698215,dieselbe,dieselbe,,,0.0,lemma,0.1440,Rest,2
7450,eG,eG,,Abkürzung,22.990444,eingetragene,eingetragene,,,0.0,lemma,0.1440,Rest,1
8077,jmdm.,jmdm.,,Abkürzung,4.435301,jemandem,jemandem,,,0.0,lemma,0.1440,Rest,1
8078,jmdn.,jmdn.,,Abkürzung,4.435301,jemanden,jemanden,,,0.0,lemma,0.1440,Rest,1


In [184]:
# Mehr gibt es nicht
mask = translate_abk.kurz_tag == ''
check_mask(translate_abk, mask, 0)

#grid(translate_abk,mask)
#msg = 'Rest'
#translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
#grid( translate_abk_trash, msg )

check_mask:  0 rows


In [185]:
# Neue kurz_id vergeben
#mask = translate_abk.kurz_id == ''
#translate_abk.loc[mask,'kurz_id'] = translate_abk[mask].kurz.str.replace(' ','_') + '_A'
#check_mask(translate_abk,mask,2983)
#grid(translate_abk,mask)

In [186]:
# member Abkürzung nachtragen
mask = ~translate_abk.kurz_member.str.contains('Abkürzung')
translate_abk.loc[mask,'kurz_member'] = (translate_abk[mask].kurz_member + ' Abkürzung').str.strip()

#### rückübertragen

In [187]:
# Alle Kurzformen aus wiktionary_lemma rauslöschen, die nicht in translate_abk erwähnt werden
# und keinen echten tag haben
mask1 =  wiktionary_lemma.member.str.contains('Abkürzung')
mask2 = ~wiktionary_lemma.member.str.contains('manuell')
mask3 = ~wiktionary_lemma.lemma_id.isin(translate_abk.kurz_id)
mask4 = ~wiktionary_lemma.lemma_id.isin(translate_abk.lang_id)
mask5 =  wiktionary_lemma.tag == 'TODO'
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
#grid(wiktionary_lemma,mask)
check_mask(wiktionary_lemma,mask,2015)

msg = 'translate_abk1'
wiktionary_lemma, wiktionary_lemma_trash = pak.move_rows( wiktionary_lemma, wiktionary_lemma_trash, mask, msg)

# anschauen
mask = wiktionary_lemma_trash.msg == msg
grid( wiktionary_lemma_trash[mask].sort_values('lemma_score', ascending=False).head(20) )

check_mask:  2084 rows
Move 2084 rows from 171220
20 rows


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score,msg
70413,OP,66325,0,OP,op,True,1,TODO,TODO,8hhu5gFrzn,X_todo,X,Abkürzung,,294.044769,translate_abk1
46613,IKT,66042,0,IKT,ikt,True,1,TODO,TODO,j9IUH7xvuR,X_todo,X,Abkürzung_verwaist,,230.687836,translate_abk1
3504,AIDS,2598,0,AIDS,aids,True,1,TODO,TODO,Q5pZo1P6rc,X_todo,X,Abkürzung_verwaist,,185.285721,translate_abk1
33793,Fr.,3519,0,Fr.,fr.,True,1,TODO,TODO,QicUrDeIIs,X_todo,X,Abkürzung,,183.475128,translate_abk1
101596,VOB,20157,0,VOB,vob,True,1,TODO,TODO,lU6kb1SBPm,X_todo,X,Abkürzung_verwaist,,169.586456,translate_abk1
3414,AAA,12177,0,AAA,aaa,True,1,TODO,TODO,lHaJalwgUX,X_todo,X,Abkürzung,,168.779648,translate_abk1
21686,DV,17392,0,DV,dv,True,1,TODO,TODO,6FCxv4HCPT,X_todo,X,Abkürzung,,159.248001,translate_abk1
82756,SPS,65443,0,SPS,sps,True,1,TODO,TODO,VpYuhBKcR2,X_todo,X,Abkürzung_verwaist,,158.823608,translate_abk1
78452,RWA,548144,0,RWA,rwa,True,1,TODO,TODO,LFCA2NkDAQ,X_todo,X,Abkürzung_verwaist,,153.982208,translate_abk1
11904,BO,34707,0,BO,bo,True,1,TODO,TODO,A2q8Ke7nNC,X_todo,X,Abkürzung,,142.694153,translate_abk1


In [188]:
# Einige Datensätze sind in translate_abk gelöscht, haben aber in wiktionary_lemma einen richtigen tag.
# Die bleiben drin.
mask1 =  wiktionary_lemma.member.str.contains('Abkürzung')
mask2 = ~wiktionary_lemma.lemma_id.isin(translate_abk.kurz_id)
mask3 = ~wiktionary_lemma.lemma_id.isin(translate_abk.lang_id)
mask4 =  wiktionary_lemma.tag != 'TODO'
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_lemma,mask,192)

# anschauen
grid( wiktionary_lemma, mask )

check_mask:  205 rows
205 rows out of 169136


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score
4,5G,975075,0,5G,5g,True,1,NN,NN,1478WOEGG2,NOM,N,Abkürzung_verwaist manuell ergänzt,,8.974190
5,9/11,697316,0,9/11,9/11,True,1,NN,NN,1nF8Eioo9V,NOM,N,Abkürzung manuell ergänzt,,55.178623
558,GmbH,3714,0,GmbH,gmbh,True,1,NN,NN,QXhlXPGFoO,NOM,N,manuell ergänzt Abkürzung_verwaist,,682.923218
588,HMM,975439,0,HMM,hmm,True,1,ITJ,ITJ,7K6MahYyEc,X_itj,X,Abkürzung_verwaist manuell ergänzt,,7.572477
1184,Opa_1,67456,1,Opa,opa,True,1,NN,NN,JoesczRgoc,NOM,N,Abkürzung_verwaist Person_pre manuell ergänzt,,8.245752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124589,dpa,102374,0,dpa,dpa,False,1,NN,NN,nzbxSu283R,NOM,N,Abkürzung_verwaist noPl,,95.411606
169761,ÖAMTC,274668,0,ÖAMTC,öamtc,True,1,NN,NN,wUwPtAimH8,NOM,N,Abkürzung_verwaist noPl,,93.568619
169762,ÖBB,274658,0,ÖBB,öbb,True,1,NN,NN,ckLvJeBMkM,NOM,N,Abkürzung_verwaist noSg,,87.414108
169767,ÖPNV,14046,0,ÖPNV,öpnv,True,1,NN,NN,gZxeYu4nLE,NOM,N,Abkürzung noPl,,65.180862


In [189]:
#a = pak.analyse_freqs(wiktionary_lemma,['tag','lemma'])
#gridt(a)

In [190]:
# kurz_tag => tag_new
wiktionary_lemma = pak.update_col( df_to =      wiktionary_lemma, 
                                   df_from =    translate_abk, 
                                   left_on =    'lemma_id',
                                   right_on =   'kurz_id',
                                   col =        'kurz_tag',
                                   col_rename = 'tag_new',
                                   func =       'first',
                                  )
wiktionary_lemma = pak.move_cols(wiktionary_lemma,'tag_new','lemma_len')

update_col: func first applied, but it was pointless!
update_col: 2238 cells written into new column


In [191]:
# prüfen: tag_new != tag
mask1 = wiktionary_lemma.tag != wiktionary_lemma.tag_new
mask2 = wiktionary_lemma.tag != 'TODO'
mask = mask1  &  mask2
check_mask(wiktionary_lemma,mask,0)
#gridt(wiktionary_lemma[mask].sort_values('lemma_score', ascending=False))

check_mask:  0 rows


In [192]:
# anschauen: tag_new != tag
mask1 = wiktionary_lemma.tag != wiktionary_lemma.tag_new
mask2 = wiktionary_lemma.tag == 'TODO'
mask = mask1  &  mask2
check_mask(wiktionary_lemma,mask,2152)

grid(wiktionary_lemma[mask].sort_values('lemma_score', ascending=False), pak.sample  )

check_mask:  2083 rows
12 rows out of 2083


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag_new,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score
2698,&c.,240350,0,&c.,&c.,False,1,ADV,TODO,TODO,yaOFLyWWDb,X_todo,X,Abkürzung,,4.049606
6816,Alt_4,1538,4,Alt,alt,True,1,NN,TODO,TODO,TFzprSKama,X_todo,X,Abkürzung,,0.901200
7453,Amtsspr.,21304,0,Amtsspr.,amtsspr.,True,1,NN,TODO,TODO,06f4pqn47X,X_todo,X,Abkürzung,,6.424513
25533,ETCS,127801,0,ETCS,etcs,True,1,NN,TODO,TODO,mdglFIpWGN,X_todo,X,Abkürzung,,587.812317
35641,GDPdU,19738,0,GDPdU,gdpdu,True,1,NN,TODO,TODO,zyl9G7uqM7,X_todo,X,Abkürzung,,27.685272
41450,HI,164,0,HI,hi,True,1,NN,TODO,TODO,t8K72vUwdx,X_todo,X,Abkürzung,,8.867126
71789,Osch.,1050201,0,Osch.,osch.,True,1,NN,TODO,TODO,5sl54Ecf46,X_todo,X,Abkürzung,,2.127967
128922,erf.,242872,0,erf.,erf.,False,1,VVFIN,TODO,TODO,YXZ02M6Wbk,X_todo,X,Abkürzung,,31.472126
131214,fm,1724,0,fm,fm,False,1,NN,TODO,TODO,vtxf5hBDCM,X_todo,X,Abkürzung,,24.073540
146635,n._Chr.,4409,0,n. Chr.,n. chr.,False,2,NN,TODO,TODO,nNOONBrQIE,X_todo,X,Abkürzung,,61.260693


In [193]:
# tag_new => tag
wiktionary_lemma.loc[mask,'tag'] = wiktionary_lemma[mask].tag_new
wiktionary_lemma = pak.drop_cols(wiktionary_lemma,'tag_new')

In [194]:
# pflege_tag_felder für die veränderten tags
wiktionary_lemma = pflege_tag_felder(wiktionary_lemma, translate_tagZ)  # tagZ, tagZZ, tag_0, tag_1, tagZ_0, tagZ_1

In [195]:
# lang_id => data_id_new
wiktionary_lemma = pak.update_col( df_to =      wiktionary_lemma, 
                                   df_from =    translate_abk, 
                                   left_on =    'lemma_id',
                                   right_on =   'kurz_id',
                                   col =        'lang_id',
                                   col_rename = 'data_id_new',
                                   func =       'first',
                                  )

update_col: func first applied, but it was pointless!
update_col: 2238 cells written into new column


In [196]:
# prüfen: data_id_new != data_id
mask1 = wiktionary_lemma.data_id != wiktionary_lemma.data_id_new
mask2 = wiktionary_lemma.data_id != ''
mask = mask1  &  mask2
check_mask(wiktionary_lemma,mask, 0)
#grid(wiktionary_lemma[mask].sort_values('lemma_score', ascending=False) )

check_mask:  0 rows


In [197]:
# anschauen: data_id_new 
mask1 = wiktionary_lemma.data_id != wiktionary_lemma.data_id_new
mask2 = wiktionary_lemma.data_id == ''
mask = mask1  &  mask2
check_mask(wiktionary_lemma,mask, 2492)
grid(wiktionary_lemma[mask].sort_values('lemma_score', ascending=False) )

check_mask:  2238 rows
2238 rows


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score,data_id_new
91137,Stasi,396199,0,Stasi,stasi,True,1,NN,NN,9x5IvmSfVM,NOM,N,Abkürzung noPl,,9214.994141,Geheimpolizei
19407,CO₂,119982,0,CO₂,co₂,True,1,NN,NN,YMeFhyt3vi,NOM,N,Abkürzung noPl,,976.711426,chemisch
102895,WM,19318,0,WM,wm,True,1,NN,NN,b3ATCltzxi,NOM,N,Abkürzung,,967.495728,Weltmeisterschaft
45603,ICE,60577,0,ICE,ice,True,1,NN,NN,JafRMvZq0O,NOM,N,Abkürzung,,874.731934,Intercity-Express
56063,Küfa,1112624,0,Küfa,küfa,True,1,NN,NN,qP1yBOhbrc,NOM,N,Abkürzung,,820.709839,öffentlich
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66208,NP_1,22234,1,NP,np,True,1,NN,NN,IJrr7HjpoQ,NOM,N,Abkürzung,,1.163690,Problem
66188,NC_1,434053,1,NC,nc,True,1,NE,NE,comPHnRYDw,NAM,N,Abkürzung,,1.123401,North_Carolina
56187,L._A.,24118,0,L. A.,l. a.,True,2,NE,NE,16098sflWY,NAM,N,Abkürzung,,1.113570,Los_Angeles
60971,Man_2,116544,2,Man,man,True,1,NN,NN,MIfP9QXra4,NOM,N,Abkürzung,,1.108692,Manual


In [198]:
# data_id_new => data_id
wiktionary_lemma.loc[mask,'data_id'] = wiktionary_lemma[mask].data_id_new
wiktionary_lemma = pak.drop_cols(wiktionary_lemma,'data_id_new')

In [199]:
# Obsolet
# Neue Datensätze für wiktionary_lemma
# (werden nicht angefügt)
mask_abk = wiktionary_lemma.member.str.contains('Abkürzung')
mask = ~translate_abk.kurz_id.isin(wiktionary_lemma[mask_abk].lemma_id)
check_mask(translate_abk,mask, 3031)

df = translate_abk[mask]

df = pak.rename_col(df, 'kurz_id',     'lemma_id')
df = pak.rename_col(df, 'kurz',        'lemma')
df = pak.rename_col(df, 'kurz_tag',    'tag')
df = pak.rename_col(df, 'kurz_member', 'member')
df = pak.rename_col(df, 'kurz_score',  'lemma_score')
df = pak.rename_col(df, 'lang_id',     'data_id')
df['page_id']   = -1
df['page_part'] = -1
df = pak.move_cols(df,['page_id','page_part'],'lemma_id')
df = pak.drop_cols(df,['konkurrenz','rank','todo','lang','lang_tag','lang_member','lang_score',])

df = ergänze_felder(df)
df = pflege_tag_felder(df, translate_tagZ)  
#assert set(wiktionary_lemma.columns) ==  set(df.columns) 
grid(df)

check_mask:  3513 rows
check_mask:  11 rows
3513 rows


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,lemma_score,data_id
0,Band~Bd,-1,-1,Bd,bd,True,1,NN,NN,hoRpnyDatV,NOM,N,manuell Abkürzung,0.0,Band
1,Februar~Feb.,-1,-1,Feb.,feb.,True,1,NN,NN,WRH7dzSg2S,NOM,N,manuell Abkürzung,0.0,Februar
2,Januar~Jan,-1,-1,Jan,jan,True,1,NN,NN,xfrGJhSu7L,NOM,N,manuell Abkürzung,0.0,Januar
3,Straße~Str,-1,-1,Str,str,True,1,NN,NN,BF2SKgmX7b,NOM,N,manuell Abkürzung,0.0,Straße
4,Straße~Str.,-1,-1,Str.,str.,True,1,NN,NN,a8hCAVIOji,NOM,N,manuell Abkürzung,0.0,Straße
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3508,per_Adresse~c/o,-1,-1,c/o,c/o,False,1,KOMBI,KOMBI,OmKwoIFjNx,X_kombi,X,Abkürzung,0.0,per_Adresse
3509,ut_supra~u s,-1,-1,u s,u s,False,2,KOMBI,KOMBI,YcTmy8NsI1,X_kombi,X,Abkürzung,0.0,ut_supra
3510,ut_supra~u. s.,-1,-1,u. s.,u. s.,False,2,KOMBI,KOMBI,fBxT6Lu6W7,X_kombi,X,Abkürzung,0.0,ut_supra
3511,ut_supra~u.s.,-1,-1,u.s.,u.s.,False,1,KOMBI,KOMBI,yVr9gtOLjh,X_kombi,X,Abkürzung,0.0,ut_supra


In [200]:
#wiktionary_lemma anfügen!
#wiktionary_lemma = pak.add_rows(wiktionary_lemma, df)

In [201]:
## lemma_id vorhanden und eindeutig?
#mask = wiktionary_lemma.lemma_id == ''
#check_mask(wiktionary_lemma,mask,0)
#
#assert wiktionary_lemma.lemma_id.is_unique

In [202]:
#mask = wiktionary_lemma.duplicated(subset='lemma_id', keep=False)
#grid(wiktionary_lemma[mask].sort_values('lemma_id'))

#### nochmal belöschen

In [203]:
# lang_id fehlt >> löschen (dürfte eigentlich nichts mehr dabei sein)
mask = translate_abk.lang_id == ''
check_mask(translate_abk, mask, 0)

msg = 'lang_id fehlt'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
#grid( translate_abk_trash, msg )

check_mask:  0 rows
No rows moved


In [204]:
# lang_id verwaist 
mask = ~translate_abk.lang_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_abk,mask,0,40)

msg = 'lang_id verwaist'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)
grid( translate_abk_trash, msg )

check_mask:  30 rows
Move 30 rows from 5751
30 rows out of 2363


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,msg,konkurrenz
7,0.,0.,CARD,Abkürzung,29.465649,nullter,nullter,,,0.0,lemma,0.1440,lang_id verwaist,1
15,1.,1.,CARD,Abkürzung,32.793232,erster,erster,,,0.0,lemma,0.1440,lang_id verwaist,1
16,10.,10.,CARD,Abkürzung,33.962463,zehnter,zehnter,,,0.0,lemma,0.1440,lang_id verwaist,1
28,2.,2.,CARD,Abkürzung,55.384083,zweiter,zweiter,,,0.0,lemma,0.1440,lang_id verwaist,1
39,3.,3.,CARD,Abkürzung,33.316307,dritter,dritter,,,0.0,lemma,0.1440,lang_id verwaist,1
44,4.,4.,CARD,Abkürzung,34.424000,vierter,vierter,,,0.0,lemma,0.1440,lang_id verwaist,1
48,5.,5.,CARD,Abkürzung,35.716309,fünfter,fünfter,,,0.0,lemma,0.1440,lang_id verwaist,1
50,6.,6.,CARD,Abkürzung,34.670155,sechster,sechster,,,0.0,lemma,0.1440,lang_id verwaist,1
52,7.,7.,CARD,Abkürzung,53.208374,siebter,siebter,,,0.0,lemma,0.1440,lang_id verwaist,1
57,9.,9.,CARD,Abkürzung,34.424000,neunter,neunter,,,0.0,lemma,0.1440,lang_id verwaist,1


In [205]:
# Bedeutungsloses ggf. löschen (dürfte eigentlich nichts mehr dabei sein)
maskA = translate_abk.kurz_id == translate_abk.lang_id
maskB = translate_abk.kurz    == translate_abk.lang
mask = maskA | maskB
check_mask(translate_abk,mask,0)

msg = 'bedeutungslos'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
#grid( translate_abk_trash, msg )

check_mask:  0 rows
No rows moved


In [206]:
# Keine neuen Datensätze für translate_lex
# mask1 = translate_abk.kurz_id.str.endswith('_A')
# mask2 = translate_abk.lang_id.isin(translate_lex.data_id)
# mask = mask1  &  mask2  
# check_mask(translate_abk,mask,0)

### wiktionary_merkmal belöschen
* alle abk-Merkmale sind schon gelöscht

In [207]:
# alle kurzformen
mask_abk =  wiktionary_lemma.member.str.contains('Abkürzung')
kurzformen = wiktionary_lemma[mask_abk].lemma_id

In [208]:
# Alle def-Merkmale aller Kurzformen löschen
mask1 = wiktionary_merkmal.merkmal == 'def'
mask2 = wiktionary_merkmal.lemma_id.isin(kurzformen)
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,4867)

msg = 'def einer Abkürzung'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg)

mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash, mask )

check_mask:  4279 rows
Move 4279 rows from 4850507
4279 rows out of 6310


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg
3437,HI,def,Herzinfarkt,Herzinfarkt,,TODO,1,Med,2.955709,False,def einer Abkürzung
3438,HI,def,Herzinsuffizienz,Herzinsuffizienz,,TODO,3,Med,2.955709,False,def einer Abkürzung
17957,Pkw,def,Personenkraftwagen,Personenkraftwagen,,NN,1,,90.900574,False,def einer Abkürzung
17985,PKW,def,Personenkraftwagen,Personenkraftwagen,,NN,1,,23.277538,False,def einer Abkürzung
18914,BP,def,Bundespost,Bundespost,,TODO,2,,9.253415,False,def einer Abkürzung
...,...,...,...,...,...,...,...,...,...,...,...
1794149,Küfa,def,Kochen,Kochen,,NN,1,,91.189987,False,def einer Abkürzung
1794855,EnSikuMaV,def,Kurzfristenergieversorgungssicherungsmaßnahmenverordnung,Kurzfristenergieversorgungssicherungsmaßnahmenverordnung,,TODO,1,,8.665539,False,def einer Abkürzung
1794859,EnSimiMaV,def,Mittelfristenergieversorgungssicherungsmaßnahmenverordnung,Mittelfristenergieversorgungssicherungsmaßnahmenverordnung,,TODO,1,,8.706564,False,def einer Abkürzung
1796003,BRAK,def,Bundesrechtsanwaltskammer,Bundesrechtsanwaltskammer,,TODO,1,,3.028793,False,def einer Abkürzung


### Abkürzungen überprüfen

In [209]:
# wiktionary_lemma: Viele Abkürzungen sind in translate_abk
mask1 = wiktionary_lemma.member.str.contains('Abkürzung')
mask2 = wiktionary_lemma.lemma_id.isin(translate_abk.kurz_id)
mask = mask1  &  mask2
check_mask(wiktionary_lemma,mask,2492)
#wiktionary_lemma[mask]

check_mask:  2208 rows


In [210]:
# wiktionary_lemma: Einige Abkürzungen sind nicht in translate_abk
# (weil es keine Auflösung gibt)
mask1 =  wiktionary_lemma.member.str.contains('Abkürzung')
mask2 = ~wiktionary_lemma.lemma_id.isin(translate_abk.kurz_id)
mask = mask1  &  mask2
check_mask(wiktionary_lemma,mask,194)
wiktionary_lemma[mask].sort_values('lemma_score',ascending=False).head(12)

check_mask:  238 rows


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score
66197,NGO,17771,0,NGO,ngo,True,1,NN,NN,A9kviKEOHu,NOM,N,Abkürzung,,2046.884277
21172,DDR,3264,0,DDR,ddr,True,1,NE,NE,HYTgj9TBi8,NAM,N,Geo Abkürzung,,1806.537842
1551,USA,5687,0,USA,usa,True,1,NE,NE,YhwITQCxUc,NAM,N,Geo Geo_land noSg Abkürzung,,1161.254395
81252,SMS,1818,0,SMS,sms,True,1,NN,NN,UobVGJE72y,NOM,N,Abkürzung,,870.798462
558,GmbH,3714,0,GmbH,gmbh,True,1,NN,NN,2WbUe3baS4,NOM,N,manuell ergänzt Abkürzung_verwaist,,682.923218
3472,ASCII,23980,0,ASCII,ascii,True,1,NN,NN,D4yZt8B2Fy,NOM,N,Abkürzung_verwaist noPl,,619.340088
3438,AG,3266,0,AG,ag,True,1,NN,NN,21PyIs5wme,NOM,N,Abkürzung,,615.005676
19361,CD-ROM,2693,0,CD-ROM,cd-rom,True,1,NN,NN,m4oBqiScUK,NOM,N,Abkürzung,,562.880005
98510,U-Bahnhof,47752,0,U-Bahnhof,u-bahnhof,True,1,NN,NN,xzeAIhlFoO,NOM,N,Abkürzung,,491.036011
99943,VBIED,452712,0,VBIED,vbied,True,1,NN,NN,6cUtL6QgVv,NOM,N,Abkürzung_verwaist,,427.921753


In [211]:
# translate_abk: Alle lang_id sind in wiktionary_lemma
mask = ~translate_abk.lang_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_abk,mask,0)
#translate_abk[mask] #.sort_values('score',ascending=False).head(12)

check_mask:  0 rows


In [212]:
# translate_abk: Viele kurz_id sind in wiktionary_lemma
mask =  translate_abk.kurz_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_abk,mask,2732)
#translate_abk[mask] #.sort_values('score',ascending=False).head(12)

check_mask:  2208 rows


In [213]:
# ... aber viele auch nicht
mask = ~translate_abk.kurz_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_abk,mask,3031)
#translate_abk[mask] #.sort_values('score',ascending=False).head(12)

check_mask:  3513 rows


## wiktionary_lemma ==========

### pflegen

In [214]:
# manuelle_kürzel erstellen: Welche Sorten Sonder-IDs gibt es?
# alt: ('_V', '_M', '_N', '_Z', '_O', '_NV')
# ('_V', '_A', '_M', '_N', '_MADJ', '_Z', '_T')

wiktionary_lemma['kürzel'] = wiktionary_lemma.lemma_id.str.split('_').str[-1]

mask = wiktionary_lemma.lemma_id.str.contains('_')
a = pak.analyse_freqs(wiktionary_lemma[mask], ['kürzel','lemma_id'])

mask1 = a.kürzel_count > 30
mask2 = a.kürzel.str.isalpha() 
mask3 = a.kürzel.str.len() < 5 
mask  = mask1  &  mask2  &  mask3
manuelle_kürzel = tuple(['_'+kürzel for kürzel in list(a[mask].kürzel)])  # um manuelle lemma_id erkennen zu können 
print(manuelle_kürzel)
assert len(manuelle_kürzel) == 6
grid(a,mask)

wiktionary_lemma = pak.drop_cols(wiktionary_lemma,'kürzel')

('_V', '_M', '_N', '_MADJ', '_Z', '_T')
6 rows out of 2245


### tag

In [215]:
# sichten
mask = wiktionary_lemma.tag == 'TODO'
pak.analyse_freqs( wiktionary_lemma[mask], ['member','lemma_id'], splits=[' ', None] )

,member,member_count,member_percent,lemma_id,lemma_id_count
0,Redewendung,268,64.4,"[A-a_machen, Aa_machen, Abbitte_leisten, Abbitte_tun, Achtung_gebietend, Anstalten_machen, Anstalten_treffen, Asbach_uralt, Atem_schöpfen, Aufsehen_erregen, Ausschau_halten, Bacchus_huldigen, Bauklötze_staunen, Bauklötzer_staunen, Bedenken_tragen, Bindfäden_regnen, Blut_lecken, Bände_sprechen, Chomez_batteln, Dampf_ablassen]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,Wortverbindung,146,35.1,"[Anno_Domini, Artes_liberales, Besorgnis_erregend, Druck_ausüben, Furcht_erregend, I_bims, Klein_Lieschen, Seine_Eminenz, Seine_Heiligkeit, Seine_Majestät, Wäller_Platt, a_la, a_lallemande, a_lalsacienne, a_lespagnole, a_secco, a_tempo, ab_irato, ad_libitum, ad_spectatores]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,,2,0.5,"[Tempi_passati, variatio_delectat]","[1, 1]"


In [216]:
# NE-Datensätze, die mit hoher Wahrscheinlichkeit was anderes als einen Namen bezeichnen
geschützte_namen = ['Martin','Baum','Junge','Klein','Keller','Mann','Peter','Schwarz','Simon',
                    'Thomas','Weiß','Winter','Sommer','Wolf','Stein','Henry','Frank','Müller',
                    'Meier','Huber','Schneider','Bauer','Conrad','Kramer','Grimm',
                    'Heinrich','Wagner','Pfeifer','Zimmer','Bach','Krämer','Fischer','Arnold',
                    'Lorenz','Fiedler','Weber','Walther','Busch','Schäfer','Michel','Ritter',
                    'Beck',
                   ]
mask1 =  wiktionary_lemma.member.str.contains('Person_nachname')
mask2 =  wiktionary_lemma.tag == 'NE'
mask3 =  wiktionary_lemma.page_part > 0
mask4 = ~wiktionary_lemma.lemma.isin(geschützte_namen)
mask5 = ~wiktionary_lemma.lemma.str.endswith(('mann','haus','meier','lein','berg','bruch','bach',))
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5

spalten = ['lemma_id','lemma','tag','member','lemma_score']
a = wiktionary_lemma[mask].sort_values('lemma_score').head(200).sort_values('lemma_score',ascending=False)[spalten]
gridt(a)

200 rows


In [217]:
# löschen
mask = wiktionary_lemma.lemma_id.isin(a.lemma_id)
check_mask( wiktionary_lemma, mask, 200 )

msg = 'NE1'
wiktionary_lemma, wiktionary_lemma_trash = pak.move_rows(wiktionary_lemma, wiktionary_lemma_trash, mask, msg)
grid(wiktionary_lemma_trash,msg)

check_mask:  200 rows
Move 200 rows from 169136
200 rows out of 2284


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score,msg
3559,Aalst_1,35489,1,Aalst,aalst,True,1,NE,NE,daN095xRWQ,NAM,N,Person_nachname,,4.795716,NE1
3573,Aar_2,2510,2,Aar,aar,True,1,NE,NE,7HB6U0BNQp,NAM,N,Person_nachname,,3.321496,NE1
3629,Abakus_1,704,1,Abakus,abakus,True,1,NE,NE,9g4IYX5gOn,NAM,N,Person_nachname,,2.555865,NE1
3639,Abart_1,2937,1,Abart,abart,True,1,NE,NE,LROPpUUh2X,NAM,N,Person_nachname,,2.852306,NE1
3649,Abate_2,14859,2,Abate,abate,True,1,NE,NE,HE1yGGr3Pv,NAM,N,Person_nachname,,0.654998,NE1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108268,Zeuge_1,25035,1,Zeuge,zeuge,True,1,NE,NE,6y9qRw0I3I,NAM,N,Person_nachname,,0.941703,NE1
108579,Zinn_1,10873,1,Zinn,zinn,True,1,NE,NE,K3dJGD0GzP,NAM,N,Person_nachname,,0.208227,NE1
108789,Zoll_2,20685,2,Zoll,zoll,True,1,NE,NE,cdgMvEHfNi,NAM,N,Person_nachname,,0.085637,NE1
109650,Zwölfer_1,241161,1,Zwölfer,zwölfer,True,1,NE,NE,cdcKk83wkQ,NAM,N,Person_nachname,,3.994555,NE1


In [218]:
# Prüfen: Alle Person_nachname sind NE
mask1 = wiktionary_lemma.member.str.contains('Person_nachname')
mask2 = wiktionary_lemma.tag == 'NE'
mask = mask1  &  ~mask2
check_mask(wiktionary_lemma,mask,0)

check_mask:  0 rows


In [219]:
# anschauen
mask = wiktionary_lemma.member.str.contains('Person_nachname')
a = pak.analyse_freqs( wiktionary_lemma[mask].sort_values('lemma_score',ascending=False), ['member','lemma'], splits=[' ',None] )
grid(a)

4 rows


,member,member_count,member_percent,lemma,lemma_count
0,Person_nachname,1405,96.1,"[Raab, Groß, Eder, Steinbach, Gottschalk, Rothe, Ansorge, Auer, Echtermeyer, Bichler, Nicolaus, Herrmann, Baak, Jauch, Blüml, Baumert, Schäuble, Stoiber, Crecelius, Rommel]","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]"
1,Person_vorname,44,3.0,"[Jordan, Timm, Paul, Erdmann, Götz, Oswald, Wichmann, Reinhardt, Wilhelm, Gottschalk, Barthel, Hein, Hermann, Hinz, Klaus, Ernst, Norman, Wulf, Sander, Urban]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,Geo,12,0.8,"[Jordan, Raab, Neuhaus, Bode, Eder, Steinbach, Geyer, Rothe, Metz, Offenbach, Lauer, Hagen]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,Geo_stadt,1,0.1,Offenbach,1


In [220]:
#grid(pak.analyse_cols(wiktionary_lemma))

In [221]:
# KOMBI: Datensätze umtaggen, die eine Wortverbindung oder eine Redewendung beinhalten
mask1  = wiktionary_lemma.tag == 'TODO'
mask2a = wiktionary_lemma.member.str.contains('Redewendung')
mask2b = wiktionary_lemma.member.str.contains('Wortverbindung')
mask = mask1  &  (mask2a | mask2b)
check_mask( wiktionary_lemma, mask, 412 )

change_tag( wiktionary_lemma, mask, 'KOMBI', manuelle_kürzel )

check_mask:  412 rows
412 Datensätze


In [222]:
# Rest sichten
mask = wiktionary_lemma.tag == 'TODO'
pak.analyse_freqs( wiktionary_lemma[mask], ['member', 'lemma_id'], splits=[' ', None] )

,member,member_count,member_percent,lemma_id,lemma_id_count
0,,2,100.0,"[Tempi_passati, variatio_delectat]","[1, 1]"


In [223]:
#mask = wiktionary_lemma.member.str.contains('Abkürzung')
#pak.analyse_freqs( wiktionary_lemma[mask], ['member', 'tag'],  )

In [224]:
# Rest löschen
mask = wiktionary_lemma.tag == 'TODO'
check_mask( wiktionary_lemma, mask, 0, 10 )

msg = 'TODO-Rest'
wiktionary_lemma, wiktionary_lemma_trash = pak.move_rows(wiktionary_lemma, wiktionary_lemma_trash, mask, msg)
grid(wiktionary_lemma_trash,msg)

check_mask:  2 rows
Move 2 rows from 168936
2 rows out of 2286


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score,msg
95239,Tempi_passati,484152,0,Tempi passati,tempi passati,True,2,TODO,TODO,3S60n0VONu,X_todo,X,,,4.542139,TODO-Rest
157767,variatio_delectat,747185,0,variatio delectat,variatio delectat,False,2,TODO,TODO,APYnHBunnW,X_todo,X,,,19.675831,TODO-Rest


In [225]:
# ADJD >> ADJA
wiktionary_lemma['tag'] = wiktionary_lemma.tag.str.replace('ADJD','ADJA')

In [226]:
# Einzelkorrekturen        
einzelkorrekturen = { 'sämtlich'       : 'PIAT ADJA',  
                      'besitzergreifen': 'VVFIN',    
                    }
for i,t in einzelkorrekturen.items():
    mask = (wiktionary_lemma.lemma_id == i)
    wiktionary_lemma.loc[mask,'tag'] = t

In [227]:
# Hilvsverben auf VAFIN
hilfsverben = [ 'haben', 'sein', 'werden', 'tun' ]
mask1 = wiktionary_lemma.tagZZ == 'V'
mask2 = wiktionary_lemma.lemma.isin(hilfsverben)
mask3 = wiktionary_lemma.tag != 'VAFIN'
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_lemma, mask, 1)

#grid(wiktionary_lemma,mask)
change_tag( wiktionary_lemma, mask, 'VAFIN', manuelle_kürzel )

check_mask:  1 rows
1 Datensätze


In [228]:
# Modalverben auf VMFIN
modalverben = [ 'dürfen', 'können', 'mögen', 'müssen', 'sollen', 'wollen', 'brauchen' ]
mask1 =  wiktionary_lemma.tagZZ == 'V'
mask2 =  wiktionary_lemma.lemma.isin(modalverben)
mask3 =  wiktionary_lemma.tag != 'VMFIN'
mask4 =  wiktionary_lemma.page_part == 0
mask  = mask1  &  mask2  &  mask3  &  mask4
check_mask( wiktionary_lemma, mask, 7)

#grid(wiktionary_lemma,mask)
change_tag( wiktionary_lemma, mask, 'VMFIN', manuelle_kürzel )

check_mask:  7 rows
7 Datensätze


In [229]:
# Obsolet: Alle anderen auf VVFIN 
#TODO vgl. und vergleiche müssten hier verschwinden
sonderverben = [ 'dürfen', 'können', 'mögen', 'müssen', 'sollen', 'wollen', 'brauchen', 'haben', 'sein', 'werden', 'tun' ]
mask1 =  wiktionary_lemma.tagZZ == 'V'
mask2 = ~wiktionary_lemma.lemma.isin(sonderverben)
mask3 =  wiktionary_lemma.tag != 'VVFIN'
mask  = mask1  &  mask2  &  mask3  
check_mask( wiktionary_lemma, mask, 0)

#change_tag( wiktionary_lemma, mask, 'VVFIN', manuelle_kürzel, manuelle_quellen_schützen=False )
#grid(wiktionary_lemma,mask)

check_mask:  0 rows


In [230]:
# Verben analysieren
mask = wiktionary_lemma.tagZZ == 'V'
pak.analyse_freqs(wiktionary_lemma[mask],['tag','lemma_id'])

,tag,tag_count,tag_percent,lemma_id,lemma_id_count
0,VVFIN,13853,99.9,"[Schlange_stehen, abbeißen, abberufen, abfahren, abküssen, abpassen, abreißen, abschrecken, abstoßen, abtörnen, abwerten, achten, adstringieren, affigieren, agglutinieren, alarmieren, alleinstehen, alternieren, amtieren, anbehalten]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,VMFIN,7,0.1,"[brauchen, dürfen, können, mögen, müssen, sollen, wollen]","[1, 1, 1, 1, 1, 1, 1]"
2,VAFIN,5,0.0,"[haben, sein, tun, werden, tun_1]","[1, 1, 1, 1, 1]"


### Affix und Suffix

In [231]:
# AFIX verschieben
mask1  = wiktionary_lemma.lemma.str.endswith('-')
mask2 = ~wiktionary_lemma.tag.isin(['CARD','$('])
mask = mask1  &  mask2
check_mask( wiktionary_lemma, mask, 554)

msg = 'AFIX'
wiktionary_lemma, wiktionary_lemma_fix = pak.move_rows( wiktionary_lemma, mask, msg ) 
grid(wiktionary_lemma_fix,msg)

check_mask:  548 rows
Move 548 rows from 168934
548 rows


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score,msg
6449,Allerwelts-,749499,0,Allerwelts-,allerwelts-,True,1,AFIX,AFIX,9Tptc4law5,FIX,Z,Präfixoid Präfix,,24.288923,31
6526,Allround-,668309,0,Allround-,allround-,True,1,AFIX,AFIX,eVsBOAuz32,FIX,Z,Gebundenes_Lexem,,18.702240,31
10021,Atto-,86413,0,Atto-,atto-,True,1,AFIX,AFIX,Eak8rjHCJE,FIX,Z,Gebundenes_Lexem Einheit,,11.136205,31
15534,Bilderbuch-,737083,0,Bilderbuch-,bilderbuch-,True,1,AFIX,AFIX,UDyIitK8WO,FIX,Z,Präfixoid Präfix,,31.893124,31
17664,Breitband-,684488,0,Breitband-,breitband-,True,1,AFIX,AFIX,mimBxvYVhc,FIX,Z,Gebundenes_Lexem,,16.201553,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168181,äqui-,71842,0,äqui-,äqui-,False,1,AFIX,AFIX,cLqoeIYECG,FIX,Z,Gebundenes_Lexem,,49.882153,31
168251,öko-,714607,0,öko-,öko-,False,1,AFIX,AFIX,dIi3vhxBRw,FIX,Z,Präfix,,35.807617,31
168273,öno-,129534,0,öno-,öno-,False,1,AFIX,AFIX,VIBY6gI4qF,FIX,Z,Gebundenes_Lexem,,21.613848,31
168310,über-,62111,0,über-,über-,False,1,AFIX,AFIX,Uf2IRteRfR,FIX,Z,Präfix,,148.673889,31


In [232]:
# ZFIX verschieben
mask1 = wiktionary_lemma.lemma.str.startswith('-')
mask2 = ~wiktionary_lemma.tag.isin(['CARD','$('])
mask = mask1  &  mask2
check_mask( wiktionary_lemma, mask, 389)

msg = 'ZFIX'
wiktionary_lemma, wiktionary_lemma_fix = pak.move_rows( wiktionary_lemma, wiktionary_lemma_fix, mask, msg ) 
grid(wiktionary_lemma_fix,msg)

check_mask:  384 rows
Move 384 rows from 168386
384 rows out of 932


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score,msg
2707,-ab,48801,0,-ab,-ab,False,1,ZFIX,ZFIX,90nPX6TDgu,FIX,Z,Suffix,,108.709518,ZFIX
2708,-abel,243516,0,-abel,-abel,False,1,ZFIX,ZFIX,HmOOHDfOWZ,FIX,Z,Adjektiv Adjektiv Suffix,,225.476135,ZFIX
2709,-aceae,232497,0,-aceae,-aceae,False,1,ZFIX,ZFIX,EqlQEiCFc1,FIX,Z,Suffix,,14.212000,ZFIX
2710,-ach,217804,0,-ach,-ach,False,1,ZFIX NE,ZFIX,NE,FIX,Z,Geo_ort Suffix,,45.883427,ZFIX
2711,-ade,714445,0,-ade,-ade,False,1,ZFIX,ZFIX,odMppWr6Pr,FIX,Z,Suffix,,104.712921,ZFIX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3086,-är,713795,0,-är,-är,False,1,ZFIX,ZFIX,N8H1NWngh4,FIX,Z,Suffix,,387.161377,ZFIX
3087,-ärmelig,232479,0,-ärmelig,-ärmelig,False,1,ZFIX,ZFIX,9MlHT6wNEf,FIX,Z,Gebundenes_Lexem,,17.031048,ZFIX
3088,-ästhesie,717017,0,-ästhesie,-ästhesie,False,1,ZFIX,ZFIX,JjoJMOzYJj,FIX,Z,Gebundenes_Lexem,,76.092667,ZFIX
3089,-äugig,690975,0,-äugig,-äugig,False,1,ZFIX,ZFIX,IM6HlmEF4I,FIX,Z,Suffix,,156.270752,ZFIX


In [233]:
# Sonstige Fixe
maskA = wiktionary_lemma.lemma.str.startswith('-')
maskB = wiktionary_lemma.lemma.str.endswith('-')
maskC = wiktionary_lemma.tag.str.contains('AFIX')
maskD = wiktionary_lemma.tag.str.contains('ZFIX')
maskE = wiktionary_lemma.member.str.contains('Affix')
maskF = wiktionary_lemma.member.str.contains('Suffix')
mask2 = ~wiktionary_lemma.tag.isin(['CARD','$('])
mask = (maskA | maskB | maskC | maskD | maskE | maskF)  &  mask2
check_mask( wiktionary_lemma, mask, 2)

msg = 'DIVFIX'
wiktionary_lemma, wiktionary_lemma_fix = pak.move_rows( wiktionary_lemma, wiktionary_lemma_fix, mask, msg ) 
grid(wiktionary_lemma_fix,msg)

check_mask:  1 rows
Move 1 rows from 168002
1 rows out of 933


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score,msg
121466,dar,50798,0,dar,dar,False,1,AFIX,AFIX,F0KypKvwnf,FIX,Z,Präfix,,1.5212,DIVFIX


### member

In [234]:
# member Aufzählung bei CARD
mask1 = wiktionary_lemma.lemma.str[:-1].str.isnumeric()
mask2 = wiktionary_lemma.lemma.str.endswith('.')  
mask3 = wiktionary_lemma.lemma.str.len() <= 3 
mask4 = wiktionary_lemma.tag == 'CARD'
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_lemma, mask, 11)

wiktionary_lemma.loc[mask,'member'] += ' Aufzählung'
grid(wiktionary_lemma,mask)

check_mask:  11 rows
11 rows out of 168001


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score
3098,0.,92317,0,0.,0.,True,1,CARD,CARD,EQr2QlR9Ap,X_zahl,X,Abkürzung Aufzählung,nullter,29.465649
3128,1.,53737,0,1.,1.,True,1,CARD,CARD,BvAgqY0lfu,X_zahl,X,Abkürzung Aufzählung,erster,32.793232
3152,10.,53904,0,10.,10.,True,1,CARD,CARD,usIXZoa9yY,X_zahl,X,Abkürzung Aufzählung,zehnter,33.962463
3228,2.,53711,0,2.,2.,True,1,CARD,CARD,Gm56NLV09d,X_zahl,X,Abkürzung Aufzählung,zweiter,55.384083
3272,3.,53739,0,3.,3.,True,1,CARD,CARD,KunrRbroUH,X_zahl,X,Abkürzung Aufzählung,dritter,33.316307
3292,4.,53891,0,4.,4.,True,1,CARD,CARD,l4Mw0Ns6nK,X_zahl,X,Abkürzung Aufzählung,vierter,34.424000
3315,5.,53892,0,5.,5.,True,1,CARD,CARD,kJBCl0m7a6,X_zahl,X,Abkürzung Aufzählung,fünfter,35.716309
3339,6.,53894,0,6.,6.,True,1,CARD,CARD,Kryt5i9qDg,X_zahl,X,Abkürzung Aufzählung,sechster,34.670155
3352,7.,53897,0,7.,7.,True,1,CARD,CARD,wgt95FakUm,X_zahl,X,Abkürzung Aufzählung,siebter,53.208374
3365,8.,53901,0,8.,8.,True,1,CARD,CARD,PxLZ2hYErG,X_zahl,X,Abkürzung Aufzählung,achter,34.177845


In [235]:
# member noSg und noPl nur bei Nomen
mask1a = wiktionary_lemma.member.str.contains('noPl')
mask1b = wiktionary_lemma.member.str.contains('noSg')
mask2  = wiktionary_lemma.tagZZ != 'N'
mask  = (mask1a | mask1b)  &  mask2
check_mask( wiktionary_lemma, mask, 0, 17 )

wiktionary_lemma.loc[mask,'member'] =  pak.remove_words( wiktionary_lemma[mask].member, ['noPl', 'noSg'] )
#grid(wiktionary_lemma,mask)

check_mask:  16 rows


### Allgemein

In [236]:
# Redundante Spalten
wiktionary_lemma = pflege_tag_felder(wiktionary_lemma, translate_tagZ)  # tagZ, tagZZ, tag_0, tag_1, tagZ_0, tagZ_1
wiktionary_lemma = ergänze_felder(wiktionary_lemma)                     # lemma_lower, isupper, lemma_len

# Fehler?
mask = (wiktionary_lemma.tagZZ.str.len() > 1)
check_mask(wiktionary_lemma,mask,0) # ein paar können wir tolerieren

check_mask:  291 rows
check_mask:  0 rows


In [237]:
# Keine Dups
mask = wiktionary_lemma.duplicated(subset='lemma_id', keep=False)
check_mask( wiktionary_lemma, mask, 0)

check_mask:  0 rows


In [238]:
# not any_nan
assert not pak.any_nan(wiktionary_lemma)
#pak.analyse_cols(wiktionary_lemma)

In [239]:
# suche_debug wiktionary_lemma
search_str(wiktionary_lemma, suche_debug)

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score


In [240]:
a = pak.analyse_freqs(wiktionary_lemma,['tag','member'], splits=[None,' '])
grid(a)

61 rows


,tag,tag_count,tag_percent,member,member_count
0,NN,96703,57.6,"[, noPl, adjNN, Abkürzung, Wortverbindung, noSg, manuell, Spezies, Name, lexAlt, Zahlwort, Abkürzung_verwaist, Einheit, ergänzt, Person_pre, Redewendung, Zahlklassifikator]","[80032, 12078, 3179, 1455, 1135, 1081, 673, 360, 305, 297, 217, 128, 80, 79, 31, 11, 5]"
1,ADJA,41839,24.9,"[Ptz1, Ptz2, noSteig, , Abkürzung, manuell, noFlex, Wortverbindung, ergänzt, lexAlt, Abkürzung_verwaist]","[13796, 13719, 7980, 5695, 645, 520, 380, 77, 48, 48, 1]"
2,VVFIN,13853,8.2,"[, manuell, lexAlt, Wortverbindung, ergänzt, Abkürzung, Redewendung]","[13308, 495, 84, 42, 11, 10, 2]"
3,NE,12118,7.2,"[Geo, Person_vorname, , Person_nachname, Wortverbindung, adjNN, Geo_stadt, noSg, Geo_land, Abkürzung, Geo_straße, Spezies, manuell, lexAlt, Abkürzung_verwaist, Geo_ort, ergänzt, Person_pre]","[6169, 5274, 1507, 1405, 613, 326, 293, 212, 103, 81, 46, 11, 6, 3, 3, 2, 2, 1]"
4,ADV,1562,0.9,"[, Wortverbindung, Lokaladverb, Abkürzung, manuell, Pronominaladverb, ergänzt, Gradpartikel, Temporaladverb, Modalpartikel, Konjunktionaladverb, Fokuspartikel, Modaladverb, Redewendung, Interrogativadverb, lexAlt]","[1272, 71, 57, 55, 30, 29, 18, 16, 14, 7, 7, 5, 5, 4, 3, 2]"
...,...,...,...,...,...
56,APPR DIV,1,0.0,"[Kontraktion, manuell, ergänzt]","[1, 1, 1]"
57,PRELS PDS,1,0.0,"[manuell, ergänzt]","[1, 1]"
58,PIAT ADJA,1,0.0,,1
59,PRELS PWAT,1,0.0,manuell,1


## wiktionary_merkmal ==========

### pflegen

In [241]:
# merkmal_order: Sichert die Reihenfolge der gleichnamigen Merkmale, z.B. mehrere Defs
wiktionary_merkmal = wiktionary_merkmal.reset_index()
wiktionary_merkmal = pak.rank( wiktionary_merkmal, 
                               cols_group = ['lemma_id','merkmal'], 
                               col_score = 'index',
                               find = 'min',
                               col_target = 'merkmal_order')
wiktionary_merkmal = pak.rename_col(wiktionary_merkmal,'index','i')
wiktionary_merkmal = pak.force_index(wiktionary_merkmal,'i')
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal,'merkmal_order','lemma_id')

rank: 0 rows less, now 4846228 rows


In [242]:
wiktionary_merkmal.head(1)

,lemma_id,merkmal_order,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
i,,,,,,,,,,,
0,Hallo,1,syn,Gejohle,Gejohle,,NN,1,,34.813919,False


### Schrott löschen 1
* ohne die Felder lemma und lemma_tag

In [243]:
# zu viele Sonderzeichen
mask1 =  wiktionary_merkmal.data.str.count('[A-ZÖÄÜa-zöäüß0-9]') < wiktionary_merkmal.data.str.count('[^A-ZÖÄÜa-zöäüß0-9]')-1
mask2 =  wiktionary_merkmal.data.str.len() > 2
mask3 = ~wiktionary_merkmal.merkmal.isin(['ipa','rhymes'])
mask4 = ~wiktionary_merkmal.data.isin(['u. s. w.','V. I. P.'])
mask  = mask1  & mask2  &  mask3  &  mask4
check_mask( wiktionary_merkmal, mask, 17)
#grid(wiktionary_merkmal, mask)

msg = 'zu viele Sonderzeichen'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg)

check_mask:  17 rows
Move 17 rows from 4846228
17 rows out of 6327


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order
64801,a._D.,alt_2,a._D.~a. d. D,a. d. D.,,,3,,11.471608,True,zu viele Sonderzeichen,1.0
64861,a._O.,alt_2,a._O.~a. d. O,a. d. O.,,TODO,2,,6.504342,True,zu viele Sonderzeichen,1.0
64862,a._O.,alt_2,a._O.~a. a. O,a. a. O.,,TODO,3,,6.504342,True,zu viele Sonderzeichen,2.0
68942,GmbH,alt_1,GmbH~G. m. b. H,G. m. b. H.,,NN,,,73.640892,True,zu viele Sonderzeichen,1.0
141173,BfA,up,,K. d. ö. R.,,NN,1,,18.144148,False,zu viele Sonderzeichen,1.0
176090,MdB,alt_2,MdB~M. d. B,M. d. B.,,TODO,,,7.672363,True,zu viele Sonderzeichen,1.0
176094,MdL,alt_2,MdL~M. d. L,M. d. L.,,TODO,1,,13.429291,True,zu viele Sonderzeichen,1.0
178302,aaO,alt_2,aaO~a. a. O,a. a. O.,,TODO,1,,12.276679,True,zu viele Sonderzeichen,1.0
268801,RAF,alt_2,RAF~R. A. F,R. A. F.,,TODO,3,,36.033504,True,zu viele Sonderzeichen,1.0
322500,u._V.,alt_1,u._V.~u. ü. V,u. ü. V.,,TODO,,,1.937145,True,zu viele Sonderzeichen,1.0


In [244]:
# fehlende Klammern
mask1 = wiktionary_merkmal.data.str.count('\(') != wiktionary_merkmal.data.str.count('\)')
mask2 = wiktionary_merkmal.data.str.len() > 1
mask = mask1  &  mask2
check_mask( wiktionary_merkmal, mask, 0, 10)

msg = 'fehlende_Klammern'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


In [245]:
# Fugenelemente
mask1 =  wiktionary_merkmal.data != '-'
mask2 =  wiktionary_merkmal.data.str.startswith('-')
mask3 =  wiktionary_merkmal.data.str.endswith('-')
mask  =  mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 0)

msg = 'Fugenelemente'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


### aufsplitten
wiktionary_merkmal wird in drei Teile aufgesplittet:
* wiktionary_merkmal wurzelt in wiktionary_lemma
* wiktionary_lexmerkmal wurzelt in translate_lex.<br>
  translate_lex enthält ja Lexeme (z.B. die Deklinierten Formen). <br>
  wiktionary_lexmerkmal enthält die Merkmale dieser Lexeme.<br>
* translate_gm enthält alle gm-Merkmale, also die Verweise von einer deklinierten Form auf das Lemma.<br>

In [246]:
# Spalten löschen
wiktionary_merkmal = pak.drop_cols( wiktionary_merkmal, ['lemma_lower'])

In [247]:
# anschauen: verwaistes und verschollenes lemma_id
mask1 = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
mask2 = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma_trash.lemma_id)
mask3 = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma_fix.lemma_id)
mask4 = ~wiktionary_merkmal.lemma_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4
#grid(wiktionary_merkmal,mask)

In [248]:
# verwaistes lemma_id
# Ursache: Belöschen von wiktionary_lemma
mask1 = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
mask2 = ~wiktionary_merkmal.lemma_id.isin(translate_lex.data_id)
mask = mask1  &  mask2

# mask nur für die Anzeige
mask_a1 = ~wiktionary_merkmal.lemma_id.str.startswith('-')
mask_a2 = ~wiktionary_merkmal.lemma_id.str.endswith('-')
mask_a = mask  &  mask_a1  &  mask_a2

a = pak.analyse_freqs(wiktionary_merkmal[mask_a].sort_values('data_score',ascending=False),['merkmal','lemma_id'])
grid(a)

30 rows


,merkmal,merkmal_count,merkmal_percent,lemma_id,lemma_id_count
0,ipa,2651,27.5,"[OP, IKT, Fachleute, AIDS, IS, MD, UTC, USBV, BENELUX, sprich, Pegida, MDR, KGV, BUND, AT, NP, XML, RKI, AC, multikulti]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,syllables,2529,26.2,"[Fachleute, UTC, sprich, Pegida, MSc, XML, QmP, multikulti, PISA, HDGDL, KIP, hdgdl, Hi-Fi, MP3, vorgetan, gesamt_1, betan, nachgetan, gewullackt, notgetan]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,def,1535,15.9,"[gel., o._B., Str., cui_bono?, MD, BN, Fr., gem., ges., Wa., HDGDL, gef., gew., BB, LGBTQIA, AAA, gen., Ges., BNA, geb.]","[23, 18, 17, 14, 13, 11, 10, 10, 10, 10, 10, 9, 9, 9, 9, 8, 8, 8, 8, 8]"
3,up,1320,13.7,"[E_450, E_452, E_451, E_341, E_339, E_340, E_504, E_503, E_501, E_966, E_967, E_515, E_479b, E_300, E_461, E_516, E_466, E_470a, E_468, E_464]","[16, 16, 15, 12, 12, 12, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 7, 7, 7, 7]"
4,rhymes,468,4.9,"[Fachleute, AIDS, sprich, Pegida, Legida, LEGIDA, PEGIDA, MERS, hinüberrannte, mechend, betan, mögen_1, sollen_1, gesamt_1, verspeist, hören_1, getan, unterminiert_1, betroffen_1, versiert_2]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
5,alt_1,390,4.0,"[Dipl.-Kff., as., PersAuswG, kartle, kartele, odle, anheimle, anheimele, odele, kartel, Fexen, odel, ab-_1, heimele_an, heimle_an, Baumgarten_1, anheimel, heimel_an, YOLO, EBIT]","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1]"
6,alt_2,263,2.7,"[Az., gef., bez., am., Ges., geb., MSc, A.C.A.B., B_42, Qu., Hi-Fi, hapax_legomenon, Hapaxlegomenon, Lo-Fi, it., FUBAR, me., HiFi, Hifi, s._Br.]","[6, 4, 4, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]"
7,gm,96,1.0,"[verspeist, unwirschen_1, hinüberrannte, Einers, geheimgetan, leichtgetan, schwergetan, schöngetan, dickgetan, großgetan, hartgetan, genuggetan, zugutegetan, zuvorgetan, zusammengetan, ausgetan, hervorgetan, aufgetan, hineingetan, hinzugetan]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
8,gegen,78,0.8,"[QbA, QmP, 2_Mos, 4_Mos, 5_Mos, 3_Mos, MDR, Hi-Fi, Lo-Fi, 3_Makk, 2_Makk, 4_Makk, 1_Makk, OOD, OOA, FWDL, SaZ, GWDL, OLAP, B2C]","[9, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 1, 1]"
9,familie,66,0.7,"[be-…-t, Ge-…-e, top-_1, multikulti, zer-…-t, OP, AIDS, DV, IS, Pegida, ASL, NP, EEG, MP3, Hiat, FDJ, VAE, GEMA, DAX, BRICS]","[24, 13, 7, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"


In [249]:
# verwaistes lemma_id löschen
check_mask(wiktionary_merkmal,mask,17724) 

msg = 'verwaist_lemma_id'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg)
#grid( wiktionary_merkmal_trash, msg )

check_mask:  19260 rows
Move 19260 rows from 4846211


In [250]:
# anschauen
mask = wiktionary_merkmal_trash.msg == msg
a = pak.analyse_freqs( wiktionary_merkmal_trash[mask], ['merkmal','lemma_tag','data',] )
grid(a)

43 rows


,merkmal,merkmal_count,merkmal_percent,lemma_tag,lemma_tag_count,data,data_count
0,familie,4885,25.4,"[AFIX, ZFIX, ZFIX NE, , TODO, NE]","[2355, 2344, 96, 71, 18, 1]","[Schizophrenie, Logopädie, Homöopathie, Empathie, Autograph, Autokrat, Autokratie, Orthopädie, desinfizieren, klassifizieren, zertifizieren, Hypophyse, Hypotonie, Heliostat, Hygrostat, Mesopotamien, Mikronesien, Ichthyophage, fernab, herab]","[3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]"
1,ipa,3553,18.4,"[, AFIX, ZFIX, TODO, NE, ZFIX NE, ZFIX PPER]","[2258, 538, 276, 198, 189, 93, 1]","[ap, -ɛk, ˈhaɪˌfi, ˈloʊˌfaɪ, at, an, ak, aʁc, tsyklo, tsykl, foːn, ˈɡʁaːfo, ɡʁaf, ˈɡlyko, ˈvaɪtɐfɛɐˌhandlt, ˈklasəstn, ˈklasɐ, ˈfiːl, ˈfoto, əʁɪc]","[4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]"
2,syllables,3420,17.8,"[, AFIX, ZFIX, NE, ZFIX NE, TODO, ZFIX PPER]","[2290, 522, 281, 197, 93, 36, 1]","[in, an, ab, gon, top, te⇀le, at, ent, heim, ier, Lo⇀Fi, Hi⇀Fi, en, bar, tel, mar, rich, el, ver⇀siert, fort]","[4, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]"
3,def,2628,13.6,"[TODO, AFIX, ZFIX, NE, ZFIX NE, , ZFIX PPER]","[1252, 665, 296, 218, 131, 64, 2]","[Familienname, Ortsnamengrundwort, Nachname, Gesetz, Kraftfahrzeugkennzeichen, Bundeswehr, Dienstgrad, deutsch, Bundesministerium, Landkreis, Abkürzung, Siedlung, Verordnung, ohne, gegen, bei, deutschsprachig, weg, Kfz-Kennzeichen, hinzu]","[129, 49, 47, 39, 35, 33, 32, 30, 24, 23, 21, 17, 17, 13, 13, 13, 12, 11, 11, 11]"
4,up,1330,6.9,"[TODO, ZFIX, , AFIX]","[1317, 8, 4, 1]","[Chemie, Lebensmittelzusatzstoff, chemisch, Verbindung, chemische Verbindung, Konservierungsmittel, Farbstabilisator, Trennmittel, Emulgator, Antioxidans, Stabilisator, Säureregulator, Trägerstoff, Füllstoff, Geschmacksverstärker, Komplexbildner, Verdickungsmittel, Mehlbehandlungsmittel, Festigungsmittel, Überzugsmittel]","[210, 176, 113, 110, 92, 36, 36, 36, 33, 32, 24, 23, 23, 23, 22, 20, 19, 18, 15, 14]"
5,alt_1,750,3.9,"[, AFIX, ZFIX, TODO, ZFIX NE, NE]","[355, 288, 53, 30, 22, 2]","[glyco, sur, glyk, glyc, glyz, gluco, gluko, glyko, kom, suf, sug, suk, sup, sub, kol, kor, kon, -ation, -ition, -tion]","[6, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3]"
6,lateral,686,3.6,"[AFIX, ZFIX, TODO, ZFIX NE, ]","[459, 159, 32, 25, 11]","[weg, nicht, über, sehr, gegen, ober, nieder, vor, völlig, hinein, miss, unter, -isch, -lich, zusammen, fort, runter, -er, herunter, anti]","[12, 11, 9, 8, 8, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5]"
7,rhymes,479,2.5,"[, NE, TODO, AFIX, ZFIX, ZFIX NE]","[300, 143, 24, 7, 4, 1]","[iːda, asəstn, iːɐ, aːɐ, aːl, iːɐtn, iːɐtə, ɛːsɐ, iːɐst, iːʁəst, iːɐtəst, iːʁət, iːɐtət, iːʁə, aːzn, aːn, aks, eː, aɪtə, ɛlə]","[4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]"
8,alt_2,323,1.7,"[, TODO, AFIX, ZFIX, NE]","[141, 120, 43, 15, 4]","[nördl. Br., nördl. Breite, südl. Br., südl. Breite, s. Br., n. Br., Hapax Legomenon, Hapax legomenon, rasser, rassesten, rässer, rässesten, Gld, Hifi, HiFi, Hi-Fi, Lofi, LoFi, Lo-Fi, P. T.]","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]"
9,gegen,256,1.3,"[AFIX, TODO, ZFIX, ]","[161, 76, 18, 1]","[weg, raus, herauf, hinein, herunter, fremd, pro, vor, hinauf, rauf, rein, hoch, 1 Mos, hetero, unter, nach, ober, gegen, -phob, 2 Makk]","[10, 6, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3]"


In [251]:
# verwaiste data_id (früher wurden die gelöscht) 
mask1 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask2 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask3 =  wiktionary_merkmal.data_id != ''
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask, 49440)
grid(wiktionary_merkmal,mask)

check_mask:  49724 rows
49724 rows out of 4826951


,lemma_id,merkmal_order,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
i,,,,,,,,,,,
96,Dezember,1,alt,Dezember~December,December,,NN,,,212.693924,True
345,August_1,1,alt_abk,August_1~Gust,Gust,,NE,1,,9.259675,True
346,August_1,1,alt_klein,August_1~Gustel,Gustel,,NE,1,,9.259675,True
399,Oktober,1,alt,Oktober~October,October,,NN,,,240.864517,True
785,Litschi,1,alt_1,Litschi~Lychee,Lychee,,NN,,,39.416626,True
...,...,...,...,...,...,...,...,...,...,...,...
3181959,ober-_1,1,Superlativ,ober-_1~oberst,oberst,,ADJA,,,30.302841,True
3182046,allerliebst,1,Superlativ,allerliebst~am allerliebsten,am allerliebsten,,ADJA,,,22.669069,True
4198298,den_1,1,syllables,den_2,den,,,,,0.679771,False


In [252]:
# msg = 'verwaist_data_id'
# wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg)
# grid( wiktionary_merkmal_trash, msg )

In [253]:
# # anschauen
# mask = wiktionary_merkmal_trash.msg == msg
# a = pak.analyse_freqs( wiktionary_merkmal_trash[mask], ['merkmal','lemma_tag','data',] )
# grid(a)

In [254]:
# translate_gm abspalten
mask = wiktionary_merkmal.merkmal == 'gm'
check_mask(wiktionary_merkmal, mask, 694604)
wiktionary_merkmal, translate_gm = pak.move_rows(wiktionary_merkmal, mask, msg=None)

check_mask:  694604 rows
Move 694604 rows from 4826951


In [255]:
# wiktionary_lexmerkmal abspalten
mask1 = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
mask2 =  wiktionary_merkmal.lemma_id.isin(translate_lex.data_id)
mask = mask1  &  mask2
check_mask(wiktionary_merkmal, mask, 1714508)

wiktionary_merkmal, wiktionary_lexmerkmal = pak.move_rows(wiktionary_merkmal, mask, msg=None)
wiktionary_lexmerkmal = pak.rename_col(wiktionary_lexmerkmal, 'lemma_id','lex_id')

check_mask:  1714508 rows
Move 1714508 rows from 4132347


### pflegen

In [256]:
# suche_debug wiktionary_merkmal
a = search_str(wiktionary_merkmal, suche_debug)
grid(a)

No rows


In [257]:
# lemma nach wiktionary_merkmal übertragen
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, wiktionary_lemma, on='lemma_id', col='lemma')

update_col: 2417839 cells written into new column


In [258]:
# lemma_tag nach wiktionary_merkmal übertragen
wiktionary_merkmal['lemma_tag'] =  wiktionary_merkmal.lemma_tag.fillna('')
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, wiktionary_lemma, on='lemma_id', col='tag', col_rename='lemma_tag')
wiktionary_merkmal['lemma_tag'] =  wiktionary_merkmal.lemma_tag.fillna('')

update_col: 2417839 cells written into existing column


In [259]:
# sicherstellen: lemma_tag ist gesetzt
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.lemma_tag == ''
mask  =  mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows


### Schrott löschen 2
* diesmal mit den Feldern lemma und lemma_tag

In [260]:
# beginnt mit Sonderzeichen
erlaubt = ["'nen"]
mask0 = ~wiktionary_merkmal.data.isin(erlaubt)
mask1 =  wiktionary_merkmal.data.str.len() > 1
mask2 = ~wiktionary_merkmal.data.str[0].str.isalpha()  
mask3 = ~wiktionary_merkmal.data.str[0].str.isnumeric()    
mask4 = ~wiktionary_merkmal.lemma_tag.isin([ '$(', '$.', '$,', 'ZFIX NE', 'ZFIX', 'CARD' ])
mask5 =  wiktionary_merkmal.is_lex == True
mask  =  mask0  &  mask1  &  mask2  &  mask3  &  mask4  &  mask5    
check_mask( wiktionary_merkmal, mask, 88)

msg = 'beginnt mit Sonderzeichen'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg ) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  88 rows
Move 88 rows from 2417839
88 rows out of 25675


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
17926,wa_1,alt_2,wa_1~-wa,-wa,,PPER,,,1.455715,True,beginnt mit Sonderzeichen,1.0,wa
841743,Ache,alt_3,-ach,-ach,,NE,1,,26.709286,True,beginnt mit Sonderzeichen,5.0,Ache
1549949,m-RNA,alt_2,m-RNA~(m)RNA,(m)RNA,,NN,,,48.221127,True,beginnt mit Sonderzeichen,2.0,m-RNA
1598833,mRNA,alt_2,mRNA~(m)RNA,(m)RNA,,NN,,,7.196318,True,beginnt mit Sonderzeichen,2.0,mRNA
2305416,Freidich,Nom Sg,Freidich~(der) Freidich,(der) Freidich,,NN,,,13.335077,True,beginnt mit Sonderzeichen,1.0,Freidich
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3014217,Sonndich,Akk Pl,Sonndich~(die) Sonndich,(die) Sonndich,,NN,,,13.335077,True,beginnt mit Sonderzeichen,1.0,Sonndich
3014260,Dunschdich,Akk Pl,Dunschdich~(die) Dunschdich,(die) Dunschdich,,NN,,,15.029414,True,beginnt mit Sonderzeichen,1.0,Dunschdich
3014269,Samschdich,Akk Pl,Samschdich~(die) Samschdich,(die) Samschdich,,NN,,,13.627384,True,beginnt mit Sonderzeichen,1.0,Samschdich
3014283,Freidich,Akk Pl,Freidich~(die) Freidich,(die) Freidich,,NN,,,13.335077,True,beginnt mit Sonderzeichen,1.0,Freidich


In [261]:
# nicht_erlaubte_sonderzeichen 
mask1 = wiktionary_merkmal.data.str.contains(nicht_erlaubte_sonderzeichen)   
mask2 = ~wiktionary_merkmal.lemma_tag.isin(['$(','$.','$,'])
mask  =  mask1  &  mask2 
check_mask( wiktionary_merkmal, mask, 2)

msg = 'nicht_erlaubte_sonderzeichen'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg)

check_mask:  2 rows
Move 2 rows from 2417751
2 rows out of 25677


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
1830810,umeinanderhetzen,Befehl du,umeinanderhetzen~hetz umeinander!,hetz umeinander!,,VVFIN,,,23.235119,True,nicht_erlaubte_sonderzeichen,1.0,umeinanderhetzen
1830811,umeinanderhetzen,Befehl ihr,umeinanderhetzen~hetzt umeinander!,hetzt umeinander!,,VVFIN,,,23.235119,True,nicht_erlaubte_sonderzeichen,1.0,umeinanderhetzen


In [262]:
# zu viele Wörter 
mask1 =  wiktionary_merkmal.data.str.split().str.len() > 3
mask2 =  wiktionary_merkmal.is_lex == True
mask3 = ~wiktionary_merkmal.merkmal.str.startswith('alt') 
mask  =  mask1  &  mask2  &  mask3 
check_mask( wiktionary_merkmal, mask, 0)

msg = 'zu viele Wörter'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


In [263]:
# Verbotene Fragmente in data, die nicht im lemma auftreten
verbotene_fragmente = ['Konjugation','Genitiv','Dativ','(',')',':','»','<','!','#',',',';','!','?']
wiktionary_merkmal['error'] = None
for verboten in verbotene_fragmente:
    mask1 =  wiktionary_merkmal.data.str.contains(verboten, regex=False)    
    mask2 = ~wiktionary_merkmal.lemma.str.contains(verboten, regex=False)  
    mask3 =  wiktionary_merkmal.data.str.len() > 1       
    mask4 =  wiktionary_merkmal.is_lex == True   
    mask5 = ~wiktionary_merkmal.merkmal.str.startswith('alt') 
    mask  =  mask1  &  mask2  &  mask3  &  mask4  &  mask5
    wiktionary_merkmal.loc[ mask, 'error'] = 'Verbotenes Fragment ' + verboten

In [264]:
mask = ~wiktionary_merkmal.error.isnull()  
check_mask( wiktionary_merkmal, mask, 0, 5)
#grid( wiktionary_merkmal, mask)

wiktionary_merkmal = pak.drop_cols( wiktionary_merkmal, 'error' )
msg = 'Verbotene Fragmente'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


In [265]:
# Überflüssige lemmagleiche Lexeme löschen
mask1  = wiktionary_merkmal.is_lex == True
mask2  = wiktionary_merkmal.data == wiktionary_merkmal.lemma
mask3A = wiktionary_merkmal.merkmal.str.startswith('alt') 
mask3B = wiktionary_merkmal.merkmal.isin( ['lex'] )
mask  = mask1  &  mask2  &  (mask3A | mask3B) 
check_mask( wiktionary_merkmal, mask, 210)

msg = 'lemmagleiche Lexeme'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  209 rows
Move 209 rows from 2417749
209 rows out of 25886


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
40468,Dojinshi,alt_2,Dojinshi,Dojinshi,,NN,,,47.115898,True,lemmagleiche Lexeme,2.0,Dojinshi
60510,Eiscreme,alt_2,Eiscreme,Eiscreme,,NN,,,71.216179,True,lemmagleiche Lexeme,1.0,Eiscreme
69528,Piezokeramik,alt_2,Piezokeramik,Piezokeramik,,NN,,,20.034588,True,lemmagleiche Lexeme,1.0,Piezokeramik
74078,Abbe_1,alt_3,Abbe_1,Abbe,,NE,1,,2.571619,True,lemmagleiche Lexeme,1.0,Abbe
77529,Chisinau,alt_2,Chisinau,Chisinau,,NE,,,48.291943,True,lemmagleiche Lexeme,1.0,Chisinau
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1761767,Enquete-Kommission,alt_2,Enquete-Kommission,Enquete-Kommission,,NN,,,40.098053,True,lemmagleiche Lexeme,1.0,Enquete-Kommission
1763657,Derrieregarde,alt_2,Derrieregarde,Derrieregarde,,NN,,,79.929085,True,lemmagleiche Lexeme,1.0,Derrieregarde
1788509,Schuhcreme,alt_2,Schuhcreme,Schuhcreme,,NN,,,47.697083,True,lemmagleiche Lexeme,1.0,Schuhcreme
1792201,Rasiercreme,alt_2,Rasiercreme,Rasiercreme,,NN,,,36.922779,True,lemmagleiche Lexeme,1.0,Rasiercreme


In [266]:
# Überflüssige lemmagleiche Nichtlexeme löschen
mask1 =  wiktionary_merkmal.is_lex == False
mask2 =  wiktionary_merkmal.data == wiktionary_merkmal.lemma
mask3 = ~wiktionary_merkmal.merkmal.isin( ['syllables','Hilfsverb'] )
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 1034)

msg = 'lemmagleiche Nichtlexeme'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  1034 rows
Move 1034 rows from 2417540
1034 rows out of 26920


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
400,November,herkunft,November,November,,NN,,,199.928970,False,lemmagleiche Nichtlexeme,1.0,November
426,ich,herkunft,ich,ich,,PPER,,,2407.714844,False,lemmagleiche Nichtlexeme,2.0,ich
456,lieben,herkunft,lieben,lieben,,VVFIN,,,173.774323,False,lemmagleiche Nichtlexeme,1.0,lieben
2919,klieben,herkunft,klieben,klieben,,VVFIN,,,106.576828,False,lemmagleiche Nichtlexeme,1.0,klieben
3362,See_1,gegen,See,See,,NN,1,,44.082108,False,lemmagleiche Nichtlexeme,6.0,See
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3996224,alt,ipa,alt,alt,,ADJA,,,326.570221,False,lemmagleiche Nichtlexeme,1.0,alt
3998460,an,ipa,an,an,,APPR,,,58.081432,False,lemmagleiche Nichtlexeme,1.0,an
3999164,an_1,ipa,an_1,an,,ADV,,,4.214169,False,lemmagleiche Nichtlexeme,1.0,an
4000400,als_1,ipa,als_1,als,,ADV,,,0.993464,False,lemmagleiche Nichtlexeme,1.0,als


In [267]:
# Benannte Lexeme
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.merkmal != 'lex'
mask  = mask1  &  mask2
benannte_lexeme = wiktionary_merkmal[mask]

In [268]:
# Überflüssige lex: Die gibt es schon unter einem konkreteren Merkmal
# Das sind nicht so viele, weil die alle in translate_lex sind
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.merkmal == 'lex'
mask3 = pak.isin( wiktionary_merkmal, benannte_lexeme, on=['lemma','data'] )
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 14)

del benannte_lexeme
msg = 'Überflüssige lex'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  22 rows
Move 22 rows from 2416506
22 rows out of 26942


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
1830812,vergleichen,lex,vergleiche,vergleiche,VVIMP,VVFIN,,manuell,96.039993,True,Überflüssige lex,1.0,vergleichen
1831413,dies,lex,dieses,dieses,,PRELS PDS,,manuell,55.655846,True,Überflüssige lex,1.0,dies
1831420,dies,lex,dieser,dieser,,PRELS PDS,,manuell,55.655846,True,Überflüssige lex,2.0,dies
1831422,dies,lex,diesen,diesen,,PRELS PDS,,manuell,55.655846,True,Überflüssige lex,3.0,dies
1831446,was,lex,wessen,wessen,,PWAT PRO,,manuell,144.917435,True,Überflüssige lex,1.0,was
1831462,Versprechung,lex,Versprechungen,Versprechungen,,NN,,manuell,39.175423,True,Überflüssige lex,1.0,Versprechung
1831514,euer,lex,euer~eurem,eurem,,PPOSS,,manuell,94.830978,True,Überflüssige lex,1.0,euer
1831515,euer,lex,euer~euren,euren,,PPOSS,,manuell,94.830978,True,Überflüssige lex,2.0,euer
1831516,euer,lex,euer~eurer,eurer,,PPOSS,,manuell,94.830978,True,Überflüssige lex,3.0,euer
1831517,euer,lex,eures,eures,,PPOSS,,manuell,94.830978,True,Überflüssige lex,4.0,euer


In [269]:
# TRUNC löschen
mask =  wiktionary_merkmal.lemma_tag == 'TRUNC'
check_mask( wiktionary_merkmal, mask, 0, 4)

msg = 'TRUNC'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  2 rows
Move 2 rows from 2416484
3 rows out of 26944


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
439905,www,def,WWW,WWW,,TRUNC,1,,2.239461,False,def einer Abkürzung,NaN,<NA>
569109,http,kombi,www,www,,TRUNC,1,http,4.404000,False,TRUNC,1.0,http
3778626,http,ipa,,haː teː teː ˈpeː,,TRUNC,,,4.404000,False,TRUNC,1.0,http


In [270]:
#search_str(wiktionary_lemma, 'http')

In [271]:
# suche_debug
maskA = wiktionary_merkmal_trash.data.isin(suche_debug)
maskB = wiktionary_merkmal_trash.lemma.isin(suche_debug)
mask = maskA | maskB
grid(wiktionary_merkmal_trash,mask, color='green')

No rows, mask filters them all away


In [272]:
# suche_debug
#mask = wiktionary_merkmal.data.isin(suche_debug)
mask = wiktionary_merkmal.lemma.isin(suche_debug)
#mask = maskA | maskB
grid(wiktionary_merkmal,mask)

No rows, mask filters them all away


In [273]:
# Nochmal sicherstellen: Alle Datensätze in wiktionary_merkmal haben einen passenden DS in wiktionary_lemma
mask = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows


In [274]:
# Spalten sortieren
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal, ['lemma_id','merkmal','is_lex','data_id','data','data_tag','lemma','lemma_tag','num','meta',])
wiktionary_merkmal.head(1)

,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
i,,,,,,,,,,,,
0,Hallo,syn,False,Gejohle,Gejohle,,Hallo,NN,1,,1,34.813919


### Grundformen anfügen

In [275]:
# Grundformen (merkmal 'lemma')
spalten = ['lemma_id', 'lemma', 'tag_0', 'lemma_score' ]
grundformen = wiktionary_lemma[spalten].copy()
grundformen = pak.rename_col(grundformen, 'tag_0', 'lemma_tag')
grundformen = pak.rename_col(grundformen, 'lemma_score', 'data_score')
grundformen['merkmal']  = 'lemma'
grundformen['is_lex']   = True
grundformen['data']     = grundformen.lemma.copy()
grundformen['data_id']  = grundformen.lemma_id.copy()
grundformen['data_tag'] = grundformen.lemma_tag.copy()
grundformen['meta']     = ''
grundformen['num']      = ''


# Spaltenvergleich
bpy.minivenn( grundformen.columns, wiktionary_merkmal.columns, format='print2')

left_only:  {}
right_only: {'merkmal_order'}



In [276]:
# add_rows
wiktionary_merkmal = pak.add_rows(wiktionary_merkmal, grundformen)
pak.sample(grundformen)

168001 rows added, now a total of 2584483


,lemma_id,lemma,lemma_tag,data_score,merkmal,is_lex,data,data_id,data_tag,meta,num
0,24/7,24/7,ADJA,14.942436,lemma,True,24/7,24/7,ADJA,,
670,Hugo_1,Hugo,NE,154.971725,lemma,True,Hugo,Hugo_1,NE,,
1358,Schlange_stehen,Schlange stehen,VVFIN,18.997892,lemma,True,Schlange stehen,Schlange_stehen,VVFIN,,
2696,!_M,!,$.,6.718384,lemma,True,!,!_M,$.,,
2702,",_M",",","$,",6.718384,lemma,True,",",",_M","$,",,
2703,-1_Z,-1,CARD,6.718384,lemma,True,-1,-1_Z,CARD,,
3122,Aadolf_V,Aadolf,NE,6.708904,lemma,True,Aadolf,Aadolf_V,NE,,
9133,Asch,Asch,NN,189.068604,lemma,True,Asch,Asch,NN,,
41414,Haus,Haus,NN,108960.914062,lemma,True,Haus,Haus,NN,,
79452,Riß_2,Riß,NN,0.000300,lemma,True,Riß,Riß_2,NN,,


In [277]:
# Sonderfall Mag.a löschen
mask = wiktionary_merkmal.data_id == 'Mag.a'
check_mask(wiktionary_merkmal,mask,0,2)

msg = 'Sonderfall Mag.a' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  2 rows
Move 2 rows from 2584483
2 rows out of 26946


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
527914,Mag.,weib,Mag.a,Mag.a.,,NN,1,,56.269928,True,Sonderfall Mag.a,1.0,Mag.
2476473,Mag.a,lemma,Mag.a,Mag.a,NN,NN,,,18.292961,True,Sonderfall Mag.a,NaN,Mag.a


In [278]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


In [279]:
# Verwaiste Merkmale: Gibt es nicht, weil die abgetrennt oder gelöscht wurden
mask = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask( wiktionary_merkmal, mask, 0)
#grid( wiktionary_merkmal, mask, )

# löschen
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal, wiktionary_merkmal_trash, mask, 'verwaist4')
grid(wiktionary_merkmal_trash, color='green')

check_mask:  0 rows
No rows moved
26946 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
2152141,Außerm_Pfarrgarten,Genus,,,,NE,,,26.970110,False,Leerstring data Müll,NaN,<NA>
2178055,Covid,Genus,,,,NE,,,44.336594,False,Leerstring data Müll,NaN,<NA>
2178457,Heisei,Genus,,,,NN,,,44.313446,False,Leerstring data Müll,NaN,<NA>
2178464,Reiwa,Genus,,,,NN,,,45.144207,False,Leerstring data Müll,NaN,<NA>
2200952,Am_Bahnhof,Genus,,,,NE,,,26.080570,False,Leerstring data Müll,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831748,Kid,lex,Kids,Kids,,NN,,manuell,51.648682,True,Überflüssige lex,1.0,Kid
569109,http,kombi,www,www,,TRUNC,1,http,4.404000,False,TRUNC,1.0,http
3778626,http,ipa,,haː teː teː ˈpeː,,TRUNC,,,4.404000,False,TRUNC,1.0,http
527914,Mag.,weib,Mag.a,Mag.a.,,NN,1,,56.269928,True,Sonderfall Mag.a,1.0,Mag.


In [280]:
# fillna
wiktionary_merkmal['meta']          = wiktionary_merkmal.meta.fillna('')
wiktionary_merkmal['data_tag']      = wiktionary_merkmal.data_tag.fillna('')
wiktionary_merkmal['lemma_tag']     = wiktionary_merkmal.lemma_tag.fillna('')
wiktionary_merkmal['num']           = wiktionary_merkmal.num.fillna('')
wiktionary_merkmal['merkmal_order'] = wiktionary_merkmal.merkmal_order.fillna(0)

In [281]:
#wiktionary_merkmal

In [282]:
# update_col aus wiktionary_lemma
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, wiktionary_lemma, on='lemma_id', col='lemma')
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, wiktionary_lemma, on='lemma_id', col='tag_0', col_rename='lemma_tag')
wiktionary_merkmal['lemma_tag'] =  wiktionary_merkmal.lemma_tag.fillna('')

update_col: 2584481 cells written into existing column
update_col: 2584481 cells written into existing column


In [283]:
# sicherstellen: lemma_tag ist gesetzt
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.lemma_tag == ''
mask  =  mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows


In [284]:
#pak.analyse_cols(wiktionary_merkmal)

### Leer oder ultrakurz

In [285]:
# Prüfen: Keine Leerstrings
spalten = ['merkmal','data','lemma','lemma_tag']
for s in spalten:
    print(s)
    mask = wiktionary_merkmal[s] == ''
    check_mask( wiktionary_merkmal, mask, 0)

merkmal
check_mask:  0 rows
data
check_mask:  0 rows
lemma
check_mask:  0 rows
lemma_tag
check_mask:  0 rows


In [286]:
# noSteig ermitteln und löschen
# data ist nur '-'
mask1 = wiktionary_merkmal.data.str.len() <= 1
mask2 = wiktionary_merkmal.lemma.str.len() > 1
mask3 = wiktionary_merkmal.merkmal.isin(['Komparativ','Superlativ'])
mask4 = wiktionary_merkmal.is_lex == True
mask  =  mask1  &  mask2  &  mask3  &  mask4 
noSteig = wiktionary_merkmal[mask].lemma_id # merken
check_mask( wiktionary_merkmal, mask, 16369)

msg = 'noSteig' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  16369 rows
Move 16369 rows from 2584481
16369 rows out of 43315


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
2052413,südbairisch,Komparativ,-_M,-,,ADJA,,,33.256210,True,noSteig,1.0,südbairisch
2052414,bezugnehmend,Komparativ,-_M,-,,ADJA,,,13.494154,True,noSteig,1.0,bezugnehmend
2052418,monosemantisch,Komparativ,-_M,-,,ADJA,,,14.983089,True,noSteig,1.0,monosemantisch
2052419,zweigleisig,Komparativ,-_M,-,,ADJA,,,46.364429,True,noSteig,1.0,zweigleisig
2052420,ungewünscht,Komparativ,-_M,-,,ADJA,,,23.117388,True,noSteig,1.0,ungewünscht
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080433,torffrei,Superlativ,-_M,-,,ADJA,,,157.889801,True,noSteig,1.0,torffrei
2080434,bayernweit,Superlativ,-_M,-,,ADJA,,,11.556718,True,noSteig,1.0,bayernweit
2080436,blunzen,Superlativ,-_M,-,,ADJA,,,22.609211,True,noSteig,1.0,blunzen
2080437,unpfändbar,Superlativ,-_M,-,,ADJA,,,13.287289,True,noSteig,1.0,unpfändbar


In [287]:
# noSteig in wiktionary_lemma kennzeichnen
mask1 =  wiktionary_lemma.lemma_id.isin(noSteig)
mask2 = ~wiktionary_lemma.member.str.contains('noSteig')
mask3 =  wiktionary_lemma.tagZ == 'ADJ'
mask  =  mask1  &  mask2  &  mask3
check_mask( wiktionary_lemma, mask, 154)

wiktionary_lemma.loc[mask,'member'] += ' noSteig' 
grid( wiktionary_lemma, mask )

check_mask:  154 rows
154 rows out of 168001


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score
1732,amtierend,105889,0,amtierend,amtierend,False,1,ADJA,ADJA,kF1YmJnHfT,ADJ,A,Ptz1 manuell noSteig,amtieren,188.885895
1780,ausnehmend,426867,0,ausnehmend,ausnehmend,False,1,ADJA,ADJA,XOAezqlnjW,ADJ,A,Ptz1 manuell noSteig,ausnehmen,45.377594
1888,dauernd,242274,0,dauernd,dauernd,False,1,ADJA,ADJA,CHxFtos1A0,ADJ,A,Ptz1 manuell noSteig,dauern,140.483337
2059,fünfte,119583,0,fünfte,fünfte,False,1,ADJA CARD,ADJA,CARD,ADJ,A,manuell ergänzt noSteig,,154.422852
2341,schwimmend,242155,0,schwimmend,schwimmend,False,1,ADJA,ADJA,KfU1dH54he,ADJ,A,Ptz1 manuell noSteig,schwimmen,268.757294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166201,zweiköpfig,538260,0,zweiköpfig,zweiköpfig,False,1,ADJA,ADJA,64kcwdjBAO,ADJ,A,noSteig,,55.199440
166268,zwiegespalten,417229,0,zwiegespalten,zwiegespalten,False,1,ADJA,ADJA,UWsMnjLiqS,ADJ,A,noSteig,,20.141333
167095,äußer_glex,681179,0,äußer,äußer,False,1,ADJA,ADJA,5XcSA1BwPB,ADJ,A,noSteig,,304.442383
167125,ökonometrisch,393779,0,ökonometrisch,ökonometrisch,False,1,ADJA,ADJA,zztBprrzuD,ADJ,A,noSteig,,51.162460


In [288]:
# Genus korrigieren
genus_m = ['Stromboli','Micky_Maus','Jesus_Christus','Jehova','Daniel_Düsentrieb','Dagobert_Duck','Tacken','Donald_Duck']

mask1 = wiktionary_merkmal.merkmal == 'Genus'
mask2 = wiktionary_merkmal.data == '0'
mask3 = wiktionary_merkmal.lemma_id.isin(genus_m)
mask  =  mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask,len(genus_m))

wiktionary_merkmal.loc[mask,'data'] = 'm'
wiktionary_merkmal[mask]

check_mask:  8 rows


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
1067625,Jesus_Christus,Genus,False,,m,,Jesus Christus,NE,,,1.0,47.556717
1098206,Stromboli,Genus,False,,m,,Stromboli,NE,,,1.0,82.534531
1098792,Tacken,Genus,False,,m,,Tacken,NN,,,1.0,93.550880
1111427,Jehova,Genus,False,,m,,Jehova,NE,,,1.0,139.055496
1114590,Daniel_Düsentrieb,Genus,False,,m,,Daniel Düsentrieb,NE,,,1.0,19.459846
1114641,Dagobert_Duck,Genus,False,,m,,Dagobert Duck,NE,,,1.0,40.256443
1114679,Donald_Duck,Genus,False,,m,,Donald Duck,NE,,,1.0,22.199026
1114713,Micky_Maus,Genus,False,,m,,Micky Maus,NE,,,1.0,44.799252


In [289]:
# Genus korrigieren
genus_f = ['Arminia','Trauerwoche']

mask1 = wiktionary_merkmal.merkmal == 'Genus'
mask2 = wiktionary_merkmal.data == '0'
mask3 = wiktionary_merkmal.lemma_id.isin(genus_f)
mask  =  mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask,len(genus_f))

wiktionary_merkmal.loc[mask,'data'] = 'f'
wiktionary_merkmal[mask]

check_mask:  2 rows


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
1080286,Trauerwoche,Genus,False,,f,,Trauerwoche,NN,,,1.0,30.284775
1115064,Arminia,Genus,False,,f,,Arminia,NE,,,1.0,22.124098


In [290]:
# Genus n falsch
mask1 =  wiktionary_merkmal.merkmal == 'Genus'
mask2 =  wiktionary_merkmal.data == 'n'
mask3 = ~wiktionary_merkmal.lemma_tag.isin(['NN','NE'])
mask  =  mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 0, 3)

msg = 'Genus n falsch' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


In [291]:
# Ungültige Genus löschen
mask1 =  wiktionary_merkmal.merkmal == 'Genus'
mask2 = ~wiktionary_merkmal.data.isin(['m','f','n'])
mask  =  mask1  &  mask2 
genus_ungültig = wiktionary_merkmal[mask].lemma_id # merken
check_mask( wiktionary_merkmal, mask, 725)

msg = 'Genus ungültig' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 

mask = wiktionary_merkmal_trash.msg == msg
pak.analyse_freqs( wiktionary_merkmal_trash[mask],['data','lemma'])

check_mask:  725 rows
Move 725 rows from 2568112


,data,data_count,data_percent,lemma,lemma_count
0,0,610,84.1,"[Pipox, Pipock, Reisekosten, Metadaten, Hauptleute, Nullerjahre, Landesfarben, Acta Apostolorum, Liparische Inseln, Acta Sanctorum, Acta Martyrum, Spompanadeln, Wirren, Svear, Freundschaftsbande, Mittelsleute, Feuerwehrleute, Fusilli, Kachin, Katschin]","[2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,x,98,13.5,"[Great Plains, Admiralitätsinseln, Aytac, Frana, Acar, Aiko, Akira, Alexis, Alfke, Alison, Alkan, Aydan, Azar, Balter, Bente, Berna, Billy, Bobbi, Bälter, Carol]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,m+n,13,1.8,"[Port Erin, Niger, Tschad, Südsudan, Sudan, Iran, Irak, Senegal, Kongo, Jemen, Libanon, Oman, Kosovo]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,f+n,2,0.3,"[Hinterbrühl, Antarktika]","[1, 1]"
4,o,1,0.1,Mixed Media,1
5,pl,1,0.1,Seschellen,1


In [292]:
# genus_ungültig in wiktionary_lemma betrachten
# Sind da Kandidaten für manuelle Löschung oder Genuss setzen dabei?
mask1 =  wiktionary_lemma.lemma_id.isin(genus_ungültig)
mask2 = ~wiktionary_lemma.member.str.contains('noSg')
mask3 = ~wiktionary_lemma.member.str.contains('noPl')
mask4 = ~wiktionary_lemma.lemma_id.str.endswith('_V')
mask5 = ~wiktionary_lemma.member.str.contains('Person_vorname')
mask6 = ~wiktionary_lemma.member.str.contains('Geo')
mask  =  mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6
check_mask( wiktionary_lemma, mask, 17)

spalten = ['lemma_id','tag','lemma_score','member']
a = wiktionary_lemma[mask][spalten].sort_values(['tag'])
grid(a)

check_mask:  17 rows
17 rows


,lemma_id,tag,lemma_score,member
5987,Allah,NE,429.435516,
81424,Sanssouci,NE,291.748779,
80669,SARS-CoV-2,NE,242.369843,
74228,Post_Covid,NE,124.849808,
74219,Post-Covid-19,NE,283.675659,
74218,Post-Covid,NE,875.175293,
68635,Obamacare,NE,54.890327,
74210,Post-COVID,NE,210.853485,
38335,Google,NE,476.271973,
19907,Christi_Himmelfahrt,NE,95.063065,Wortverbindung


In [293]:
# data ist nur '-'
mask1 =  wiktionary_merkmal.data == '-'
mask2 =  wiktionary_merkmal.lemma_id != '-_M'
#mask3 = ~wiktionary_merkmal.lemma_tag.isin(['VVFIN'])
mask = mask1  &  mask2 # &  mask3
check_mask( wiktionary_merkmal, mask, 1053)

msg = 'data ist nur -' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 

mask = wiktionary_merkmal_trash.msg == msg
pak.analyse_freqs(wiktionary_merkmal_trash[mask], ['merkmal','lemma_id'] ).head(7)

check_mask:  1053 rows
Move 1053 rows from 2567387


,merkmal,merkmal_count,merkmal_percent,lemma_id,lemma_id_count
0,Präsens ich,361,34.3,"[verfirnen, immanieren, schiefgehen, beknospen, versteppen, verfließen, fohlen, luften, aufbrisen, niederrinnen, verölen, kugelstoßen, abferkeln, rückwirken, plattern, stären, schwanen, rückziehen, maßschneidern, bauchreden]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,Präsens du,361,34.3,"[dahinströmen, abferkeln, aufschwellen_1, schmitzen, zwangsräumen, wahrhaben, überaltern, beknospen, einherfließen, dahinplätschern, urformen, abheilen, auflodern, hausschlachten, kunststopfen, aufflammen, preiskegeln, aufkeimen, ansamen, verjähren]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,Konj2 ich,92,8.7,"[rechtschreiben, vorfallen, wahrhaben, chronifizieren, vorbelasten, voranmelden, erstaufführen, unterversichern, zueinanderstehen, zueinanderdürfen, zueinanderkönnen, überbelegen, zwangsräumen, garnen_1, unterversorgen, drecken_1, drießen, größern, wärmebehandeln, bruchrechnen]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,Präteritum ich,92,8.7,"[drießen, wettschwimmen, drecken_1, erstaufführen, bauchreden, größern, angedeihen, wärmebehandeln, sippen, wettturnen, wettlaufen, wettsegeln, wettstreiten, wettrennen, rechtschreiben, grundsanieren, rückschreiben, strafversetzen, rückschreiten, rückrufen]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
4,Präsens ersiees,83,7.9,"[rückdenken, rückbringen, rückfahren, vorbelasten, rückfallen, voranmelden, wahrhaben, rücklaufen, rücklenken, rücklassen, rückhalten, rückgeben, milben, erstaufführen, tiefkühlen, umfragen, weißglühen, feuerverzinken, dienstverpflichten, bruchlanden]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
5,Ptz2,40,3.8,"[rückspringen, rücksinken, rückdenken, garnen_1, angedeihen, rückstoßen, lauben_1, größern, wahrhaben, interdependieren, gegenstreben, rückfahren, drießen, rückfallen, drecken_1, rücktreten, gegenfahren, rückgeben, rückhalten, rückrufen]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
6,Positiv,11,1.0,"[weltgrößte, viertgrößte, zweitgrößte, neuntgrößte, allerbeste, allerliebst, allerniedrigste, zweitkleinste, allerunglücklichste, allerheiligste, zweitbeste]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"


In [294]:
# superkurz >> alles bleibt wie es ist
mask1 =  wiktionary_merkmal.data.str.len() <= 1
mask2 =  wiktionary_merkmal.lemma.str.len() > 1   
mask3 = ~wiktionary_merkmal.lemma_tag.isin(['$(','$.','$,'])
mask  =  mask1  &  mask2  &  mask3     

#check_mask( wiktionary_merkmal, mask, 1292)
#wiktionary_merkmal, wiktionary_merkmal_trash = pak.drop_rows( wiktionary_merkmal, mask, wiktionary_merkmal_trash, msg='superkurz' ) 

pak.analyse_freqs(wiktionary_merkmal[mask],['merkmal','data','lemma_id','lemma_tag'])
#grid( wiktionary_merkmal, mask )

,merkmal,merkmal_count,merkmal_percent,data,data_count,lemma_id,lemma_id_count,lemma_tag,lemma_tag_count
0,Genus,100884,99.9,"[f, m, n]","[43299, 37373, 20212]","[Nudossi, Spam, Benelux, Queue, Nutella, Triangel, Werd, Gaokao, Quargel, Quandel, Ischias, Flipchart, Fruchtjoghurt, Zigarillo, Dingsbums, Halfter, Unbill, Butter, Gun, Jus]","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]","[NN, NE, CARD]","[95280, 5387, 217]"
1,rhymes,33,0.0,"[a, ɛ, ɔ]","[23, 5, 5]","[Ucha, Abasa, Isba, Yue, Sobriquet, voila, Niamey, Duschanbe, a_bas, Chardonnay, ha, Fauxpas, baba, so_4, aha, Aa_2, tja, na, Cha-Cha-Cha, Omerta]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[NN, ITJ, NE, ADV]","[17, 9, 6, 1]"
2,def,20,0.0,"[2, 5, 0, 6, 9, 7, 3, 1, 8, 4]","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[Zwei, Fünf, null_1, sechs, neun, Sieben, drei, eins, Null, Neun, Sechs, acht, fünf, Acht, Eins, zwei, Drei, Vier, vier, sieben]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[NN, CARD]","[10, 10]"
3,lex,10,0.0,"[3, 5, 1, 2, 0, 6, 7, 9, 8, 4]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[drei, fünf, eins, zwei, null_1, sechs, sieben, neun, acht, vier]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",CARD,10
4,ipa,2,0.0,ɛ,2,"[Ain, Ain_1]","[1, 1]",NE,2


### Allgemein

In [295]:
# Explizit gesetzte Tags werden Merkmale
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.data_tag != ''
mask3 = wiktionary_merkmal.data_tag != wiktionary_merkmal.lemma_tag
mask4 = ~((wiktionary_merkmal.data_tag == 'ADJA')  &   (wiktionary_merkmal.lemma_tag == 'VVFIN')) # Partizipien
mask5 = wiktionary_merkmal.merkmal == 'lex'
mask  = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(wiktionary_merkmal,mask,1,50)

# Merkmal erstellen
wiktionary_merkmal.loc[mask,'merkmal'] = wiktionary_merkmal[mask].data_tag

grid(wiktionary_merkmal,mask)
#a = pak.analyse_freqs(wiktionary_merkmal[mask], ['lemma_tag','data_tag'])
#grid(a)

check_mask:  8 rows
8 rows out of 2566334


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
975762,angucken_M,VVIZU,True,anzugucken,anzugucken,VVIZU,angucken,VVFIN,,manuell,1.0,2.239461
975763,herauslassen_M,VVIZU,True,herauslassen_M~herauszulassen,herauszulassen,VVIZU,herauslassen,VVFIN,,manuell,1.0,2.239461
975764,hereinlassen_M,VVIZU,True,hereinlassen_M~hereinzulassen,hereinzulassen,VVIZU,hereinlassen,VVFIN,,manuell,1.0,2.239461
975765,herumpimpeln_M,VVIZU,True,herumzupimpeln,herumzupimpeln,VVIZU,herumpimpeln,VVFIN,,manuell,1.0,2.239461
975766,loswerden,VVIZU,True,loszuwerden,loszuwerden,VVIZU,loswerden,VVFIN,,manuell,1.0,520.682129
975767,einreden,VVIZU,True,einzureden,einzureden,VVIZU,einreden,VVFIN,,manuell,1.0,77.859055
975768,anstaunen,VVIZU,True,anzustaunen,anzustaunen,VVIZU,anstaunen,VVFIN,,manuell,1.0,26.530531
975769,aufrütteln_M,VVIZU,True,aufzurütteln,aufzurütteln,VVIZU,aufrütteln,VVFIN,,manuell,1.0,2.239461


In [296]:
# manuelle Korrekturen
mask = wiktionary_merkmal.merkmal == 'genitiv Sg'
wiktionary_merkmal.loc[mask,'merkmal'] = 'Gen Sg'
mask = wiktionary_merkmal.merkmal ==         'Sg'
wiktionary_merkmal.loc[mask,'merkmal'] = 'Nom Sg'

In [297]:
# Überblick wiktionary_merkmal
a = pak.analyse_freqs(wiktionary_merkmal, ['merkmal', 'data'] )
gridt(a)

73 rows


In [298]:
# alle Merkmale sind in plan_merkmal erfasst
mask = ~a.merkmal.isin(plan_merkmal.merkmal)
check_mask(a,mask,0)

check_mask:  0 rows


In [299]:
#grid(a,mask, color='green')

In [300]:
# suche_debug wiktionary_merkmal
mask = wiktionary_merkmal.is_lex == True
search_str(wiktionary_merkmal[mask], suche_debug)

,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score


In [301]:
assert not pak.any_nan(wiktionary_merkmal)
#pak.nnan(wiktionary_merkmal)

In [302]:
#mask = wiktionary_merkmal.data_score.isnull()
#pak.sample(wiktionary_merkmal[mask])

### Wortarten -------

#### Partizipien 

In [303]:
# Alle Adjektive
mask = wiktionary_lemma.tagZZ == 'A'
adj = wiktionary_lemma[mask].lemma
#adj

In [304]:
# Alle Partizipien
mask = wiktionary_merkmal.merkmal == 'Ptz1'
ptz1 = wiktionary_merkmal[mask]
mask = wiktionary_merkmal.merkmal == 'Ptz2'
ptz2 = wiktionary_merkmal[mask].copy()
ptz_all = pak.add_rows(ptz1, ptz2)
#ptz_all

14213 rows added, now a total of 27848


In [305]:
# Alle Ptz1 sind auch als Adjektiv registriert
mask = ~ptz1.data.isin(adj)
check_mask(ptz1,mask,0)

check_mask:  0 rows


In [306]:
# Die meisten Ptz2 sind auch als Adjektiv registriert
mask = ~ptz2.data.isin(adj)
check_mask(ptz2,mask,0,120)
grid(ptz2,mask)

check_mask:  109 rows
109 rows out of 14213


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
949081,tun,Ptz2,True,getan,getan,,tun,VAFIN,,,1.0,1.053309
963952,nottun,Ptz2,True,notgetan,notgetan,,nottun,VVFIN,,,1.0,0.391496
965048,dartun,Ptz2,True,dargetan,dargetan,,dartun,VVFIN,,,1.0,0.711443
966125,abtun,Ptz2,True,abgetan,abgetan,,abtun,VVFIN,,,1.0,0.496868
966130,betun,Ptz2,True,betan,betan,,betun,VVFIN,,,1.0,0.536124
...,...,...,...,...,...,...,...,...,...,...,...,...
2023545,erschallen,Ptz2,True,erschollen,erschollen,,erschallen,VVFIN,,,2.0,144.076263
2023548,hängen_lassen,Ptz2,True,hängen_lassen,hängen lassen,,hängen lassen,VVFIN,,,2.0,27.317692
2023549,zurückwollen,Ptz2,True,zurückwollen,zurückwollen,,zurückwollen,VVFIN,,,2.0,195.421127
2023550,wegwollen,Ptz2,True,wegwollen,wegwollen,,wegwollen,VVFIN,,,2.0,167.690308


#### Verben

In [307]:
# Kontrolle: Es gibt keine Doppeltags in lemma_tag und data_tag
mask1 =  wiktionary_merkmal.is_lex    == True
mask2 =  wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3A = wiktionary_merkmal.lemma_tag.str.contains(' ')
mask3B = wiktionary_merkmal.data_tag.str.contains(' ')
mask  =  mask1  &  mask2  &  (mask3A | mask3B)
check_mask( wiktionary_merkmal, mask, 0)

check_mask:  0 rows


In [308]:
# Übersicht: Alle V-Tags in wiktionary_lemma
mask = wiktionary_lemma.tag.str.startswith('V')
a = pak.analyse_freqs(wiktionary_lemma[mask],['tag','lemma'])
grid(a)

3 rows


,tag,tag_count,tag_percent,lemma,lemma_count
0,VVFIN,13853,99.9,"[strippen, scheren, überhängen, erschrecken, docken, durchbrechen, eingeben, gichten, keilen, kollern, losen, löschen, nicken, ramschen, reihen, schlacken, tüdern, umhängen, durchfahren, durchlaufen]","[6, 5, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2]"
1,VMFIN,7,0.1,"[brauchen, dürfen, können, mögen, müssen, sollen, wollen]","[1, 1, 1, 1, 1, 1, 1]"
2,VAFIN,5,0.0,"[tun, haben, sein, werden]","[2, 1, 1, 1]"


In [309]:
# Übersicht: Alle V-Tags in wiktionary_merkmal
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask = mask1  &  mask2
a = pak.analyse_freqs(wiktionary_merkmal[mask],['data_tag','lemma_tag','data','lemma'])
a = pak.drop_cols(a,['data_tag_percent','data_count','lemma_count'])
grid(a)

6 rows


,data_tag,data_tag_count,lemma_tag,lemma_tag_count,data,lemma
0,,136559,"[VVFIN, VMFIN, VAFIN]","[136414, 75, 70]","[strippe, strippt, strippte, schere, schert, lost, scherte, erschreckt, löscht, verschleißt, vermisst, beschert, vertiert, konzertiert, befestigt, verführt, detoniert, graviert, laviert, gestrippt]","[strippen, scheren, erschrecken, anbellen, überhängen, löschen, nicken, umhängen, durchkämmen, keilen, reihen, unterschieben, docken, ramschen, tüdern, durchbrechen, umfassen, backen, losen, murmeln]"
1,ADJA,16480,VVFIN,16480,"[aufzuwärmende, aufzuwärmendes, aufzuwärmender, aufzuwärmenden, aufzuwärmendem, abzugießende, abzugießendes, abzugießender, abzugießenden, abzugießendem, nasszuschwitzende, nasszuschwitzendes, nasszuschwitzender, nasszuschwitzenden, nasszuschwitzendem, entzweizureißende, entzweizureißendes, entzweizureißender, entzweizureißenden, entzweizureißendem]","[aufwärmen, abgießen, nassschwitzen, entzweireißen, festnageln, weiterführen, runtersetzen, herausoperieren, abfliegen, abgrenzen, abholzen, weiterforschen, abisolieren, ablassen, abnabeln, abnutzen, abnützen, abringen, abschieben, absegeln]"
2,VVFIN,13853,VVFIN,13853,"[strippen, scheren, überhängen, erschrecken, docken, durchbrechen, eingeben, gichten, keilen, kollern, losen, löschen, nicken, ramschen, reihen, schlacken, tüdern, umhängen, durchfahren, durchlaufen]","[strippen, scheren, überhängen, erschrecken, docken, durchbrechen, eingeben, gichten, keilen, kollern, losen, löschen, nicken, ramschen, reihen, schlacken, tüdern, umhängen, durchfahren, durchlaufen]"
3,VVIZU,8,VVFIN,8,"[anzugucken, herauszulassen, hereinzulassen, herumzupimpeln, loszuwerden, einzureden, anzustaunen, aufzurütteln]","[angucken, herauslassen, hereinlassen, herumpimpeln, loswerden, einreden, anstaunen, aufrütteln]"
4,VMFIN,7,VMFIN,7,"[brauchen, dürfen, können, mögen, müssen, sollen, wollen]","[brauchen, dürfen, können, mögen, müssen, sollen, wollen]"
5,VAFIN,5,VAFIN,5,"[tun, haben, sein, werden]","[tun, haben, sein, werden]"


In [310]:
# Infinitiv
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 = wiktionary_merkmal.merkmal == 'lemma'
mask  = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask, 13888)

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal.lemma_tag.str[0:2] + 'INF'
grid(wiktionary_merkmal,mask)

check_mask:  13865 rows
13865 rows out of 2566334


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
2417840,Schlange_stehen,lemma,True,Schlange_stehen,Schlange stehen,VVINF,Schlange stehen,VVFIN,,,0.0,18.997892
2418170,abbeißen,lemma,True,abbeißen,abbeißen,VVINF,abbeißen,VVFIN,,,0.0,279.654022
2418171,abberufen,lemma,True,abberufen,abberufen,VVINF,abberufen,VVFIN,,,0.0,233.622742
2418172,abfahren,lemma,True,abfahren,abfahren,VVINF,abfahren,VVFIN,,,0.0,2072.352051
2418174,abküssen,lemma,True,abküssen,abküssen,VVINF,abküssen,VVFIN,,,0.0,85.658691
...,...,...,...,...,...,...,...,...,...,...,...,...
2584453,überzählen,lemma,True,überzählen,überzählen,VVINF,überzählen,VVFIN,,,0.0,237.076874
2584457,überzüchten,lemma,True,überzüchten,überzüchten,VVINF,überzüchten,VVFIN,,,0.0,57.731117
2584466,übrigbleiben,lemma,True,übrigbleiben,übrigbleiben,VVINF,übrigbleiben,VVFIN,,,0.0,168.506989
2584472,übrighaben,lemma,True,übrighaben,übrighaben,VVINF,übrighaben,VVFIN,,,0.0,78.089584


In [311]:
# Imperativ
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 = wiktionary_merkmal.merkmal.str.contains('Imp')
mask  = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask, 37471)

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal.lemma_tag.str[0:2] + 'IMP'
grid(wiktionary_merkmal,mask)

check_mask:  37471 rows
37471 rows out of 2566334


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
1570141,abbleiben,Imp Sg,True,bleib_ab,bleib ab,VVIMP,abbleiben,VVFIN,,,1.0,95.814873
1570142,hintermauern,Imp Sg,True,hintermauere,hintermauere,VVIMP,hintermauern,VVFIN,,,1.0,30.213717
1570143,beiseiteschaffen,Imp Sg,True,schaff_beiseite,schaff beiseite,VVIMP,beiseiteschaffen,VVFIN,,,1.0,176.383514
1570144,wiederschreiben,Imp Sg,True,schreibe_wieder,schreibe wieder,VVIMP,wiederschreiben,VVFIN,,,1.0,50.379482
1570145,vorwegentscheiden,Imp Sg,True,entscheid_vorweg,entscheid vorweg,VVIMP,vorwegentscheiden,VVFIN,,,1.0,105.478287
...,...,...,...,...,...,...,...,...,...,...,...,...
1952099,gehen,Imp Pl,True,gehet,gehet,VVIMP,gehen,VVFIN,,,2.0,1871.558228
1952100,stierln,Imp Pl,True,stierlet,stierlet,VVIMP,stierln,VVFIN,,,2.0,58.932964
1952101,bräunen,Imp Pl,True,bräunet,bräunet,VVIMP,bräunen,VVFIN,,,2.0,42.459415
1952102,nachvollziehen,Imp Pl,True,vollziehet_nach,vollziehet nach,VVIMP,nachvollziehen,VVFIN,,,2.0,42.160534


In [312]:
# Imperativ: VMIMP gibt es gar nicht
mask = wiktionary_merkmal.data_tag == 'VMIMP'
check_mask(wiktionary_merkmal,mask, 3)
wiktionary_merkmal.loc[mask,'data_tag'] = 'VVIMP'
#grid(wiktionary_merkmal,mask)

check_mask:  3 rows


In [313]:
# Partizip2
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 = wiktionary_merkmal.merkmal == 'Ptz2' 
mask  = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask,14214)

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal.lemma_tag.str[0:2] + 'PP'
grid(wiktionary_merkmal,mask)

check_mask:  14213 rows
14213 rows out of 2566334


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
948801,dezidieren,Ptz2,True,dezidiert_2,dezidiert,VVPP,dezidieren,VVFIN,,,1.0,0.057074
948802,dedizieren,Ptz2,True,dediziert_1,dediziert,VVPP,dedizieren,VVFIN,,,1.0,0.056685
948803,verrücken,Ptz2,True,verrückt_2,verrückt,VVPP,verrücken,VVFIN,,,1.0,0.041750
948804,designieren,Ptz2,True,designiert_1,designiert,VVPP,designieren,VVFIN,,,1.0,0.058751
948805,verfluchen,Ptz2,True,verflucht_2,verflucht,VVPP,verfluchen,VVFIN,,,1.0,0.039684
...,...,...,...,...,...,...,...,...,...,...,...,...
2023838,missgestalten,Ptz2,True,mißgestaltet_MADJ,mißgestaltet,VVPP,missgestalten,VVFIN,,manuell,2.0,35.198692
2023839,missraten,Ptz2,True,mißraten_MADJ,mißraten,VVPP,missraten,VVFIN,,manuell,2.0,25.746058
2023840,misslingen,Ptz2,True,mißlungen_MADJ,mißlungen,VVPP,misslingen,VVFIN,,manuell,2.0,61.479610
2023841,instandsetzen,Ptz2,True,instandgesetzt,instandgesetzt,VVPP,instandsetzen,VVFIN,,manuell,1.0,2.239461


In [314]:
# Partizip1 
mask1 =  wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 = wiktionary_merkmal.merkmal == 'Ptz1'
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 13635) 

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal[mask].lemma_tag
grid(wiktionary_merkmal,mask)

check_mask:  13635 rows
13635 rows out of 2566334


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
948806,bedeuten,Ptz1,True,bedeutend_1,bedeutend,VVFIN,bedeuten,VVFIN,,,1.0,0.059268
948807,belasten,Ptz1,True,belastend_1,belastend,VVFIN,belasten,VVFIN,,,1.0,0.070373
948811,nachtragen,Ptz1,True,nachtragend_1,nachtragend,VVFIN,nachtragen,VVFIN,,,1.0,0.062625
948813,rezidivieren,Ptz1,True,rezidivierend_1,rezidivierend,VVFIN,rezidivieren,VVFIN,,,1.0,0.064949
948816,retardieren,Ptz1,True,retardierend_1,retardierend,VVFIN,retardieren,VVFIN,,,1.0,0.075280
...,...,...,...,...,...,...,...,...,...,...,...,...
2023803,entgegenkommen,Ptz1,True,entgegenkommend,entgegenkommend,VVFIN,entgegenkommen,VVFIN,,manuell,2.0,53.561970
2023804,ionisieren,Ptz1,True,ionisierend,ionisierend,VVFIN,ionisieren,VVFIN,,manuell,2.0,116.584099
2023805,intermittieren,Ptz1,True,intermittierend,intermittierend,VVFIN,intermittieren,VVFIN,,manuell,2.0,46.668831
2023806,irreführen,Ptz1,True,irreführend,irreführend,VVFIN,irreführen,VVFIN,,manuell,2.0,34.594280


In [315]:
# (1) Gerundivum: Schon fertig
mask = wiktionary_merkmal.merkmal == 'Gerundivum'
grid(wiktionary_merkmal,mask)

16480 rows out of 2566334


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
2400002,aufwärmen,Gerundivum,True,aufzuwärmende,aufzuwärmende,ADJA,aufwärmen,VVFIN,,,1.0,1.813820
2400003,aufwärmen,Gerundivum,True,aufzuwärmendes,aufzuwärmendes,ADJA,aufwärmen,VVFIN,,,2.0,1.133051
2400004,aufwärmen,Gerundivum,True,aufzuwärmender,aufzuwärmender,ADJA,aufwärmen,VVFIN,,,3.0,1.254846
2400005,aufwärmen,Gerundivum,True,aufzuwärmenden,aufzuwärmenden,ADJA,aufwärmen,VVFIN,,,4.0,2.143308
2400006,aufwärmen,Gerundivum,True,aufzuwärmendem,aufzuwärmendem,ADJA,aufwärmen,VVFIN,,,5.0,0.804719
...,...,...,...,...,...,...,...,...,...,...,...,...
2416477,entgegenwerfen,Gerundivum,True,entgegenzuwerfende,entgegenzuwerfende,ADJA,entgegenwerfen,VVFIN,,,1.0,1.839462
2416478,entgegenwerfen,Gerundivum,True,entgegenzuwerfendem,entgegenzuwerfendem,ADJA,entgegenwerfen,VVFIN,,,2.0,0.846041
2416479,entgegenwerfen,Gerundivum,True,entgegenzuwerfenden,entgegenzuwerfenden,ADJA,entgegenwerfen,VVFIN,,,3.0,2.233051
2416480,entgegenwerfen,Gerundivum,True,entgegenzuwerfender,entgegenzuwerfender,ADJA,entgegenwerfen,VVFIN,,,4.0,1.299718


In [316]:
# Adjektivformen der Verben: ADJA ergänzen 
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.merkmal.isin(['Ptz1','Ptz2'])
mask3 = ~wiktionary_merkmal.data_tag.str.contains('ADJA')
mask  = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal, mask, 27849)

adjaformen = wiktionary_merkmal[mask].copy()
adjaformen['data_tag'] = 'ADJA'
adjaformen['merkmal_order'] += 10

wiktionary_merkmal = pak.add_rows(wiktionary_merkmal,adjaformen)
grid(adjaformen)

check_mask:  27848 rows
27848 rows added, now a total of 2594182
27848 rows


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
948801,dezidieren,Ptz2,True,dezidiert_2,dezidiert,ADJA,dezidieren,VVFIN,,,11.0,0.057074
948802,dedizieren,Ptz2,True,dediziert_1,dediziert,ADJA,dedizieren,VVFIN,,,11.0,0.056685
948803,verrücken,Ptz2,True,verrückt_2,verrückt,ADJA,verrücken,VVFIN,,,11.0,0.041750
948804,designieren,Ptz2,True,designiert_1,designiert,ADJA,designieren,VVFIN,,,11.0,0.058751
948805,verfluchen,Ptz2,True,verflucht_2,verflucht,ADJA,verfluchen,VVFIN,,,11.0,0.039684
...,...,...,...,...,...,...,...,...,...,...,...,...
2023838,missgestalten,Ptz2,True,mißgestaltet_MADJ,mißgestaltet,ADJA,missgestalten,VVFIN,,manuell,12.0,35.198692
2023839,missraten,Ptz2,True,mißraten_MADJ,mißraten,ADJA,missraten,VVFIN,,manuell,12.0,25.746058
2023840,misslingen,Ptz2,True,mißlungen_MADJ,mißlungen,ADJA,misslingen,VVFIN,,manuell,12.0,61.479610
2023841,instandsetzen,Ptz2,True,instandgesetzt,instandgesetzt,ADJA,instandsetzen,VVFIN,,manuell,11.0,2.239461


In [317]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


In [318]:
# Übersicht: Partizipien und Gerundivum
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.merkmal.isin(['Ptz1','Ptz2','Gerundivum'])
mask  = mask1  &  mask2
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['merkmal','data_tag','data'] )
a

,merkmal,merkmal_count,merkmal_percent,data_tag,data_tag_count,data,data_count
0,Ptz2,28426,39.4,"[ADJA, VVPP, VMPP, VAPP]","[14213, 14199, 8, 6]","[gestrippt, geschert, erschrocken, gewahrt, gekeilt, gelost, genickt, eingegeben, gekollert, gereiht, geschnoben, gedockt, gegichtet, geschlackt, geramscht, getüdert, verwunden, gedacht, geraten, gehört]","[12, 8, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 4, 4, 4]"
1,Ptz1,27270,37.8,"[ADJA, VVFIN, VMFIN, VAFIN]","[13635, 13624, 7, 4]","[bedeutend, belastend, nachtragend, rezidivierend, retardierend, reizend, alternierend, quälend, gravierend, veraltend, irisierend, abwertend, dringend, anstrengend, überbordend, wütend, vermögend, oszillierend, beißend, heranwachsend]","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]"
2,Gerundivum,16480,22.8,ADJA,16480,"[aufzuwärmende, aufzuwärmendes, aufzuwärmender, aufzuwärmenden, aufzuwärmendem, abzugießende, abzugießendes, abzugießender, abzugießenden, abzugießendem, nasszuschwitzende, nasszuschwitzendes, nasszuschwitzender, nasszuschwitzenden, nasszuschwitzendem, entzweizureißende, entzweizureißendes, entzweizureißender, entzweizureißenden, entzweizureißendem]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"


In [319]:
# Alle wiktionary_merkmal.data die aussehen wie Erweiterte Infinitive sind merkmal VVIZU 
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 =  wiktionary_merkmal.merkmal != 'lemma'
mask4 =  wiktionary_merkmal.data.str.replace('zu','') == wiktionary_merkmal.lemma.str.replace('zu','') # erst wenn alle ZU entfernt werden ist es wieder gleich
mask5 =  wiktionary_merkmal.data.str.len() == wiktionary_merkmal.lemma.str.len() + 2
mask6 = ~wiktionary_merkmal.data.str.startswith('zu')
mask7 =  wiktionary_merkmal.merkmal != 'VVIZU'
mask  =  mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6  &  mask7
check_mask(wiktionary_merkmal,mask, 0)
#grid(wiktionary_merkmal,mask)

check_mask:  0 rows


In [320]:
# Alle merkmal VVIZU sehen aus wie Erweiterte Infinitive
mask1  = wiktionary_merkmal.merkmal == 'VVIZU'
mask2N =  wiktionary_merkmal.data.str.replace('zu','') == wiktionary_merkmal.lemma.str.replace('zu','') # erst wenn alle ZU entfernt werden ist es wieder gleich
mask3N =  wiktionary_merkmal.data.str.len() == wiktionary_merkmal.lemma.str.len() + 2
mask  =  mask1  &  ~(mask2N  &  mask3N)  
check_mask(wiktionary_merkmal,mask, 0)
#grid(wiktionary_merkmal,mask)

check_mask:  0 rows


In [321]:
# translate_lex suche_debug
#grid(search_str(translate_lex, suche_debug))

In [322]:
# TODO_VVIZU
# VVIZU in translate_lex
mask1 = translate_lex.lemma_tag.str.startswith('V')
mask2 = translate_lex.data.str.replace('zu','') == translate_lex.lemma.str.replace('zu','') # erst wenn alle ZU entfernt werden ist es wieder gleich
mask3 = translate_lex.data.str.len() == translate_lex.lemma.str.len() + 2
mask  =  mask1  &  mask2  &  mask3  
check_mask(translate_lex,mask, 5715)

vvizuformen = translate_lex[mask].copy()

check_mask:  5715 rows


In [323]:
vvizuformen

,data_id,data,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower
4659,aufzustocken,aufzustocken,,0.538190,wiki Erweiterter_Infinitiv Verb,aufstocken,aufstocken,VVFIN,V,536.487732,aufstocken
7460,auszusprechen,auszusprechen,,0.478273,wiki Erweiterter_Infinitiv Verb,aussprechen,aussprechen,VVFIN,V,418.778046,aussprechen
7461,aufzukleben,aufzukleben,,0.461744,wiki Erweiterter_Infinitiv Verb,aufkleben,aufkleben,VVFIN,V,101.949570,aufkleben
7462,aufzukommen,aufzukommen,,0.544388,wiki Erweiterter_Infinitiv Verb,aufkommen,aufkommen,VVFIN,V,422.478607,aufkommen
7463,aufzukreuzen,aufzukreuzen,,0.486537,wiki Erweiterter_Infinitiv Verb,aufkreuzen,aufkreuzen,VVFIN,V,160.325668,aufkreuzen
...,...,...,...,...,...,...,...,...,...,...,...
667021,vollzusaugen,vollzusaugen,,0.439017,wiki Erweiterter_Infinitiv Verb,vollsaugen,vollsaugen,VVFIN,V,22.681517,vollsaugen
667062,entgegenzuwerfen,entgegenzuwerfen,,0.478273,wiki Erweiterter_Infinitiv Verb,entgegenwerfen,entgegenwerfen,VVFIN,V,475.184418,entgegenwerfen
667186,beieinanderzuhocken,beieinanderzuhocken,,0.507198,wiki Erweiterter_Infinitiv Verb,beieinanderhocken,beieinanderhocken,VVFIN,V,315.757355,beieinanderhocken
667234,einzusargen,einzusargen,,0.432818,wiki Erweiterter_Infinitiv Verb,einsargen,einsargen,VVFIN,V,86.188210,einsargen


In [324]:
# vvizuformen anpassen 
vvizuformen = pak.drop_cols(vvizuformen,['lemma_tagZZ','lemma_lower','lemma_score','data_tagZZ'])
vvizuformen = pak.rename_col(vvizuformen,'member',    'meta'     )
#vvizuformen = pak.rename_col(vvizuformen,'data_score', 'score'     )
vvizuformen['merkmal']       = 'VVIZU'
vvizuformen['is_lex']        = True
vvizuformen['data_tag']      = 'VVIZU'
vvizuformen['num']           = ''
vvizuformen['merkmal_order'] = 0

# Spaltenvergleich
bpy.minivenn( vvizuformen.columns, wiktionary_merkmal.columns, format='print2')

left_only:  {}
right_only: {}



In [325]:
#mask = wiktionary_merkmal.merkmal == 'VVIZU'
#wiktionary_merkmal[mask]

In [326]:
# vvizuformen an wiktionary_merkmal anfügen
wiktionary_merkmal = pak.add_rows(wiktionary_merkmal, vvizuformen, only_new=['lemma_id','lemma','data_id','data'])

9 rows not attached
5706 rows added, now a total of 2599888


In [327]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


In [328]:
# Kontrolle: Keine anderen Infinitiv-like
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 =  wiktionary_merkmal.data == wiktionary_merkmal.lemma
mask4 = ~wiktionary_merkmal.merkmal.isin( ['lemma','Ptz2'] )
mask  = mask1  &  mask2  &  mask3  &  mask4

check_mask( wiktionary_merkmal, mask, 0)
#wiktionary_merkmal, wiktionary_merkmal_trash = pak.drop_rows( wiktionary_merkmal, mask, wiktionary_merkmal_trash, msg='Infinitiv-like' ) 
#grid(wiktionary_merkmal,mask)

check_mask:  0 rows


In [329]:
# Kontrolle: Es gibt keine Doppeltags in lemma_tag und data_tag
mask1 =  wiktionary_merkmal.is_lex    == True
mask2 =  wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3A = wiktionary_merkmal.lemma_tag.str.contains(' ')
mask3B = wiktionary_merkmal.data_tag.str.contains(' ')
mask  =  mask1  &  mask2  &  (mask3A | mask3B)
check_mask( wiktionary_merkmal, mask, 0)

check_mask:  0 rows


In [330]:
# data_tag setzen (auf lemma_tag)
mask1 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask2 = wiktionary_merkmal.is_lex    == True
mask3 = wiktionary_merkmal.data_tag  == ''
mask  = mask1  &  mask2  &  mask3

#grid(wiktionary_merkmal,mask)
wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal[mask].lemma_tag

In [331]:
# Übersicht: Alle V-Tags in wiktionary_merkmal
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask = mask1  &  mask2
a = pak.analyse_freqs(wiktionary_merkmal[mask],['data_tag','lemma_tag','data','lemma'])
a = pak.drop_cols(a,['data_tag_percent','data_count','lemma_count'])
grid(a)

13 rows


,data_tag,data_tag_count,lemma_tag,lemma_tag_count,data,lemma
0,VVFIN,84761,VVFIN,84761,"[strippte, scherte, strippst, lost, strippt, strippe, gichtete, dockte, schlackte, nickte, ramschte, kollerte, tüderte, loste, keilte, scherst, schert, schere, reihte, öst]","[strippen, scheren, erschrecken, anbellen, löschen, überhängen, nicken, reihen, losen, durchbrechen, eingeben, kollern, keilen, unterschieben, durchkämmen, umhängen, docken, gichten, ramschen, tüdern]"
1,ADJA,44328,"[VVFIN, VMFIN, VAFIN]","[44303, 15, 10]","[gestrippt, geschert, erschrocken, gewahrt, gekeilt, gelost, genickt, eingegeben, gekollert, gereiht, geschnoben, gedockt, gegichtet, geschlackt, geramscht, getüdert, bedeutend, belastend, nachtragend, rezidivierend]","[überhängen, umfassen, umhängen, durchschlagen, durchkämmen, durchbrechen, anreihen, zusammenhängen, durchwühlen, durchpflügen, umspannen, umbauen, überstreichen, anhängen, untergraben, überwerfen, unterschieben, durchjagen, durchnagen, überholen]"
2,VVIMP,37457,"[VVFIN, VMFIN]","[37454, 3]","[strippe, strippt, schere, schert, keil, koller, tüder, ramsch, gib ein, dock, gichte, reih, lose, nick, docke, tüdere, ramsche, nicke, keile, tüdert]","[strippen, scheren, keilen, unterschieben, anbellen, tüdern, ramschen, umhängen, docken, durchkämmen, nicken, murmeln, erschrecken, sporteln, anwandeln, umspannen, durchschnüffeln, aufschrecken, löschen, einwiegen]"
3,VVPP,14199,VVFIN,14199,"[gestrippt, geschert, erschrocken, gewahrt, gekeilt, gelost, genickt, eingegeben, gekollert, gereiht, geschnoben, gedockt, gegichtet, geschlackt, geramscht, getüdert, verwunden, gedacht, geraten, gehört]","[überhängen, scheren, strippen, erschrecken, reihen, umhängen, löschen, bescheren, umfassen, backen, durchbrechen, heißen, bleichen, durchkämmen, anreihen, raten, untersetzen, schaffen, wiederholen, umwandeln]"
4,VVINF,13853,VVFIN,13853,"[strippen, scheren, überhängen, erschrecken, docken, durchbrechen, eingeben, gichten, keilen, kollern, losen, löschen, nicken, ramschen, reihen, schlacken, tüdern, umhängen, durchfahren, durchlaufen]","[strippen, scheren, überhängen, erschrecken, docken, durchbrechen, eingeben, gichten, keilen, kollern, losen, löschen, nicken, ramschen, reihen, schlacken, tüdern, umhängen, durchfahren, durchlaufen]"
5,VVIZU,5714,VVFIN,5714,"[auszupowern, anzugucken, herauszulassen, hereinzulassen, herumzupimpeln, loszuwerden, einzureden, anzustaunen, aufzurütteln, aufzustocken, auszusprechen, aufzukleben, aufzukommen, aufzukreuzen, aufzuladen, aufzulassen, aufzulegen, abzudanken, abzudecken, abzudichten]","[auspowern, angucken, herauslassen, hereinlassen, herumpimpeln, loswerden, einreden, anstaunen, aufrütteln, aufstocken, aussprechen, aufkleben, aufkommen, aufkreuzen, aufladen, auflassen, auflegen, abdanken, abdecken, abdichten]"
6,VMFIN,64,VMFIN,64,"[will, muss, darf, mag, soll, kann, wollte, sollte, brauchte, könnend, wollend, brauchend, müssend, mögend, dürfend, sollend, darfs, durft, wills, solls]","[müssen, können, brauchen, dürfen, wollen, mögen, sollen]"
7,VAFIN,50,VAFIN,50,"[tust, tut, tue, tu, tat, täte, seyn, syn, frühneuhochdeutsch, ham, thun, seiend, habend, werdend, tuend, werds, bins, biste, seis, wars]","[sein, tun, haben, werden]"
8,VAIMP,14,VAFIN,14,"[tue, tu, tut, sei, werde, habe, hab, werd, habt, werdet, seid]","[tun, werden, haben, sein]"
9,VMPP,8,VMFIN,8,"[gewollt, gebraucht, gesollt, gemocht, gekonnt, gemusst, gedurft, wollen]","[wollen, brauchen, sollen, mögen, können, müssen, dürfen]"


#### Nomen

In [332]:
# Übersicht über alle explizit gesetzten tags (Tagquelle: lexeme_manuell)
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('N')
mask3 = wiktionary_merkmal.data_tag != ''
mask  = mask1  &  mask2  &  mask3
#grid(wiktionary_merkmal,mask)
a = pak.analyse_freqs(wiktionary_merkmal[mask], ['lemma_tag','data_tag'])
(a)

,lemma_tag,lemma_tag_count,lemma_tag_percent,data_tag,data_tag_count
0,NN,96708,88.9,NN,96708
1,NE,12118,11.1,NE,12118


In [333]:
# Keine Doppeltags
mask1 =  wiktionary_merkmal.is_lex    == True
mask2 =  wiktionary_merkmal.lemma_tag.str.startswith('N')
mask3 =  wiktionary_merkmal.lemma_tag.str.contains(' ')
mask  =  mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 0)

check_mask:  0 rows


In [334]:
# Irgendwelche unbekannten tags?
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.lemma_tag.str.startswith('N')
mask3 = ~wiktionary_merkmal.data_tag.isin( ['NN','NE',''] )
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 0)
#grid(wiktionary_merkmal,mask)

check_mask:  0 rows


In [335]:
# data_tag setzen (auf lemma_tag)
mask1 = wiktionary_merkmal.lemma_tag.str.startswith('N')
mask2 = wiktionary_merkmal.is_lex    == True
mask3 = wiktionary_merkmal.data_tag  == ''
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 761247)

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal[mask].lemma_tag
#grid(wiktionary_merkmal,mask)

check_mask:  807811 rows


In [336]:
# Nomen analysieren
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('N')
mask  = mask1  &  mask2  
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['merkmal','data_tag'] )
grid(a)

27 rows


,merkmal,merkmal_count,merkmal_percent,data_tag,data_tag_count
0,Gen Sg,119275,13.0,"[NN, NE]","[112557, 6718]"
1,Dat Sg,112400,12.3,"[NN, NE]","[109538, 2862]"
2,lemma,108826,11.9,"[NN, NE]","[96708, 12118]"
3,Nom Sg,99640,10.9,"[NN, NE]","[92935, 6705]"
4,Akk Sg,95642,10.4,"[NN, NE]","[92962, 2680]"
5,Nom Pl,86255,9.4,"[NN, NE]","[84231, 2024]"
6,Akk Pl,86246,9.4,"[NN, NE]","[84226, 2020]"
7,Gen Pl,86246,9.4,"[NN, NE]","[84223, 2023]"
8,Dat Pl,85481,9.3,"[NN, NE]","[83502, 1979]"
9,alt_2,8518,0.9,"[NN, NE]","[7398, 1120]"


#### Rest

In [337]:
# data_tag setzen (auf lemma_tag)
mask1 = wiktionary_merkmal.is_lex    == True
mask2 = wiktionary_merkmal.data_tag  == ''
mask3 = ~wiktionary_merkmal.lemma_tag.str[0].isin(['V','N'])
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 31447)

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal[mask].lemma_tag
#a = pak.analyse_freqs( wiktionary_merkmal[mask], ['lemma_tag','data_tag'] )
#grid(a)

check_mask:  31520 rows


#### zweittag

In [338]:
spalten = ['lemma_id','tag_1']
mask = wiktionary_lemma.tag.str.contains(' ')
check_mask( wiktionary_lemma, mask, 179)

zweitags = wiktionary_lemma[mask][spalten]

check_mask:  175 rows


In [339]:
# Merkmale anfügen mit zweittag
mask1 = wiktionary_merkmal.lemma_id.isin(zweitags.lemma_id)
mask2 = wiktionary_merkmal.lemma_tag == wiktionary_merkmal.data_tag
mask = mask1  &  mask2
check_mask( wiktionary_merkmal, mask, 606)

zweitlexeme = wiktionary_merkmal[mask].copy()
zweitlexeme = pak.update_col(zweitlexeme, zweitags, on='lemma_id', col='tag_1', col_rename='data_tag')
zweitlexeme.meta = (zweitlexeme.meta + ' zweittag').str.strip()
wiktionary_merkmal = pak.add_rows(wiktionary_merkmal,zweitlexeme)
grid(zweitlexeme)

check_mask:  624 rows
update_col: 624 cells written into existing column
624 rows added, now a total of 2600512
624 rows


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
88989,viel,alt_2,True,Viel,Viel,PIAT,viel,ADJA,,zweittag,1.0,430.210968
148687,dich,alt_2,True,Dich,Dich,PRF,dich,PPER,,zweittag,1.0,13.250154
168868,aberhundert,alt_1,True,aberhundert~aberhunderte,aberhunderte,CARD,aberhundert,ADJA,,zweittag,1.0,28.599972
168869,aberhundert,alt_2,True,aberhundert~Aberhundert,Aberhundert,CARD,aberhundert,ADJA,,zweittag,1.0,28.599972
169695,beide,alt,True,beide~beyde,beyde,CARD,beide,PIAT,,zweittag,1.0,100.831711
...,...,...,...,...,...,...,...,...,...,...,...,...
2564586,zweiundzwanzigste,lemma,True,zweiundzwanzigste,zweiundzwanzigste,CARD,zweiundzwanzigste,ADJA,,zweittag,0.0,175.004791
2564648,zwote,lemma,True,zwote,zwote,CARD,zwote,ADJA,,zweittag,0.0,102.335976
2564654,zwölffach,lemma,True,zwölffach,zwölffach,CARD,zwölffach,ADJA,,zweittag,0.0,66.035995
2564669,zwölfte,lemma,True,zwölfte,zwölfte,CARD,zwölfte,ADJA,,zweittag,0.0,401.481995


In [340]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


#### Adjektive

In [341]:
# Merkmal Positiv ist sinnlos
mask1 = wiktionary_merkmal.merkmal == 'Positiv'
mask2 = wiktionary_merkmal.data != wiktionary_merkmal.lemma
mask = mask1  &  mask2
check_mask( wiktionary_merkmal, mask, 0,10)
wiktionary_merkmal[mask]

check_mask:  1 rows


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
2046369,Wiesbadner,Positiv,True,Wiesbadener,Wiesbadener,ADJA,Wiesbadner,ADJA,,,1.0,12.086063


In [342]:
# Merkmal Positiv löschen
mask = wiktionary_merkmal.merkmal == 'Positiv'
check_mask( wiktionary_merkmal, mask, 14036)

msg = 'Merkmal Positiv ist sinnlos' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  14036 rows
Move 14036 rows from 2600512
14036 rows out of 59129


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
2036711,gesüßt,Positiv,gesüßt,gesüßt,ADJA,ADJA,,,32.092140,True,Merkmal Positiv ist sinnlos,1.0,gesüßt
2036712,passager,Positiv,passager,passager,ADJA,ADJA,,,19.041552,True,Merkmal Positiv ist sinnlos,1.0,passager
2036713,gesellig,Positiv,gesellig,gesellig,ADJA,ADJA,,,48.439751,True,Merkmal Positiv ist sinnlos,1.0,gesellig
2036714,liebeskrank,Positiv,liebeskrank,liebeskrank,ADJA,ADJA,,,25.672915,True,Merkmal Positiv ist sinnlos,1.0,liebeskrank
2036715,körnerfressend,Positiv,körnerfressend,körnerfressend,ADJA,ADJA,,,31.704203,True,Merkmal Positiv ist sinnlos,1.0,körnerfressend
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2600316,keinerlei,Positiv,keinerlei,keinerlei,CARD,ADJA,,zweittag,27.678524,True,Merkmal Positiv ist sinnlos,1.0,keinerlei
2600317,faktisch,Positiv,faktisch,faktisch,ADJA,ADV,,zweittag,73.586159,True,Merkmal Positiv ist sinnlos,1.0,faktisch
2600318,zwote,Positiv,zwote,zwote,CARD,ADJA,,zweittag,34.111992,True,Merkmal Positiv ist sinnlos,1.0,zwote
2600319,hundertundzweite,Positiv,hundertundzweite,hundertundzweite,CARD,ADJA,,zweittag,56.782925,True,Merkmal Positiv ist sinnlos,1.0,hundertundzweite


In [343]:
# Ist kein Adjektiv
mask1 =  wiktionary_merkmal.data_tag.isin(['ADJA','ADJD'])
mask2 = ~wiktionary_merkmal.merkmal.isin(['lemma','Ptz1','Ptz2','Gerundivum','Positiv','Komparativ','Superlativ','alt','alt_1','alt_2','lexAlt','lex','alt_abk','klein','weib','männ'])
mask3 =  wiktionary_merkmal.is_lex == True
mask  = mask1  &  mask2  &  mask3   
check_mask( wiktionary_merkmal, mask, 20)

msg = 'Ist kein Adjektiv' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  16 rows
Move 16 rows from 2586476
16 rows out of 59145


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
2599987,sämtlich,Nom Sg f,sämtliche,sämtliche,ADJA,PIAT,,zweittag,85.837029,True,Ist kein Adjektiv,1.0,sämtlich
2599992,sämtlich,Nom Sg m,sämtlicher,sämtlicher,ADJA,PIAT,,zweittag,85.837029,True,Ist kein Adjektiv,1.0,sämtlich
2600007,sämtlich,Nom Sg n,sämtliches,sämtliches,ADJA,PIAT,,zweittag,85.837029,True,Ist kein Adjektiv,1.0,sämtlich
2600030,sämtlich,Gen Sg f,sämtlicher,sämtlicher,ADJA,PIAT,,zweittag,85.837029,True,Ist kein Adjektiv,1.0,sämtlich
2600039,sämtlich,Gen Sg m,sämtliches,sämtliches,ADJA,PIAT,,zweittag,85.837029,True,Ist kein Adjektiv,1.0,sämtlich
2600060,sämtlich,Gen Sg n,sämtliches,sämtliches,ADJA,PIAT,,zweittag,85.837029,True,Ist kein Adjektiv,1.0,sämtlich
2600075,sämtlich,Dat Sg f,sämtlicher,sämtlicher,ADJA,PIAT,,zweittag,85.837029,True,Ist kein Adjektiv,1.0,sämtlich
2600086,sämtlich,Dat Sg m,sämtlichem,sämtlichem,ADJA,PIAT,,zweittag,85.837029,True,Ist kein Adjektiv,1.0,sämtlich
2600094,sämtlich,Dat Sg n,sämtlichem,sämtlichem,ADJA,PIAT,,zweittag,85.837029,True,Ist kein Adjektiv,1.0,sämtlich
2600112,sämtlich,Akk Sg f,sämtliche,sämtliche,ADJA,PIAT,,zweittag,85.837029,True,Ist kein Adjektiv,1.0,sämtlich


In [344]:
# Adjektiv-only Merkmale
mask1 = ~wiktionary_merkmal.data_tag.isin(['ADJA','ADJD','ADV','PIAT'])
mask2 =  wiktionary_merkmal.merkmal.isin(['Gerundivum','Komparativ','Superlativ'])
mask3 =  wiktionary_merkmal.is_lex == True
mask  = mask1  &  mask2  &  mask3   
check_mask( wiktionary_merkmal, mask, 0)

#a = wiktionary_merkmal[mask].sort_values(['data_tag','lemma_id'])
#grid(a)

msg = 'Adjektiv-only Merkmale' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


In [345]:
# Anschauen: Steigerbare Adverbien (TODO_Steigerbare Adverbien)
mask1 =  wiktionary_merkmal.data_tag == 'ADV'
mask2 =  wiktionary_merkmal.merkmal.isin(['Komparativ','Superlativ'])
mask3 =  wiktionary_merkmal.is_lex == True
mask4 = ~wiktionary_merkmal.lemma.isin(['sehr','gern','gerne','oft', 'bald','wohl'])
    
mask  = mask1  &  mask2  &  mask3  &  mask4 
check_mask( wiktionary_merkmal, mask, )

a = wiktionary_merkmal[mask].sort_values(['merkmal','data_tag','lemma_id'])
grid(a)

# 'sehr', 'mehr','am meisten'
# 'gern', 'lieber','am liebsten'
# 'bald', 'eher','am ehesten'
## 'oft',  'öfter','am öftesten'  # drin
## 'wohl', 'besser','am besten'   # drin

check_mask:  16 rows
16 rows


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
2052887,gewiss,Komparativ,True,gewisser,gewisser,ADV,gewiss,ADV,,,1.0,109.957718
2055003,hoch_1,Komparativ,True,höher,höher,ADV,hoch,ADV,,,1.0,4.868667
2056384,inwendig,Komparativ,True,inwendiger,inwendiger,ADV,inwendig,ADV,,,1.0,24.664410
2053050,lange,Komparativ,True,länger,länger,ADV,lange,ADV,,,1.0,45.577663
2052077,schal,Komparativ,True,schaler,schaler,ADV,schal,ADV,,,1.0,41.949757
2055740,unumstößlich,Komparativ,True,unumstößlicher,unumstößlicher,ADV,unumstößlich,ADV,,,1.0,35.523693
2055268,wahrlich,Komparativ,True,wahrlicher,wahrlicher,ADV,wahrlich,ADV,,,1.0,103.879677
2052037,äbich,Komparativ,True,äbichor,äbichor,ADV,äbich,ADV,,,1.0,37.883972
2062134,gewiss,Superlativ,True,gewissesten,gewissesten,ADV,gewiss,ADV,,,1.0,109.957718
2059749,hoch_1,Superlativ,True,höchsten,höchsten,ADV,hoch,ADV,,,1.0,4.868667


In [346]:
# Die gleichen lemmas als Adjektive
mask1 = wiktionary_merkmal.lemma.isin(a.lemma)
mask2 = wiktionary_merkmal.lemma_tag == 'ADJA'
mask3 =  wiktionary_merkmal.merkmal.isin(['Komparativ','Superlativ'])
mask = mask1  &  mask2  &  mask3
wiktionary_merkmal[mask]

,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
2055000,hoch,Komparativ,True,höher,höher,ADJA,hoch,ADJA,,,1.0,1241.171753
2059580,hoch,Superlativ,True,höchsten,höchsten,ADJA,hoch,ADJA,,,1.0,1241.171753


#### analysieren

In [347]:
# Merkmale analysieren
mask1 = wiktionary_merkmal.is_lex == True
mask  = mask1   
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['merkmal','data_tag'] )
grid(a)

53 rows


,merkmal,merkmal_count,merkmal_percent,data_tag,data_tag_count
0,lemma,168175,14.2,"[NN, ADJA, VVINF, NE, ADV, KOMBI, CARD, ITJ, APPR, FM, PIAT, PTKANT, KOUS, KON, DIV, PDAT, PPER, ART, $(, APPO]","[96708, 41958, 13853, 12121, 1599, 445, 419, 297, 182, 103, 71, 67, 63, 53, 38, 36, 32, 16, 15, 13]"
1,Gen Sg,119298,10.1,"[NN, NE, PIAT, PWAT, PRF, PRELAT, PTKANT, PRO]","[112557, 6723, 7, 3, 3, 2, 2, 1]"
2,Dat Sg,112417,9.5,"[NN, NE, PIAT, PTKANT, PRELAT, PWAT, PRF]","[109538, 2867, 7, 2, 1, 1, 1]"
3,Nom Sg,99655,8.4,"[NN, NE, PIAT, PWAT, PTKANT, PRELAT, PRO]","[92935, 6709, 5, 2, 2, 1, 1]"
4,Akk Sg,95660,8.1,"[NN, NE, PIAT, PWAT, PTKANT, PRF, PRELAT, PRO]","[92962, 2684, 7, 2, 2, 1, 1, 1]"
5,Nom Pl,86322,7.3,"[NN, NE, PDAT, PIAT, ART, PPOSS, PRELAT, CARD, PRELS, PWAT, PPER, PRO, PTKANT, PDS]","[84231, 2025, 19, 15, 10, 6, 4, 3, 2, 2, 2, 1, 1, 1]"
6,Gen Pl,86316,7.3,"[NN, NE, PDAT, PIAT, ART, PPOSS, PRELAT, CARD, PRF, PPER, PWAT, PRELS, PTKANT, PDS, PRO]","[84223, 2024, 19, 15, 10, 6, 4, 3, 3, 2, 2, 2, 1, 1, 1]"
7,Akk Pl,86314,7.3,"[NN, NE, PDAT, PIAT, ART, PPOSS, PRELAT, CARD, PPER, PRELS, PWAT, PRF, PDS, PTKANT, PRO]","[84226, 2021, 19, 15, 10, 6, 4, 3, 2, 2, 2, 1, 1, 1, 1]"
8,Dat Pl,85552,7.2,"[NN, NE, PDAT, PIAT, ART, PPOSS, PRELAT, CARD, PWAT, PRELS, PPER, PRF, PRO, PTKANT, PDS]","[83502, 1980, 20, 16, 11, 6, 4, 3, 2, 2, 2, 1, 1, 1, 1]"
9,Ptz2,28426,2.4,"[ADJA, VVPP, VMPP, VAPP]","[14213, 14199, 8, 6]"


In [348]:
# Keine Verwaisten lemma_id
mask = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask( wiktionary_merkmal, mask, 0)
#wiktionary_merkmal, wiktionary_merkmal_trash = pak.drop_rows( wiktionary_merkmal, mask, wiktionary_merkmal_trash, msg='verwaist' ) 
#grid( wiktionary_merkmal, mask)

check_mask:  0 rows


### Felder -------

#### data

In [349]:
# Prüfen: Keine mehrfache Spaces
mask = wiktionary_merkmal.data.str.contains('  ')
check_mask(wiktionary_merkmal,mask,0)
wiktionary_merkmal['data'] = pak.replace_str( wiktionary_merkmal.data, {'  ':' '})

check_mask:  0 rows


In [350]:
# Prüfen: nicht leer
mask = wiktionary_merkmal.data.str.len() == 0
check_mask( wiktionary_merkmal, mask, 0)
#wiktionary_merkmal, wiktionary_merkmal_trash = pak.drop_rows( wiktionary_merkmal, mask, wiktionary_merkmal_trash, msg='leer3' ) 

check_mask:  0 rows


In [351]:
# '⇀' ist nur in syllables
mask1 =  wiktionary_merkmal.data.str.contains('⇀')
mask2 = ~wiktionary_merkmal.merkmal.isin(['syllables'])
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows


In [352]:
# Apostroph anschauen
mask =  wiktionary_merkmal.data.str.contains("'")
check_mask(wiktionary_merkmal,mask,32)
grid(wiktionary_merkmal,mask)

check_mask:  32 rows
32 rows out of 2586460


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
48524,Maat_1,alt_1,True,Maat_1~Ma'at,Ma'at,NE,Maat,NE,,,1.0,2.849264
93974,ein,alt_abk,True,ein~'nen,'nen,ART,ein,ART,,,1.0,80.404617
204159,Scharia,alt_2,True,Scharia~Schari'a,Schari'a,NN,Scharia,NN,,,1.0,61.271812
495871,Micmac,alt_2,True,Micmac~Mi'kmaq,Mi'kmaq,NN,Micmac,NN,,,1.0,35.002872
553379,ausm,alt_2,True,ausm~aus'm,aus'm,DIV,ausm,DIV,,,1.0,9.454154
560630,Cabernet,down,False,Cabernet_d'Anjou,Cabernet d'Anjou,,Cabernet,NN,2,,6.0,47.284927
584537,Faradaykäfig,alt_2,True,Faradaykäfig~Faraday'sche Käfig,Faraday'sche Käfig,NN,Faradaykäfig,NN,,,2.0,76.690163
668560,Fraunhoferlinie,alt_1,True,Fraunhoferlinie~Fraunhofer'sche Linie,Fraunhofer'sche Linie,NN,Fraunhoferlinie,NN,,,1.0,42.633888
1219352,Cabernet_d'Anjou,Nom Sg,True,Cabernet_d'Anjou,Cabernet d'Anjou,NN,Cabernet d'Anjou,NN,,,1.0,12.235624
1248362,Linus,Gen Sg,True,Linus~Linus',Linus',NE,Linus,NE,,,1.0,42.800297


In [353]:
# Belöschen: Komma_in_data
mask1 =  wiktionary_merkmal.data.str.contains(",")
mask2 = ~wiktionary_merkmal.lemma.str.contains(",")
mask3 =  wiktionary_merkmal.data.str.endswith(('säure','ol','in','an'))
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask,16)
#grid(wiktionary_merkmal,mask)

msg = 'Komma_in_data' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  16 rows
Move 16 rows from 2586460
16 rows out of 59161


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
165730,Abietinsäure,syn,"Abieta-7,14-dien-19-carbonsäure","Abieta-7,14-dien-19-carbonsäure",,NN,1,,55.998070,False,Komma_in_data,1.0,Abietinsäure
217823,moin,alt_1,"moin~Moin, Moin","Moin, Moin",PTKANT,PTKANT,,,58.166126,True,Komma_in_data,2.0,moin
232398,Aconitsäure,syn,"1,2,3-Propentricarbonsäure","1,2,3-Propentricarbonsäure",,NN,1,,44.564732,False,Komma_in_data,1.0,Aconitsäure
299601,Tetramethylenglycol,def,"1,4-Butandiol","1,4-Butandiol",,NN,1,Chemie,12.677977,False,Komma_in_data,1.0,Tetramethylenglycol
315394,Weinsäure,syn,"2,3-Dihydroxybutandisäure","2,3-Dihydroxybutandisäure",,NN,1,,55.846737,False,Komma_in_data,1.0,Weinsäure
315396,Weinsäure,syn,"2,3-Dihydroxybernsteinsäure","2,3-Dihydroxybernsteinsäure",,NN,1,,55.846737,False,Komma_in_data,3.0,Weinsäure
315405,Terephthalsäure,syn,"1,4-Benzoldicarbonsäure","1,4-Benzoldicarbonsäure",,NN,1,,35.536873,False,Komma_in_data,1.0,Terephthalsäure
315449,Pikrinsäure,syn,"2,4,6-Trinitrophenol","2,4,6-Trinitrophenol",,NN,1,,48.590359,False,Komma_in_data,1.0,Pikrinsäure
315454,Phthalsäure,syn,"1,2-Benzoldicarbonsäure","1,2-Benzoldicarbonsäure",,NN,1,,45.471306,False,Komma_in_data,1.0,Phthalsäure
315514,Gluconsäure,syn,"2,3,4,5,6-Pentahydroxyhexansäure","2,3,4,5,6-Pentahydroxyhexansäure",,NN,1,,31.847654,False,Komma_in_data,1.0,Gluconsäure


In [354]:
# Restliche Kommas anschauen
mask1 =  wiktionary_merkmal.data.str.contains(",")
mask2 = ~wiktionary_merkmal.lemma.str.contains(",")
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,19)
grid(wiktionary_merkmal,mask)

check_mask:  19 rows
19 rows out of 2586444


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
34627,Flasche,down,False,"0,2-Liter-Flasche","0,2-Liter-Flasche",,Flasche,NN,1,Fassungsvermögen,46.0,1516.371216
34628,Flasche,down,False,"0,33-Liter-Flasche","0,33-Liter-Flasche",,Flasche,NN,1,Fassungsvermögen,47.0,1516.371216
34629,Flasche,down,False,"0,5-Liter-Flasche","0,5-Liter-Flasche",,Flasche,NN,1,Fassungsvermögen,48.0,1516.371216
34630,Flasche,down,False,"0,7-Liter-Flasche","0,7-Liter-Flasche",,Flasche,NN,1,Fassungsvermögen,49.0,1516.371216
34631,Flasche,down,False,"0,75-Liter-Flasche","0,75-Liter-Flasche",,Flasche,NN,1,Fassungsvermögen,50.0,1516.371216
51328,paffen,familie,False,"piff,_paff","piff, paff",,paffen,VVFIN,,,1.0,71.216179
149329,Ziel,down,False,"1,5-Grad-Ziel","1,5-Grad-Ziel",,Ziel,NN,1,,1.0,609.851990
303933,ruckzuck,alt_2,True,"ruck,_zuck","ruck, zuck",ADV,ruckzuck,ADV,,,1.0,58.453896
402355,predigen,kombi,False,"landauf,_landab","landauf, landab",,predigen,VVFIN,2,figurativ,29.0,228.334396
415663,bundesweit,lateral,False,"landauf,_landab","landauf, landab",,bundesweit,ADJA,1,,2.0,115.302048


#### num

In [355]:
# num ist bei den meisten Lexemen sinnlos
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.num.str.len() > 0
#mask3 = wiktionary_merkmal.merkmal.str.startswith('alt') 
mask  =  mask1  &  mask2  #&  mask3

check_mask( wiktionary_merkmal, mask, 19930)
wiktionary_merkmal.loc[mask,'num'] = ''   
#grid(wiktionary_merkmal,mask, color='green')

check_mask:  19939 rows


In [356]:
#pak.analyse_freqs(wiktionary_merkmal[mask], ['num','data',])

#### meta

In [357]:
# remove_words
d = [ '(', ')', 'auch','nicht','bis','Adjektive','Adjektiv','Kombination','Substantive','Substantiv',
      'allgemein','Adverbien','Verb','Adverbien','Adverb','andere','über','Erweiterter_Infinitiv','wiki',
    ]       
wiktionary_merkmal['meta'] = pak.remove_words( wiktionary_merkmal.meta, d )

In [358]:
# replace_str
r = {       
    'seltener':'selten',
    'Schweiz':'schweizerisch',    
    'Österreich':'österreichisch', 
    'Süddeutschland':'süddeutsch',    
    'bairisch':'bayrisch',        
    'bairisch':'bayrisch',       
     
    }
wiktionary_merkmal['meta'] = pak.replace_str( wiktionary_merkmal.meta, r )

In [359]:
# leeren: meta == lemma 
mask = wiktionary_merkmal.lemma == wiktionary_merkmal.meta
check_mask(wiktionary_merkmal,mask,61000)

wiktionary_merkmal.loc[mask,'meta'] = ''
#wiktionary_merkmal[mask]
#pak.analyse_freqs(wiktionary_merkmal[mask],['merkmal'])

check_mask:  61033 rows


In [360]:
# meta ist typisch für ein einzelnes lemma
mask = wiktionary_merkmal.meta.str.len() > 1

a = pak.analyse_freqs(wiktionary_merkmal[mask],['meta','lemma'])
mask = a.lemma.str.isalpha().notnull() # meta ist typisch für ein einzelnes lemma
löschmich = list(a[mask].meta)

# meta belöschen
mask = wiktionary_merkmal.is_lex == True
wiktionary_merkmal.loc[mask,'meta'] = pak.remove_words( wiktionary_merkmal[mask].meta, löschmich )

In [361]:
# meta Überblick Lexeme
mask = wiktionary_merkmal.is_lex == True
a = pak.analyse_freqs(wiktionary_merkmal[mask], ['meta','merkmal','lemma_id'], splits=[' ', None])
a = pak.drop_cols(a,['lemma_id_count'])
gridt(a)

119 rows


In [362]:
# Seltene meta löschen
mask = a.meta_count < 20
löschmich = list(a[mask].meta)
#löschmich

# meta belöschen
mask = wiktionary_merkmal.is_lex == True
wiktionary_merkmal.loc[mask,'meta'] = pak.remove_words( wiktionary_merkmal[mask].meta, löschmich )

In [363]:
# meta Überblick Nichtlexeme
mask = wiktionary_merkmal.is_lex == False
a = pak.analyse_freqs(wiktionary_merkmal[mask], ['meta','merkmal','lemma_id'], splits=[' ', None])
a = pak.drop_cols(a,['lemma_id_count'])
gridt(a)

8031 rows


In [364]:
# Seltene meta löschen
mask = a.meta_count < 20
löschmich = list(a[mask].meta)
#löschmich

# meta belöschen
mask = wiktionary_merkmal.is_lex == False
wiktionary_merkmal.loc[mask,'meta'] = pak.remove_words( wiktionary_merkmal[mask].meta, löschmich )

#### tag-Felder

In [365]:
# data_tagZ
wiktionary_merkmal = pak.update_col( wiktionary_merkmal, translate_tagZ, left_on='data_tag',  right_on='tag', col='tagZ', col_rename='data_tagZ')
wiktionary_merkmal = pak.update_col( wiktionary_merkmal, translate_tagZ, left_on='lemma_tag', right_on='tag', col='tagZ', col_rename='data_tagZ', cond='null')
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal,'data_tagZ','data_tag')

update_col: 1180505 cells written into new column
update_col: 1405939 cells written into existing column


In [366]:
# data_tagZZ
wiktionary_merkmal = pak.update_col( wiktionary_merkmal, translate_tagZ, left_on='data_tag',  right_on='tag', col='tagZZ', col_rename='data_tagZZ')
wiktionary_merkmal = pak.update_col( wiktionary_merkmal, translate_tagZ, left_on='lemma_tag', right_on='tag', col='tagZZ', col_rename='data_tagZZ', cond='null')
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal,'data_tagZZ','data_tagZ')

update_col: 1180505 cells written into new column
update_col: 1405939 cells written into existing column


In [367]:
# Wieder löschen wenn kein Lexem
mask = wiktionary_merkmal.is_lex != True
wiktionary_merkmal.loc[mask,'data_tagZ']  = ''
wiktionary_merkmal.loc[mask,'data_tagZZ'] = ''

In [368]:
# data_tagZ und data_tagZZ überall gesetzt
mask1  = wiktionary_merkmal.is_lex == True
mask2A = wiktionary_merkmal.data_tagZ  == ''
mask2B = wiktionary_merkmal.data_tagZZ == ''
mask = mask1  &  (mask2A | mask2B)
check_mask(wiktionary_merkmal,mask, 0)

check_mask:  0 rows


In [369]:
# alle verwendeten tags
mask = wiktionary_merkmal.is_lex == True
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['data_tag','merkmal'], splits=[' ', None] )
grid(a)

46 rows


,data_tag,data_tag_count,data_tag_percent,merkmal,merkmal_count
0,NN,868331,73.6,"[Gen Sg, Dat Sg, lemma, Akk Sg, Nom Sg, Nom Pl, Akk Pl, Gen Pl, Dat Pl, alt_2, weib, männ, alt_1, alt, klein, alt_abk, lexAlt, alt_3, alt_weib, alt_männ]","[112557, 109538, 96708, 92962, 92935, 84231, 84226, 84223, 83502, 7398, 6829, 5592, 3739, 2017, 1042, 352, 343, 56, 26, 25]"
1,ADJA,101689,8.6,"[lemma, Gerundivum, Ptz2, Ptz1, Superlativ, Komparativ, alt_2, alt_1, alt, lexAlt, lex, alt_abk, männ, klein, weib]","[41958, 16480, 14213, 13635, 5904, 5875, 1957, 1016, 499, 133, 10, 5, 2, 1, 1]"
2,VVFIN,84761,7.2,"[Präsens ich, Konj2 ich, Präteritum ich, Präsens ersiees, Präsens du, Ptz1, alt_2, alt_1, alt, lexAlt, alt_abk, lex, Konj1 ich]","[14423, 13983, 13909, 13792, 13627, 13624, 562, 448, 176, 157, 33, 26, 1]"
3,NE,48331,4.1,"[lemma, Gen Sg, Nom Sg, alt_3, Dat Sg, Akk Sg, Nom Pl, Gen Pl, Akk Pl, Dat Pl, alt_2, alt_klein, alt_abk, alt_männ, alt_weib, alt_1, klein, alt, Pl, Gen]","[12121, 6723, 6709, 5768, 2867, 2684, 2025, 2024, 2021, 1980, 1120, 661, 390, 349, 334, 259, 163, 75, 38, 6]"
4,VVIMP,37457,3.2,"[Imp Sg, Imp Pl]","[23924, 13533]"
5,VVPP,14199,1.2,Ptz2,14199
6,VVINF,13853,1.2,lemma,13853
7,VVIZU,5714,0.5,VVIZU,5714
8,ADV,2086,0.2,"[lemma, alt_1, alt_2, alt, Komparativ, Superlativ, lex, lexAlt, alt_abk]","[1599, 210, 208, 26, 14, 14, 8, 4, 3]"
9,CARD,823,0.1,"[lemma, lex, alt_2, alt_1, alt, Nom Sg f, Nom Sg m, Nom Sg n, Gen Sg f, Gen Sg m, Gen Sg n, Dat Sg f, Dat Sg m, Dat Sg n, Akk Sg f, Akk Sg m, Akk Sg n, Nom Pl, Gen Pl, Dat Pl]","[419, 238, 58, 35, 13, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3]"


In [370]:
# Gibt es unregistrierte tags?
alle_realen_tags = set(alle_tags) - {'AFIX','ZFIX','TODO','LEX_A','LEX_N', 'APPRART','MFIX','NNE','XY','_SP',}
seltsame_tags = set(a.data_tag) - alle_realen_tags
assert len(seltsame_tags) == 0

#mask = wiktionary_merkmal.tag.isin(seltsame_tags)
#wiktionary_merkmal[mask]

In [371]:
# studieren
mask1 = wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.data_tag == 'TRUNC'
mask  =  mask1  &  mask2

grid(wiktionary_merkmal,mask, color='green')

3 rows out of 2586444


,lemma_id,merkmal,is_lex,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,num,meta,merkmal_order,data_score
2400464,http,lemma,True,http,http,TRUNC,z,Z,http,TRUNC,,,0.0,19.930384
2400913,www,lemma,True,www,www,TRUNC,z,Z,www,TRUNC,,,0.0,7.598153
2535725,https_M,lemma,True,https_M,https,TRUNC,z,Z,https,TRUNC,,,0.0,6.718384


In [372]:
# Alle merkmale und ihre tags
mask = wiktionary_merkmal.is_lex == True
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['merkmal','data_tag'] )
grid(a)

53 rows


,merkmal,merkmal_count,merkmal_percent,data_tag,data_tag_count
0,lemma,168175,14.2,"[NN, ADJA, VVINF, NE, ADV, KOMBI, CARD, ITJ, APPR, FM, PIAT, PTKANT, KOUS, KON, DIV, PDAT, PPER, ART, $(, APPO]","[96708, 41958, 13853, 12121, 1599, 445, 419, 297, 182, 103, 71, 67, 63, 53, 38, 36, 32, 16, 15, 13]"
1,Gen Sg,119298,10.1,"[NN, NE, PIAT, PWAT, PRF, PRELAT, PTKANT, PRO]","[112557, 6723, 7, 3, 3, 2, 2, 1]"
2,Dat Sg,112417,9.5,"[NN, NE, PIAT, PTKANT, PRELAT, PWAT, PRF]","[109538, 2867, 7, 2, 1, 1, 1]"
3,Nom Sg,99655,8.4,"[NN, NE, PIAT, PWAT, PTKANT, PRELAT, PRO]","[92935, 6709, 5, 2, 2, 1, 1]"
4,Akk Sg,95660,8.1,"[NN, NE, PIAT, PWAT, PTKANT, PRF, PRELAT, PRO]","[92962, 2684, 7, 2, 2, 1, 1, 1]"
5,Nom Pl,86322,7.3,"[NN, NE, PDAT, PIAT, ART, PPOSS, PRELAT, CARD, PRELS, PWAT, PPER, PRO, PTKANT, PDS]","[84231, 2025, 19, 15, 10, 6, 4, 3, 2, 2, 2, 1, 1, 1]"
6,Gen Pl,86316,7.3,"[NN, NE, PDAT, PIAT, ART, PPOSS, PRELAT, CARD, PRF, PPER, PWAT, PRELS, PTKANT, PDS, PRO]","[84223, 2024, 19, 15, 10, 6, 4, 3, 3, 2, 2, 2, 1, 1, 1]"
7,Akk Pl,86314,7.3,"[NN, NE, PDAT, PIAT, ART, PPOSS, PRELAT, CARD, PPER, PRELS, PWAT, PRF, PDS, PTKANT, PRO]","[84226, 2021, 19, 15, 10, 6, 4, 3, 2, 2, 2, 1, 1, 1, 1]"
8,Dat Pl,85552,7.2,"[NN, NE, PDAT, PIAT, ART, PPOSS, PRELAT, CARD, PWAT, PRELS, PPER, PRF, PRO, PTKANT, PDS]","[83502, 1980, 20, 16, 11, 6, 4, 3, 2, 2, 2, 1, 1, 1, 1]"
9,Ptz2,28426,2.4,"[ADJA, VVPP, VMPP, VAPP]","[14213, 14199, 8, 6]"


In [373]:
# studieren
mask1 = wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.data_tag == 'ADV'
mask3 =  wiktionary_merkmal.merkmal == 'Superlativ'
mask  =  mask1  &  mask2  &  mask3

grid(wiktionary_merkmal,mask, color='green')

14 rows out of 2586444


,lemma_id,merkmal,is_lex,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,num,meta,merkmal_order,data_score
2057010,äbich,Superlativ,True,äbichsten,äbichsten,ADV,ADV,A,äbich,ADV,,,1.0,37.883972
2057387,oft,Superlativ,True,öftesten,öftesten,ADV,ADV,A,oft,ADV,,,1.0,200.598923
2058492,wahrlich,Superlativ,True,wahrlichsten,wahrlichsten,ADV,ADV,A,wahrlich,ADV,,,1.0,103.879677
2058589,wohl,Superlativ,True,wohlsten,wohlsten,ADV,ADV,A,wohl,ADV,,,1.0,141.056320
2058649,inwendig,Superlativ,True,inwendigsten,inwendigsten,ADV,ADV,A,inwendig,ADV,,,1.0,24.664410
2058803,unumstößlich,Superlativ,True,unumstößlichsten,unumstößlichsten,ADV,ADV,A,unumstößlich,ADV,,,1.0,35.523693
2059749,hoch_1,Superlativ,True,höchsten,höchsten,ADV,ADV,A,hoch,ADV,,,1.0,4.868667
2060474,gerne,Superlativ,True,liebsten,liebsten,ADV,ADV,A,gerne,ADV,,,1.0,108.105972
2061162,schal,Superlativ,True,schalsten,schalsten,ADV,ADV,A,schal,ADV,,,1.0,41.949757
2062134,gewiss,Superlativ,True,gewissesten,gewissesten,ADV,ADV,A,gewiss,ADV,,,1.0,109.957718


In [374]:
# lemma_tagZZ schreiben
wiktionary_merkmal = pak.update_col( wiktionary_merkmal, translate_tagZ, left_on='lemma_tag', right_on='tag', col='tagZZ', col_rename='lemma_tagZZ', verbose=False )
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal,'lemma_tagZZ','lemma_tag')
wiktionary_merkmal['lemma_tagZZ'] = wiktionary_merkmal.lemma_tagZZ.fillna('')

In [375]:
# check
mask1 = wiktionary_merkmal.lemma_tag != ''
mask2 = wiktionary_merkmal.lemma_tagZZ == ''
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows


In [376]:
mask = wiktionary_merkmal.is_lex == True
wiktionary_merkmal[mask].head(1)

,lemma_id,merkmal,is_lex,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,lemma_tagZZ,num,meta,merkmal_order,data_score
46,Subspezies,alt_2,True,Subspecies,Subspecies,NN,NOM,N,Subspezies,NN,N,,,1.0,46.296391


#### hunspell

In [377]:
# Hunspell instance
from hunspell import Hunspell
hun = Hunspell('de_DE', hunspell_data_dir='/usr/share/hunspell') 

In [378]:
# verify_hunspell
# liefert 1=verifiziert, 0=nicht verifiziert, -1=Besonderes Unicode-Zeichen
wiktionary_merkmal['hunspell'] = -2
mask = wiktionary_merkmal.is_lex == True
wiktionary_merkmal[mask] = wiktionary_merkmal[mask].swifter.apply(verify_hunspell, col_check='data', col_result='hunspell', hunspellinstance=hun, axis=1)
#wiktionary_merkmal = pak.change_datatype(wiktionary_merkmal, verbose=False)  

Pandas Apply:   0%|          | 0/1180505 [00:00<?, ?it/s]

In [379]:
# prüfen: alle erfasst?
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.hunspell < 0
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows


In [380]:
# abwerten
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.hunspell == 0
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,521518)
wiktionary_merkmal.loc[mask,'data_score'] *= 0.1
grid(wiktionary_merkmal,mask)

check_mask:  518039 rows
518039 rows out of 2586444


,lemma_id,merkmal,is_lex,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,lemma_tagZZ,num,meta,merkmal_order,data_score,hunspell
46,Subspezies,alt_2,True,Subspecies,Subspecies,NN,NOM,N,Subspezies,NN,N,,,1.0,4.629639,0
93,Dezember,alt,True,Dezember~December,December,NN,NOM,N,Dezember,NN,N,,,1.0,21.269392,0
248,Mai,alt_1,True,Maien,Maien,NN,NOM,N,Mai,NN,N,,veraltet,1.0,26.315842,0
334,August_1,alt_abk,True,August_1~Gust,Gust,NE,NAM,N,August,NE,N,,,1.0,0.925968,0
335,August_1,alt_klein,True,August_1~Gustel,Gustel,NE,NAM,N,August,NE,N,,,1.0,0.925968,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2600494,zehntausendste,lemma,True,zehntausendste,zehntausendste,CARD,X_zahl,X,zehntausendste,ADJA,A,,zweittag,0.0,11.541921,0
2600497,zig,lemma,True,zig,zig,ADJA,ADJ,A,zig,NN,N,,zweittag,0.0,10.829743,0
2600505,zweihundertste,lemma,True,zweihundertste,zweihundertste,CARD,X_zahl,X,zweihundertste,ADJA,A,,zweittag,0.0,14.701342,0
2600508,zwote,lemma,True,zwote,zwote,CARD,X_zahl,X,zwote,ADJA,A,,zweittag,0.0,10.233598,0


## translate_lex aufbauen ==========
* translate_lex enthält derzeit alle Wiktionary-Seitentitel, die keine Lemmas sind.
* Diverse Listen werden hier zusätzlich angefügt.
* Nach jeder Anfügung werden Dups entfernt.

In [381]:
pak.specials(translate_lex)

,data_id,data,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower
0,lieben_1,lieben,,0.311824,wiki Deklinierte_Form,lieb,lieb,ADJA,A,82.432930,lieb
1,Polen_1,Polen,,0.113761,wiki Deklinierte_Form,Pole,Pole,NN,N,473.835083,pole
823,gehest,gehest,,0.777860,wiki Konjugierte_Form Verb,gehen,gehen,VVFIN,V,5621.393066,gehen
1491,Häuser,Häuser,,0.812983,wiki Deklinierte_Form,Haus,Haus,NN,N,108960.914062,haus
38706,sinkenden,sinkenden,,2.690744,wiki Deklinierte_Form,sinkend,sinkend,ADJA,A,1.509198,sinkend
149655,fahr_Rad,fahr Rad,,0.585970,wiki Konjugierte_Form Verb,Rad_fahren,Rad fahren,VVFIN,V,512.580444,rad fahren
188180,überführte,überführte,,1.812539,wiki Deklinierte_Form,überführt_4,überführt,ADJA,A,0.032591,überführt
188314,α-Zerfällen,α-Zerfällen,,0.441083,wiki Deklinierte_Form,α-Zerfall,α-Zerfall,NN,N,345.928619,α-zerfall
238517,abzäunte,abzäunte,,1.175359,wiki Konjugierte_Form Verb,abzäunen,abzäunen,VVFIN,V,111.581848,abzäunen
370258,1._Petrusbriefs,1. Petrusbriefs,,0.682977,wiki Deklinierte_Form,1._Petrusbrief,1. Petrusbrief,NN,N,74.730980,1. petrusbrief


In [382]:
from x32_Funktionen import *
translate_lex = pflege_translate_tabelle(translate_lex, translate_tagZ, wiktionary_lemma, translate_lex )
#pak.sample(translate_lex)

rank: 0 rows less, now 667534 rows
update_col: 667534 cells written into existing column
update_col: 667445 cells written into existing column
update_col: 0 cells written into existing column
update_col: 0 cells written into existing column
update_col: 667445 cells written into new column
update_col: 667445 cells written into existing column


In [383]:
# translate_lex suche_debug
#grid(search_str(translate_lex, suche_debug))

In [384]:
# data_id vorhanden und eindeutig?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

assert translate_lex.data_id.is_unique

#a = pak.same_but_different(translate_lex,same='data_id',different='data')
#assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


### translate_abk

In [385]:
# Abkürzungen, die nicht als Lemma registriert sind
# >> abtrennen. Der Rest wird translate_lemma hinzugefügt.
mask = ~translate_abk.kurz_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_abk,mask,3031)

translate_abk, df = pak.move_rows(translate_abk, mask, msg=None)

check_mask:  3513 rows
Move 3513 rows from 5721


In [386]:
# Spalten anpassen
df = pak.rename_col(df, 'kurz_id',     'data_id')
df = pak.rename_col(df, 'kurz',        'data')
df = pak.rename_col(df, 'kurz_tag',    'data_tag')
df = pak.rename_col(df, 'kurz_score',  'data_score')
df = pak.rename_col(df, 'lang_id',     'lemma_id')
df = pak.rename_col(df, 'lang',        'lemma')
df = pak.rename_col(df, 'lang_tag',    'lemma_tag')
df = pak.rename_col(df, 'lang_score',  'lemma_score')

df['member'] = (df.kurz_member + ' ' + df.lang_member).str.strip() 
df = pak.move_cols(df,'member','data_tag')

df = pak.drop_cols(df, ['todo','rank','konkurrenz','kurz_member','lang_member'] )

In [387]:
df = pflege_translate_tabelle(df, translate_tagZ, wiktionary_lemma, translate_lex )
#pak.sample(df)

rank: 0 rows less, now 667534 rows
update_col: 0 cells written into existing column
update_col: 3510 cells written into existing column
update_col: 0 cells written into existing column
update_col: 0 cells written into new column
update_col: 3510 cells written into new column
update_col: 3510 cells written into existing column


In [388]:
# score fehlt überall
mask = df.data_score != 0
check_mask(df,mask,0)

check_mask:  0 rows


In [389]:
# lemma_score übernehmen
df['data_score'] = df.lemma_score.copy()

# abwerten, wenn nicht manuell
mask = ~df.member.str.contains('manuell')
df.loc[mask,'data_score'] *= 0.8

In [390]:
df

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower
0,Band~Bd,,Bd,NN,,41571.363281,manuell Abkürzung manuell,Band,L,Band,NN,N,41571.363281,band
1,Februar~Feb.,,Feb.,NN,,15263.912109,manuell Abkürzung manuell,Februar,L,Februar,NN,N,15263.912109,februar
2,Januar~Jan,,Jan,NN,,13696.132812,manuell Abkürzung manuell,Januar,L,Januar,NN,N,13696.132812,januar
3,Straße~Str,,Str,NN,,12764.211914,manuell Abkürzung manuell,Straße,L,Straße,NN,N,12764.211914,straße
4,Straße~Str.,,Str.,NN,,12764.211914,manuell Abkürzung manuell,Straße,L,Straße,NN,N,12764.211914,straße
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3508,per_Adresse~c/o,,c/o,KOMBI,,10.986852,Abkürzung Wortverbindung,per_Adresse,L,per Adresse,KOMBI,X,13.733564,per adresse
3509,ut_supra~u s,,u s,KOMBI,,4.921723,Abkürzung Wortverbindung,ut_supra,L,ut supra,KOMBI,X,6.152153,ut supra
3510,ut_supra~u. s.,,u. s.,KOMBI,,4.921723,Abkürzung Wortverbindung,ut_supra,L,ut supra,KOMBI,X,6.152153,ut supra
3511,ut_supra~u.s.,,u.s.,KOMBI,,4.921723,Abkürzung Wortverbindung,ut_supra,L,ut supra,KOMBI,X,6.152153,ut supra


In [391]:
# unterschiedliche Abkürzungen sind ohne Statistik ununterscheidbar #TODO_Statistik
mask = df.lemma == 'Krankenhaus'
grid(df,mask)

3 rows out of 3513


,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower
6,Krankenhaus~Krh,,Krh,NN,,41620.988281,Abkürzung,Krankenhaus,L,Krankenhaus,NN,N,52026.234375,krankenhaus
7,Krankenhaus~Krhs,,Krhs,NN,,41620.988281,Abkürzung,Krankenhaus,L,Krankenhaus,NN,N,52026.234375,krankenhaus
8,Krankenhaus~Krkhs,,Krkhs,NN,,41620.988281,Abkürzung,Krankenhaus,L,Krankenhaus,NN,N,52026.234375,krankenhaus


In [392]:
pak.specials(df)

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower
0,Band~Bd,,Bd,NN,,41571.363281,manuell Abkürzung manuell,Band,L,Band,NN,N,41571.363281,band
5,Wasser~H2O,,H2O,NN,,44848.921875,Abkürzung,Wasser,L,Wasser,NN,N,56061.152344,wasser
93,erkundigen~erk,,erk,VVFIN,,696.154114,Abkürzung,erkundigen,L,erkundigen,VVFIN,V,870.192627,erkundigen
237,Euro~€,,€,NN,,1403.598999,manuell Abkürzung Einheit manuell,Euro,L,Euro,NN,N,1403.598999,euro
267,Prozent~%,,%,NN,,1309.592773,manuell Abkürzung Einheit manuell,Prozent,L,Prozent,NN,N,1309.592773,prozent
338,täglich~tgl,,tgl,ADJA,,245.535614,manuell Abkürzung noSteig manuell,täglich,L,täglich,ADJA,A,245.535614,täglich
1649,üblich~übl,,übl,ADJA,,180.241989,Abkürzung,üblich,L,üblich,ADJA,A,225.302490,üblich
2904,Diplom-Volkswirt~Dipl Vw,,Dipl Vw,NN,,188.272827,Abkürzung,Diplom-Volkswirt,L,Diplom-Volkswirt,NN,N,235.341034,diplom-volkswirt
3135,üblicherweise~üblw,,üblw,ADV,,159.146286,Abkürzung,üblicherweise,L,üblicherweise,ADV,A,198.932861,üblicherweise
3258,-prozentig~-%ig,,-%ig,ZFIX,,11.886984,Abkürzung Gebundenes_Lexem,-prozentig,,-prozentig,<NA>,,14.858729,-prozentig


In [393]:
# Spaltenvergleich
bpy.minivenn( df.columns, translate_lex.columns, format='print2')

left_only:  {'data_tag'}
right_only: {}



In [394]:
# anfügen
translate_lex = pak.add_rows( translate_lex, df, only_new=['data', 'lemma'])

0 rows not attached
3513 rows added, now a total of 671047


In [395]:
# check score
maskA = translate_lex.data_score == 0
maskB = translate_lex.lemma_score == 0
mask = maskA | maskB
check_mask(translate_lex,mask,0)

check_mask:  0 rows


In [396]:
# data_tag
translate_lex = pak.move_cols(translate_lex,'data_tag','data')
translate_lex['data_tag'] = translate_lex.data_tag.fillna('')
translate_lex

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower
0,lieben_1,T,lieben,,,0.311824,wiki Deklinierte_Form,lieb,L,lieb,ADJA,A,82.432930,lieb
1,Polen_1,T,Polen,,,0.113761,wiki Deklinierte_Form,Pole,L,Pole,NN,N,473.835083,pole
2,Polen_2,T,Polen,,,0.037273,wiki Deklinierte_Form,Pol,L,Pol,NN,N,1914.818848,pol
3,lieb_1,T,lieb,,,0.170379,wiki Konjugierte_Form Verb,lieben,L,lieben,VVFIN,V,528.041321,lieben
4,Kriegen_1,T,Kriegen,,,0.079412,wiki Deklinierte_Form,Krieg,L,Krieg,NN,N,26028.195312,krieg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671042,per_Adresse~c/o,,c/o,KOMBI,,10.986852,Abkürzung Wortverbindung,per_Adresse,L,per Adresse,KOMBI,X,13.733564,per adresse
671043,ut_supra~u s,,u s,KOMBI,,4.921723,Abkürzung Wortverbindung,ut_supra,L,ut supra,KOMBI,X,6.152153,ut supra
671044,ut_supra~u. s.,,u. s.,KOMBI,,4.921723,Abkürzung Wortverbindung,ut_supra,L,ut supra,KOMBI,X,6.152153,ut supra
671045,ut_supra~u.s.,,u.s.,KOMBI,,4.921723,Abkürzung Wortverbindung,ut_supra,L,ut supra,KOMBI,X,6.152153,ut supra


### translate_gm
* gm-Merkmale sind die Merkmale auf den Wiktionary-Seiten, die keine Lemmas sind. Also diejenigen, die unter wiktionary_nolemma bzw. translate_lex gespeichert sind.
* Daher weisen ihre lemma_id nicht auf ein Lemma, sondern auf ein Lexem. lemma_id und data_id müssen also hier vertauscht werden.

In [397]:
# translate_lex 
# Dies sind die Seiten, deren Merkmale translate_gm enthält.
# data_id ist die ID der Nicht-Lemma-Wiktionary-Seite.
(search_str(translate_lex, ['besonne']))

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower
532797,besonne,T,besonne,,,1.644078,wiki Konjugierte_Form Verb,besonnen_2,L,besonnen,VVFIN,V,16.730038,besonnen


In [398]:
# lemma und data vertauschen
translate_gm = pak.rename_col(translate_gm, 'lemma_id',    'data_id_neu')
translate_gm = pak.rename_col(translate_gm, 'data_id',     'lemma_id')
translate_gm = pak.rename_col(translate_gm, 'data_id_neu', 'data_id')
translate_gm = pak.rename_col(translate_gm, 'data',        'lemma')
#translate_gm = pak.rename_col(translate_gm, 'score',       'data_score')

In [399]:
# lemma_id nachschlagen (und dabei korrigieren!)
translate_gm = pak.update_col(translate_gm, translate_lex, on='data_id', col='lemma_id' ) 

update_col: 667461 cells written into existing column


In [400]:
# translate_gm umformen
translate_gm = pak.rename_col(translate_gm, 'merkmal', 'member')
translate_gm['member'] = (translate_gm.member + ' ' + translate_gm.meta).str.strip()

translate_gm = pak.drop_cols(translate_gm, ['is_lex','data_tag','num','meta',])

In [401]:
# translate_gm
# data_id ist die ID der Nicht-Lemma-Wiktionary-Seite.
mask = translate_gm.lemma == 'besonnen'
a = translate_gm[mask].sort_values(['lemma_id','lemma'])
grid(a)

29 rows


,data_id,merkmal_order,member,lemma_id,lemma,lemma_tag,data_score
i,,,,,,,
716606,besonnenem,1,gm,besonnen,besonnen,,0.786124
716607,besonnenen,1,gm,besonnen,besonnen,,2.242026
716608,besonnener,1,gm,besonnen,besonnen,,1.233051
716609,besonnener_1,1,gm,besonnen,besonnen,,0.068565
716610,besonnenes,1,gm,besonnen,besonnen,,1.131769
716611,besonnene,1,gm,besonnen,besonnen,,1.790744
716612,besonnenerem,1,gm,besonnen,besonnen,,0.740669
716613,besonneneren,1,gm,besonnen,besonnen,,2.193308
716614,besonnenerer,1,gm,besonnen,besonnen,,1.242026


In [402]:
from x32_Funktionen import *
translate_gm = pflege_translate_tabelle(translate_gm, translate_tagZ, wiktionary_lemma, translate_lex )
#pak.sample(translate_gm)

rank: 0 rows less, now 671047 rows
update_col: 667461 cells written into new column
update_col: 694514 cells written into existing column
update_col: 27143 cells written into existing column
update_col: 27143 cells written into new column
update_col: 694514 cells written into new column
update_col: 694514 cells written into new column


In [403]:
# translate_gm
# data_id ist die ID der Nicht-Lemma-Wiktionary-Seite.
mask = translate_gm.lemma == 'besonnen'
a = translate_gm[mask].sort_values(['lemma_id','lemma'])
grid(a)

29 rows


,data_id,data_home,data,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,merkmal_order,lemma_lower
101266,besonnenem,T,besonnenem,,0.786124,gm,besonnen,L,besonnen,ADJA,A,105.049240,1,besonnen
101267,besonnenen,T,besonnenen,,2.242026,gm,besonnen,L,besonnen,ADJA,A,105.049240,1,besonnen
101268,besonnener,T,besonnener,,1.233051,gm,besonnen,L,besonnen,ADJA,A,105.049240,1,besonnen
101269,besonnener_1,T,besonnener,,0.068565,gm,besonnen,L,besonnen,ADJA,A,105.049240,1,besonnen
101270,besonnenes,T,besonnenes,,1.131769,gm,besonnen,L,besonnen,ADJA,A,105.049240,1,besonnen
101271,besonnene,T,besonnene,,1.790744,gm,besonnen,L,besonnen,ADJA,A,105.049240,1,besonnen
101272,besonnenerem,T,besonnenerem,,0.740669,gm,besonnen,L,besonnen,ADJA,A,105.049240,1,besonnen
101273,besonneneren,T,besonneneren,,2.193308,gm,besonnen,L,besonnen,ADJA,A,105.049240,1,besonnen
101274,besonnenerer,T,besonnenerer,,1.242026,gm,besonnen,L,besonnen,ADJA,A,105.049240,1,besonnen
101275,besonneneres,T,besonneneres,,1.111256,gm,besonnen,L,besonnen,ADJA,A,105.049240,1,besonnen


In [404]:
assert translate_gm.lemma_home.nunique() == 2 # stand mal auf 1
assert translate_gm.data_home.nunique() == 2

In [405]:
pak.analyse_freqs(translate_gm, ['data_home','lemma_home'])

,data_home,data_home_count,data_home_percent,lemma_home,lemma_home_count
0,T,667461,96.1,"[L, ]","[667372, 89]"
1,L,27143,3.9,"[L, ]","[27142, 1]"


In [406]:
# translate_lex_trash pflegen
translate_lex_trash = pak.drop_cols(translate_lex_trash,['tag','tag_0','tag_1','tagZ','tagZZ'])
translate_lex_trash = pak.move_cols( translate_lex_trash, translate_lex.columns)
#translate_lex_trash

In [407]:
pak.nnan(translate_gm)

lemma_tag      90
lemma_score    90
dtype: int64

In [408]:
mask = translate_gm.lemma_tag.isnull()
check_mask(translate_gm,mask,90)

msg = 'verwaist_gm'
translate_gm, translate_lex_trash = pak.move_rows(translate_gm, translate_lex_trash, mask, msg)
grid(translate_lex_trash,msg)

check_mask:  90 rows
Move 90 rows from 694604
90 rows out of 104


,data_id,data,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower,nolemma_id,msg,data_home,lemma_home,merkmal_order
2145,Quintchen,Quintchen,N,27.678524,gm,Quintchen~Quentchen,Quentchen,<NA>,,NaN,quentchen,<NA>,verwaist_gm,L,,1.0
3738,-innen,-innen,,0.637364,gm,-in,-in,<NA>,,NaN,-in,<NA>,verwaist_gm,T,,1.0
201588,-sale,-sale,,1.126357,gm,-sal,-sal,<NA>,,NaN,-sal,<NA>,verwaist_gm,T,,1.0
201589,-sales,-sales,,0.588722,gm,-sal,-sal,<NA>,,NaN,-sal,<NA>,verwaist_gm,T,,1.0
201590,-salen,-salen,,0.350174,gm,-sal,-sal,<NA>,,NaN,-sal,<NA>,verwaist_gm,T,,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597977,-ablerem,-ablerem,,0.695215,gm,-abel,-abel,<NA>,,NaN,-abel,<NA>,verwaist_gm,T,,1.0
597978,-ableren,-ableren,,2.125359,gm,-abel,-abel,<NA>,,NaN,-abel,<NA>,verwaist_gm,T,,1.0
597979,-ablerer,-ablerer,,1.202282,gm,-abel,-abel,<NA>,,NaN,-abel,<NA>,verwaist_gm,T,,1.0
597980,-ableres,-ableres,,1.075359,gm,-abel,-abel,<NA>,,NaN,-abel,<NA>,verwaist_gm,T,,1.0


In [409]:
# Seltene Kombination (gibt's nicht mehr)
mask1 = translate_gm.lemma_home == 'T'
mask2 = translate_gm.data_home == 'T'
mask = mask1  &  mask2
check_mask(translate_gm,mask,0,51)

#grid(translate_gm,mask)
msg = 'Kombi TT'
translate_gm, translate_lex_trash = pak.move_rows(translate_gm, translate_lex_trash, mask, msg)
grid(translate_lex_trash,msg)

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


In [410]:
# Bedeutungsloses löschen
mask1 = translate_gm.data == translate_gm.lemma
mask2A = translate_gm.data_tagZZ == translate_gm.lemma_tagZZ
mask2B = translate_gm.data_tagZZ == ''
mask = mask1  &  (mask2A | mask2B)
check_mask(translate_gm,mask,235)

translate_gm, df = pak.move_rows(translate_gm,mask)
grid(df)

check_mask:  236 rows
Move 236 rows from 694514
236 rows


,data_id,data_home,data,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,merkmal_order,lemma_lower,msg
250,übertragen_2,L,übertragen,A,0.229672,gm,übertragen,L,übertragen,ADJA,A,149.543640,1,übertragen,57
255,Handlungsreisender_1,T,Handlungsreisender,,0.119443,gm,Handlungsreisender,L,Handlungsreisender,NN,N,171.500916,1,handlungsreisender,57
389,vergessen_1,L,vergessen,A,0.279426,gm,vergessen_1,L,vergessen,ADJA,A,0.838277,1,vergessen,57
429,verlassen_2,L,verlassen,A,0.205567,gm,verlassen_1,L,verlassen,ADJA,A,11.249609,1,verlassen,57
1059,vergeben_2,L,vergeben,A,0.131876,gm,vergeben,L,vergeben,ADJA,A,476.063232,1,vergeben,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680644,Schwächster_1,T,Schwächster,,0.109113,gm,Schwächster,L,Schwächster,NN,N,1481.802246,1,schwächster,57
680645,Schwächster_2,T,Schwächster,,0.047948,gm,Schwächster,L,Schwächster,NN,N,1481.802246,1,schwächster,57
681761,Hitzetoter_1,T,Hitzetoter,,0.096974,gm,Hitzetoter,L,Hitzetoter,NN,N,1436.636230,1,hitzetoter,57
682057,Bedienter_1,T,Bedienter,,0.133779,gm,Bedienter,L,Bedienter,NN,N,126.533546,1,bedienter,57


In [411]:
# Schrott rauslöschen
löschmich = ['Nominativ','Genitiv','Dativ','Akkusativ','Maskulinum','Femininum','Neutrum','Singular','Plural',
            'Hauptsatzkonjugation','Nebensatzkonjugation','Artikel','unbestimmt','Fugenelement','Allomorph',
             'Deklinationsendung','Komposita','Astronomischer Kalender','Konjugationsendung','Wortbildung',
             'am Lateinischen angelehnte Deklination','Hilfsverb','backen (regelmäßig)','Hauptsatz','Endung',
             'lateinisch','unbestimmter','Maskulinums','vernetzt','Derivatem','Ableitungsmorphem','griechisch',
             'Hapaxlegomenon','Interfix','Vollverb','unbestimmt','Artikel',
            ]
mask1A = translate_gm.lemma.isin(löschmich)
mask1B = translate_gm.data.isin(löschmich)
mask1C = translate_gm.lemma_id.isin(löschmich)
mask1D = translate_gm.data_id.isin(löschmich)
mask2 = translate_gm.lemma_id.str[0] != translate_gm.data_id.str[0]
mask = (mask1A | mask1B | mask1C | mask1D)  &  mask2
check_mask(translate_gm,mask,9)

msg = 'Schrott1'
translate_gm, translate_lex_trash = pak.move_rows(translate_gm, translate_lex_trash, mask, msg)
grid(translate_lex_trash,msg)

check_mask:  9 rows
Move 9 rows from 694278
9 rows out of 113


,data_id,data,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower,nolemma_id,msg,data_home,lemma_home,merkmal_order
139,ein,ein,X,80.404617,gm,unbestimmt,unbestimmt,ADJA,A,71.649826,unbestimmt,<NA>,Schrott1,L,L,1.0
140,ein,ein,X,80.404617,gm,Artikel,Artikel,NN,N,5720.057617,artikel,<NA>,Schrott1,L,L,2.0
141,ein,ein,X,80.404617,gm,Maskulinum,Maskulinum,NN,N,952.761658,maskulinum,<NA>,Schrott1,L,L,3.0
142,ein,ein,X,80.404617,gm,Nominativ,Nominativ,NN,N,511.155334,nominativ,<NA>,Schrott1,L,L,4.0
143,ein,ein,X,80.404617,gm,Neutrum,Neutrum,NN,N,371.873871,neutrum,<NA>,Schrott1,L,L,5.0
144,ein,ein,X,80.404617,gm,Akkusativ,Akkusativ,NN,N,560.514771,akkusativ,<NA>,Schrott1,L,L,6.0
2672,worden,worden,A,1.875900,gm,Hilfsverb,Hilfsverb,NN,N,908.798218,hilfsverb,<NA>,Schrott1,L,L,2.0
2675,geworden,geworden,A,1.272141,gm,Vollverb,Vollverb,NN,N,444.328613,vollverb,<NA>,Schrott1,L,L,2.0
3129,PR,PR,N,87.091766,gm,Femininum,Femininum,NN,N,321.927002,femininum,<NA>,Schrott1,L,L,1.0


In [412]:
# Weiteren Schrott finden
mask1 = translate_gm.lemma_id.str[0] != translate_gm.data_id.str[0]
mask2 = translate_gm.lemma_tagZZ != 'V'    # keine Verben
mask3 = ~translate_gm.lemma.str.endswith(('en','ern','eln','sein','tun')) # und sieht auch nicht aus wie ein Verb
mask = mask1  &  mask2  &  mask3
a = pak.analyse_freqs(translate_gm[mask],['lemma','data',])
grid(a)

50 rows


,lemma,lemma_count,lemma_percent,data,data_count
0,gut,12,9.4,"[besten, beste, besser, bestes, bester, bessere, besseren, besserem, besserer, besseres, bestem]","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,arg,12,9.4,"[ärgsten, ärgere, ärgerem, ärgeren, ärgerer, ärgeres, ärgstem, ärgster, ärgstes, ärgste, ärger]","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,alt,12,9.4,"[ältesten, älteste, älter, älteren, ältester, älterer, älterem, älteres, ältere, ältestem, ältestes]","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,arm,12,9.4,"[ärmsten, ärmste, ärmstes, ärmster, ärmstem, ärmere, ärmeres, ärmerer, ärmeren, ärmerem, ärmer]","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
4,gutgelaunt,7,5.5,"[bestgelaunte, bestgelauntes, bestgelaunter, bestgelaunten, bestgelauntem, besser gelaunt, bestgelaunt]","[1, 1, 1, 1, 1, 1, 1]"
5,gutbesucht,7,5.5,"[bestbesuchte, bestbesuchtes, bestbesuchter, bestbesuchten, bestbesuchtem, bestbesucht, besser besucht]","[1, 1, 1, 1, 1, 1, 1]"
6,viel,3,2.4,"[meisten, mehr]","[2, 1]"
7,gut gelaunt,2,1.6,"[besser gelaunt, besten gelaunt]","[1, 1]"
8,ich,2,1.6,"[mich, meiner]","[1, 1]"
9,Axt,2,1.6,"[Äxten, Äxte]","[1, 1]"


In [413]:
# suche_debug translate_gm
#search_str(translate_gm, suche_debug)

In [414]:
# not any_nan
assert not pak.any_nan(translate_gm)

In [415]:
# Spalten anpassen
translate_gm = pak.drop_cols(translate_gm,['merkmal_order'])

In [416]:
# Spaltenvergleich
bpy.minivenn( translate_gm.columns, translate_lex.columns, format='print2')

left_only:  {}
right_only: {'data_tag'}



In [417]:
# check score
maskA = translate_lex.data_score == 0
maskB = translate_lex.lemma_score == 0
mask = maskA | maskB
check_mask(translate_lex,mask,0)

check_mask:  0 rows


In [418]:
# check score
maskA = translate_gm.data_score == 0
maskB = translate_gm.lemma_score == 0
mask = maskA | maskB
check_mask(translate_gm,mask,0)

check_mask:  0 rows


In [419]:
# add_rows
translate_lex = pak.add_rows(translate_lex,translate_gm)

694269 rows added, now a total of 1365316


In [420]:
# check score
maskA = translate_lex.data_score == 0
maskB = translate_lex.lemma_score == 0
mask = maskA | maskB
check_mask(translate_lex,mask,0)

check_mask:  0 rows


In [421]:
pak.specials(translate_lex)

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower
0,lieben_1,T,lieben,,,0.311824,wiki Deklinierte_Form,lieb,L,lieb,ADJA,A,82.432930,lieb
526,Gotte,T,Gotte,,,0.465876,wiki Deklinierte_Form,Gott,L,Gott,NN,N,15700.294922,gott
1491,Häuser,T,Häuser,,,0.812983,wiki Deklinierte_Form,Haus,L,Haus,NN,N,108960.914062,haus
7661,abzufinden,T,abzufinden,,,0.447281,wiki Erweiterter_Infinitiv Verb,abfinden,L,abfinden,VVFIN,V,143.596985,abfinden
13923,Abklatschen,T,Abklatschen,,,0.525793,wiki Deklinierte_Form,Abklatsch,L,Abklatsch,NN,N,186.794449,abklatsch
188178,überführte,T,überführte,,,1.812539,wiki Deklinierte_Form,überführt_4,L,überführt,ADJA,A,0.032591,überführt
188312,α-Zerfällen,T,α-Zerfällen,,,0.441083,wiki Deklinierte_Form,α-Zerfall,L,α-Zerfall,NN,N,345.928619,α-zerfall
195070,10-Euro-Scheinen,T,10-Euro-Scheinen,,,1.125418,wiki Deklinierte_Form,10-Euro-Schein,L,10-Euro-Schein,NN,N,165.746552,10-euro-schein
362411,vermarktende,T,vermarktende,,,1.799718,wiki Deklinierte_Form,vermarktend,L,vermarktend,ADJA,A,1.261264,vermarktend
440699,einzuspinnende,T,einzuspinnende,,,1.795872,wiki Dekliniertes_Gerundivum Adjektiv,einspinnen,L,einspinnen,VVFIN,V,85.306229,einspinnen


In [422]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

1 rows


,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower
667771,Euro~€,,€,NN,,1403.598999,manuell Abkürzung Einheit manuell,Euro,L,Euro,NN,N,1403.598999,euro


In [423]:
#pak.sample(translate_lex)

In [424]:
# nan entfernen
translate_lex['lemma_tag'] = translate_lex.lemma_tag.fillna('')
translate_lex['data_tag']  = translate_lex.data_tag.fillna('')

In [425]:
#pak.nnan(translate_lex)

In [426]:
# Konfiguration Dups entfernen
aggwords    =  pak.agg_words_nodup
col_origins =  [ 'data_id','data','data_tag','data_tagZZ','data_score','member', 'lemma_id','lemma','lemma_tag','lemma_tagZZ','lemma_lower','lemma_score','data_home','lemma_home', ]
col_funcs   =  [ '',       '',    '',        '',          'sum',       aggwords, '',        '',     '',         '',           '',           'sum',        '',         ''          , ]   
col_names   =  [ '',       '',    '',        '',          'data_score','member', '',        '',     '',         '',           '',           'lemma_score','',         ''          , ] 

In [427]:
# Gruppieren!
translate_lex = pak.group_and_agg( translate_lex, col_origins, col_funcs, col_names, optimize=True)

31092 unique rows and 1334224 rows with duplicates
667151 rows less, now 698165 rows


In [428]:
# data_id vorhanden und nicht widerspüchlich?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

#assert translate_lex.data_id.is_unique

a = pak.same_but_different(translate_lex,same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


In [429]:
gridt(a)

No rows


In [430]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

1 rows


,data_id,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_lower,lemma_score,data_home,lemma_home
667771,Euro~€,€,NN,,1403.598999,manuell Abkürzung Einheit manuell,Euro,Euro,NN,N,euro,1403.598999,,L


### translate_m
* Alle Lexeme aus wiktionary_merkmal

In [431]:
# Bei lemma: lemma_id und data_id müssen gleich sein
mask1 = wiktionary_merkmal.merkmal == 'lemma'
mask2 = wiktionary_merkmal.lemma_id != wiktionary_merkmal.data_id
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows


In [432]:
# Alle Lexeme aus wiktionary_merkmal
mask = wiktionary_merkmal.is_lex == True
translate_m = wiktionary_merkmal[mask].copy()
#translate_m

In [433]:
# translate_m umformen
translate_m = pak.rename_col(translate_m, 'merkmal', 'member')
translate_m['member'] = ('merkmal ' + translate_m.member + ' ' + translate_m.meta).str.strip()

translate_m = pak.drop_cols(translate_m, ['is_lex','num','meta'])

In [434]:
# lemma_tagZZ schreiben
translate_m = pak.update_col( translate_m, translate_tagZ, left_on='lemma_tag', right_on='tag', col='tagZZ', col_rename='lemma_tagZZ', verbose=False )
translate_m = pak.move_cols(  translate_m,'lemma_tagZZ','lemma_tag')
translate_m['lemma_tagZZ'] =  translate_m.lemma_tagZZ.fillna('')

In [435]:
# temp
mask1 = translate_m.data  == 'ausgerutscht'
mask2 = translate_m.lemma == 'ausrutschen'
mask = mask1  &  mask2
translate_m[mask1]

,lemma_id,member,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,lemma_tagZZ,merkmal_order,data_score,hunspell
953665,ausrutschen,merkmal Ptz2,ausgerutscht,ausgerutscht,VVPP,ADJ,A,ausrutschen,VVFIN,V,1.0,0.432818,1
2513671,ausgerutscht,merkmal lemma,ausgerutscht,ausgerutscht,ADJA,ADJ,A,ausgerutscht,ADJA,A,0.0,1.298455,1
2571199,ausrutschen,merkmal Ptz2,ausgerutscht,ausgerutscht,ADJA,ADJ,A,ausrutschen,VVFIN,V,11.0,0.432818,1


In [436]:
# data_id löschen, wenn nötig
mask1 = translate_m.data_id != translate_m.data
mask2 = translate_m.data_id == translate_m.lemma_id
mask = mask1  &  mask2
check_mask(translate_m,mask,22392)

check_mask:  22388 rows


In [437]:
translate_m.loc[mask,'data_id'] = ''
grid(translate_m,mask)
#a = pak.analyse_freqs(translate_m[mask],['member','data'])
#grid(a)

22388 rows out of 1180505


,lemma_id,member,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,lemma_tagZZ,merkmal_order,data_score,hunspell
263078,Pro_1,merkmal männ,,Pro,NN,NOM,N,Pro,NN,N,1.0,3.135333,1
1141678,Evelyn_V,merkmal Nom Sg,,Evelyn,NE,NAM,N,Evelyn,NE,N,1.0,1.293374,0
1141680,Angelina_V,merkmal Nom Sg,,Angelina,NE,NAM,N,Angelina,NE,N,1.0,1.129354,0
1141683,Fritaun_V,merkmal Nom Sg,,Fritaun,NE,NAM,N,Fritaun,NE,N,1.0,0.074543,0
1141684,Aaliyah_V,merkmal Nom Sg,,Aaliyah,NE,NAM,N,Aaliyah,NE,N,1.0,0.737553,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2600446,nämlich_1,merkmal lemma zweittag,,nämlich,KON,X_konj,X,nämlich,ADV,A,0.0,65.382156,1
2600476,unbeschadet_1,merkmal lemma zweittag,,unbeschadet,ADJA,ADJ,A,unbeschadet,ADV,A,0.0,10.897341,1
2600478,ungeachtet_1,merkmal lemma zweittag,,ungeachtet,APPR,X_prep,X,ungeachtet,APPO,X,0.0,12.538471,1
2600488,welch_M,merkmal lemma zweittag,,welch,PWAT,PRO,P,welch,PRELS,P,0.0,2.239461,1


In [438]:
# wiktionary_lemma suche_debug
grid(search_str(translate_lex, suche_debug))

1 rows


,data_id,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_lower,lemma_score,data_home,lemma_home
667771,Euro~€,€,NN,,1403.598999,manuell Abkürzung Einheit manuell,Euro,Euro,NN,N,euro,1403.598999,,L


In [439]:
# data_id: Wenn leer, neu schreiben
mask =  translate_m.data_id == ''
check_mask(translate_m,mask,22339)
translate_m[mask] = pak.update_col( translate_m[mask], wiktionary_lemma, cond='null', left_on=['data','lemma_tagZZ'], right_on=['lemma','tagZZ'], col='lemma_id', col_rename='data_id', col_score='lemma_score') 
translate_m[mask] = pak.update_col( translate_m[mask], translate_lex,    cond='null',      on=['data','lemma_tagZZ'],                             col='data_id',                        col_score='data_score') 

check_mask:  22388 rows
update_col: col_score lemma_score applied, 4048 records less!
update_col: 22388 cells written into existing column
update_col: col_score data_score applied, 20782 records less!
update_col: 0 cells written into existing column


In [440]:
# Nicht nötig: data_id neu erzeugen
mask =  translate_m.data_id == ''
check_mask(translate_m,mask,0)

translate_m.loc[mask,'data_id'] = translate_m[mask].lemma + '~' + translate_m[mask].data + '~N'

check_mask:  0 rows


In [441]:
search_str(translate_m, suche_debug)

,lemma_id,member,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,lemma_tagZZ,merkmal_order,data_score,hunspell


In [442]:
#translate_m_bak = translate_m.copy()
#translate_m = translate_m_bak.copy()

In [443]:
from x32_Funktionen import *
translate_m = pflege_translate_tabelle(translate_m, translate_tagZ, wiktionary_lemma, translate_lex )
#pak.sample(translate_m)

rank: 5 rows less, now 698160 rows
update_col: 609677 cells written into existing column
update_col: 1180505 cells written into existing column
update_col: 633200 cells written into existing column
update_col: 0 cells written into existing column
update_col: 1180505 cells written into new column
update_col: 1180505 cells written into new column


In [444]:
# Bedeutungsloses löschen
mask1 = ~translate_m.member.str.contains('lemma')
mask2 =  translate_m.data == translate_m.lemma
mask3A = translate_m.data_tagZZ == translate_m.lemma_tagZZ
mask3B = translate_m.data_tagZZ == ''
mask = mask1  &  mask2  &  (mask3A | mask3B)
#grid(translate_m,mask)
check_mask(translate_m,mask,374075)

translate_m, df = pak.move_rows(translate_m,mask)
grid(df)

check_mask:  374068 rows
Move 374068 rows from 1180505
374068 rows


,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,data_tagZ,merkmal_order,hunspell,lemma_lower,msg
475,Mangaka,L,Mangaka,NN,N,3.680828,merkmal weib,Mangaka,L,Mangaka,NN,N,110.424843,NOM,1.0,0,mangaka,58
2094,Weiße,L,Weiße,NN,N,74.658867,merkmal männ,Weiße,L,Weiße,NN,N,671.929810,NOM,2.0,1,weiße,58
5156,Manx,L,Manx,NN,N,0.752407,merkmal männ,Manx_1,L,Manx,NN,N,67.716667,NOM,1.0,0,manx,58
5157,Manx,L,Manx,NN,N,0.275540,merkmal weib,Manx_2,L,Manx,NN,N,24.798576,NOM,1.0,0,manx,58
6234,Israeli,L,Israeli,NN,N,56.027298,merkmal weib,Israeli,L,Israeli,NN,N,504.245667,NOM,1.0,1,israeli,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1180223,Christus,L,Christus,NE,N,150.754379,merkmal Akk Sg zweittag,Christus,L,Christus,NN NE,N,458.981506,NAM,1.0,1,christus,58
1180230,die_1,L,die,PRO,P,5.123206,merkmal Akk Sg f zweittag,die_1,L,die,PRELS PRO,P,7.362668,PRO,1.0,1,die,58
1180260,dies,L,dies,PDS,P,55.655846,merkmal Akk Sg n zweittag,dies,L,dies,PRELS PDS,P,60.134766,PRO,2.0,1,dies,58
1180261,die_1,L,die,PRO,P,5.123206,merkmal Nom Pl zweittag,die_1,L,die,PRELS PRO,P,7.362668,PRO,1.0,1,die,58


In [445]:
pak.analyse_freqs(translate_m, ['lemma_home','data_home'])

,lemma_home,lemma_home_count,lemma_home_percent,data_home,data_home_count
0,L,778418,96.5,"[T, L, B, ]","[520515, 169153, 62581, 26169]"
1,B,28019,3.5,"[B, L, , T]","[26413, 985, 531, 90]"


In [446]:
# Neuzugänge
mask = translate_m.data_home == ''
grid(translate_m,mask)

26700 rows out of 806437


,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,data_tagZ,merkmal_order,hunspell,lemma_lower
1,Dezember~December,,December,NN,N,21.269392,merkmal alt,Dezember,L,Dezember,NN,N,1934.400513,NOM,1.0,0,dezember
4,August_1~Gust,,Gust,NE,N,0.925968,merkmal alt_abk,August_1,L,August,NE,N,109.221161,NAM,1.0,0,august
5,August_1~Gustel,,Gustel,NE,N,0.925968,merkmal alt_klein,August_1,L,August,NE,N,109.221161,NAM,1.0,0,august
9,Oktober~October,,October,NN,N,24.086452,merkmal alt,Oktober,L,Oktober,NN,N,2187.935547,NOM,1.0,0,oktober
11,Litschi~Lychee,,Lychee,NN,N,3.941663,merkmal alt_1,Litschi,L,Litschi,NN,N,118.249878,NOM,1.0,0,litschi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1180091,oder~und/oder,,und/oder,ITJ,X,11.460547,merkmal lex manuell zweittag,oder,L,oder,KON ITJ,X,1051.604370,X_itj,1.0,0,oder
1180092,beide~beidem,,beidem,CARD,X,100.831711,merkmal lex manuell zweittag,beide,L,beide,PIAT CARD,P,309.213531,X_zahl,1.0,1,beide
1180093,gewiss~gewiß,,gewiß,ADJA,A,10.995772,merkmal lexAlt manuell zweittag,gewiss,L,gewiss,ADV ADJA,A,343.309937,ADJ,1.0,0,gewiss
1180094,gewiss~gewiße,,gewiße,ADJA,A,10.995772,merkmal lexAlt manuell zweittag,gewiss,L,gewiss,ADV ADJA,A,343.309937,ADJ,2.0,0,gewiss


In [447]:
assert not pak.any_nan(translate_m)

In [448]:
# Spalten anpassen
translate_m = pak.drop_cols(translate_m,['data_tagZ', 'hunspell', 'merkmal_order'])
#translate_m = pak.rename_col(translate_m, 'score', 'data_score')

In [449]:
# Spaltenvergleich
bpy.minivenn( translate_m.columns, translate_lex.columns, format='print2')

left_only:  {}
right_only: {}



In [450]:
search_str(translate_m, suche_debug)

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower


In [451]:
# add_rows
translate_lex = pak.add_rows(translate_lex,translate_m)

806437 rows added, now a total of 1504602


In [452]:
# check score
maskA = translate_lex.data_score == 0
maskB = translate_lex.lemma_score == 0
mask = maskA | maskB
check_mask(translate_lex,mask,0)

check_mask:  0 rows


In [453]:
# Beispiel anzeigen: Mehrdeutigkeit
# Welcher Lemma soll einem Lexem (data) zugeordnet werden?
# Die Zeile mit dem höchsten data_score wird ausgewählt.
mask = translate_lex.data == 'Alben'
translate_lex[mask].sort_values('data_score', ascending=False)

,data_id,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_lower,lemma_score,data_home,lemma_home
1147994,Alben_4,Alben,NN,N,128.670151,merkmal Akk Pl,Album,Album,NN,N,album,1158.031250,L,L
938318,Alben_4,Alben,NN,N,128.670151,merkmal Nom Pl,Album,Album,NN,N,album,1158.031250,L,L
1043640,Alben_4,Alben,NN,N,128.670151,merkmal Dat Pl,Album,Album,NN,N,album,1158.031250,L,L
1024882,Alben_4,Alben,NN,N,128.670151,merkmal Gen Pl,Album,Album,NN,N,album,1158.031250,L,L
1039507,Alben_4,Alben,NN,N,59.462894,merkmal Dat Pl,Alb,Alb,NN,N,alb,535.166077,L,L
1177265,Alben_4,Alben,NN,N,59.462894,merkmal Akk Pl,Alb,Alb,NN,N,alb,535.166077,L,L
892485,Alben_4,Alben,NN,N,59.462894,merkmal Nom Pl,Alb,Alb,NN,N,alb,535.166077,L,L
984838,Alben_4,Alben,NN,N,59.462894,merkmal Gen Pl,Alb,Alb,NN,N,alb,535.166077,L,L
986322,Alben_4,Alben,NN,N,55.461880,merkmal Gen Pl,Albe,Albe,NN,N,albe,166.385635,L,L
1176002,Alben_4,Alben,NN,N,55.461880,merkmal Akk Pl,Albe,Albe,NN,N,albe,166.385635,L,L


In [454]:
# nan entfernen
translate_lex['data_tag']   = translate_lex.data_tag.fillna('')
translate_lex['data_tagZZ'] = translate_lex.data_tagZZ.fillna('')

In [455]:
pak.nnan(translate_lex)

Series([], dtype: int64)

In [456]:
# Bleibt: Konfiguration Dups entfernen
# aggwords    =  pak.agg_words_nodup
# col_origins =  [ 'data_id','data','data_tag','data_tagZZ','data_score','member', 'lemma_id','lemma','lemma_tag','lemma_tagZZ','lemma_lower','lemma_score','data_home','lemma_home', ]
# col_funcs   =  [ '',       '',    '',        '',          'sum',       aggwords, '',        '',     '',         '',           '',           'sum',        '',         ''          , ]   
# col_names   =  [ '',       '',    '',        '',          'data_score','member', '',        '',     '',         '',           '',           'lemma_score','',         ''          , ] 

In [457]:
# Gruppieren!
translate_lex = pak.group_and_agg( translate_lex, col_origins, col_funcs, col_names, optimize=True)

1188880 unique rows and 315722 rows with duplicates
228506 rows less, now 1276096 rows


In [458]:
# data_id vorhanden und nicht widersprüchlich?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

a = pak.same_but_different(translate_lex,same='data_id',different='data')
assert a.shape[0] == 0

check_mask:  0 rows


In [459]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

1 rows


,data_id,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_lower,lemma_score,data_home,lemma_home
754987,Euro~€,€,NN,,1403.598999,manuell Abkürzung Einheit manuell,Euro,Euro,NN,N,euro,1403.598999,,L


In [460]:
#pak.sample(translate_lex)

In [461]:
# Beispiel anzeigen
mask = translate_lex.data == 'Alben'
translate_lex[mask]

,data_id,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_lower,lemma_score,data_home,lemma_home
2003,Alben_4,Alben,NN,N,237.851578,merkmal Nom Pl Gen Dat Akk,Alb,Alb,NN,N,alb,2140.664307,L,L
2004,Alben_4,Alben,NN,N,9.647217,merkmal Nom Pl Gen Dat Akk,Alba_1,Alba,NN,N,alba,28.941650,L,L
2005,Alben_4,Alben,NN,N,221.847519,merkmal Nom Pl Gen Dat Akk,Albe,Albe,NN,N,albe,665.542542,L,L
2006,Alben_4,Alben,NN,N,514.680603,merkmal Nom Pl Gen Dat Akk,Album,Album,NN,N,album,4632.125000,L,L
92354,Alben,Alben,,,1.506132,wiki Deklinierte_Form gm,Album,Album,NN,N,album,2316.062500,T,L
92355,Alben_1,Alben,,,0.183101,wiki Deklinierte_Form gm,Alb,Alb,NN,N,alb,1070.332153,T,L
92356,Alben_2,Alben,,,0.123789,wiki Deklinierte_Form gm,Alba,Alba,NN,N,alba,184.357941,T,L
92357,Alben_3,Alben,,,0.092842,wiki Deklinierte_Form gm,Albe,Albe,NN,N,albe,332.771271,T,L
1079996,Alben_4,Alben,NN,N,14.287867,merkmal lemma,Alben_4,Alben,NN,N,alben,14.287867,L,L


In [462]:
mask = translate_lex.data_tag != ''
check_mask(translate_lex,mask,562350)

check_mask:  581444 rows


### translate_lma und translate_lmb
* aus wiktionary_lexmerkmal

In [463]:
# zunächst wiktionary_lexmerkmal: Nichtlexeme löschen. Das sind vor allem syllables, ipa und rhymes.
mask = wiktionary_lexmerkmal.is_lex == False
check_mask( wiktionary_lexmerkmal, mask, 1575569) # 1523133

msg = 'Nichtlexeme'
wiktionary_lexmerkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_lexmerkmal, wiktionary_merkmal_trash, mask, msg )
#grid(wiktionary_merkmal_trash, msg)

check_mask:  1575569 rows
Move 1575569 rows from 1714508


In [464]:
#pak.analyse_freqs(wiktionary_lexmerkmal,['merkmal','data'])

In [465]:
# wiktionary_lexmerkmal
wiktionary_lexmerkmal.head(1)

,lex_id,merkmal_order,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
i,,,,,,,,,,,
1759,lieb_1,1,alt_1,liebe,liebe,,,,,0.170379,True


In [466]:
# lemma_id für data_id finden
wiktionary_lexmerkmal['lemma_id'] = ''
wiktionary_lexmerkmal = pak.update_col(wiktionary_lexmerkmal, translate_lex, on='data_id',  col='lemma_id')

update_col: 205844 cells written into existing column
update_col: WARNING: df_from identifier not unique. I call this again with func="max"
update_col: func max applied, 416327 records less!
update_col: 116648 cells written into existing column


In [467]:
# translate_lma erstellen
mask = wiktionary_lexmerkmal.lemma_id != ''
translate_lma = wiktionary_lexmerkmal[mask].copy()

translate_lma = pak.rename_col(translate_lma, 'merkmal', 'member')
translate_lma['member'] = ('lma ' + translate_lma.member + ' ' + translate_lma.meta ).str.strip()

translate_lma = pak.drop_cols(translate_lma, ['lex_id','is_lex','data_tag','num','meta','data'])

In [468]:
# lemma_id für lex_id finden
wiktionary_lexmerkmal['lemma_id'] = ''
wiktionary_lexmerkmal = pak.update_col(wiktionary_lexmerkmal, translate_lex, left_on='lex_id', right_on='data_id',  col='lemma_id', col_score='data_score')

update_col: col_score data_score applied, 416327 records less!
update_col: 138939 cells written into existing column


In [469]:
# translate_lmb erstellen
mask = wiktionary_lexmerkmal.lemma_id != ''
translate_lmb = wiktionary_lexmerkmal[mask].copy()

translate_lmb = pak.rename_col(translate_lmb, 'merkmal', 'member')
translate_lmb['member'] = ('lmb ' + translate_lmb.member + ' ' + translate_lmb.meta ).str.strip()

translate_lmb = pak.drop_cols(translate_lmb, ['data_id','data','data_tag','is_lex','num','meta','data'])

translate_lmb = pak.rename_col(translate_lmb, 'lex_id', 'data_id')

In [470]:
# pflege_translate_tabelle
from x32_Funktionen import *
translate_lma = pflege_translate_tabelle(translate_lma, translate_tagZ, wiktionary_lemma, translate_lex )
translate_lmb = pflege_translate_tabelle(translate_lmb, translate_tagZ, wiktionary_lemma, translate_lex )

rank: 416327 rows less, now 859769 rows
update_col: 116648 cells written into new column
update_col: 116646 cells written into new column
update_col: 17408 cells written into existing column
update_col: 17408 cells written into new column
update_col: 116646 cells written into new column
update_col: 116646 cells written into new column
rank: 416327 rows less, now 859769 rows
update_col: 138939 cells written into new column
update_col: 138935 cells written into new column
update_col: 0 cells written into existing column
update_col: 0 cells written into new column
update_col: 138935 cells written into new column
update_col: 138935 cells written into new column


In [471]:
# Bedeutungsloses löschen
mask1  = translate_lma.data       == translate_lma.lemma
mask2A = translate_lma.data_tagZZ == translate_lma.lemma_tagZZ
mask2B = translate_lma.data_tagZZ == ''
mask = mask1  &  (mask2A | mask2B)
check_mask(translate_lma,mask,16606)

translate_lma, df = pak.move_rows(translate_lma,mask)
grid(df)

check_mask:  16602 rows
Move 16602 rows from 116648
16602 rows


,data_id,data_home,data,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,merkmal_order,lemma_lower,msg
32,Buch,B,Buch,N,0.161979,lma alt_1,Buch,B,Buch,NN,N,19298.539062,1,buch,59
46,ne,B,ne,X,36.811817,lma alt_abk,ne_1,L,ne,ITJ,X,7.864007,1,ne,59
48,ner,B,ner,X,46.378887,lma alt_abk,ner,B,ner,ART,X,22.673538,1,ner,59
58,Mare,B,Mare,N,0.091428,lma alt_1,Mare_2,L,Mare,NE,N,0.922488,1,mare,59
59,Chordatier,B,Chordatier,N,1.289985,lma alt_1 Dativ,Chordatier,B,Chordatier,NN,N,133.904037,1,chordatier,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116643,die,B,die,X,5.924480,lma Akk Pl,die,B,die,ART,X,201.369171,1,die,59
116644,die,B,die,X,38.961239,lma Akk Pl,die,B,die,ART,X,201.369171,1,die,59
116645,die,B,die,X,0.679771,lma Akk Pl,die,B,die,ART,X,201.369171,1,die,59
116646,die,B,die,X,31.915634,lma Akk Pl,die,B,die,ART,X,201.369171,1,die,59


In [472]:
# Bedeutungsloses löschen
mask1  = translate_lmb.data       == translate_lmb.lemma
mask2A = translate_lmb.data_tagZZ == translate_lmb.lemma_tagZZ
mask2B = translate_lmb.data_tagZZ == ''
mask = mask1  &  (mask2A | mask2B)
check_mask(translate_lmb,mask,9)

translate_lmb, df = pak.move_rows(translate_lmb,mask)
grid(df)

check_mask:  10 rows
Move 10 rows from 138939
10 rows


,data_id,data_home,data,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,merkmal_order,lemma_lower,msg
781,Zweitplatzierter_1,T,Zweitplatzierter,,0.221164,lmb alt,Zweitplatzierter,B,Zweitplatzierter,NN,N,226.211151,1,zweitplatzierter,59
782,Zweitplatzierter_1,T,Zweitplatzierter,,0.221164,lmb alt,Zweitplatzierter,B,Zweitplatzierter,NN,N,226.211151,2,zweitplatzierter,59
6651,Ausschussvorsitzender_1,T,Ausschussvorsitzender,,0.184432,lmb alt,Ausschussvorsitzender,B,Ausschussvorsitzender,NN,N,60.538273,1,ausschussvorsitzender,59
8381,Spielende_1,T,Spielende,,3.134705,lmb männ,Spielende,B,Spielende,NN,N,354.233612,1,spielende,59
20860,schlapp_1,T,schlapp,,0.113402,lmb alt_1,schlapp,B,schlapp,ADJA,A,176.206070,1,schlapp,59
23007,Russlanddeutsche_1,T,Russlanddeutsche,,0.181033,lmb alt,Russlanddeutsche,B,Russlanddeutsche,NN,N,76.839951,1,russlanddeutsche,59
25532,Großindustrieller_1,T,Großindustrieller,,0.185792,lmb alt_2 schweizerisch,Großindustrieller,B,Großindustrieller,NN,N,263.612091,1,großindustrieller,59
26738,Vermisster_1,T,Vermisster,,0.167457,lmb alt,Vermisster,B,Vermisster,NN,N,266.342865,1,vermisster,59
29339,Außerirdischer_1,T,Außerirdischer,,0.178313,lmb alt_2 schweizerisch,Außerirdischer,B,Außerirdischer,NN,N,48.231888,1,außerirdischer,59
74825,Kongressabgeordneter_1,T,Kongressabgeordneter,,0.191912,lmb alt,Kongressabgeordneter,B,Kongressabgeordneter,NN,N,221.026123,1,kongressabgeordneter,59


In [473]:
# Spalten anpassen
translate_lma = pak.drop_cols(translate_lma,['merkmal_order'])
translate_lma['data_score'] = 0

In [474]:
# Spaltenvergleich
bpy.minivenn( translate_lma.columns, translate_lex.columns, format='print2')

left_only:  {}
right_only: {'data_tag'}



In [475]:
# score fehlt überall
mask = translate_lma.data_score != 0
check_mask(translate_lma,mask,0)

# lemma_score übernehmen
translate_lma['data_score'] = translate_lma.lemma_score.copy()

# abwerten, wenn nicht manuell
mask = ~translate_lma.member.str.contains('manuell')
translate_lma.loc[mask,'data_score'] *= 0.33

check_mask:  0 rows


In [476]:
# unterschiedliche Genitive sind ohne Statistik ununterscheidbar #TODO_Statistik
mask = translate_lma.lemma == 'Land'
grid(translate_lma,mask)

2 rows out of 100046


,data_id,data_home,data,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower
430,Lands,T,Lands,,12541.646484,lma alt_1,Land,B,Land,NN,N,38004.988281,land
6823,Landes,T,Landes,,12541.646484,lma alt_1,Land,B,Land,NN,N,38004.988281,land


In [477]:
# Spalten anpassen
translate_lmb = pak.drop_cols(translate_lmb,['merkmal_order'])

In [478]:
# Spaltenvergleich
bpy.minivenn( translate_lmb.columns, translate_lex.columns, format='print2')

left_only:  {}
right_only: {'data_tag'}



In [479]:
# check score
maskA = translate_lmb.data_score == 0
maskB = translate_lmb.lemma_score == 0
mask = maskA | maskB
check_mask(translate_lmb,mask,0)

check_mask:  0 rows


In [480]:
#pak.specials(translate_lmb)

In [481]:
# add_rows
translate_lex = pak.add_rows(translate_lex,translate_lma)
translate_lex = pak.add_rows(translate_lex,translate_lmb)

100046 rows added, now a total of 1376142
138929 rows added, now a total of 1515071


In [482]:
# check score
maskA = translate_lex.data_score == 0
maskB = translate_lex.lemma_score == 0
mask = maskA | maskB
check_mask(translate_lex,mask,0)

check_mask:  0 rows


In [483]:
# unterschiedliche Genitive sind ohne Statistik ununterscheidbar #TODO_Statistik
mask1 = translate_lex.lemma == 'Land'
mask2 = translate_lex.data.str.endswith('s')
mask = mask1  &  mask2
grid(translate_lex,mask)

8 rows out of 1515071


,data_id,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_lower,lemma_score,data_home,lemma_home
178720,Landes,Landes,,,1.452105,wiki Deklinierte_Form gm,Land,Land,NN,N,land,76009.976562,T,L
179003,Lands,Lands,,,1.434574,wiki Deklinierte_Form gm,Land,Land,NN,N,land,76009.976562,T,L
870353,Lands,Lands,NN,N,4222.776367,merkmal Gen Sg,Land,Land,NN,N,land,38004.988281,T,L
903358,Landes,Landes,NN,N,4222.776367,merkmal Gen Sg,Land,Land,NN,N,land,38004.988281,T,L
1276496,Lands,Lands,<NA>,,12541.646484,lma alt_1,Land,Land,NN,N,land,38004.988281,T,B
1281383,Landes,Landes,<NA>,,12541.646484,lma alt_1,Land,Land,NN,N,land,38004.988281,T,B
1376599,Landes,Landes,<NA>,,0.726053,lmb alt_1,Land,Land,NN,N,land,38004.988281,T,B
1384188,Lands,Lands,<NA>,,0.717287,lmb alt_1,Land,Land,NN,N,land,38004.988281,T,B


### pflegen

In [484]:
# data_id vorhanden und nicht widersprüchlich?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

a = pak.same_but_different(translate_lex,same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


In [485]:
# verwaiste lemma
mask = translate_lex.lemma.isnull()
check_mask(translate_lex,mask,0,10)

msg = 'verwaist_lemma_10'
translate_lex, translate_lex_trash = pak.move_rows( translate_lex, translate_lex_trash, mask, msg )
grid(translate_lex_trash,msg)

check_mask:  6 rows
Move 6 rows from 1515071
6 rows out of 119


,data_id,data,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower,nolemma_id,msg,data_home,lemma_home,merkmal_order,data_tag
1302040,-sals,-sals,,NaN,lma alt_1,-sal,<NA>,<NA>,,NaN,<NA>,<NA>,verwaist_lemma_10,T,,NaN,<NA>
1302041,-sales,-sales,,NaN,lma alt_1,-sal,<NA>,<NA>,,NaN,<NA>,<NA>,verwaist_lemma_10,T,,NaN,<NA>
1417259,-sale,-sale,,1.126357,lmb alt_1 Dativ,-sal,<NA>,<NA>,,NaN,<NA>,<NA>,verwaist_lemma_10,T,,NaN,<NA>
1417260,-sales,-sales,,0.588722,lmb alt_1,-sal,<NA>,<NA>,,NaN,<NA>,<NA>,verwaist_lemma_10,T,,NaN,<NA>
1417261,-sals,-sals,,0.582878,lmb alt_1,-sal,<NA>,<NA>,,NaN,<NA>,<NA>,verwaist_lemma_10,T,,NaN,<NA>
1457206,-stans,-stans,,0.626707,lmb alt_1,-stan,<NA>,<NA>,,NaN,<NA>,<NA>,verwaist_lemma_10,T,,NaN,<NA>


In [486]:
# verwaiste lemma_id
mask = ~translate_lex.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lex,mask,89)

msg = 'verwaist_lemma_id_100'
translate_lex, translate_lex_trash = pak.move_rows( translate_lex, translate_lex_trash, mask, msg )
grid(translate_lex_trash,msg)

check_mask:  92 rows
Move 92 rows from 1515065
92 rows out of 211


,data_id,data,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower,nolemma_id,msg,data_home,lemma_home,merkmal_order,data_tag
754314,-innen,-innen,,0.637364,wiki Deklinierte_Form,-in,-in,,,164.909531,-in,<NA>,verwaist_lemma_id_100,T,,NaN,
754493,-sale,-sale,,1.126357,wiki Deklinierte_Form,-sal,-sal,,,170.996689,-sal,<NA>,verwaist_lemma_id_100,T,,NaN,
754494,-sales,-sales,,0.588722,wiki Deklinierte_Form,-sal,-sal,,,170.996689,-sal,<NA>,verwaist_lemma_id_100,T,,NaN,
754495,-salen,-salen,,0.350174,wiki Deklinierte_Form,-sal,-sal,,,170.996689,-sal,<NA>,verwaist_lemma_id_100,T,,NaN,
754496,-sals,-sals,,0.582878,wiki Deklinierte_Form,-sal,-sal,,,170.996689,-sal,<NA>,verwaist_lemma_id_100,T,,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754712,-ableres,-ableres,,1.075359,wiki Deklinierte_Form,-abel,-abel,,,225.476135,-abel,<NA>,verwaist_lemma_id_100,T,,NaN,
754713,-ables,-ables,,1.049718,wiki Deklinierte_Form,-abel,-abel,,,225.476135,-abel,<NA>,verwaist_lemma_id_100,T,,NaN,
756328,-schaft~-sch,-sch,,62.831264,Abkürzung Substantiv Substantiv Suffix Suffix,-schaft,-schaft,,,78.539078,-schaft,<NA>,verwaist_lemma_id_100,,,NaN,ZFIX
756329,-schaft~-schft,-schft,,62.831264,Abkürzung Substantiv Substantiv Suffix Suffix,-schaft,-schaft,,,78.539078,-schaft,<NA>,verwaist_lemma_id_100,,,NaN,ZFIX


In [487]:
#data_tag fillna
translate_lex['data_tag'] = translate_lex.data_tag.fillna('')

In [488]:
assert not pak.any_nan(translate_lex)
#pak.nnan(translate_lex)

In [489]:
# pflege_translate_tabelle
#from x32_Funktionen import *
translate_lex = pflege_translate_tabelle(translate_lex, translate_tagZ, wiktionary_lemma, translate_lex )

rank: 655296 rows less, now 859677 rows
update_col: 1514973 cells written into existing column
update_col: 1514973 cells written into existing column
update_col: 284006 cells written into existing column
update_col: 0 cells written into existing column
update_col: 1514973 cells written into new column
update_col: 1514973 cells written into existing column


### analyse_stringdiff

In [490]:
from strsimpy.qgram import QGram 

qgram = QGram(2)
a = 'herbeispringen'
b = 'spring herbei'
c = 'herbeispringendes'
d = 'xfalsche Anfänger'
print(qgram.distance(a,b))
print(qgram.distance(a,c))
print(qgram.distance(a,d))

5
3
21


In [491]:
from strsimpy.metric_lcs import MetricLCS
metric_lcs = MetricLCS()

print(metric_lcs.distance(a,b))
print(metric_lcs.distance(a,c))
print(metric_lcs.distance(a,d))

0.5
0.17647058823529416
0.7058823529411764


In [492]:
# data_swaped: dächtest um >> umdächtest
translate_lex['data_swaped'] = translate_lex.data.str.split()
mask = translate_lex.data_swaped.str.len() == 2
translate_lex.loc[ mask, 'data_swaped'] = translate_lex[mask].data_swaped.str[-1] + translate_lex[mask].data_swaped.str[0]
translate_lex.loc[~mask, 'data_swaped'] = ''

In [493]:
# Ermittelt die Differenz zwischen lex und lemma

def analyse_stringdiff(zeile):
    if len(zeile.data_swaped) > 0:
        result_a = metric_lcs.distance( zeile.data.lower(), zeile.lemma.lower() ) 
        result_b = metric_lcs.distance( zeile.data_swaped.lower(), zeile.lemma.lower() )     
        zeile['same_lexlemma'] = 1.0 - min(result_a, result_b)
    else:
        result   = metric_lcs.distance( zeile.data.lower(), zeile.lemma.lower() ) 
        zeile['same_lexlemma'] = 1.0 - result      
    return zeile

In [494]:
# apply analyse_stringdiff 
translate_lex['same_lexlemma']   = 0.0
translate_lex = translate_lex.swifter.apply( analyse_stringdiff, axis=1 )
#translate_lex = pak.change_datatype(translate_lex, verbose=False)  

Pandas Apply:   0%|          | 0/1514973 [00:00<?, ?it/s]

In [495]:
# data_swaped wieder löschen
translate_lex = pak.drop_cols(translate_lex,'data_swaped')

In [496]:
# same_lexlemma auf scores übertragen
translate_lex['data_score']  = translate_lex.data_score  * ((translate_lex.same_lexlemma + 0.1)**0.5)
translate_lex['lemma_score'] = translate_lex.lemma_score * ((translate_lex.same_lexlemma + 0.1)**0.5)

### data_id eindeutig

In [497]:
# Beispiel anzeigen: Mehrdeutigkeit
# Welcher Lemma soll einem Lexem (data) zugeordnet werden?
# Die Zeile mit dem höchsten data_score wird ausgewählt.
mask = translate_lex.data == 'Alben'
translate_lex[mask].sort_values('data_score', ascending=False)

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower,same_lexlemma
2006,Alben_4,B,Alben,NN,N,430.612687,merkmal Nom Pl Gen Dat Akk,Album,B,Album,NN,N,968.878456,album,0.6
2005,Alben_4,B,Alben,NN,N,210.463036,merkmal Nom Pl Gen Dat Akk,Albe,B,Albe,NN,N,157.847273,albe,0.8
2003,Alben_4,B,Alben,NN,N,199.000907,merkmal Nom Pl Gen Dat Akk,Alb,B,Alb,NN,N,447.752064,alb,0.6
1079904,Alben_4,B,Alben,NN,N,14.985241,merkmal lemma,Alben_4,B,Alben,NN,N,14.985241,alben,1.0
2004,Alben_4,B,Alben,NN,N,8.071441,merkmal Nom Pl Gen Dat Akk,Alba_1,L,Alba,NN,N,6.053580,alba,0.6
92354,Alben,T,Alben,,,1.260121,wiki Deklinierte_Form gm,Album,B,Album,NN,N,968.878456,album,0.6
92355,Alben_1,T,Alben,,,0.153193,wiki Deklinierte_Form gm,Alb,B,Alb,NN,N,447.752064,alb,0.6
92356,Alben_2,T,Alben,,,0.103570,wiki Deklinierte_Form gm,Alba,B,Alba,NN,N,77.122460,alba,0.6
92357,Alben_3,T,Alben,,,0.088078,wiki Deklinierte_Form gm,Albe,B,Albe,NN,N,157.847273,albe,0.8


In [498]:
# Konfiguration Dups entfernen
aggwords    =  pak.agg_words_nodup
col_origins =  ['data_id','data','data_tag','data_tagZZ','data_score','member', 'lemma_id','lemma','lemma_tag','lemma_tagZZ','lemma_lower','lemma_score','data_home','lemma_home','same_lexlemma',]
col_funcs   =  ['',       '',    '',        '',          'sum',       aggwords, '',        '',     '',         '',           '',           'sum',        '',         ''          ,'first',        ]   
col_names   =  ['',       '',    '',        '',          'data_score','member', '',        '',     '',         '',           '',           'lemma_score','',         ''          ,'same_lexlemma',] 

In [499]:
# Gruppieren!
lookup_lex_tag = pak.group_and_agg( translate_lex, col_origins, col_funcs, col_names, optimize=True)

1152848 unique rows and 362125 rows with duplicates
236877 rows less, now 1278096 rows


In [500]:
# Beispiel anzeigen
mask = translate_lex.data == 'Alben'
translate_lex[mask]

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower,same_lexlemma
2003,Alben_4,B,Alben,NN,N,199.000907,merkmal Nom Pl Gen Dat Akk,Alb,B,Alb,NN,N,447.752064,alb,0.6
2004,Alben_4,B,Alben,NN,N,8.071441,merkmal Nom Pl Gen Dat Akk,Alba_1,L,Alba,NN,N,6.053580,alba,0.6
2005,Alben_4,B,Alben,NN,N,210.463036,merkmal Nom Pl Gen Dat Akk,Albe,B,Albe,NN,N,157.847273,albe,0.8
2006,Alben_4,B,Alben,NN,N,430.612687,merkmal Nom Pl Gen Dat Akk,Album,B,Album,NN,N,968.878456,album,0.6
92354,Alben,T,Alben,,,1.260121,wiki Deklinierte_Form gm,Album,B,Album,NN,N,968.878456,album,0.6
92355,Alben_1,T,Alben,,,0.153193,wiki Deklinierte_Form gm,Alb,B,Alb,NN,N,447.752064,alb,0.6
92356,Alben_2,T,Alben,,,0.103570,wiki Deklinierte_Form gm,Alba,B,Alba,NN,N,77.122460,alba,0.6
92357,Alben_3,T,Alben,,,0.088078,wiki Deklinierte_Form gm,Albe,B,Albe,NN,N,157.847273,albe,0.8
1079904,Alben_4,B,Alben,NN,N,14.985241,merkmal lemma,Alben_4,B,Alben,NN,N,14.985241,alben,1.0


### weniger data_ids 

In [501]:
translate_lex['data_id_bak'] = '' # Sicherheitskopie von data_id

In [502]:
# Eine halbe Million data_ids, die nirgendwohin verweisen. 
# Sichern und Leeren!
mask1 =  translate_lex.data != ''
mask2 = ~translate_lex.data_id.isin(wiktionary_lemma.lemma_id)
mask3 = ~translate_lex.data_id.isin(wiktionary_merkmal.data_id)
mask = mask1  &  mask2  &  mask3  
check_mask(translate_lex,mask,470316)

# kennzeichnen
translate_lex.loc[mask,'data_id_bak'] = translate_lex[mask].data_id  # sichern
translate_lex.loc[mask,'data_id']     = ''                           # leeren

#grid(translate_lex[mask],pak.sample )

check_mask:  473906 rows


In [503]:
# Sicherheitskopie
#translate_lex_bak = translate_lex.copy()
#translate_lex = translate_lex_bak.copy()

In [504]:
# Zweites data_id-Feld
translate_lex['data_id2'] = translate_lex.data_id.copy()
translate_lex = pak.move_cols(translate_lex,'data_id2','data_id')
#translate_lex

In [505]:
# Konfiguration Dups entfernen
aggwords    =  pak.agg_words_nodup
col_origins =  ['data_id','data_id2','data_id_bak','data','data_tag','data_tagZZ','data_score','member','lemma_id','lemma','lemma_tag','lemma_tagZZ','lemma_lower','lemma_score','data_home','lemma_home','same_lexlemma',]
col_funcs   =  ['max',    'min',     'max',        '',    '',        '',          'sum',       aggwords,'',        '',     '',         '',           '',           'sum',        'first',    'first',     'first',        ]   
col_names   =  ['data_id','data_id2','data_id_bak','',    '',        '',          'data_score','member','',        '',     '',         '',           '',           'lemma_score','data_home','lemma_home','same_lexlemma',] 

In [506]:
# Gruppieren!
translate_lex = pak.group_and_agg( translate_lex, col_origins, col_funcs, col_names, optimize=True)

1125543 unique rows and 389430 rows with duplicates
251963 rows less, now 1263010 rows


In [507]:
# Beide data_id leer? Dann data_id_bak verwenden
mask1 = translate_lex.data_id == ''
mask2 = translate_lex.data_id2 == ''
mask = mask1  &  mask2
check_mask(translate_lex,mask,354478)
translate_lex.loc[mask,'data_id'] = translate_lex[mask].data_id_bak
#grid(translate_lex,mask)

check_mask:  380026 rows


In [508]:
# Prüfen: data_id immer belegt
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

check_mask:  0 rows


In [509]:
# data_ids sind verschieden, aber nahezu gleich
# z.B. abstoßend_1 und abstoßend
mask1 = translate_lex.data_id != translate_lex.data_id2
mask2 = translate_lex.data_id2 != ''
mask3 = pak.remove_str( translate_lex.data_id, ['_1','_2','_3','_4','_MADJ',], safemode=False )  ==  pak.remove_str( translate_lex.data_id2, ['_1','_2','_3','_4','_MADJ',], safemode=False )
mask = mask1  &  mask2  &  mask3
check_mask(translate_lex,mask,393)

translate_lex.loc[mask,'data_id'] = translate_lex[mask].data_id2 # das Minimum, also die kürzere id
translate_lex.loc[mask,'data_id2'] = ''
#grid(translate_lex,mask)

check_mask:  562 rows


In [510]:
# Jetzt gibt es keine widersprüchliche data_id mehr
mask1 = translate_lex.data_id != translate_lex.data_id2
mask2 = translate_lex.data_id2 != ''
mask = mask1  &  mask2
check_mask(translate_lex,mask,0)
#grid(translate_lex,mask,)

check_mask:  0 rows


In [511]:
# Hilfsspalten löschen
translate_lex = pak.drop_cols(translate_lex,['data_id2','data_id_bak'])

In [512]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

1 rows


,data_id,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_lower,lemma_score,data_home,lemma_home,same_lexlemma
741446,Euro~€,€,NN,,443.856976,manuell Abkürzung Einheit manuell,Euro,Euro,NN,N,euro,443.856976,T,B,0.0


In [513]:
# pflege_translate_tabelle
translate_lex_ranked  = pak.rank(translate_lex, col_score='data_score', cols_group='data_id', on_conflict='first')
translate_lex         = pflege_translate_tabelle(translate_lex, translate_tagZ, wiktionary_lemma, translate_lex_ranked )

rank: 418040 rows less, now 844970 rows
rank: 0 rows less, now 844970 rows
update_col: 1263010 cells written into existing column
update_col: 1263010 cells written into existing column
update_col: 283379 cells written into existing column
update_col: 0 cells written into existing column
update_col: 1263010 cells written into new column
update_col: 1263010 cells written into existing column


In [514]:
# translate_lex: data_id vorhanden und nicht widersprüchlich?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

#assert translate_lex.data_id.is_unique

a = pak.same_but_different(translate_lex,same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


In [515]:
assert not pak.any_nan(translate_lex)

In [516]:
# Beispiel anzeigen
mask = translate_lex.data == 'Alben'
translate_lex[mask]

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower
139470,Alben_4,B,Alben,NN,N,199.000907,merkmal Nom Pl Gen Dat Akk,Alb,B,Alb,NN,N,535.166077,0.6,alb
139471,Alben_4,B,Alben,NN,N,8.071441,merkmal Nom Pl Gen Dat Akk,Alba_1,L,Alba,NN,N,7.235413,0.6,alba
139472,Alben_4,B,Alben,NN,N,210.463036,merkmal Nom Pl Gen Dat Akk,Albe,B,Albe,NN,N,166.385635,0.8,albe
139473,Alben_4,B,Alben,NN,N,430.612687,merkmal Nom Pl Gen Dat Akk,Album,B,Album,NN,N,1158.031250,0.6,album
228026,Alben,T,Alben,,,1.260121,wiki Deklinierte_Form gm,Album,B,Album,NN,N,1158.031250,0.6,album
228027,Alben_1,T,Alben,,,0.153193,wiki Deklinierte_Form gm,Alb,B,Alb,NN,N,535.166077,0.6,alb
228028,Alben_2,T,Alben,,,0.103570,wiki Deklinierte_Form gm,Alba,B,Alba,NN,N,92.178970,0.6,alba
228029,Alben_3,T,Alben,,,0.088078,wiki Deklinierte_Form gm,Albe,B,Albe,NN,N,166.385635,0.8,albe
1065706,Alben_4,B,Alben,NN,N,14.985241,merkmal lemma,Alben_4,B,Alben,NN,N,14.287867,1.0,alben


In [517]:
# Beispiel anzeigen
mask = translate_lex.data == 'ausführt'
translate_lex[mask]

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower
62134,ausführt_1,T,ausführt,,,0.298841,wiki Konjugierte_Form Verb gm lmb alt_1,ausfahren,B,ausfahren,VVFIN,V,387.210602,0.666667,ausfahren
62135,ausführt,T,ausführt,,,141.066666,wiki Konjugierte_Form Verb gm lma alt_1,ausführen,B,ausführen,VVFIN,V,451.414001,0.777778,ausführen


In [518]:
# Beispiel anzeigen
#finde = ['eine','einer','eines','einen','ein','einem']
#mask = translate_lex.data.isin(finde)
#translate_lex[mask]

In [519]:
#pak.analyse_freqs(translate_lex,['data_home','lemma_home'])

## wiktionary_merkmal

In [520]:
# check data_id
mask1 =  wiktionary_merkmal.data_id == ''
mask2 =  wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows


In [521]:
# Sicherheitskopie
#wiktionary_merkmal_bak = wiktionary_merkmal.copy()
#wiktionary_merkmal = wiktionary_merkmal_bak.copy()

In [522]:
wiktionary_merkmal['data_id_bak'] = '' # Sicherheitskopie von data_id

In [523]:
#translate_lex

In [524]:
# verwaiste data_id:
# leeren und neu belegen
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.data != ''
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4 
check_mask(wiktionary_merkmal,mask, 0, 34)

# kennzeichnen
wiktionary_merkmal.loc[mask,'data_id_bak'] = wiktionary_merkmal[mask].data_id  # sichern
wiktionary_merkmal.loc[mask,'data_id']     = ''                                # leeren

# data_id schreiben 
on = ['data','lemma_tagZZ']
wiktionary_merkmal[mask] = pak.update_col( wiktionary_merkmal[mask], wiktionary_lemma, cond='null', left_on=on, right_on=['lemma','tagZZ'], col='lemma_id', col_rename='data_id', col_score='lemma_score')
wiktionary_merkmal[mask] = pak.update_col( wiktionary_merkmal[mask], translate_lex,    cond='null',      on=on,                             col='data_id',  col_score='data_score', )

# Kontrolle
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.data != ''
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4  
check_mask(wiktionary_merkmal,mask, 0)

# anschauen
mask = wiktionary_merkmal.data_id_bak != ''
grid(wiktionary_merkmal,mask)

check_mask:  1 rows
update_col: col_score lemma_score applied, 4048 records less!
update_col: 0 cells written into existing column
update_col: col_score data_score applied, 401222 records less!
update_col: 1 cells written into existing column
check_mask:  0 rows
1 rows out of 2586444


,lemma_id,merkmal,is_lex,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,lemma_tagZZ,num,meta,merkmal_order,data_score,hunspell,data_id_bak
2595970,auspowern,VVIZU,True,auszupowern,auszupowern,VVIZU,VER,V,auspowern,VVFIN,V,,,0.0,0.005901,0,auszupowern_1


In [525]:
mask = wiktionary_merkmal.data_id == 'Petersen~Peterson'
wiktionary_merkmal[mask]

,lemma_id,merkmal,is_lex,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,lemma_tagZZ,num,meta,merkmal_order,data_score,hunspell,data_id_bak
835572,Petersen,alt_3,True,Petersen~Peterson,Peterson,NE,NAM,N,Petersen,NE,N,,,5.0,4.700129,0,


In [526]:
# check data_id
mask1 =  wiktionary_merkmal.data_id == ''
mask2 =  wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows


In [527]:
# Verlinkung auf data prüfen 
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, translate_lex, on='data_id', col='data', col_rename='data_check', col_score='data_score')
mask_kaputt = wiktionary_merkmal.data != wiktionary_merkmal.data_check
check_mask(wiktionary_merkmal,mask_kaputt,0)
#grid(wiktionary_merkmal,mask_kaputt)

update_col: col_score data_score applied, 418040 records less!
update_col: 2093456 cells written into new column
check_mask:  0 rows


In [528]:
# meta == lex 
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, translate_lex, left_on='meta', right_on='data', col='lemma', col_rename='löschmich_lemma', col_score='data_score')

mask = wiktionary_merkmal.lemma == wiktionary_merkmal.löschmich_lemma
check_mask(wiktionary_merkmal,mask,2742)
wiktionary_merkmal.loc[mask,'meta'] = ''

wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal,'löschmich_lemma')

update_col: col_score data_score applied, 437711 records less!
update_col: 134399 cells written into new column
check_mask:  2695 rows


In [529]:
# Hilfsdaten löschen
wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal,['data_id_bak','notiz','data_check'])

In [530]:
grid(wiktionary_merkmal,pak.sample)

21 rows out of 2586444


,lemma_id,merkmal,is_lex,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,lemma_tagZZ,num,meta,merkmal_order,data_score,hunspell
0,Hallo,syn,False,Gejohle,Gejohle,,,,Hallo,NN,N,1,,1.0,34.813919,-2
1157,sein_1,weib,True,ihr_1,ihr,PPOSS,PRO,P,sein,PPOSS,P,,,1.0,16.740204,1
6134,Haus,def,False,Gebäude,Gebäude,,,,Haus,NN,N,1,,1.0,12106.767578,-2
6740,Haus,kombi,False,königlich,königlich,,,,Haus,NN,N,9,,323.0,12106.767578,-2
7448,rot,alt,True,rot~roth,roth,ADJA,ADJ,A,rot,ADJA,A,,,1.0,84.052872,0
54428,äbich,def,False,falsch,falsch,,,,äbich,ADV,A,1,meißnisch,1.0,37.883972,-2
56305,Steuer_1,down,False,Märchensteuer,Märchensteuer,,,,Steuer,NN,N,1,übrige,36.0,126.621284,-2
72955,Punkt,up,False,Ort,Ort,,,,Punkt,NN,N,"9,14",,6.0,1526.807129,-2
240674,Geruch,def,False,Geruchsorgan,Geruchsorgan,,,,Geruch,NN,N,1,,1.0,2919.970215,-2
351914,äbsch,syn,False,äbich,äbich,,,,äbsch,ADJA,A,1,rheinisch,1.0,21.519691,-2


In [531]:
#mask = translate_lex.data.isin(['besonnte','überlegte'])
#grid(translate_lex,mask)

In [532]:
#translate_lex_bak = translate_lex.copy()

# rückgängig
# translate_lex = translate_lex_bak.copy()

### Kontrolle wiktionary_merkmal (aus d01)
* Diverse Kontrollen

In [533]:
# plan_merkmal Steuertabelle neu erstellen
try:
    del plan_merkmal
except:
    pass

from Steuertabellen.s01_Steuertabellen import *
plan_merkmal = plan_merkmal_erstellen()
#pak.sample(plan_merkmal)
grid(plan_merkmal)

103 rows


,merkmal,template,collect,plan,sort,is_lex
0,noSteig,kSt.,E,1,,False
1,noSing,kSg.,E,1,,False
2,nurWortart,Wortart fehlt,P,1,,False
3,istName,Navigationsleiste Anthroponyme,E,1,,False
4,Wikispecies,Wikispecies,E,1,,False
...,...,...,...,...,...,...
98,Superlativ,,,3,x3 Superlativ,True
99,Stamm,,,3,z1 Stamm,False
100,ipa,,,3,z1 ipa,False
101,noFlex,,E,3,z1 noFlex,True


In [534]:
# Merkmale lt. Steuertabelle und lt. Realität
mask = (plan_merkmal.plan != 2)
merkmale_plan  = set(plan_merkmal[mask].merkmal) - set(member_replaces.keys())
merkmale_data  = set(wiktionary_merkmal.merkmal.unique())
merkmale_trash = set(wiktionary_merkmal_trash.merkmal.unique())

In [535]:
# left_only:  Merkmale, die in der Realität nicht auftreten
# right_only: Merkmale, die nicht geplant sind
a = bpy.minivenn(merkmale_plan,merkmale_data)
bpy.minivenn(merkmale_plan,merkmale_data,format='print2')

left_only:  {'Suffix', 'Imp', 'Präfix', 'Konj1', 'Konj', 'gm', 'istName', 'Befehl ihr', 'abk', 'Befehl du', 'Positiv', 'übersicht', 'Taxonomie', 'down_part', 'noPl', 'noSteig', 'noFlex', 'Konj2', 'PTKVZ', 'noSg', 'noSing'}
right_only: {}



In [536]:
# Welche vermissten Merkmale treten in wiktionary_merkmal_trash auf?
a = bpy.minivenn( a['left_only'] , merkmale_trash)
mask = wiktionary_merkmal_trash.merkmal.isin(a['both'])
pak.analyse_freqs(wiktionary_merkmal_trash[mask],['merkmal','lemma'])

,merkmal,merkmal_count,merkmal_percent,lemma,lemma_count
0,Positiv,14051,86.7,"[<NA>, griawig, wahrlich, bankähnlich, siebenfach, hybrid, hunderterste, schal, sechsfach, einhunderterste, hundertzweite, kleinkariert, elffach, zwölffach, dreißigfach, zwanzigfach, hunderttausendfach, millionenfach, hundertfach, neunfach]","[4, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]"
1,abk,2019,12.5,<NA>,2019
2,gm,131,0.8,<NA>,131
3,Befehl du,1,0.0,umeinanderhetzen,1
4,Befehl ihr,1,0.0,umeinanderhetzen,1


In [537]:
# Veränderungen lemma_tag >> data_tag
# nach Merkmal
mask1 = wiktionary_merkmal.data_tag != wiktionary_merkmal.lemma_tag
mask2 = wiktionary_merkmal.data_tag != ''
mask = mask1  &  mask2
pak.analyse_freqs(wiktionary_merkmal[mask], ['merkmal','lemma_tag','data_tag',])

,merkmal,merkmal_count,merkmal_percent,lemma_tag,lemma_tag_count,data_tag,data_tag_count
0,Ptz2,28426,24.5,"[VVFIN, VMFIN, VAFIN]","[28398, 16, 12]","[ADJA, VVPP, VMPP, VAPP]","[14213, 14199, 8, 6]"
1,Imp Sg,23933,20.6,"[VVFIN, VAFIN, VMFIN]","[23922, 9, 2]","[VVIMP, VAIMP]","[23924, 9]"
2,Gerundivum,16480,14.2,VVFIN,16480,ADJA,16480
3,lemma,14038,12.1,"[VVFIN, ADJA, ADV, PDAT, VMFIN, APPO, NN, VAFIN, PRELS, APPR, PWAT, PRELAT, PIAT, KON, PPER]","[13853, 92, 37, 8, 7, 7, 6, 5, 4, 4, 4, 4, 3, 3, 1]","[VVINF, CARD, ADJA, APPR, ART, VMINF, PRO, PIAT, VAINF, KOUS, ITJ, PDAT, NE, KON, PTKANT, PDS, DIV, PRF, PWAT]","[13853, 87, 27, 14, 8, 7, 6, 6, 5, 5, 4, 4, 3, 3, 2, 1, 1, 1, 1]"
4,Ptz1,13635,11.7,"[VVFIN, VMFIN, VAFIN]","[13624, 7, 4]",ADJA,13635
5,Imp Pl,13538,11.7,"[VVFIN, VAFIN, VMFIN]","[13532, 5, 1]","[VVIMP, VAIMP]","[13533, 5]"
6,VVIZU,5714,4.9,VVFIN,5714,VVIZU,5714
7,alt_2,56,0.0,"[ADJA, ADV, NN, PPER, APPO]","[48, 4, 2, 1, 1]","[CARD, PIAT, APPR, ADJA, PTKANT, PRF]","[45, 3, 3, 2, 2, 1]"
8,Dat Pl,14,0.0,"[PDAT, PRELS, NN, PIAT]","[9, 2, 2, 1]","[ART, PRO, CARD, NE, PTKANT, PDS]","[9, 1, 1, 1, 1, 1]"
9,Gen Pl,13,0.0,"[PDAT, NN, PRELS, PIAT]","[8, 2, 2, 1]","[ART, PTKANT, NE, PDS, CARD, PRO]","[8, 1, 1, 1, 1, 1]"


In [538]:
# TODO: Zeile lemma untersuchen
raise Stop

Stop Time:  23:13:24
Elapsed:    36 mins, 44 secs


In [ ]:
# TODO_TAGS_IN_MERKMALE
bgc('orange')
mask = wiktionary_merkmal.merkmal.isin({'VAFIN', 'VMFIN', 'VVIMP', 'VVIZU',})
wiktionary_merkmal[mask]

In [539]:
# Keine nan in data
assert pak.nnan(wiktionary_merkmal.data) == 0

In [540]:
# Eindeutige Datentypen
for col in wiktionary_merkmal.columns:
    print(col)
    if col == 'num':
        continue
    assert pak.ntypes(wiktionary_merkmal[col]) == 1

lemma_id
merkmal
is_lex
data_id
data
data_tag
data_tagZ
data_tagZZ
lemma
lemma_tag
lemma_tagZZ
num
meta
merkmal_order
data_score
hunspell


In [541]:
wiktionary_merkmal.shape

(2586444, 16)

## translate_lemma ==========
* Datensätze aus translate_lex, deren IDs von Lemma zu Lemma zeigen
* Aber keine Lemmas (also keine Verweise auf sich selbst)
* Datensätze aus translate_abk

In [542]:
# Beispiel anzeigen
mask = translate_lex.data == 'Ackermann'
translate_lex[mask]

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower
792954,Ackermann,B,Ackermann,NN,N,31.476034,merkmal alt_1,Ackersmann,B,Ackersmann,NN,N,94.428101,0.9,ackersmann
1060035,Ackermann_1,B,Ackermann,NE,N,471.455069,merkmal lemma,Ackermann_1,B,Ackermann,NE,N,449.514771,1.0,ackermann
1064575,Ackermann,B,Ackermann,NN,N,242.296289,merkmal lemma,Ackermann,B,Ackermann,NN,N,231.020447,1.0,ackermann


In [543]:
# translate_lemma erzeugen
keine_lemmas = ['einen','beigen','Montage',]

mask1 =  translate_lex.data_home == 'B'             # beides sind lemmas
mask2 =  translate_lex.data != translate_lex.lemma  # sie sind verschieden
mask3 = ~translate_lex.data.isin(keine_lemmas)      # gehören nicht zu den Ausnahmen
mask4 = ~translate_lex.member.str.contains('Ptz1')  # kein Partizip1
mask5 = ~translate_lex.member.str.contains('Ptz2')  # kein Partizip2
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(translate_lex,mask,60004)
#grid(translate_lex,mask)
translate_lex, translate_lemma = pak.move_rows(translate_lex,mask, msg=None)

check_mask:  60001 rows
Move 60001 rows from 1263010


In [544]:
pak.sample(translate_lemma)

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower
11281,Eis,B,Eis,,N,48.841379,lma alt_1 meist,eislaufen,B,eislaufen,VVFIN,V,112.417236,0.333333,eislaufen
72503,durchfahrend,B,durchfahrend,,A,0.740692,gm,durchfahren_MADJ,B,durchfahren,ADJA,A,6.718384,0.916667,durchfahren
143116,Azubi,B,Azubi,NN,N,3.115557,merkmal alt_3 alt_weib,Lehrmädchen,B,Lehrmädchen,NN,N,443.351532,0.000000,lehrmädchen
163401,Hanke_N,B,Hanke,NE,N,86.482301,merkmal alt_3 alt_klein,Johannes,B,Johannes,NE,N,5441.716797,0.500000,johannes
191898,Schnecken,B,Schnecken,NN,N,28715.024786,merkmal Nom Pl Gen Dat Akk,Schnecke,B,Schnecke,NN,N,21656.921875,0.888889,schnecke
211737,dasjenige,B,dasjenige,ART,X,85.037130,merkmal Nom Sg n zweittag Akk,demjenigen,B,demjenigen,PDAT ART,P,142.611603,0.700000,demjenigen
211914,der,B,der,ART,X,235.090453,merkmal Nom Sg m Gen f Dat Pl,die,B,die,ART,X,201.369171,0.666667,die
745632,vorkommend,B,vorkommend,,A,0.412157,gm,vorkommen,B,vorkommen,VVFIN,V,333.464844,0.900000,vorkommen
748029,entfloht_2,B,entfloht,,A,0.044761,gm,entflohen_2,B,entflohen,ADJA,A,0.045710,0.777778,entflohen
771541,Häuschen,B,Häuschen,NN,N,8344.013726,merkmal klein,Haus,B,Haus,NN,N,108960.914062,0.375000,haus


### translate_abk

In [545]:
# Alle Kurzformen sind in wiktionary_lemma
mask_abk = wiktionary_lemma.member.str.contains('Abkürzung')

mask = ~translate_abk.kurz_id.isin(wiktionary_lemma[mask_abk].lemma_id)
check_mask(translate_abk,mask,0)

#msg = 'kurz_id inzwischen verwaist'
#translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg )
#grid(translate_abk_trash,msg)

check_mask:  0 rows


In [546]:
# Beispiel
mask = translate_abk.lang == 'Krankenhaus'
grid(translate_abk,mask)

4 rows out of 2208


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,konkurrenz
3802,Kh.,Kh.,NN,Abkürzung,4.933620,Krankenhaus,Krankenhaus,NN,,52026.234375,lemma,187295.15625,1
3932,Krh.,Krh.,NN,Abkürzung,5.775660,Krankenhaus,Krankenhaus,NN,,52026.234375,lemma,187295.15625,1
3935,Krhs.,Krhs.,NN,Abkürzung,6.462754,Krankenhaus,Krankenhaus,NN,,52026.234375,lemma,187295.15625,1
3940,Krkhs.,Krkhs.,NN,Abkürzung,5.818603,Krankenhaus,Krankenhaus,NN,,52026.234375,lemma,187295.15625,1


In [547]:
assert not pak.any_nan(translate_abk)

In [548]:
# Umbenennungen. data und lemma sind nicht passende Bezeichnungen, aber pflege_translate_tabelle wünscht diese. 
# Wird später durch passendere ersetzt.
spalten = ['kurz_id','lang_id','todo']
df = translate_abk[spalten].copy()
df = pak.rename_col(df, 'kurz_id', 'data_id')
df = pak.rename_col(df, 'lang_id', 'lemma_id') 
df = pak.rename_col(df, 'todo', 'member') 
df['member'] = ('Abkürzung ' + df.member).str.strip()
df.head(1)

,data_id,lemma_id,member
1092,Bd.,Band,Abkürzung lemma


In [549]:
# lang_id war kein Lemma >> translate_lex nachschlagen
df['lemma_id_new'] = ''
mask = ~df.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(df,mask,0, 30)

df.loc[mask] = pak.update_col( df[mask],          
                               translate_lex,
                               col_score='data_score', 
                               left_on='lemma_id',
                               right_on='data_id',
                               col='lemma_id',
                               col_rename='lemma_id_new',
                               
                       )   

mask = df.lemma_id_new != ''
df.loc[mask,'lemma_id'] = df[mask].lemma_id_new

df = pak.drop_cols(df, 'lemma_id_new')
#df[mask]

check_mask:  0 rows
update_col: col_score data_score applied, 358353 records less!
update_col: 0 cells written into existing column


In [550]:
# Müll löschen
löschmich = ['PLURV','PIMS','MRSA','EBRG','BodSchätzDB','ZÄPrO','SJZ','Mmes.','RÄ','GDPdU','E_570','E_163']
mask = df.data_id.isin(löschmich)
df = pak.drop_rows(df,mask)

No rows deleted


In [551]:
# dups löschen
mask = df.duplicated(subset=['data_id','lemma_id'])
df = pak.drop_rows(df,mask)

No rows deleted


In [552]:
assert not pak.any_nan(df)

In [553]:
#pak.nnan(df)

In [554]:
# lemma_score nachschlagen
df = pak.update_col(df,          
                    wiktionary_lemma,
                    left_on='data_id',
                    right_on='lemma_id',
                    col='lemma_score',
                       )   

update_col: 2208 cells written into new column


In [555]:
# lemma_score übernehmen
df['data_score'] = df.lemma_score.copy()

# abwerten, wenn nicht manuell
mask = ~df.member.str.contains('manuell')
df.loc[mask,'data_score'] *= 0.8

In [556]:
# pflege_translate_tabelle
df = pflege_translate_tabelle(df, translate_tagZ, wiktionary_lemma, translate_lex )

rank: 358353 rows less, now 844656 rows
update_col: 2188 cells written into new column
update_col: 2208 cells written into new column
update_col: 2208 cells written into existing column
update_col: 2208 cells written into new column
update_col: 2208 cells written into new column
update_col: 2208 cells written into existing column


In [557]:
# data_swaped: dächtest um >> umdächtest
df['data_swaped'] = df.data.str.split()
mask = df.data_swaped.str.len() == 2
check_mask(df,mask)
df.loc[ mask, 'data_swaped'] = df[mask].data_swaped.str[-1] + df[mask].data_swaped.str[0]
df.loc[~mask, 'data_swaped'] = ''

check_mask:  102 rows


In [558]:
#pak.analyse_cols(df)

In [559]:
# apply analyse_stringdiff 
df['same_lexlemma']   = 0.0
df = df.apply( analyse_stringdiff, axis=1 )

In [560]:
# Beispiel
mask = df.lemma == 'Krankenhaus'
grid(df,mask)

4 rows out of 2208


,data_id,data_home,data,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower,data_swaped,same_lexlemma
2,Kh.,B,Kh.,N,3.946896,Abkürzung lemma,Krankenhaus,B,Krankenhaus,NN,N,52026.234375,krankenhaus,,0.181818
3,Krh.,B,Krh.,N,4.620528,Abkürzung lemma,Krankenhaus,B,Krankenhaus,NN,N,52026.234375,krankenhaus,,0.272727
4,Krhs.,B,Krhs.,N,5.170203,Abkürzung lemma,Krankenhaus,B,Krankenhaus,NN,N,52026.234375,krankenhaus,,0.363636
5,Krkhs.,B,Krkhs.,N,4.654882,Abkürzung lemma,Krankenhaus,B,Krankenhaus,NN,N,52026.234375,krankenhaus,,0.454545


In [561]:
# same_lexlemma auf scores übertragen
df['data_score']  = df.data_score  * ((df.same_lexlemma + 0.1)**0.5)
df['lemma_score'] = df.lemma_score * ((df.same_lexlemma + 0.1)**0.5)

In [562]:
# data_swaped wieder löschen
df = pak.drop_cols(df,'data_swaped')

In [563]:
# data_tag
df['data_tag'] = df.lemma_tag.copy()

In [564]:
# Spaltenvergleich
bpy.minivenn( df.columns, translate_lemma.columns, format='print2')

left_only:  {}
right_only: {}



In [565]:
# Beispiel
mask = df.lemma == 'Krankenhaus'
grid(df,mask)

4 rows out of 2208


,data_id,data_home,data,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower,same_lexlemma,data_tag
2,Kh.,B,Kh.,N,2.095271,Abkürzung lemma,Krankenhaus,B,Krankenhaus,NN,N,27618.933058,krankenhaus,0.181818,NN
3,Krh.,B,Krh.,N,2.820897,Abkürzung lemma,Krankenhaus,B,Krankenhaus,NN,N,31762.741408,krankenhaus,0.272727,NN
4,Krhs.,B,Krhs.,N,3.520435,Abkürzung lemma,Krankenhaus,B,Krankenhaus,NN,N,35425.104379,krankenhaus,0.363636,NN
5,Krkhs.,B,Krkhs.,N,3.466389,Abkürzung lemma,Krankenhaus,B,Krankenhaus,NN,N,38742.796733,krankenhaus,0.454545,NN


In [566]:
# add_rows
translate_lemma = pak.add_rows(translate_lemma, df)

2208 rows added, now a total of 62209


In [567]:
# rename_col
translate_lemma = pak.rename_col(translate_lemma, 'data_id',     'lem_id')
translate_lemma = pak.rename_col(translate_lemma, 'data_home',   'lem_home')
translate_lemma = pak.rename_col(translate_lemma, 'data',        'lem')
translate_lemma = pak.rename_col(translate_lemma, 'data_score',  'lem_score')

In [568]:
translate_lemma = pak.rename_col(translate_lemma, 'data_tag',  'lem_tag')
translate_lemma = pak.rename_col(translate_lemma, 'data_tagZZ','lem_tagZZ')

In [569]:
grid(translate_lemma, pak.sample)

16 rows out of 62209


,lem_id,lem_home,lem,lem_tag,lem_tagZZ,lem_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower
0,Eis,B,Eis,,N,48.841379,lma alt_1 meist,eislaufen,B,eislaufen,VVFIN,V,112.417236,0.333333,eislaufen
30,durchfahrend,B,durchfahrend,,A,0.740692,gm,durchfahren_MADJ,B,durchfahren,ADJA,A,6.718384,0.916667,durchfahren
216,Aaken,B,Aaken,NN,N,8.363235,merkmal Nom Pl Gen Dat Akk,Aake,B,Aake,NN,N,66.117172,0.800000,aake
290,Azubi,B,Azubi,NN,N,3.115557,merkmal alt_3 alt_weib,Lehrmädchen,B,Lehrmädchen,NN,N,443.351532,0.000000,lehrmädchen
550,Hanke_N,B,Hanke,NE,N,86.482301,merkmal alt_3 alt_klein,Johannes,B,Johannes,NE,N,5441.716797,0.500000,johannes
603,Ivan,B,Ivan,NE,N,52.391504,merkmal alt_2 alt_weib,Iwan_1,B,Iwan,NE,N,262.428467,0.750000,iwan
778,Mendelsch,B,Mendelsch,ADJA,A,3.619957,merkmal alt alt_2,mendelsch,B,mendelsch,ADJA,A,51.772400,1.000000,mendelsch
986,Schnecken,B,Schnecken,NN,N,28715.024786,merkmal Nom Pl Gen Dat Akk,Schnecke,B,Schnecke,NN,N,21656.921875,0.888889,schnecke
1270,dasjenige,B,dasjenige,ART,X,85.037130,merkmal Nom Sg n zweittag Akk,demjenigen,B,demjenigen,PDAT ART,P,142.611603,0.700000,demjenigen
2970,angehörend,B,angehörend,,A,0.414223,gm,angehören,B,angehören,VVFIN,V,264.909546,0.900000,angehören


In [570]:
# lem_tag
translate_lemma = pak.update_col( translate_lemma,          
                                  wiktionary_lemma,
                                  left_on='lem_id',
                                  right_on='lemma_id',
                                  col='tag',
                                  col_rename = 'lem_tag',
                       )   

translate_lemma = pak.update_col( translate_lemma,          
                                  wiktionary_lemma,
                                  left_on='lem_id',
                                  right_on='lemma_id',
                                  col='tagZZ',
                                  col_rename = 'lem_tagZZ',
                       )   

translate_lemma['lem_tag']   = translate_lemma.lem_tag.fillna('')
translate_lemma['lem_tagZZ'] = translate_lemma.lem_tagZZ.fillna('')
#translate_lemma = pak.move_cols( translate_lemma, ['lem_tag','lem_tagZZ'],'lem')      

update_col: 62209 cells written into existing column
update_col: 62209 cells written into existing column


### belöschen

In [571]:
# translate_lemma_trash
translate_lemma_trash = translate_lemma.head(0)
translate_lemma_trash['msg'] = ''

In [572]:
a = pak.analyse_freqs(translate_lemma,['lemma','lem'])
grid(a)

34454 rows


,lemma,lemma_count,lemma_percent,lem,lem_count
0,Johannes,52,0.1,"[Hanke, Hannes, Hans, John, Johnny, Evan, Giovanni, Hanne, Honza, Ion, Ioannis, Ionel, Ivan, Ivo, Iwan, Jack, Jahn, Jake, Jan, Janek]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,Iwan,45,0.1,"[Ivan, Ivana, Wanja, Evan, Giovanni, Hanne, Hannes, Hanke, Hans, Honza, Ion, Ioannis, Ionel, Ivo, Jack, Jahn, Jake, Jan, Janek, Janika]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,Johanna,22,0.0,"[Joana, Joanna, Janna, Jente, Hansi, Ivana, Jana, Jane, Janet, Janina, Janine, Janka, Jean, Jeanette, Joan, Jojo, Joke, Jona, Jonna, Hanna]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,Peter,20,0.0,"[Peters, Petrus, Peet, Pietsch, Peer, Pidder, Pete, Piet, Pit, Petra, Petrine, Pedersen, Petereit, Peterling, Petersen]","[2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
4,Nikolaus,19,0.0,"[Kola, Nikoläusin, Nicolaus, Nikolai, Nikolas, Klaus, Niklas, Niko, Klaas, Niels, Nils, Klose, Nickel, Nicki, Nicky, Niki, Nicole, Nikola]","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
...,...,...,...,...,...
34449,Zinn,1,0.0,Zinne,1
34450,Freundschaftsband,1,0.0,Freundschaftsbande,1
34451,Talk,1,0.0,Talke,1
34452,Hub,1,0.0,Hube,1


In [573]:
#translate_lemma

In [574]:
# aussortiert1
erlaubt = ['H2O','CAD','CH','GAU','DIN','UN','UNO','LCD','IGS','2D','3D','4D','ISP','IR','DRK','MPU','OOP','ÖPNV','° C','°C','HDTV','° F','°F']
mask1 =  translate_lemma['same_lexlemma'] < 0.3
mask2 =  translate_lemma.lem.str[0] != translate_lemma.lemma.str[0]
mask3 =  translate_lemma.lem.str.len() <= 4
mask4 =  translate_lemma.lem.str.isupper()
mask5 = ~translate_lemma.lem.isin(erlaubt)
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(translate_lemma,mask,0,212)

msg = 'aussortiert1'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg)
grid(translate_lemma_trash,msg)

check_mask:  143 rows
Move 143 rows from 62209
143 rows


,lem_id,lem_home,lem,lem_tag,lem_tagZZ,lem_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower,msg
50141,ACAB,B,ACAB,NN,N,0.936565,merkmal alt_2,1312,B,1312,NN,N,88.850334,0.000000,1312,aussortiert1
60018,ISBN,B,ISBN,NN,N,42.556359,Abkürzung lemma,Nummer,B,Nummer,NN,N,3756.431762,0.166667,nummer,aussortiert1
60035,ABC_1,L,ABC,NN,N,1.638859,Abkürzung lemma,Kampf,B,Kampf,NN,N,7620.133918,0.200000,kampf,aussortiert1
60036,DOS,B,DOS,NN,N,8.595088,Abkürzung lemma,System,B,System,NN,N,6672.337382,0.166667,system,aussortiert1
60056,VIE,B,VIE,NN,N,3.617953,Abkürzung lemma,Kreis,B,Kreis,NN,N,4896.151314,0.200000,kreis,aussortiert1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62165,SCNR,B,SCNR,FM,Z,10.634649,Abkürzung lemma,could_M,B,could,FM,Z,3.679811,0.200000,could,aussortiert1
62167,KH,B,KH,NE,N,59.234021,Abkürzung lemma,Bad_Kreuznach,B,Bad Kreuznach,NE,N,6.543065,0.153846,bad kreuznach,aussortiert1
62194,EOS,B,EOS,NN,N,26.047257,Abkürzung lemma,erweitert,B,erweitert,ADJA,A,0.221650,0.111111,erweitert,aussortiert1
62203,ASU,B,ASU,NN,N,66.576150,Abkürzung lemma,eingeführt,B,eingeführt,ADJA,A,0.189138,0.000000,eingeführt,aussortiert1


In [575]:
# aussortiert2
# 

löschmich = ['-','Hag','DrIng','Deik','Fritsch','Frerk', 'Fricke','nomen nominandum',
             'wullacken','Diastema mediale','BM', 'JO', 'NB',
             'Rosch ha-Schana','Wiktionary','Fuzo', 'Fuzo.',
             'Intern', 'Intern.', 'iNet', 'iNet.', 'ZUM', 'ut supra','Art', 'GSSD', 
             'Met','DE', 'DFV', 'DHB', 'DSV','AN', 'AS', 'AUR','GVBl', 'GWB', 'GenG', 'Ges',
             'BKG', 'BVT', 'BfN', 'BfS','ritenuto','Folio','Aserbaidschan-Manat',
             'sensu lato','nes.','ceterum censeo',
             'pro rata','GGVS','IRPA','Tau','ABC',
             'ISA','DOS','ABF','ÖFB','VOC','KPÖ','IN','ZIF','PVP','BDO','g. e.','AUT','EUV',
            ]
behalten  = ['nen','ne','nes', 'geeint','einend', ]
mask1A =  translate_lemma.lem.isin(löschmich)
mask1B =  translate_lemma.lemma.isin(löschmich)
mask2  = ~translate_lemma.lem.isin(behalten)
mask = (mask1A | mask1B)  &  mask2
check_mask(translate_lemma,mask,36)

msg = 'aussortiert2'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg)
grid(translate_lemma_trash,msg)

check_mask:  36 rows
Move 36 rows from 62066
36 rows out of 179


,lem_id,lem_home,lem,lem_tag,lem_tagZZ,lem_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower,msg
29447,ABC,B,ABC,NN,N,113.490156,merkmal alt_2,Abc,B,Abc,NN,N,973.877563,1.000000,abc,aussortiert2
29449,ABC,B,ABC,NN,N,37.728275,merkmal alt_2,Abece,B,Abece,NN,N,405.845215,0.600000,abece,aussortiert2
29522,Frerk,B,Frerk,NE,N,0.915965,merkmal alt_3,Friedrich_1,L,Friedrich,NE,N,111.723572,0.444444,friedrich,aussortiert2
29523,Fricke_N,B,Fricke,NE,N,0.915965,merkmal alt_3,Friedrich_1,L,Friedrich,NE,N,111.723572,0.444444,friedrich,aussortiert2
29528,Fritsch_N,B,Fritsch,NE,N,1.005095,merkmal alt_3,Friedrich_1,L,Friedrich,NE,N,111.723572,0.555556,friedrich,aussortiert2
30174,Deik_V,B,Deik,NE,N,7.851753,merkmal alt_klein,Adelheid,B,Adelheid,NE,N,344.011871,0.375000,adelheid,aussortiert2
30624,Abc,B,Abc,NN,N,2.545074,merkmal alt_2,ABC,B,ABC,NN,N,72.798981,1.000000,abc,aussortiert2
30625,Abece,B,Abece,NN,N,2.030267,merkmal alt_2,ABC,B,ABC,NN,N,72.798981,0.600000,abc,aussortiert2
33030,Hag_1,B,Hag,NN,N,0.141378,merkmal alt_1,Hagen_2,B,Hagen,NN,N,5.069370,0.600000,hagen,aussortiert2
34555,Roscheschone,B,Roscheschone,NN,N,5.404711,merkmal alt_1,Rosch_ha-Schana,B,Rosch ha-Schana,NN,N,193.795959,0.600000,rosch ha-schana,aussortiert2


In [576]:
# unabhängige_lexeme

unabhängige_lexeme  = ['Alu','ALU','Arbeitslosenunterstützung','CO2', 'CO₂','chemisch','USK','vergl', 'verglasen','vergl.','24/7','Stunde','Schriftwerk','Buch','und','E 962','Verbindung',
                       'km/h','Kilometer','weiblich','männlich','Elefant','frühneuhochdeutsch','sein','Disko','modern','tun','Handlung',
                      ]
mask1 = translate_lemma.lem.isin(unabhängige_lexeme)
mask2 = translate_lemma.lemma.isin(unabhängige_lexeme)
mask = mask1  &  mask2
check_mask(translate_lemma,mask,9)

msg = 'unabhängige_lexeme'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg)
grid(translate_lemma_trash,msg)

check_mask:  9 rows
Move 9 rows from 62030
9 rows out of 188


,lem_id,lem_home,lem,lem_tag,lem_tagZZ,lem_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower,msg
28351,frühneuhochdeutsch,B,frühneuhochdeutsch,ADJA,A,10.204680,merkmal alt,sein,B,sein,VAFIN,V,782.926025,0.055556,sein,unabhängige_lexeme
28620,weiblich,B,weiblich,ADJA,A,186.078926,merkmal weib,Elefant,B,Elefant,NN,N,943.592407,0.250000,elefant,unabhängige_lexeme
28622,männlich,B,männlich,ADJA,A,149.195055,merkmal männ,Elefant,B,Elefant,NN,N,943.592407,0.125000,elefant,unabhängige_lexeme
34840,CO2,B,CO2,NN,N,28.506790,merkmal alt_2,CO₂,B,CO₂,NN,N,976.711426,0.666667,co₂,unabhängige_lexeme
50362,CO₂,B,CO₂,NN,N,2.165834,merkmal lex manuell,CO2,B,CO2,NN,N,80.925072,0.666667,co2,unabhängige_lexeme
60015,24/7,B,24/7,ADJA,A,3.780171,Abkürzung lemma,Stunde,B,Stunde,NN,N,1948.119300,0.000000,stunde,unabhängige_lexeme
60070,km/h,B,km/h,NN,N,4.886722,Abkürzung lemma,Kilometer,B,Kilometer,NN,N,1693.510942,0.222222,kilometer,unabhängige_lexeme
60094,E_962,B,E 962,NN,N,3.832166,Abkürzung lemma,Verbindung,B,Verbindung,NN,N,2682.317705,0.100000,verbindung,unabhängige_lexeme
60110,USK,B,USK,KON,X,83.566152,Abkürzung lemma,und,B,und,KON,X,1464.938522,0.333333,und,unabhängige_lexeme


In [577]:
# Mögliche Probleme #TODO
# 
spalten = ['lem','lemma','member','lemma_score']
mask1 =  translate_lemma['same_lexlemma'] < 0.3
mask2 = translate_lemma.lem.str[0] != translate_lemma.lemma.str[0]
mask = mask1  &  mask2
a = translate_lemma[mask][spalten].sort_values(['lemma_score'], ascending=[False])
grid(a)

357 rows


,lem,lemma,member,lemma_score
29544,weiblich,Taube,merkmal weib,28314.085938
29546,Weibchen,Taube,merkmal weib,28314.085938
29547,männlich,Taube,merkmal männ,28314.085938
29552,Männchen,Taube,merkmal männ,28314.085938
32724,Entbindungspfleger,Hebamme,merkmal männ,21891.761719
...,...,...,...,...
50347,5.,fünftens,merkmal lex manuell,6.718384
62174,HDTV,hochauflösendes Fernsehen,Abkürzung lemma,6.153261
62171,k. u.,Transleithanien,Abkürzung lemma,2.616268
50357,6.,sechstens,merkmal lex manuell,2.239461


In [578]:
# lem_tag übertragen, falls nötig
mask = translate_lemma.lem_tag.isnull()
check_mask(translate_lemma,mask,0,10)

translate_lemma.loc[mask,'lem_tag']   = translate_lemma[mask].lemma_tag
translate_lemma.loc[mask,'lem_tagZZ'] = translate_lemma[mask].lemma_tagZZ

check_mask:  0 rows


In [579]:
# alle getaggt
maskA = translate_lemma.lem_tag == ''
maskB = translate_lemma.lem_tagZZ == ''
mask = maskA | maskB
check_mask(translate_lemma,mask,0)

check_mask:  0 rows


In [580]:
mask = translate_lemma.isna().any(axis=1)
translate_lemma[mask]

,lem_id,lem_home,lem,lem_tag,lem_tagZZ,lem_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower


In [581]:
#assert not pak.any_nan(translate_lemma)
pak.nnan(translate_lemma)

Series([], dtype: int64)

In [582]:
pak.sample(translate_lemma)

,lem_id,lem_home,lem,lem_tag,lem_tagZZ,lem_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower
0,Eis,B,Eis,NN,N,48.841379,lma alt_1 meist,eislaufen,B,eislaufen,VVFIN,V,112.417236,0.333333,eislaufen
30,durchfahrend,B,durchfahrend,ADJA,A,0.740692,gm,durchfahren_MADJ,B,durchfahren,ADJA,A,6.718384,0.916667,durchfahren
290,Azubi,B,Azubi,NN,N,3.115557,merkmal alt_3 alt_weib,Lehrmädchen,B,Lehrmädchen,NN,N,443.351532,0.000000,lehrmädchen
550,Hanke_N,B,Hanke,NE,N,86.482301,merkmal alt_3 alt_klein,Johannes,B,Johannes,NE,N,5441.716797,0.500000,johannes
603,Ivan,B,Ivan,NE,N,52.391504,merkmal alt_2 alt_weib,Iwan_1,B,Iwan,NE,N,262.428467,0.750000,iwan
778,Mendelsch,B,Mendelsch,ADJA,A,3.619957,merkmal alt alt_2,mendelsch,B,mendelsch,ADJA,A,51.772400,1.000000,mendelsch
986,Schnecken,B,Schnecken,ITJ,X,28715.024786,merkmal Nom Pl Gen Dat Akk,Schnecke,B,Schnecke,NN,N,21656.921875,0.888889,schnecke
1310,der,B,der,ART,X,235.090453,merkmal Nom Sg m Gen f Dat Pl,die,B,die,ART,X,201.369171,0.666667,die
3072,auskennend,B,auskennend,ADJA,A,0.410091,gm,auskennen,B,auskennen,VVFIN,V,155.924927,0.900000,auskennen
28373,Häuschen,B,Häuschen,NN,N,8344.013726,merkmal klein,Haus,B,Haus,NN,N,108960.914062,0.375000,haus


## wiktionary_merkmal_text ==========

In [583]:
# lemma übertragen
assert not pak.any_nan(wiktionary_lemma, without=['page_id','page_part','data_id',])
wiktionary_merkmal_text = pak.update_col( wiktionary_merkmal_text, wiktionary_lemma, on='lemma_id', col='lemma'  )

update_col: 646116 cells written into new column


In [584]:
# verwaise_lemma_id finden
mask = wiktionary_merkmal_text.lemma.isnull()
check_mask(wiktionary_merkmal_text,mask)
#verwaise_lemma_id = wiktionary_merkmal_text[mask].lemma_id.drop_duplicates()
#verwaise_lemma_id

check_mask:  11392 rows


In [585]:
# Warum sind diese Datensätze verwaist? Löschursachen
mask = wiktionary_merkmal_text.lemma.isnull()
check_mask(wiktionary_merkmal_text,mask)
l = wiktionary_merkmal_text[mask].lemma_id

mask = wiktionary_lemma_trash.lemma_id.isin(l)
pak.analyse_freqs(wiktionary_lemma_trash[mask], ['msg','lemma'])

check_mask:  11392 rows


,msg,msg_count,msg_percent,lemma,lemma_count
0,translate_abk1,2084,91.2,"[FDA, PC, 1 Chr, 1 Kön, 1 Makk, 2 Chr, 2 Kön, 2 Makk, 2 Mos, 3 Makk, 3 Mos, 4 Makk, 4 Mos, 5 Mos, A.C.A.B., A. B., AAA, AAB, AABG, AAG]","[2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,NE1,200,8.7,"[Aalst, Aar, Abakus, Abart, Abate, Ablaß, Abzug, Alzheimer, Ammer, Apfel, Asi, Atlas, Babekuhl, Baumeister, Baumgarten, Bausch, Becher, Berber, Bildner, Birke]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,TODO-Rest,2,0.1,"[Tempi passati, variatio delectat]","[1, 1]"


In [586]:
# Einzelne Löschursache untersuchen
mask = wiktionary_lemma_trash.msg == 'NE1'
l = wiktionary_lemma_trash[mask].lemma_id
mask = wiktionary_merkmal_text.lemma_id.isin(l)
grid(wiktionary_merkmal_text[mask])

769 rows


,lemma_id,merkmal,num,meta,data,rank_merkmal,lemma
604,Gelb_1,def,1,,"deutsch er Nachname, Familienname",100,<NA>
605,Gelb_1,bsp,1,Gelb,Herr Gelb ist als Kaninchenzüchter erfolgreich.,100,<NA>
606,Gelb_1,bsp,1,Gelb,Herr Gelb heiratet im September auf Hawaii.,200,<NA>
607,Gelb_1,bsp,1,Gelb,"Frau Gelb liebtes, an Weihnachten Plätzchen zu backen.",300,<NA>
608,Gelb_1,bsp,1,Gelbs,Die Gelbs lieben die Natur und die Tiere.,400,<NA>
...,...,...,...,...,...,...,...
631257,Schießer_1,def,1,,Familienname,100,<NA>
631258,Schießer_1,bsp,1,Schießer,Marion Schießer wird die Führung durch das Museum übernehmen.,100,<NA>
649902,Scharschmied_1,def,1,,deutscher Familienname,100,<NA>
649903,Scharschmied_1,herkunft,,,"Benennung nach Beruf, siehe oben",100,<NA>


In [587]:
# Warum sind diese Datensätze verwaist? Unbekannt
mask1 =  wiktionary_merkmal_text.lemma.isnull()
mask2 = ~wiktionary_merkmal_text.lemma_id.isin(wiktionary_lemma_trash.lemma_id)
mask = mask1  &  mask2
check_mask(wiktionary_merkmal_text,mask)
gridt(wiktionary_merkmal_text,mask)

check_mask:  5495 rows
5495 rows out of 657508


In [588]:
# Verwaiste Lemmas löschen
mask = wiktionary_merkmal_text.lemma.isnull()
check_mask(wiktionary_merkmal_text,mask, 11347)

msg = 'verwaist1'
wiktionary_merkmal_text, wiktionary_merkmal_text_trash = pak.move_rows(wiktionary_merkmal_text, wiktionary_merkmal_text_trash, mask, msg)
grid(wiktionary_merkmal_text, msg)

check_mask:  11392 rows
Move 11392 rows from 657508
No rows, mask filters them all away


In [589]:
# Striche entfernen, auch in wiktionary_merkmal_text
mask1 = wiktionary_merkmal_text.data.str.count("\'") > 1
mask2 = wiktionary_merkmal_text.merkmal.isin(['def','abk']) 
mask  =  mask1  &  mask2 
check_mask(wiktionary_merkmal_text,mask,1674)
wiktionary_merkmal_text.loc[mask,'data'] = wiktionary_merkmal_text[mask].data.str.replace("'", '')
#grid( wiktionary_merkmal_text, mask )

check_mask:  1646 rows


In [590]:
# not any_nan
assert not pak.any_nan(wiktionary_merkmal_text)
#pak.analyse_cols(wiktionary_merkmal_text)

In [591]:
# Verwaiste löschen
mask = ~wiktionary_merkmal_text.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(wiktionary_merkmal_text,mask,0)

msg = 'verwaist2'
wiktionary_merkmal_text, wiktionary_merkmal_text_trash = pak.move_rows(wiktionary_merkmal_text, wiktionary_merkmal_text_trash, mask, msg)
grid(wiktionary_merkmal_text, msg)

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


In [592]:
# meta == lemma
mask = wiktionary_merkmal_text.meta == wiktionary_merkmal_text.lemma
check_mask(wiktionary_merkmal_text,mask,219137)
wiktionary_merkmal_text.loc[mask,'meta'] = ''

check_mask:  219137 rows


In [595]:
# data leer
mask = wiktionary_merkmal_text.data == ''
check_mask(wiktionary_merkmal_text,mask,0,30)

msg = 'leer'
wiktionary_merkmal_text, wiktionary_merkmal_text_trash = pak.move_rows(wiktionary_merkmal_text, wiktionary_merkmal_text_trash, mask, msg)
grid(wiktionary_merkmal_text, msg)

check_mask:  26 rows
Move 26 rows from 646116
34 rows out of 646090


,lemma_id,merkmal,num,meta,data,rank_merkmal,lemma
67936,leer,def,1,,(fast) ohne ( physisch en) Inhalt,100,leer
67937,leer,def,2,,ohne Mensch en,200,leer
67938,leer,def,3,übertragen,übertragen ohne Mimik und Geste n; ohne geistigen Gehalt ; ohne Substanz / Tiefgang,300,leer
67939,leer,herkunft,,,"mittelhochdeutsch : lære, von althochdeutsch : lari QS",100,leer
67940,leer,bsp,1,,Die Flasche ist leer.,100,leer
67941,leer,bsp,1,,"»Ich trinke nur noch mein Glas leer, dann gehe ich.«",200,leer
67942,leer,bsp,1,,"»Mein Magen ist leer, ich muss unbedingt etwas essen.«",300,leer
67943,leer,bsp,1,,"»Die Großen schicken ihre Diener nach Wasser; aber wenn sie zum Brunnen kommen, finden sie kein Wasser und bringen ihre Gefäße leer zurück.«",400,leer
67944,leer,bsp,2,,Die Straßen sind heute leer.,500,leer
67945,leer,bsp,2,leerer,Das Kino ist heute noch leerer als sonst.,600,leer


In [596]:
#grid(wiktionary_merkmal_text, msg)

In [597]:
# suche_debug
mask = wiktionary_merkmal_text.lemma.isin(suche_debug)
grid(wiktionary_merkmal_text,mask)
#wiktionary_lemma[mask].lemma

No rows, mask filters them all away


In [598]:
# Was ist drin?
a = pak.analyse_freqs(wiktionary_merkmal_text,['merkmal','data'] )
grid(a)

6 rows


,merkmal,merkmal_count,merkmal_percent,data,data_count
0,bsp,350488,54.2,"[Beispiele, »Trifft das noch zu im Zeitalter der Weltraumfahrt? Dieses Wort ist eine maßlose Übertreibung, ebenso wie sein Synonym Astronautik. Es gibt eine Reihe von Vergleichen, die das anschaulich machen. Einer mag genügen: Wenn jemand am Alpenrand, etwa bei Grainau unweit Garmisch, in 900 m Höhe über dem Meeresspiegel steht und zum Gipfel der Zugspitze (2000 m über ihm) hinaufblickt, und wenn er seinen Hals reckt, um dort oben etwas zu erkennen, und wenn ihm dann ein hilfsbereiter Freund ein Blatt von 0,2 mm Stärke unter die Schuhe schiebt, »damit du etwas näher dran bist und besser sehen kannst«, so hat sich dieser Jemand damit dem Zugspitzgipfel nur um ein Zehnmillionstel der gegebenen Distanz genähert (1 km = 1 Million mm). Ein »Astronaut«, der den Mond betritt, hat mit der Reise von der Erde bis dorthin von der Entfernung, die uns vom sonnennächsten Fixstern trennt, noch weniger, nämlich 1 Hundertmillionstel, zurückgelegt. Andere Fixsterne sind millionen- und milliardenmal weiter entfernt.«, »Dieses den jungen Goethe ergreifende Verlangen, sich in den unendlichen Luftraum zu stürzen und über den schauerlichen Abgründen zu schweben, frei zu werden wie der Adler im Äther, beflügelt und bewegt auch die heutigen Delta- und Gleitschirmpiloten, die dank des opferbereiten Wagemuts Lilienthals und anderer Flugpioniere diesen Traum heute zu realisieren vermögen.«, »Während die Extrembergsteiger sowohl in ihren Veröffentlichungen als auch in ihren Auskünften bei meinen Befragungen eine große Offenheit und ein hohes Reflexionsniveau gegenüber diesen Problemstellungen beweisen, offenbart sich bei vielen anderen Risikogruppen noch ein erheblicher Nachholbedarf an Selbstkritik und Bereitschaft, das Gefahrenpotenzial der eigenen Sportart unvoreingenommen objektivieren zu lassen.«, »Das Misstrauen gegenüber gentechnisch veränderten Lebensmitteln ist zwar groß, allerdings sind »Gene« natürlich in jedem Lebewesen und damit auch in allen Nahrungsmitteln enthalten.«, »Der F...","[10996, 18, 16, 14, 13, 13, 12, 12, 12, 11, 11, 11, 11, 11, 11, 10, 10, 10, 10, 10]"
1,def,183819,28.5,"[männlicher Vorname, weiblicher Vorname, transitiv, eine Stadt in Baden-Württemberg, Deutschland, intransitiv, eine Stadt in Bayern, Deutschland, eine Stadt in Nordrhein-Westfalen, Deutschland, deutschsprachiger Familienname, Nachname, eine Stadt in Hessen, Deutschland, Familienname, Nachname, eine Stadt in Sachsen, Deutschland, deutscher Nachname, Familienname, eine Stadt in Niedersachsen, Deutschland, Familienname, eine Stadt in Thüringen, Deutschland, eine Stadt in Sachsen-Anhalt, Deutschland, deutsch er Nachname, Familienname, eine Stadt in Brandenburg, Deutschland, umgangssprachlich, eine Stadt in Rheinland-Pfalz, Deutschland]","[789, 567, 451, 259, 247, 243, 212, 178, 155, 137, 135, 124, 117, 115, 100, 99, 87, 85, 84, 82]"
2,herkunft,102092,15.8,"[QS, Entlehnung aus dem japanischen Üt, Entlehnung aus dem altgriechischen Üt, siehe oben, Entlehnung aus dem russisch en, Entlehnung aus dem sanskritischen Üt, von griechisch, aus dem Englischen, Das Wort ist seit dem 17. Jahrhundert belegt., Lehnwort aus dem gleichbedeutenden Hebräisch : Hebr, aus dem Lateinisch en, lateinisch, griechisch, Das Wort ist im Deutschen ein Neologismus der 2010er Jahre Online-Wortschatz-Informationssystem Deutsch -, QS Das Wort ist im Deutschen ein Neologismus der 2010er Jahre Online-Wortschatz-Informationssystem Deutsch -, unklar, seit dem 19. Jahrhundert bezeugt, friesische Kurzform von Namen, die mit dem althochdeutschen ban oder mit dem altfriesischen ban oder bon ( Aufgebot, Bann ) gebildet sind, von altgriechisch, Das Wort ist seit dem 15. Jahrhundert belegt.]","[128, 51, 26, 24, 22, 16, 15, 11, 11, 11, 10, 9, 9, 7, 7, 6, 6, 6, 6, 5]"
3,bsp_re,8844,1.4,"[umgangssprachlich, zu, den, Ref-DWDS, Ref-Duden, jemandem, die, jemanden, etwas, sterben, Iller, Lech, Isar, Inn, fließen rechts zur Donau hin. Altmühl, 

## wiktionary_lemma =====

In [599]:
# Anzahl der Lexeme untersuchen

mask = wiktionary_merkmal.is_lex == True
count_lex = pak.group_and_agg(wiktionary_merkmal[mask], 
              ['lemma_id', 'data' ],
              ['group',    'count'],
              ['',         'lex_anz'   ],                              
             )

# eintragen
wiktionary_lemma['lex_anz'] = 0
wiktionary_lemma = pak.update_col(wiktionary_lemma, count_lex, on='lemma_id', col='lex_anz')

1012504 rows less, now 168001 rows
update_col: 168001 cells written into existing column


In [600]:
# Anzahl der Nichtlexeme untersuchen

mask = wiktionary_merkmal.is_lex == False
count_lex = pak.group_and_agg(wiktionary_merkmal[mask], 
              ['lemma_id', 'data' ],
              ['group',    'count'],
              ['',         'nolex_anz'   ],                              
             )

# eintragen
wiktionary_lemma['nolex_anz'] = 0
wiktionary_lemma = pak.update_col(wiktionary_lemma, count_lex, on='lemma_id', col='nolex_anz')

1240395 rows less, now 165544 rows
update_col: 165544 cells written into existing column


In [601]:
# Dups
mask = wiktionary_lemma.duplicated(subset='lemma_id', keep=False)
grid(wiktionary_lemma,mask, color='green')

No rows, mask filters them all away


In [602]:
#TODO_member: member überprüfen wie in d01

## wiktionary_merkmal ==========

In [603]:
# verwaiste wiktionary_merkmal data_id?
mask1 =  wiktionary_merkmal.data_id != ''
mask2 =  wiktionary_merkmal.is_lex == True
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal,mask, 0)

check_mask:  0 rows


In [604]:
#a = pak.analyse_freqs(wiktionary_merkmal[mask], ['merkmal','data_id'])
#grid(a)

In [605]:
# Fehlende data_id?
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2 
check_mask(wiktionary_merkmal,mask,0, 10)

check_mask:  0 rows


### Überschneidungen
* Lexeme, die sich in translate_lex finden aber auch direkt in wiktionary_lemma >> entweder oder!

In [606]:
# mask_LL erstellen
mask1 = pak.isin(translate_lex, wiktionary_lemma, left_on=['data','lemma_tagZZ'], right_on=['lemma','tagZZ'])
mask2 = translate_lex.data != translate_lex.lemma
mask_LL = mask1  &  mask2
check_mask(translate_lex,mask_LL,0,1700)
grid(translate_lex[mask_LL].sort_values('data_score',ascending=False))

check_mask:  1567 rows
1567 rows


,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower
741211,Januar~Jan,T,Jan,NN,,10608.978858,manuell Abkürzung manuell,Januar,B,Januar,NN,N,13696.132812,0.500000,januar
741241,Gesetz~Ges,T,Ges,NN,,9914.118746,Abkürzung,Gesetz,B,Gesetz,NN,N,15998.839844,0.500000,gesetz
741245,Polizei~Pol,T,Pol,NN,,8281.904625,Abkürzung,Polizei,B,Polizei,NN,N,14239.291992,0.428571,polizei
741298,Information~Info,T,Info,NN,,4532.979003,Abkürzung,Information,B,Information,NN,N,8321.564453,0.363636,information
741365,Grammatik~Gramm,T,Gramm,NN,,3747.606633,Abkürzung,Grammatik,B,Grammatik,NN,N,5785.744629,0.555556,grammatik
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740874,Aas_2,T,Aas,,,0.005118,wiki Deklinierte_Form,Aa,B,Aa,NE,N,545.752502,0.666667,aa
740984,Aren_1,T,Aren,,,0.004151,wiki Deklinierte_Form,Ar,B,Ar,NN,N,571.012878,0.500000,ar
1228204,wegbegeben_2,B,wegbegeben,ADJA,A,0.003814,merkmal Ptz2,wegbewegen,B,wegbewegen,VVFIN,V,94.999748,0.800000,wegbewegen
812125,wegbegeben_2,B,wegbegeben,VVPP,A,0.003814,merkmal Ptz2,wegbewegen,B,wegbewegen,VVFIN,V,94.999748,0.800000,wegbewegen


In [607]:
# wiktionary_lemma belöschen: Abkürzungen wieder löschen
mask1 =  wiktionary_lemma.lemma.isin(translate_lex[mask_LL].data)
mask2 =  wiktionary_lemma.member.str.contains('Abkürzung')
mask3 = ~wiktionary_lemma.member.str.contains('manuell')
mask4 =  wiktionary_lemma.page_part == -1
#mask5 = ~wiktionary_lemma.lemma.isin(['pos',])
mask = mask1  &  mask2  &  mask3  &  mask4  
check_mask(wiktionary_lemma, mask, 0, 150)

msg = 'Überschneidungen_1'
wiktionary_lemma, wiktionary_lemma_trash = pak.move_rows(wiktionary_lemma, wiktionary_lemma_trash, mask, msg)
grid(wiktionary_lemma_trash,msg)

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


In [608]:
# wiktionary_merkmal: verwaiste_Lemmas löschen
mask1 =  wiktionary_merkmal.merkmal == 'lemma'
mask2 =  wiktionary_merkmal.data_id != ''
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3 &  mask4
check_mask(wiktionary_merkmal,mask,0,150)

msg = 'verwaiste_Lemmas'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg )
grid(wiktionary_merkmal_trash,msg)

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


In [609]:
# translate_lemma lem_id
mask = ~translate_lemma.lem_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lemma,mask,0,150)
# translate_lemma[mask]

msg = 'verwaist_lem_id'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg )
grid(translate_lemma_trash,msg)

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


In [610]:
# translate_lemma lemma_id
mask = ~translate_lemma.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lemma,mask,0)
# translate_lemma[mask]

#msg = 'verwaist_lemma_id'
#translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg )
#grid(translate_lemma_trash,msg)

check_mask:  0 rows


In [611]:
# Löschen: Einige sonstige Verwaisungen
mask1 =  wiktionary_merkmal.data_id != ''
mask2 =  wiktionary_merkmal.is_lex == True
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal,mask,0,30)
#grid(wiktionary_merkmal,mask)

msg = 'verwaist_data_id_24'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg )
grid(wiktionary_merkmal_trash,msg)

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


In [612]:
# TODO Ungelöste Probleme 
mask1 = pak.isin(translate_lex, wiktionary_lemma, left_on=['data','lemma_tagZZ'], right_on=['lemma','tagZZ'])
mask2 = translate_lex.data != translate_lex.lemma
mask_LL = mask1  &  mask2
check_mask(translate_lex,mask_LL,0,1600)
grid(translate_lex[mask_LL].sort_values('data_score',ascending=False))

check_mask:  1567 rows
1567 rows


,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower
741211,Januar~Jan,T,Jan,NN,,10608.978858,manuell Abkürzung manuell,Januar,B,Januar,NN,N,13696.132812,0.500000,januar
741241,Gesetz~Ges,T,Ges,NN,,9914.118746,Abkürzung,Gesetz,B,Gesetz,NN,N,15998.839844,0.500000,gesetz
741245,Polizei~Pol,T,Pol,NN,,8281.904625,Abkürzung,Polizei,B,Polizei,NN,N,14239.291992,0.428571,polizei
741298,Information~Info,T,Info,NN,,4532.979003,Abkürzung,Information,B,Information,NN,N,8321.564453,0.363636,information
741365,Grammatik~Gramm,T,Gramm,NN,,3747.606633,Abkürzung,Grammatik,B,Grammatik,NN,N,5785.744629,0.555556,grammatik
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740874,Aas_2,T,Aas,,,0.005118,wiki Deklinierte_Form,Aa,B,Aa,NE,N,545.752502,0.666667,aa
740984,Aren_1,T,Aren,,,0.004151,wiki Deklinierte_Form,Ar,B,Ar,NN,N,571.012878,0.500000,ar
1228204,wegbegeben_2,B,wegbegeben,ADJA,A,0.003814,merkmal Ptz2,wegbewegen,B,wegbewegen,VVFIN,V,94.999748,0.800000,wegbewegen
812125,wegbegeben_2,B,wegbegeben,VVPP,A,0.003814,merkmal Ptz2,wegbewegen,B,wegbewegen,VVFIN,V,94.999748,0.800000,wegbewegen


## translate_lex.data_tag 

In [613]:
# temp
mask1 = translate_lex.data  == 'ausgerutscht'
mask2 = translate_lex.lemma == 'ausrutschen'
mask = mask1  &  mask2
translate_lex[mask1]

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower
816612,ausgerutscht,B,ausgerutscht,VVPP,A,0.399039,merkmal Ptz2,ausrutschen,B,ausrutschen,VVFIN,V,666.286865,0.75,ausrutschen
1175331,ausgerutscht,B,ausgerutscht,ADJA,A,1.361831,merkmal lemma,ausgerutscht,B,ausgerutscht,ADJA,A,1.298455,1.00,ausgerutscht
1232691,ausgerutscht,B,ausgerutscht,ADJA,A,0.399039,merkmal Ptz2,ausrutschen,B,ausrutschen,VVFIN,V,666.286865,0.75,ausrutschen


In [614]:
# Beispiel anzeigen
mask = translate_lex.data == 'Ackermann'
translate_lex[mask]

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower
1060035,Ackermann_1,B,Ackermann,NE,N,471.455069,merkmal lemma,Ackermann_1,B,Ackermann,NE,N,449.514771,1.0,ackermann
1064575,Ackermann,B,Ackermann,NN,N,242.296289,merkmal lemma,Ackermann,B,Ackermann,NN,N,231.020447,1.0,ackermann


In [615]:
# Beispiel anzeigen
mask = translate_lemma.lem == 'Ackermann'
translate_lemma[mask]

,lem_id,lem_home,lem,lem_tag,lem_tagZZ,lem_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower
39409,Ackermann,B,Ackermann,NN,N,31.476034,merkmal alt_1,Ackersmann,B,Ackersmann,NN,N,94.428101,0.9,ackersmann


In [616]:
# pflege_translate_tabelle
#from x32_Funktionen import *
translate_lex   = pflege_translate_tabelle(translate_lex,   translate_tagZ, wiktionary_lemma)
translate_lemma = pflege_translate_tabelle(translate_lemma, translate_tagZ, wiktionary_lemma, translate_lex )

update_col: 1203009 cells written into existing column
update_col: 223378 cells written into existing column
update_col: 0 cells written into existing column
update_col: 1203009 cells written into new column
update_col: 1203009 cells written into existing column
rank: 358353 rows less, now 844656 rows
update_col: 62021 cells written into existing column
update_col: 62021 cells written into new column
update_col: 62021 cells written into existing column


In [617]:
# Nachschlagetabelle vorbereiten
spalten = ['data_id','data','data_tag','lemma_id','lemma','lemma_tag','lemma_tagZZ']
mask = wiktionary_merkmal.is_lex == True
df = wiktionary_merkmal[mask][spalten].drop_duplicates().sort_values(spalten)
#df

In [618]:
# Prüfe Anzahl data_tag
mask = translate_lex.data_tag != ''
check_mask(translate_lex,mask,562350)

check_mask:  548736 rows


In [619]:
# Vorhandene data_tag übertragen
# sonst leeren
translate_lex['data_tag_0'] = translate_lex.data_tag
translate_lex['data_tag_1'] = translate_lex.data_tag
translate_lex = pak.move_cols(translate_lex, ['data_tag','data_tag_0','data_tag_1'],'data')

In [620]:
# Beispiel anzeigen
mask = translate_lex.data == 'Ackermann'
translate_lex[mask]

,data_id,data_home,data,data_tag,data_tag_0,data_tag_1,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower
1000770,Ackermann_1,B,Ackermann,NE,NE,NE,N,471.455069,merkmal lemma,Ackermann_1,B,Ackermann,NE,N,449.514771,1.0,ackermann
1005310,Ackermann,B,Ackermann,NN,NN,NN,N,242.296289,merkmal lemma,Ackermann,B,Ackermann,NN,N,231.020447,1.0,ackermann


In [621]:
# data_tag übertragen 
translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma_id'],            col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma_id'],            col='data_tag', col_rename='data_tag_1', func='last',  cond='null')
           
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_id'],               col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_id'],               col='data_tag', col_rename='data_tag_1', func='last',  cond='null')
  
translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma','lemma_tag'],   col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma','lemma_tag'],   col='data_tag', col_rename='data_tag_1', func='last',  cond='null')

translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma','lemma_tagZZ'], col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma','lemma_tagZZ'], col='data_tag', col_rename='data_tag_1', func='last',  cond='null')

print()
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma','lemma_tag'],      col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma','lemma_tag'],      col='data_tag', col_rename='data_tag_1', func='last',  cond='null')
   
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma','lemma_tagZZ'],    col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma','lemma_tagZZ'],    col='data_tag', col_rename='data_tag_1', func='last',  cond='null')

translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_tag'],              col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_tag'],              col='data_tag', col_rename='data_tag_1', func='last',  cond='null')

translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_tagZZ'],            col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_tagZZ'],            col='data_tag', col_rename='data_tag_1', func='last',  cond='null')

update_col: func first applied, 57910 records less!
update_col: 269228 cells written into existing column
update_col: func last applied, 57910 records less!
update_col: 269228 cells written into existing column
update_col: func first applied, 59792 records less!
update_col: 10621 cells written into existing column
update_col: func last applied, 59792 records less!
update_col: 10621 cells written into existing column
update_col: func first applied, 61634 records less!
update_col: 1258 cells written into existing column
update_col: func last applied, 61634 records less!
update_col: 1258 cells written into existing column
update_col: func first applied, 61995 records less!
update_col: 166 cells written into existing column
update_col: func last applied, 61995 records less!
update_col: 166 cells written into existing column

update_col: func first applied, 65522 records less!
update_col: 71 cells written into existing column
update_col: func last applied, 65522 records less!
update_col: 71

In [622]:
# data_tag fehlt
maskA = translate_lex.data_tag_0 == ''
maskB = translate_lex.data_tag_1 == ''
mask = maskA | maskB
check_mask(translate_lex,mask,356699)
translate_lex.loc[mask,'data_tag']   = translate_lex[mask].lemma_tag
translate_lex.loc[mask,'data_tag_0'] = translate_lex[mask].lemma_tag
translate_lex.loc[mask,'data_tag_1'] = ''
#grid(translate_lex,mask)

check_mask:  372534 rows


In [623]:
# data_tag eindeutig
mask = translate_lex.data_tag_0 == translate_lex.data_tag_1
check_mask(translate_lex,mask,804272)
translate_lex.loc[mask,'data_tag']   = translate_lex[mask].data_tag_0
translate_lex.loc[mask,'data_tag_0'] = translate_lex[mask].lemma_tag
translate_lex.loc[mask,'data_tag_1'] = ''
#grid(translate_lex,mask)

check_mask:  804048 rows


In [624]:
# data_tag uneindeutig
mask1 = translate_lex.data_tag_0 != translate_lex.data_tag_1
mask2 = translate_lex.data_tag_0 != ''
mask3 = translate_lex.data_tag_1 != ''
mask = mask1  &  mask2  &  mask3
check_mask(translate_lex,mask, 26421)
translate_lex.loc[mask,'data_tag']   = translate_lex[mask].data_tag_0 + ' ' + translate_lex[mask].data_tag_1
#grid(translate_lex,mask)

check_mask:  26427 rows


In [625]:
# Sicherstellen: data_tag fehlt nirgens komplett
mask = translate_lex.data_tag == ''
check_mask(translate_lex,mask,0)
#grid(translate_lex,mask)

check_mask:  0 rows


In [626]:
# nach Priorität sortieren (laut Steuertabellen)
translate_lex['data_tag'] = tag_sortieren(translate_lex.data_tag)

In [627]:
# translate_lex vorbereiten
translate_lex = pak.drop_cols(translate_lex,['data_tag_0','data_tag_1','data_tagZZ'])    
translate_lex = pak.rename_col(translate_lex,'data_tag','tag')    

In [628]:
# pflege_tag_felder
translate_lex = pflege_tag_felder(translate_lex, translate_tagZ)

pflege_tag_felder Error: 381 Datensätze haben kein eindeutiges tagZZ


In [629]:
# # translate_lex nachbereiten
translate_lex = pak.rename_col(translate_lex,'tag',   'data_tag')    
translate_lex = pak.rename_col(translate_lex,'tag_0', 'data_tag_0')   
translate_lex = pak.rename_col(translate_lex,'tag_1', 'data_tag_1')   
translate_lex = pak.rename_col(translate_lex,'tagZZ', 'data_tagZZ')   
translate_lex = pak.drop_cols(translate_lex,['tagZ'])    
grid(translate_lex,pak.sample)

19 rows out of 1203009


,data_id,data_home,data,data_tag,data_tag_0,data_tag_1,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower
0,"1,5-Grad-Zieles",T,"1,5-Grad-Zieles",NN,NN,vJUZNl6IBS,N,816.456153,wiki Deklinierte_Form gm lma alt_1 lmb,"1,5-Grad-Ziel",B,"1,5-Grad-Ziel",NN,N,2508.954590,0.866667,"1,5-grad-ziel"
19,1-minütige,T,1-minütige,ADJA,ADJA,VZszKeXYPB,A,36.979410,wiki Deklinierte_Form gm lma alt_2,1-minütig,B,1-minütig,ADJA,A,101.337914,0.900000,1-minütig
1673,Ahornblattes,T,Ahornblattes,NN,NN,ej27UhgfDd,N,117.835626,wiki Deklinierte_Form gm lma alt_1 lmb,Ahornblatt,B,Ahornblatt,NN,N,362.903748,0.833333,ahornblatt
5088,Bearbeitungsstaus,T,Bearbeitungsstaus,NN,NN,Dnr2FEYp1G,N,140.510450,wiki Deklinierte_Form gm lma alt_1 lmb Genitiv,Bearbeitungsstau,B,Bearbeitungsstau,NN,N,403.343994,0.941176,bearbeitungsstau
56806,aase,T,aase,VVFIN VVIMP,VVFIN,VVIMP,V,52.993995,wiki Konjugierte_Form Verb gm lma alt_1 lmb Imperativ,aasen,B,aasen,VVFIN,V,155.432648,0.800000,aasen
63099,aß,T,aß,VVFIN,VVFIN,x36IH4otls,V,1.331895,wiki Konjugierte_Form Verb gm lmb alt_2 schweizerisch,essen,B,essen,VVFIN,V,2889.576904,0.000000,essen
63926,beeinflusst_2,T,beeinflusst,VVPP VVFIN,VVPP,VVFIN,AV,0.357671,wiki Konjugierte_Form Verb gm lmb alt,beeinflussen,B,beeinflussen,VVFIN,V,356.379883,0.833333,beeinflussen
78929,falschlägest,T,falschlägest,VVFIN,VVFIN,JnY5Waghbb,V,27.104041,wiki Konjugierte_Form Verb gm lma alt_1 lmb,falschliegen,B,falschliegen,VVFIN,V,80.918549,0.750000,falschliegen
79013,fang_ein,T,fang ein,VVIMP,VVIMP,obvkweBVLz,V,77.243549,wiki Konjugierte_Form Verb gm lma alt_1 lmb,einfangen,B,einfangen,VVFIN,V,246.456619,0.777778,einfangen
106989,ruf_hinein,T,ruf hinein,VVIMP,VVIMP,fM4QONU3Sh,V,81.244708,wiki Konjugierte_Form Verb gm lma alt_1 lmb,hineinrufen,B,hineinrufen,VVFIN,V,253.338730,0.818182,hineinrufen


In [630]:
mask = translate_lex.data == 'vervollständigt'
translate_lex[mask]

,data_id,data_home,data,data_tag,data_tag_0,data_tag_1,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower
687142,vervollständigt,T,vervollständigt,ADJA VVPP,ADJA,VVPP,A,1.989476,wiki Konjugierte_Form Verb gm,vervollständigen,B,vervollständigen,VVFIN,V,59.944149,0.875,vervollständigen
972064,vervollständigt_MADJ,B,vervollständigt,VVPP,VVPP,3fMuEwoUBr,A,56.978815,merkmal Ptz2 manuell,vervollständigen,B,vervollständigen,VVFIN,V,59.944149,0.875,vervollständigen
1160404,vervollständigt_MADJ,B,vervollständigt,ADJA,ADJA,zGgjflwvEV,A,2.348767,merkmal lemma,vervollständigt_MADJ,B,vervollständigt,ADJA,A,2.239461,1.000,vervollständigt
1195767,vervollständigt_MADJ,B,vervollständigt,ADJA,ADJA,N6s2XaW90K,A,56.978815,merkmal Ptz2 manuell,vervollständigen,B,vervollständigen,VVFIN,V,59.944149,0.875,vervollständigen


In [631]:
#mask = wiktionary_lemma.tag.str.contains(' ')
#wiktionary_lemma[mask]

In [632]:
# Beispiel anzeigen
mask = translate_lex.data == 'lachen'
translate_lex[mask]

,data_id,data_home,data,data_tag,data_tag_0,data_tag_1,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower
1002903,lachen,B,lachen,VVINF,VVINF,l1rHrE9gis,V,717.660129,merkmal lemma,lachen,B,lachen,VVFIN,V,684.262085,1.0,lachen


In [633]:
# temp
mask1 = translate_lex.data  == 'ausgerutscht'
mask2 = translate_lex.lemma == 'ausrutschen'
mask = mask1  &  mask2
translate_lex[mask]

,data_id,data_home,data,data_tag,data_tag_0,data_tag_1,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower
766267,ausgerutscht,B,ausgerutscht,VVPP,VVPP,VjZX6l76RJ,A,0.399039,merkmal Ptz2,ausrutschen,B,ausrutschen,VVFIN,V,666.286865,0.75,ausrutschen
1173426,ausgerutscht,B,ausgerutscht,ADJA,ADJA,oQPMqrR15j,A,0.399039,merkmal Ptz2,ausrutschen,B,ausrutschen,VVFIN,V,666.286865,0.75,ausrutschen


## Prüfen ==========

### Unique

In [634]:
# wiktionary_lemma: lemma_id vorhanden und eindeutig?
mask = wiktionary_lemma.lemma_id == ''
check_mask(wiktionary_lemma,mask,0)

assert wiktionary_lemma.lemma_id.is_unique

check_mask:  0 rows


In [635]:
# wiktionary_merkmal: data_id vorhanden und nicht widersprüchlich?
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

# Vergebene data_id nicht widersprüchlich?
mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


In [636]:
# translate_lex: data_id vorhanden und eindeutig?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

#assert translate_lex.data_id.is_unique

a = pak.same_but_different(translate_lex,same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


### Mögliche Probleme

In [637]:
# Mögliche Probleme
spalten = ['data','lemma','lemma_score']
mask =  translate_lex['same_lexlemma'] < 0.3
a = translate_lex[mask][spalten].sort_values(['lemma_score'], ascending=[False])
gridt(a)

1946 rows


In [638]:
mask = translate_lex.same_lexlemma < 0.5
a = pak.analyse_freqs(translate_lex[mask],['lemma','data'])
gridt(a)

2638 rows


### Verwaisungen

In [639]:
# translate_lex lemma_id
mask = ~translate_lex.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lex,mask,0,160)

msg = 'verwaist_lemma_id_endkontrolle'
translate_lex, translate_lex_trash = pak.move_rows( translate_lex, translate_lex_trash, mask, msg )
grid(translate_lex_trash,msg)

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


In [640]:
# translate_lemma lem_id
mask = ~translate_lemma.lem_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lemma,mask,0,32)
# translate_lemma[mask]

msg = 'verwaist_lem_id_endkontrolle'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg )
grid(translate_lemma_trash,msg)

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


In [641]:
# translate_lemma lemma_id
mask = ~translate_lemma.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lemma,mask,0)
# translate_lemma[mask]

msg = 'verwaist_lemma_id_endkontrolle'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg )
grid(translate_lemma_trash,msg)

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


In [642]:
# wiktionary_lemma.data_id (nur leeren)
mask1 =  wiktionary_lemma.data_id != ''
mask2 = ~wiktionary_lemma.data_id.isin(wiktionary_lemma.lemma_id)
mask3 = ~wiktionary_lemma.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_lemma,mask,1)

wiktionary_lemma.loc[mask,'data_id'] = ''
grid(wiktionary_lemma,mask)

check_mask:  0 rows
No rows, mask filters them all away


In [643]:
# wiktionary_merkmal lemma_id
mask = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(wiktionary_merkmal,mask,0,160)
# wiktionary_merkmal[mask]

msg = 'verwaist_lemma_id_endkontrolle'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg )
grid(wiktionary_merkmal_trash,msg)

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


In [644]:
# verwaist wiktionary_merkmal data_id?
mask1 =  wiktionary_merkmal.data_id != ''
mask2 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask3 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask4 = wiktionary_merkmal.data_id.isin(translate_lex_trash.data_id)
mask = mask1  &  mask2  &  mask3 # &  mask4
check_mask(wiktionary_merkmal,mask)

#grid(wiktionary_merkmal,mask)

check_mask:  319 rows


In [645]:
# wiktionary_merkmal data_id
mask1 =  wiktionary_merkmal.data_id != ''
mask2 =  wiktionary_merkmal.is_lex == True
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal,mask,0,40)

msg = 'verwaist_data_id_endkontrolle'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg )
grid(wiktionary_merkmal_trash,msg)

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


In [646]:
#grid(wiktionary_merkmal,mask)

In [647]:
#pak.analyse_freqs(wiktionary_merkmal[mask], ['merkmal','data_id'])

In [648]:
# wiktionary_merkmal_text lemma_id
mask = ~wiktionary_merkmal_text.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(wiktionary_merkmal_text,mask,0)
# wiktionary_merkmal_text[mask]

#msg = 'verwaist_lemma_id_endkontrolle'
#wiktionary_merkmal_text, df = pak.move_rows( wiktionary_merkmal_text, mask, msg )
#grid(df)

check_mask:  0 rows


### NaN

In [649]:
assert not pak.any_nan(wiktionary_lemma)
#pak.nnan(wiktionary_lemma)

In [650]:
assert not pak.any_nan(wiktionary_merkmal)
#pak.nnan(wiktionary_merkmal)

In [651]:
assert not pak.any_nan(wiktionary_merkmal_text)
#pak.nnan(wiktionary_merkmal_text)

In [652]:
assert not pak.any_nan(translate_lemma)
#pak.nnan(translate_lemma)

In [653]:
assert not pak.any_nan(translate_lex)
#pak.nnan(translate_lex)

### Leer

In [654]:
# Kein tag
mask = wiktionary_lemma.tag == ''
check_mask(wiktionary_lemma,mask,0)

check_mask:  0 rows


In [655]:
translate_lex

,data_id,data_home,data,data_tag,data_tag_0,data_tag_1,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower
0,"1,5-Grad-Zieles",T,"1,5-Grad-Zieles",NN,NN,vJUZNl6IBS,N,816.456153,wiki Deklinierte_Form gm lma alt_1 lmb,"1,5-Grad-Ziel",B,"1,5-Grad-Ziel",NN,N,2508.954590,0.866667,"1,5-grad-ziel"
1,"1,5-Grad-Ziels",T,"1,5-Grad-Ziels",NN,NN,1gOYVwnSBL,N,842.157625,wiki Deklinierte_Form gm lma alt_1 lmb,"1,5-Grad-Ziel",B,"1,5-Grad-Ziel",NN,N,2508.954590,0.928571,"1,5-grad-ziel"
2,1-Cent-Münzen,T,1-Cent-Münzen,NN,NN,EOlVsQ9t5H,N,37.159276,wiki Deklinierte_Form gm lma alt_2 lmb,1-Cent-Münze,B,1-Cent-Münze,NN,N,45.220615,0.923077,1-cent-münze
3,1-Cent-Stücke,T,1-Cent-Stücke,NN,NN,X61WNE5mw4,N,46.409997,wiki Deklinierte_Form gm Dativ-e lma alt_2 lmb alt_1 Dativ,1-Cent-Stück,B,1-Cent-Stück,NN,N,53.207848,0.923077,1-cent-stück
4,1-Cent-Stücken,T,1-Cent-Stücken,NN,NN,3Lw5EdPVLI,N,38.718567,wiki Deklinierte_Form gm lma alt_2 lmb,1-Cent-Stück,B,1-Cent-Stück,NN,N,53.207848,0.857143,1-cent-stück
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1203004,stund,T,stund,VVFIN,VVFIN,Cc38duCVPc,V,0.725357,lmb alt_1,stehen,B,stehen,VVFIN,V,6346.693359,0.500000,stehen
1203005,veralt,T,veralt,ADV,ADV,jP6RTlyxXQ,A,0.630297,lmb alt_1,vlt.,B,vlt.,ADV,A,87.709984,0.500000,vlt.
1203006,Ainus,T,Ainus,NN,NN,KAopbUNzqy,N,1.173891,lmb alt_1 Genitiv,Ainu_1,L,Ainu,NN,N,17.936787,0.800000,ainu
1203007,Wiccas,T,Wiccas,NN,NN,yNChe4SXxK,N,1.483233,lmb alt_1 Genitiv,Wicca_1,L,Wicca,NN,N,24.625883,0.833333,wicca


### Div

In [656]:
# lemma_id und data_id müssen gleich sein
mask1 = wiktionary_merkmal.merkmal == 'lemma'
mask2 = wiktionary_merkmal.lemma_id != wiktionary_merkmal.data_id
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows


In [657]:
# lemma und data müssen gleich sein
mask1 = wiktionary_merkmal.merkmal == 'lemma'
mask2 = wiktionary_merkmal.lemma != wiktionary_merkmal.data
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows


### Partizipien prüfen

In [658]:
#suche_L = 'missraten' 
#suche_P = 'missraten'  

suche_L = 'gefriertrocknen' 
suche_P = 'gefriergetrocknet'  

In [659]:
# wiktionary_merkmal
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.data  == suche_P
mask = mask1  &  mask2
wiktionary_merkmal[mask]

,lemma_id,merkmal,is_lex,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,lemma_tagZZ,num,meta,merkmal_order,data_score,hunspell
2021566,gefriertrocknen,Ptz2,True,gefriergetrocknet,gefriergetrocknet,VVPP,ADJ,A,gefriertrocknen,VVFIN,V,,manuell,1.0,11.211962,0
2400407,gefriergetrocknet,lemma,True,gefriergetrocknet,gefriergetrocknet,ADJA,ADJ,A,gefriergetrocknet,ADJA,A,,,0.0,1.731048,0
2593672,gefriertrocknen,Ptz2,True,gefriergetrocknet,gefriergetrocknet,ADJA,ADJ,A,gefriertrocknen,VVFIN,V,,manuell,11.0,11.211962,0


In [660]:
# das Verb in wiktionary_lemma
mask1 = wiktionary_lemma.lemma == suche_L
mask2 = wiktionary_lemma.tagZZ == 'V'
mask = mask1  &  mask2

suche_L_id = wiktionary_lemma[mask].iloc[0].lemma_id
wiktionary_lemma[mask]

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score,lex_anz,nolex_anz
2074,gefriertrocknen,1055347,0,gefriertrocknen,gefriertrocknen,False,1,VVFIN,VVFIN,0dFJzT9Nt6,VER,V,manuell,,114.359077,18,18


In [661]:
# das Adjektiv in wiktionary_lemma
mask1 = wiktionary_lemma.lemma == suche_P
mask2 = wiktionary_lemma.tagZZ == 'A'
mask3 = wiktionary_lemma.member.str.contains('Ptz')
mask4 = wiktionary_lemma.data_id == suche_L_id
mask = mask1  &  mask2  &  mask3  &  mask4

suche_P_id = wiktionary_lemma[mask].iloc[0].lemma_id
wiktionary_lemma[mask]

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score,lex_anz,nolex_anz
2073,gefriergetrocknet,274097,0,gefriergetrocknet,gefriergetrocknet,False,1,ADJA,ADJA,O8FNnXgrtG,ADJ,A,Ptz2 noSteig manuell,gefriertrocknen,17.31048,1,4


In [662]:
# Das Ptz-Merkmal in wiktionary_lemma
mask1 = wiktionary_merkmal.data == suche_P
mask2 = wiktionary_merkmal.merkmal.str.startswith('Ptz')
mask3 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask4 = wiktionary_merkmal.lemma_id == suche_L_id   # zeigt aufs Verb
mask5 = wiktionary_merkmal.data_id  == suche_P_id   # zeigt aufs Adjektiv
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
wiktionary_merkmal[mask]

,lemma_id,merkmal,is_lex,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,lemma_tagZZ,num,meta,merkmal_order,data_score,hunspell
2021566,gefriertrocknen,Ptz2,True,gefriergetrocknet,gefriergetrocknet,VVPP,ADJ,A,gefriertrocknen,VVFIN,V,,manuell,1.0,11.211962,0
2593672,gefriertrocknen,Ptz2,True,gefriergetrocknet,gefriergetrocknet,ADJA,ADJ,A,gefriertrocknen,VVFIN,V,,manuell,11.0,11.211962,0


### Testlexeme

In [697]:
# lex_test, lemma_test
from x32_Funktionen import *
lex_test, lemma_test = lade_lex_text( lex_test_filename, level=3 )
#grid(lemma_test)

185 rows loaded
Delete 3 rows from 185


In [698]:
# lemma_test
from x32_Funktionen import *
lemma_test = check_lemma_test(lemma_test, wiktionary_lemma)
#lemma_test

In [699]:
# Fehler anzeigen
maskA = lemma_test.lemma_id == ''
maskB = lemma_test.check_tag == ''
maskC = lemma_test.check_member == ''
mask = maskA | maskB | maskC
check_mask(lemma_test,mask)

probleme = list(lemma_test[mask].lemma)
lemma_test[mask]

check_mask:  13 rows


,level,lemma_id,lemma,lemma_tag_soll,lemma_tag_ist,check_tag,lemma_member_soll,lemma_member_ist,check_member
36,3,,d. h.,KON,,,,,ok
37,3,,d.h.,KON,,,,,ok
57,3,erregend,erregend,ADJA,ADJA,OK,Ptz1,,
59,3,,EUR,NN,,,,,ok
94,3,naheliegend,naheliegend,ADJA,ADJA,OK,Ptz1,,
121,3,,tugendhaftestes,ADJA,,,,,ok
150,3,,z.B.,KOUS,,,,,ok
151,3,,z.T.,KOMBI,,,,,ok
152,3,,z.T.,KOMBI,,,manuell,,
155,3,,zu viele,ADV,,,,,ok


In [690]:
gridt(lemma_test)

162 rows


In [666]:
# Sicherstellen: Keine Fehler
check_mask(lemma_test,mask,0)

Exception: ERROR: 17 rows, but it should be a maximum of 0

### lex_test

In [ ]:
# lex_test
from x32_Funktionen import *
lex_test = check_lex_test(lex_test, translate_lex)
#lex_test

In [ ]:
# Fehler
maskA = lex_test.lex_id == ''
maskB = lex_test.check_tag == ''
mask = maskA | maskB 
check_mask(lex_test,mask)

probleme = list(lex_test[mask].lemma)
lex_test[mask]

In [ ]:
# Sicherstellen: Keine Fehler
check_mask(lex_test,mask,0)

In [ ]:














bgc('green')

### Fehler analysieren

In [720]:
sucheL = ['schwimmend']
suche  = ['schwimmend']

In [721]:
mask = lemma_test.lemma.isin(suche + sucheL)
lemma_test[mask]

,level,lemma_id,lemma,lemma_tag_soll,lemma_tag_ist,check_tag,lemma_member_soll,lemma_member_ist,check_member
110,3,schwimmend,schwimmend,ADJA,ADJA,OK,Ptz1,Ptz1 manuell noSteig,OK


In [722]:
mask = lex_test.lex.isin(suche + sucheL)
lex_test[mask]

,level,lex,lemma,lex_tag_soll,lex_member_soll,notiz,check_tag
125,3,schwimmend,schwimmend,ADJA,Ptz1,Partizip,


In [723]:
mask = translate_lex.data.isin(suche + sucheL)
translate_lex[mask]

,data_id,data_home,data,data_tag,data_tag_0,data_tag_1,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower
111828,schwimmend,B,schwimmend,ADJA,ADJA,MVoS3CxYIB,A,8328.683751,merkmal Ptz1 manuell,schwimmen,B,schwimmen,VVFIN,V,24992.593750,0.9,schwimmen
111829,schwimmend,B,schwimmend,VVFIN,VVFIN,h1qTNU73dG,V,8328.683751,merkmal Ptz1 manuell,schwimmen,B,schwimmen,VVFIN,V,24992.593750,0.9,schwimmen
1003071,schwimmend,B,schwimmend,ADJA,ADJA,ukubuPI7s4,A,281.875028,merkmal lemma,schwimmend,B,schwimmend,ADJA,A,268.757294,1.0,schwimmend
1151359,schwimmend,B,schwimmend,ADJA,ADJA,dbMHyZHsN7,A,0.184856,merkmal lemma,schwimmend_1,B,schwimmend,ADJA,A,0.176253,1.0,schwimmend


In [724]:
mask = translate_lex_trash.data.isin(suche + sucheL)
translate_lex_trash[mask]

,data_id,data,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower,nolemma_id,msg,data_home,lemma_home,merkmal_order,data_tag


In [725]:
mask = translate_lemma.lem.isin(suche + sucheL)
translate_lemma[mask]

,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,lem_id,lem_home,lem,lem_tag,lem_tagZZ,lem_score,same_lexlemma,lemma_lower
3633,gm,schwimmen,B,schwimmen,VVFIN,V,24992.59375,schwimmend_1,B,schwimmend,ADJA,A,0.058751,0.9,schwimmen


In [726]:
mask = wiktionary_lemma.lemma.isin( sucheL + suche )
wiktionary_lemma[mask]

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score,lex_anz,nolex_anz
2341,schwimmend,242155,0,schwimmend,schwimmend,False,1,ADJA,ADJA,KfU1dH54he,ADJ,A,Ptz1 manuell noSteig,schwimmen,268.757294,1,15
150631,schwimmend_1,242155,1,schwimmend,schwimmend,False,1,ADJA,ADJA,j7BOd5xDKF,ADJ,A,Ptz1,schwimmen,0.176253,1,3


In [727]:
mask1 = wiktionary_merkmal.data.isin(suche + sucheL)
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
wiktionary_merkmal[mask].sort_values(['lemma_id','merkmal','data'])

,lemma_id,merkmal,is_lex,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,lemma_tagZZ,num,meta,merkmal_order,data_score,hunspell
950839,schwimmen,Ptz1,True,schwimmend_1,schwimmend,VVFIN,VER,V,schwimmen,VVFIN,V,,,1.0,0.058751,1
2021905,schwimmen,Ptz1,True,schwimmend,schwimmend,VVFIN,VER,V,schwimmen,VVFIN,V,,manuell,2.0,8328.625000,1
2568373,schwimmen,Ptz1,True,schwimmend_1,schwimmend,ADJA,ADJ,A,schwimmen,VVFIN,V,,,11.0,0.058751,1
2594011,schwimmen,Ptz1,True,schwimmend,schwimmend,ADJA,ADJ,A,schwimmen,VVFIN,V,,manuell,12.0,8328.625000,1
2400675,schwimmend,lemma,True,schwimmend,schwimmend,ADJA,ADJ,A,schwimmend,ADJA,A,,,0.0,268.757294,1
2548964,schwimmend_1,lemma,True,schwimmend_1,schwimmend,ADJA,ADJ,A,schwimmend,ADJA,A,,,0.0,0.176253,1


In [728]:
grid(lex_test)

182 rows


,level,lex,lemma,lex_tag_soll,lex_member_soll,notiz,check_tag
0,1,…,…,$(,manuell,,
1,3,°,Grad,$(,Abkürzung,,
2,3,€,Euro,NN,Abkürzung,,
3,1,Ackermann,Ackermann,NE,Person_nachname,,
5,1,Alben,Album,NN,,,
...,...,...,...,...,...,...,...
180,3,o.ä.,o.ä.,KOMBI,Abkürzung,,
181,3,jdm,jemand,PIAT,Abkürzung,kürzt nicht das Lemma ab,
182,3,jds,jemand,PIAT,Abkürzung,kürzt nicht das Lemma ab,
183,3,nen,ein,ART,Abkürzung,kürzt nicht das Lemma ab,


In [729]:
raise Stop

Stop Time:  00:09:36
Elapsed:    1 hour, 32 mins, 56 secs


## Speichern ==========

In [ ]:
assert gründlich

In [ ]:
# Speichern  
pak.dump_pickle(wiktionary_lemma,         wiktionary_lemma_filename_save)   
pak.dump_pickle(wiktionary_lemma_fix,     wiktionary_lemma_fix_filename_save)   
pak.dump_pickle(wiktionary_merkmal,       wiktionary_merkmal_filename_save) 
pak.dump_pickle(wiktionary_merkmal_text,  wiktionary_merkmal_text_filename_save)  
pak.dump_pickle(translate_lemma,          translate_lemma_filename_save) 
pak.dump_pickle(translate_lex,            translate_lex_filename_save)
pak.dump_pickle(translate_lex_trash,      translate_lex_trash_filename_save)
os.utime(verzeichnis_save)

In [ ]:
# drop_cols
wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal, ['data_id_old','data_id_bak'])

In [ ]:
# Save in Google Drive
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av {verzeichnis_save} /content/drive/MyDrive

In [ ]:
raise Stop

## suche_error ==========

In [ ]:
# Beispiel anzeigen: Mehrdeutigkeit
# Welcher Lemma soll einem Lexem (data) zugeordnet werden?
# Die Zeile mit dem höchsten data_score wird ausgewählt.
mask = translate_lex.data == 'Alben'
translate_lex[mask].sort_values('data_score', ascending=False)

In [ ]:
# Veränderungen lemma_tag >> data_tag
# nach Merkmal
mask1 = wiktionary_merkmal.data_tag != wiktionary_merkmal.lemma_tag
mask2 = wiktionary_merkmal.data_tag != ''
mask = mask1  &  mask2
pak.analyse_freqs(wiktionary_merkmal[mask], ['merkmal','lemma_tag','data_tag',])

In [ ]:
suche_debug = [suche_error[1]]
suche_debug

In [730]:
# suche_debug wiktionary_lemma
mask = wiktionary_lemma.lemma.isin(suche_debug)
(wiktionary_lemma[mask])

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score,lex_anz,nolex_anz


In [731]:
# suche_debug wiktionary_lemma_trash
mask = wiktionary_lemma_trash.lemma.isin(suche_debug)
(wiktionary_lemma_trash[mask])

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score,msg


In [732]:
# translate_lex
search_str(translate_lex, suche_debug)

,data_id,data_home,data,data_tag,data_tag_0,data_tag_1,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,same_lexlemma,lemma_lower
739854,Euro~€,T,€,NN,NN,mEK2Cgm80G,N,443.856976,manuell Abkürzung Einheit manuell,Euro,B,Euro,NN,N,1403.598999,0.0,euro


In [733]:
# suche_debug translate_lemma
search_str(translate_lemma, suche_debug)

,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,lem_id,lem_home,lem,lem_tag,lem_tagZZ,lem_score,same_lexlemma,lemma_lower


In [734]:
# wiktionary_merkmal
mask = wiktionary_merkmal.lemma.isin(suche_debug)
grid(wiktionary_merkmal,mask)

No rows, mask filters them all away


In [735]:
# wiktionary_merkmal
mask = wiktionary_merkmal.data.isin(['besonne'])
grid(wiktionary_merkmal,mask)

2 rows out of 2586444


,lemma_id,merkmal,is_lex,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,lemma_tagZZ,num,meta,merkmal_order,data_score,hunspell
1586381,besonnen_2,Imp Sg,True,besonne,besonne,VVIMP,VER,V,besonnen,VVFIN,V,,,2.0,5.576679,1
1980453,besonnen_2,Präsens ich,True,besonne,besonne,VVFIN,VER,V,besonnen,VVFIN,V,,,1.0,5.576679,1


In [736]:
# suche_debug wiktionary_merkmal
mask = wiktionary_merkmal.is_lex == True
a = search_str(wiktionary_merkmal[mask], suche_debug)
grid(a.sort_values(['lemma_id','merkmal']))

No rows


In [737]:
# suche_debug wiktionary_merkmal
mask = wiktionary_merkmal.is_lex == True
a = search_str(wiktionary_merkmal[mask], suche_debug)
grid(a.sort_values(['lemma_id','merkmal']))

No rows


In [738]:
# suche_debug wiktionary_merkmal_text
mask = wiktionary_merkmal.lemma_id.isin(suche_debug)
ids = set(wiktionary_merkmal[mask].lemma_id)
mask = wiktionary_merkmal_text.lemma_id.isin(ids)
grid(wiktionary_merkmal_text,mask)

No rows, mask filters them all away


In [739]:
raise Stop

Stop Time:  00:09:48
Elapsed:    1 hour, 33 mins, 7 secs


## Fragmente

In [ ]:
raise Stop

In [ ]:
# rename_col 
translate_lex = pak.rename_col(translate_lex, 'data_id',    'lex_id')
translate_lex = pak.rename_col(translate_lex, 'data_home',  'lex_home')
translate_lex = pak.rename_col(translate_lex, 'data',       'lex')
translate_lex = pak.rename_col(translate_lex, 'data_score', 'lex_score')

translate_lex_full = pak.rename_col(translate_lex_full, 'data_id',    'lex_id')
translate_lex_full = pak.rename_col(translate_lex_full, 'data_home',  'lex_home')
translate_lex_full = pak.rename_col(translate_lex_full, 'data',       'lex')
translate_lex_full = pak.rename_col(translate_lex_full, 'data_score', 'lex_score')

translate_lemma = pak.rename_col(translate_lemma, 'same_lexlemma', 'same_lemlemma')

In [ ]:
# rank!
#translate_lex = pak.rank(translate_lex, col_score='data_score', cols_group='data_id', on_conflict='first')

In [ ]:
# rank!
# Ein data_id pro data und lemma_tagZZ.
vorher = translate_lex.shape[0]
translate_lex = pak.rank(translate_lex, col_score='data_score', cols_group=['data','lemma_tagZZ'], on_conflict='first')
print(vorher - translate_lex.shape[0], 'Datensätze gelöscht')

In [ ]:
# Dups in data + lemma: Sind stets verschiedene lemma_tagZZ
mask = translate_lex.duplicated(subset=['data','lemma'], keep=False)
a = translate_lex[mask].sort_values('data_id')
#grid(a)

In [ ]:
# Konfiguration Dups entfernen
aggwords    =  pak.agg_words_nodup
col_origins =  [ 'data_id',  'data_id2', 'data_id_bak', 'data', 'data_tag', 'data_tagZZ', 'data_score', 'member',  'lemma_id', 'lemma', 'lemma_tag', 'lemma_tagZZ', 'lemma_lower', 'lemma_score', 'data_home', 'lemma_home',  'same_lexlemma',  'score', ]
col_funcs   =  [ 'max',      'min',      'max',         '',     '',         '',           'sum',        aggwords,  '',         '',      '',          '',            '',            'sum',         'first',     'first',       'first',          'sum',   ]   
col_names   =  [ 'data_id',  'data_id2', 'data_id_bak', '',     '',         '',           'data_score', 'member',  '',         '',      '',          '',            '',            'lemma_score', 'data_home', 'lemma_home',  'same_lexlemma',  'score', ] 

In [ ]:
# Gruppieren!
translate_lex = pak.group_and_agg( translate_lex, col_origins, col_funcs, col_names, optimize=True)

Executing shutdown due to inactivity...


2023-03-04 10:00:49,443 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2023-03-04 10:00:49,486 - INFO     - Executing shutdown...


Exception on /shutdown [GET]
Traceback (most recent call last):
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 435, in shutdown
    shutdown_server()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 421, in shutdown_server
    raise RuntimeError("Not 

2023-03-04 10:00:49,490 - ERROR    - Exception on /shutdown [GET]
Traceback (most recent call last):
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 435, in shutdown
    shutdown_server()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 421, in shutdow